In [1]:
'''Generating grassmannians tableaux'''
#Import libraries
import numpy as np
import sympy as sp
from typing import List
from sage.combinat.shuffle import ShuffleProduct
import itertools
from numpy import matrix 
from itertools import combinations as comb
import multiprocessing 
#from sage.parallel.multiprocessing_sage import Pool
from operator import add
import operator
from sys import getsizeof
import random
from numpy import array
from collections import Counter
import functools
import copy
import scipy 
import matplotlib.pyplot as plt
import networkx as nx
from multiprocessing import get_context
from sage.combinat.root_system.associahedron import Associahedra
from fractions import Fraction
from scipy.special import bernoulli, zeta 
import numbers
import decimal

    
####################################################################
#Define relevant functions

def sub_tableau(T1,l):
    r=[]
    T=TransposeOfRectangularTableau(T1)
    for i in T:
        t1=[]
        for j in i:
            if IsInList(j,l)==1:
                t1.append(j)
        if t1!=[]:
            r.append(t1)
    r=TransposeTableauSLk(r)
    return r

def is_rectangular_tableau(T): # T[i]'s can be rows or columns
    r=1
    if len(T)>=2:
        l=len(T[0])
        for i in [1..len(T)-1]:
            if l!=len(T[i]):
                r=0
                break
    return r

def cyclic_intervals(l,n): # cyclic intervals in [1,...,n] with length l
    r=[]
    for i in [1..n]:
        t1=ListAToN_cyclic(i,ModResultOneToN(i+l-1,n),n)
        r.append(t1)
    return r

def is_union_of_consecutive_tableaux(T,k,n): # check if T=T_1\cup T_2, where T_i is has consecutive content
    r=0
    m=len(T)
    for jj in [1..m-1]: 
        r1=cyclic_intervals(k*jj,n)
        for l in r1:
            T1=sub_tableau(T, l)
            #print(l,T1)
            if is_rectangular_tableau(T1)==1 and len(T1)==jj and IsSemiStandardTableauTIsList(T1)==1:
                l2=SetDifference(ListAToN(1,n),l)
                T2=sub_tableau(T, l2)
                #print(T1,T2)
                if is_rectangular_tableau(T2)==1 and len(T2)==m-jj and IsSemiStandardTableauTIsList(T2)==1 and UnionOfTwoTableaux(T1,T2)==T:
                    r=1
                    #print(T1,T2)
                    return r
    return r

def has_size_m_in_list_of_lists(l,m):
    r=0
    for i in l:
        if len(i)==m:
            r=1
            break
    return r

def tableau_m_column_to_blocks_half(T,n): # T is a m-column standard tableau, m \ge 2, at the moment T has 3 rows
    if flatten(T)==[]:
        r=[]
    else:
        k=len(T[0]) 
        r1=[]
        for i in T:
            t1=split_list_into_intervals_cyclic_intervals(i, n)
            r1=r1+t1
        i0=0
        r=[]
        while i0<=len(r1)-1:
            sn=0
            i=r1[i0]
            r2=SetDifference(r1, [i])
            #print(r1,r2)
            for j in r2:
                #print(i,j,k,n)
                #print(is_sub_tableau_a_column_which_is_standard_tableau(T,sorted(i+j)))
                if len(i+j)<=k and len(split_list_into_intervals_cyclic_intervals(sorted(i+j),n))==1 and is_sub_tableau_a_column_which_is_standard_tableau(T,sorted(i+j))==1:
                    if len(i+j)<k:
                        r.append(sorted(i+j))
                        r1=SetDifference(r2, [j])
                        i0=0
                        sn=1
                        #print('i,j,k', i, j, k)
                        break
                    elif len(i+j)==k: 
                        if IsSemiStandardTableau(TableauToMatrix(TableauDivision(T,[sorted(i+j)])))==1:
                            r.append(sorted(i+j))
                            r1=SetDifference(r2, [j])
                            i0=0 
                            sn=1
                            #print('i,j,k', i, j, k)
                            break
                    
            if sn==0:
                if len(i)<k:
                    r.append(i)
                    i0=0
                    r1=r2
                else:
                    r.append(i)
                    i0=0
                    r1=r2
    return r

def tableau_m_column_to_blocks(T,n): # T is a m-column standard tableau, m \ge 2, at the moment T has 3 rows
    r1=tableau_m_column_to_blocks_half(T,n)
    r=[]
    k=len(T[0])
    if r1!=[]:
        i0=0
        r=[]
        while i0<=len(r1)-1:
            sn=0
            i=r1[i0]
            r2=SetDifference(r1, [i])
            #print(r1,r2)
            for j in r2:
                #print(i,j,k,n)
                #print(is_sub_tableau_a_column_which_is_standard_tableau(T,sorted(i+j)))
                if len(i+j)<=k and len(split_list_into_intervals_cyclic_intervals(sorted(i+j),n))==1 and is_sub_tableau_a_column_which_is_standard_tableau(T,sorted(i+j))==1:
                    if len(i+j)<k:
                        r.append(sorted(i+j))
                        r1=SetDifference(r2, [j])
                        i0=0
                        sn=1
                        #print('i,j,k', i, j, k)
                        break
                    elif len(i+j)==k: 
                        if IsSemiStandardTableau(TableauToMatrix(TableauDivision(T,[sorted(i+j)])))==1:
                            r.append(sorted(i+j))
                            r1=SetDifference(r2, [j])
                            i0=0 
                            sn=1
                            #print('i,j,k', i, j, k)
                            break
                    
            if sn==0:
                if len(i)<k:
                    r.append(i)
                    i0=0
                    r1=r2
                else:
                    r.append(i)
                    i0=0
                    r1=r2
        
    return r


#Function for multiprocessing 
def ll_perms_compute_cluster_variables_only_standard_tableaux_and_all_numbers_distinct(lli,typ,rank,max_column,n,repeat,b6,fp1): #  
    IC=InitialClusterWithoutNumericalMatrices(rank,n)
    mat1=IC[0]
    vertices1=IC[1]
    #e1=initial_cluster_variables_DFGK(rank,n)
    
    #print('e1',e1)
    
    #print('rr')
    #ll=list(np.array(lli)-1)
    ll=list(map(lambda x: x - 1, lli))
    #print(ll)
    
    mutationSequence=[]
    for j1 in [1..repeat]: 
        mutationSequence=mutationSequence+ll 
    
    for j in range(len(mutationSequence)): 
        #print('kk')
        vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 
        
        if vertices1[1].ncols()>max_column:
            vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank) 
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
        elif IsStandardTableauInMatrix(vertices1[1])==1 and IsAllNumberDistinctInMatrix(vertices1[1])==1:
        #elif sorted(flatten(MatrixTakeColumns(vertices1[1])))==ListAToN(1,rank*vertices1[1].ncols()):
            ii0=vertices1[1]
            ii=MatrixTakeColumns(vertices1[1])
            p1=repr(ii)
            p2=b6.get(p1)
            
            #print('p1',p1)
            
            if p2==None:
                #print('ii',ii)
                
                c1=ExtendSetOfTableauxToContainPromotionsInMatrix([ii0],n) 
                #c1=[ii0]
                c1=MatrixTakeColumnsList(c1)
                f1=open(fp1, 'a+')
                for ci in c1:
                    g1=repr(ci)
                    g2=b6.get(g1)
                    if g2==None:
                        f1.write(str(ci)+'\n')
                        b6[g1]=[] 
                f1.close()
    return b6

#Function for multiprocessing 
def ll_perms_compute_cluster_variables_only_standard_tableaux_and_all_numbers_distinct_content_one_to_n(lli,typ,rank,max_column,n,repeat,b6,fp1): #  
    IC=InitialClusterWithoutNumericalMatrices(rank,n)
    mat1=IC[0]
    vertices1=IC[1]
    #e1=initial_cluster_variables_DFGK(rank,n)
    
    #print('e1',e1)
    
    #print('rr')
    #ll=list(np.array(lli)-1)
    ll=list(map(lambda x: x - 1, lli))
    #print(ll)
    
    mutationSequence=[]
    for j1 in [1..repeat]: 
        mutationSequence=mutationSequence+ll 
    
    for j in range(len(mutationSequence)): 
        #print('kk')
        vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 
        
        if vertices1[1].ncols()>max_column:
            vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank) 
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
        elif IsStandardTableauInMatrix(vertices1[1])==1 and IsAllNumberDistinctInMatrix(vertices1[1])==1:
        #elif sorted(flatten(MatrixTakeColumns(vertices1[1])))==ListAToN(1,rank*vertices1[1].ncols()):
            ii=TableauReduceEntriesToOneToN(MatrixTakeColumns(vertices1[1]))
            ii0=TableauToMatrix(ii)
            p1=repr(ii)
            p2=b6.get(p1)
            
            #print('p1',p1)
            
            if p2==None:
                #print('ii',ii)
                
                c1=ExtendSetOfTableauxToContainPromotionsInMatrix([ii0],len(ContentOfTableau(ii))) 
                #c1=[ii0]
                c1=MatrixTakeColumnsList(c1)
                f1=open(fp1, 'a+')
                for ci in c1:
                    g1=repr(ci)
                    g2=b6.get(g1)
                    if g2==None:
                        f1.write(str(ci)+'\n')
                        b6[g1]=[] 
                f1.close()
    return b6


#Function for multiprocessing 
def ll_perms_compute_cluster_variables_only_certain_contents(lli,typ,rank,max_column,n,repeat,b6,fp1,content_type,mat,vertices): #  contain_type = [1,2,3,1,2,2], number of i is content_type[i] 
    #IC=InitialClusterWithoutNumericalMatrices(rank,n)
    #mat1=IC[0]
    #vertices1=IC[1]
    #e1=initial_cluster_variables_DFGK(rank,n)
    mat1=copy.deepcopy(mat)
    vertices1=copy.deepcopy(vertices)
    
    #print('rr')
    #ll=list(np.array(lli)-1)
    ll=list(map(lambda x: x - 1, lli))
    #print(ll)
    
    mutationSequence=[]
    for j1 in [1..repeat]: 
        mutationSequence=mutationSequence+ll 
    
    for j in range(len(mutationSequence)): 
        #print('kk')
        vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 
        
        if vertices1[1].ncols()>max_column:
            vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank) 
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
        elif sorted(TableauxToListOfTimesOfOccurrenceOfNumbersNEntries(n, vertices1[1]))==sorted(content_type):
            #print(vertices1[1])
            ii0=vertices1[1]
            ii=MatrixTakeColumns(vertices1[1])
            p1=repr(ii)
            p2=b6.get(p1)
            
            #print('p1',p1)
            
            if p2==None:
                #print('ii',ii)
                
                c1=ExtendSetOfTableauxToContainPromotionsInMatrix([ii0],n) 
                c1=MatrixTakeColumnsList(c1)
                f1=open(fp1, 'a+')
                for ci in c1:
                    g1=repr(ci)
                    g2=b6.get(g1)
                    if g2==None:
                        f1.write(str(ci)+'\n')
                        b6[g1]=[] 
                f1.close()
    return [b6,mat1,vertices1]


#Function for multiprocessing 
def ll_perms_compute_cluster_variables_only_standard_tableaux(lli,typ,rank,max_column,n,repeat,b6,fp1): #  
    IC=InitialClusterWithoutNumericalMatrices(rank,n)
    mat1=IC[0]
    vertices1=IC[1]
    #e1=initial_cluster_variables_DFGK(rank,n)
    
    #print('e1',e1)
    
    #print('rr')
    #ll=list(np.array(lli)-1)
    ll=list(map(lambda x: x - 1, lli))
    #print(ll)
    
    mutationSequence=[]
    for j1 in [1..repeat]: 
        mutationSequence=mutationSequence+ll 
    
    for j in range(len(mutationSequence)): 
        #print('kk')
        vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 
        
        if vertices1[1].ncols()>max_column:
            vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank) 
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
        elif IsStandardTableauInMatrix(vertices1[1])==1:
            #print(vertices1[1])
            ii0=vertices1[1]
            ii=MatrixTakeColumns(vertices1[1])
            p1=repr(ii)
            p2=b6.get(p1)
            
            #print('p1',p1)
            
            if p2==None:
                f1=open(fp1, 'a+')
                f1.write(str(ii)+'\n')
                b6[p1]=[] 
                f1.close()
    return b6


def IsStandardTableauInMatrix(a): # a is a tableau written in the matrix form 
    n=a.nrows()
    m=a.ncols()

    r=1
    for i in [0..n-1]: 
        for j in [0..m-2]: 
            if a[i,j]>=a[i,j+1]:
                r=0
                break
    for j in [0..m-1]: 
        for i in [0..n-2]:
            if a[i,j]>=a[i+1,j]:
                r=0
                break
    return r

def IsStandardTableau(T): # a is a tableau written in list of lists form 
    a=TableauToMatrix(T)
    r=IsStandardTableauInMatrix(a)
    return r

def IsAllNumberDistinctInMatrix(M):
    T=MatrixTakeRows(M)
    t1=flatten(T)
    if len(removeDuplicates(t1))==len(t1):
        r=1
    else:
        r=0
    return r

def IsAllNumberDistinct(T):
    t1=flatten(T)
    if len(removeDuplicates(t1))==len(t1):
        r=1
    else:
        r=0
    return r

def tableau_2_column_find_cluster_variables(k,n):
    m=2
    g2=[]
    num=0
    for i in SemistandardTableaux_k_by_m(k,n,m):
        t1=TransposeOfRectangularTableau(i)
        t2=tableau_2_column_to_chain_polynomial_use_tableau_not_noncrossing_pair_with_number_of_cyclic_interval_for_row_indices(t1)
        if t2!=[] and len(t2[0])==3 and t2[1]<=1:
            g2.append(t1)
            num=num+1
            if num%1000==1:
                print(num)
    r=g2
    return r

def is_sub_tableau_a_column_and_row_indices_cyclic_consecutive(T,l): # T is rectangular tableau, subtableau 234 in [[1,3,4],[2,5,6]] is considered as in one column, l=[2,3,4]
    r=1
    k=len(T[0])
    r1=TableauToMatrix(T) 
    r2=[]
    for i in range(r1.nrows()):
        for j in range(r1.ncols()):
            if IsInList(r1[i,j], l)==1:
                r2.append(i)
    #print(r2)
    r3=removeDuplicates(r2)
    if len(r3)!=len(r2):
        r=0
    else:
        t0=[i+1 for i in r2]
        t1=is_cyclic_interval(t0,k)
        if t1==0:
            r=0
    return r


def tableau_2_column_to_chain_polynomial(T1): # T1 is a 2-column tableau
    T=tableau_to_standard_tableau(T1) 
    #print(T)
    if flatten(T)==[]:
        r=[]
    else:
        k=len(T[0])
        n=max(flatten(T))
        #print(T, k, n)
        t2=NonCrossingTupleWithSameContentAndWeightAsT(T)
        r1=split_list_into_intervals_cyclic_intervals(t2[0], n)
        r2=split_list_into_intervals_cyclic_intervals(t2[1], n)
        r3=[]
        r4=[]
        #print(r1,r2)
        for i in r1:
            sn=0
            for j in r2:
                #print(i,j,k,n)
                #print(is_sub_tableau_a_column_which_is_standard_tableau(T,sorted(i+j)))
                
                if len(i+j)<k and len(split_list_into_intervals_cyclic_intervals(sorted(i+j),n))==1 and is_sub_tableau_a_column_which_is_standard_tableau(T,sorted(i+j))==1:
#                    t1=split_list_into_intervals(sorted(i+j))
#                     if len(t1)==2:
#                         if len(t1[0])==1 or len(t1[1])==1:
#                             r3.append(sorted(i+j))
#                             r4.append(j)
#                             sn=1
#                             #print('i,j,k', i, j, k)
#                             break
#                     else:
#                         r3.append(sorted(i+j))
#                         r4.append(j)
#                         sn=1
#                         #print('i,j,k', i, j, k)
#                         break
                    r3.append(sorted(i+j))
                    r4.append(j)
                    sn=1
                    #print('i,j,k', i, j, k)
                    break
                    
            if sn==0:
                r3.append(i)
        r5=SetDifference(r2,r4)
        r3=r3+r5
        r=r3
    return r

def tableau_2_column_to_chain_polynomial_use_tableau_not_noncrossing_pair(T1): # T1 is a 2-column tableau
    T=tableau_to_standard_tableau(T1) 
    #print(T)
    if flatten(T)==[]:
        r=[]
    else:
        k=len(T[0])
        n=max(flatten(T))
        #print(T, k, n)
        t2=T
        r1=split_list_into_intervals_cyclic_intervals(t2[0], n)
        r2=split_list_into_intervals_cyclic_intervals(t2[1], n)
        r3=[]
        r4=[]
        #print(r1,r2)
        for i in r1:
            sn=0
            for j in r2:
                #print(i,j,k,n)
                #print(is_sub_tableau_a_column_which_is_standard_tableau(T,sorted(i+j)))
                if len(i+j)<k and len(split_list_into_intervals_cyclic_intervals(sorted(i+j),n))==1 and is_sub_tableau_a_column_which_is_standard_tableau(T,sorted(i+j))==1:
#                     t1=split_list_into_intervals(sorted(i+j))
#                     if len(t1)==2:
#                         if len(t1[0])==1 or len(t1[1])==1:
#                             r3.append(sorted(i+j))
#                             r4.append(j)
#                             sn=1
#                             #print('i,j,k', i, j, k)
#                             break
#                     else:
#                         r3.append(sorted(i+j))
#                         r4.append(j)
#                         sn=1
#                         #print('i,j,k', i, j, k)
#                        break
                    r3.append(sorted(i+j))
                    r4.append(j)
                    sn=1
                    #print('i,j,k', i, j, k)
                    break
                    
            if sn==0:
                r3.append(i)
        #print(r1,r2,r3)
        r5=SetDifference(r2,r4)
        r3=r3+r5
        r=r3
    return r

def tableau_2_column_to_chain_polynomial_use_tableau_not_noncrossing_pair_with_number_of_cyclic_interval_for_row_indices(T1): # T1 is a 2-column tableau
    T=tableau_to_standard_tableau(T1) 
    #print(T)
    if flatten(T)==[]:
        r=[]
    else:
        num=0
        k=len(T[0])
        n=max(flatten(T))
        #print(T, k, n)
        t2=T
        r1=split_list_into_intervals_cyclic_intervals(t2[0], n)
        r2=split_list_into_intervals_cyclic_intervals(t2[1], n)
        r3=[]
        r4=[]
        #print(r1,r2)
        for i in r1:
            sn=0
            for j in r2:
                #print(i,j,k,n)
                #print(is_sub_tableau_a_column_which_is_standard_tableau(T,sorted(i+j)))
                sij=sorted(i+j)
                if len(i+j)<k and len(split_list_into_intervals_cyclic_intervals(sij,n))==1 and is_sub_tableau_a_column_which_is_standard_tableau(T,sij)==1:
#                     t1=split_list_into_intervals(sorted(i+j))
#                     if len(t1)==2:
#                         if len(t1[0])==1 or len(t1[1])==1:
#                             r3.append(sorted(i+j))
#                             r4.append(j)
#                             sn=1
#                             #print('i,j,k', i, j, k)
#                             break
#                     else:
#                         r3.append(sorted(i+j))
#                         r4.append(j)
#                         sn=1
#                         #print('i,j,k', i, j, k)
#                        break
                    r3.append(sij)
                    r4.append(j)
                    sn=1
                    if is_sub_tableau_a_column_and_row_indices_cyclic_consecutive(T, sij)==0:
                        num=num+1
                    #print('i,j,k', i, j, k)
                    break
                    
            if sn==0:
                r3.append(i)
        #print(r1,r2,r3)
        r5=SetDifference(r2,r4)
        r3=r3+r5             
        r=[r3, num]
    return r

def tableau_2_column_to_chain_polynomial_use_tableau_not_noncrossing_pair_collect_size_3(T):
    t1=TransposeOfRectangularTableau(T)
    t2=tableau_2_column_to_chain_polynomial_use_tableau_not_noncrossing_pair(t1)
    #print(t1,t2)
    if len(t2)==3:
        r=t1
    else:
        r=[]
    return r

def BraidGroupActionSigmaI_KKOP_A2_SigmaW(lli, T, n1, max_column, max_entry, repeat, fp1, b6): 
    T1=copy.deepcopy(T)
    ll=copy.deepcopy(lli)
    braid_word=[]
    for j1 in [1..repeat]: 
        braid_word=braid_word+ll 
    for j in ReverseList(braid_word): 
        T1=BraidGroupActionSigmaI_KKOP_A2(j,T1,n1)
        if len(T1)<=max_column and max(flatten(T1))<=max_entry:
            p1=repr(T1)
            p2=b6.get(p1)
            
            if p2==None:
                f1=open(fp1, 'a+')
                f1.write(str(T1)+'\n')
                b6[p1]=[]
                f1.close()
    return b6


def BraidGroupActionSigmaI_KKOP_A2(ii,T,n1): # T is a 3-row tableau, n1 is large enough, Gr(k,n1), it seems that the result does not depend on n1
    n=2
    k=3
    l1=[[1,2],[2,1],[1,0]]
    r1=TableauToModule('Y', T)
    r2=HL_module_to_KKOP_module_An(r1, n)
    r3=braid_group_action_on_dominant_monomials_KKOP_A2(ii,r2,l1)
    r5=KKOP_module_to_HL_module_An(r3,n)
    r=ModuleToTableau('Y',r5,k,n1)
    return r

def Y_is_to_Dkab_A2(i,s,n): # b+k(n+1) = s, D(a,b)=(n+1-a, b+n+1), D^{-1}(a,b)=(n+1-a, b-n-1)
    for b in [2,1,0]:
        if (s-b)/(n+1)==int((s-b)/(n+1)):
            b1=b
            k=int((s-b)/(n+1))
            break
    a1=i
    for t1 in [1..abs(k)]:
        a1=n+1-a1
    r=[k, [a1,b1]]
    return r

def Dkab_to_Yis_A2_one(l1,n): # k=1, l1=[a,b], b+k(n+1) = s, D(a,b)=(n+1-a, b+n+1), D^{-1}(a,b)=(n+1-a, b-n-1)
    a,b=l1
    r=[n+1-a,b+n+1]
    return r

def Dkab_to_Yis_A2_minus_one(l1,n): # k=1,l1=[a,b], b+k(n+1) = s, D(a,b)=(n+1-a, b+n+1), D^{-1}(a,b)=(n+1-a, b-n-1)
    a,b=l1
    r=[n+1-a,b-n-1]
    return r

def Dkab_to_Yis_A2(l,n): # l=[k,[a,b]], b+k(n+1) = s, D(a,b)=(n+1-a, b+n+1), D^{-1}(a,b)=(n+1-a, b-n-1)
    k=l[0]
    a,b=l[1]
    r=[a,b]
    if k>0:
        for i in range(k):
            r=Dkab_to_Yis_A2_one(r,n)
    elif k<0:
        for i in range(-k):
            r=Dkab_to_Yis_A2_minus_one(r,n)
    return r

def Dkab_to_Yis_A2_list(l,n): # l=[k,[a,b]], b+k(n+1) = s, D(a,b)=(n+1-a, b+n+1), D^{-1}(a,b)=(n+1-a, b-n-1)
    r=[]
    for i in l:
        r.append(Dkab_to_Yis_A2(i,n))
    return r


def Y_is_to_Dkab_A2_list(l,n): # b+k(n+1) = s, D(a,b)=(n+1-a, b+n+1), D^{-1}(a,b)=(n+1-a, b-n-1)
    r=[]
    for t1 in l:
        i,s=t1
        t2=Y_is_to_Dkab_A2(i,s,n)
        r.append(t2)
    return r

def number_of_occurrence(i,l):
    r=0
    for j in l:
        if j==i:
            r=r+1
    return r

def dominant_monomial_to_Dke1e2e3_A2(l,n,l1): # l1=[[1,2],[2,1],[1,0]]
    r1=Y_is_to_Dkab_A2_list(l,n)
    p1=TakeIthElements(1,r1)
    p1=removeDuplicates(p1)
    r2=[]
    r3=[]
    for i in p1:
        t1=[]
        if IsInList(i,r3)==0:
            for j in r1:
                if j[0]==i:
                    t1.append(j[1])
        t2=[]
        for j in l1:
            t2.append(number_of_occurrence(j,t1))
        r2.append([i, t2])
    r=SortByIthTerms(1,r2)
    return r

def Dke1e2e3_to_Dkab_A2_one(l,l1): # l=[2, [1,2,0]], l1=[[1,2],[2,1],[1,0]]
    r=[]
    for i0 in range(len(l[1])):
        i=l[1][i0]
        t1=[]
        if i!=0:
            for j in range(i):
                t1.append([l[0], l1[i0]])
        if t1!=[]:
            r=r+t1
    return r

def Dke1e2e3_to_Dkab_A2(l,l1): # l=[[2, [1,2,0]],[3, [1,1,0]] ], l1=[[1,2],[2,1],[1,0]]
    r=[]
    for i in l:
        r=r+Dke1e2e3_to_Dkab_A2_one(i,l1)
    return r

def Dke1e2e3_to_dominant_monomials_A2(l,n,l1): # l=[[2, [1,2,0]],[3, [1,1,0]] ], l1=[[1,2],[2,1],[1,0]]
    r1=Dke1e2e3_to_Dkab_A2(l,l1)
    r=Dkab_to_Yis_A2_list(r1,n)
    return r

def filling_missing_k_in_Dke1e2e3_A2(l): 
    r1=TakeIthElements(1,l)
    r=[]
    for i in [min(r1)..max(r1)]:
        if IsInList(i,r1)==0:
            r.append([i, [0,0,0]])
    r=r+l
    r=SortByIthTerms(1,r)
    return r

def braid_group_action_on_dominant_monomials_KKOP_A2_T1_half(l,l1): # A2, l1=[[1,2],[2,1],[1,0]]
    n=2
    r1=dominant_monomial_to_Dke1e2e3_A2(l,n,l1)
    r1=filling_missing_k_in_Dke1e2e3_A2(r1)
    r=[]
    for i1 in range(len(r1)):
        i=r1[i1]
        t1=i[0]
        a,b,c=i[1]
        if i1!=0:
            a0,b0,c0=r1[i1-1][1][0], r1[i1-1][1][1], r1[i1-1][1][2]
        else:
            a0,b0,c0=0,0,0
        p1=min(a,c)
        p2=b+max(a-c,0)
        p3=b0+max(c0-a0,0)
        r.append([t1, [p1,p2,p3]])
    a0,b0,c0=a,b,c
    p3=b0+max(c0-a0,0)
    k=t1+1
    r.append([k, [0,0,p3]])
    return r

def braid_group_action_on_dominant_monomials_KKOP_A2_T1(l,l1): # A2, l1=[[1,2],[2,1],[1,0]]
    r1=braid_group_action_on_dominant_monomials_KKOP_A2_T1_half(l,l1)
    n=2
    r=Dke1e2e3_to_dominant_monomials_A2(r1,n,l1)
    return r
        
def braid_group_action_on_dominant_monomials_KKOP_A2_T2_half(l,l1): # A2, l1=[[1,2],[2,1],[1,0]]
    n=2
    r1=dominant_monomial_to_Dke1e2e3_A2(l,n,l1)
    r1=filling_missing_k_in_Dke1e2e3_A2(r1)
    r=[]
    a0,b0,c0=0,0,0
    for i1 in range(len(r1)):
        i=r1[i1]
        t1=i[0]
        a,b,c=i[1]
        if i1!=0:
            a0,b0,c0=r1[i1-1][1][0], r1[i1-1][1][1], r1[i1-1][1][2]
        p1=a0+c-min(a0,b)
        p2=min(a0,b)
        p3=b+c-min(a0,b)
        r.append([t1, [p1,p2,p3]])
    a0=copy.deepcopy(a)
    b0=copy.deepcopy(b)
    c0=copy.deepcopy(c)
    a,b,c=0,0,0
    k=t1+1
    p1=a0+c-min(a0,b)
    p2=min(a0,b)
    p3=b+c-min(a0,b)
    r.append([k, [p1,p2,p3]])
    return r

def braid_group_action_on_dominant_monomials_KKOP_A2_T2(l,l1): # A2, l1=[[1,2],[2,1],[1,0]]
    r1=braid_group_action_on_dominant_monomials_KKOP_A2_T2_half(l,l1)
    n=2
    r=Dke1e2e3_to_dominant_monomials_A2(r1,n,l1)
    return r

def braid_group_action_on_dominant_monomials_KKOP_A2(ii,l,l1): # A2, l1=[[1,2],[2,1],[1,0]]
    if ii==1:
        r=braid_group_action_on_dominant_monomials_KKOP_A2_T1(l,l1)
    elif ii==2:
        r=braid_group_action_on_dominant_monomials_KKOP_A2_T2(l,l1)
    return r


def KKOP_module_to_HL_module_An_one(l,n): # l=[i,s]
    i,s=l
    r=[n+1-i, -s]
    return r

def KKOP_module_to_HL_module_An(l,n):
    r=[]
    for i in l:
        r.append(KKOP_module_to_HL_module_An_one(i,n))
    return r

def HL_module_to_KKOP_module_An(l,n):
    r=KKOP_module_to_HL_module_An(l,n) # they have the same formula
    return r

def shifted_Plucker_monomial_half(p, a, n): # p=[1,2,3]
    r=[]
    for i in p:
        r.append(ModResultOneToN(i+a,n))
    return r
def shifted_Plucker_monomial(p,a,n): # p=[[1,2,3],[1,2,4]]
    r=[]
    for i in p:
        t1=shifted_Plucker_monomial_half(i, a, n)
        r.append(t1)
    return r

def shifted_Plucker_polynomial(p,a,n): # p=[[1,2,3],[1,2,4]]
    r=[]
    for i in p:
        t1=i[0]
        t2=i[1]
        t3=shifted_Plucker_monomial(t2,a,n)
        r.append([t1,t3])
    return r

def cyclic_shift_of_chain_polynomial_blocks(L,i,n):
    r=[]
    for j in L:
        t2=[]
        for j1 in j:
            t2.append(ModResultOneToN(j1+i,n))
        r.append(t2)
    return r

def cyclic_shift_of_chain_polynomial_blocks_all(L,n):
    r=[]
    for i in [0..n-1]:
        t1=cyclic_shift_of_chain_polynomial_blocks(L,i,n)
        r.append(t1)
    return r

def cyclic_shift_of_chain_polynomial(L,i,k,n): # L is a list of blocks
    r1=cyclic_shift_of_chain_polynomial_blocks(L,i,n)
    r=chain_polynomial_Gkn(k,n,r1)
    return r

def cyclic_shift_of_chain_polynomial_all(L,n): # L is a list of blocks
    r=[]
    for i in [0..n-1]:
        t1=cyclic_shift_of_chain_polynomial(L,i,k,n)
        r.append(t1)
    return r

def chain_polynomial_G4n(l1, d, b, l2): # not finished, l1=[a0,b0,c0], l2=[b_s,c_s,a_s], d=[[d10,d11],[d20,d21],...,[ds0,ds1]], b=[[b1,c1],...,[b_{s-1},c_{s-1}]]
    s=len(d)
    r1=CartesianProductOfListsInAList([[0,1]]*s)
    r=[]
    for t in r1:
        t1=(-1)^sum(t)
        t2=[]
        #print('t,t1',t,t1)
        t2.append(l1+[d[0][t[0]]])
        #print(t2)
        for j in [0..s-2]:
            t2.append([d[j][1-t[j]], b[j][0], b[j][1], d[j+1][t[j+1]]])
            #print(t2)
        t2.append([d[s-1][1-t[s-1]]]+l2)
        #print(t2)
        r.append([t1, t2])
    return r

def all_partition_of_list_into_two_parts_given_first_part_len(L, m): # the first part is of length m
    r=[]
    l=sorted(L)
    for i in powerset(l):
        if len(i)==m:
            r.append([i, SetDifference(l,i)])
    return r

def chain_polynomial_Gkn_half(k,n,L): # L=[B1,B2,...,B_{2r-1}]
    r=[]
    for i in range(len(L)):
        if i%2==1:
            if i==1:
                m=k-len(L[0])
                r1=all_partition_of_list_into_two_parts_given_first_part_len(L[i], m)
                r.append(r1)
                second_part=len(L[i])-m
            else:
                m=k-second_part-len(L[i-1])
                r1=all_partition_of_list_into_two_parts_given_first_part_len(L[i], m)
                r.append(r1)
                second_part=len(L[i])-m
        
    return r
            
def partition_sign(l, P): # l is a permutation of union of lists in P
    r1=NumberOfInversions(l)
    r2=NumberOfInversions(UnionOfListsMany(P))
    r=(-1)^(r1-r2)
    return r

def chain_polynomial_Gkn(k,n,L): # L=[B1,B2,...,B_{2r-1}]
    r1=chain_polynomial_Gkn_half(k,n,L)
    r2=CartesianProductOfListsInAList(r1)
    r=[]
    for i in r2:
        p1=[]
        t1=L[0]+i[0][0]
        p1.append(t1)
        sn=partition_sign(L[1], i[0])
        for j in range(3,len(L)):
            if j%2==1:
                t1=i[(j-1)/2-1][1]+L[j-1]+i[(j-1)/2][0]
                p1.append(t1)
                sn=sn*partition_sign(L[j], i[(j-1)/2])
        j=len(L)
        t1=i[(j-3)/2][1]+L[j-1]
        p1.append(t1)
        r.append([sn, p1])
    return r

def p_i_2r_each_part_between_2_and_RminusOne(i,R): #number of partitions R=r1+r2+r3 such that |\{ r1,r2,r3 \}|=i, r1,r2,r3 between 2 and R-1
    r=0
    for i1 in [2..R-1]:
        for i2 in [i1..R-1]:
            for i3 in [i2..R-1]:
                if i1+i2+i3==2*R and len(removeDuplicates([i1,i2,i3]))==i:
                    r=r+1
    return r


def p_ir(i,n): #number of partitions n=r1+r2+r3 such that |\{ r1,r2,r3 \}|=i
    r=0
    for i1 in [1..n]:
        for i2 in [i1..n]:
            for i3 in [i2..n]:
                if i1+i2+i3==n and len(removeDuplicates([i1,i2,i3]))==i:
                    r=r+1
    return r

def number_of_rank_2_cluster_variables_Grkn(k,n): # k \le n/2
    r=0
    for r1 in [3..k]:
        t1=2*r1/3*p_ir(1,r1)
        t2=2*r1*p_ir(2,r1)
        t3=4*r1*p_ir(3,r1)
        c1=binomial(n,2*r1)*binomial(n-2*r1,k-r1)
        print(t1*c1,t2*c1,t3*c1)
        t4=t1+t2+t3
        t5=t4*binomial(n,2*r1)*binomial(n-2*r1,k-r1)
        print(t5)
        r=r+t5
    return r

def are_list_all_numbers(l):
    r=1
    for i in l:
        t1=isinstance(i, numbers.Number)
        if t1==False:
            r=0
            break
    return r

def tableau_to_standard_tableau(T):    # start from a 2-columes 
    #print('T', T)
    r1=SetDifference(T[0], T[1])
    r2=SetDifference(T[1], T[0])
    S=[r1,r2]
    g1=len(flatten(S))
    c1=ListAToN(1,g1) 
    r=[]
    d2={}
    num=1
    for i in sorted(flatten(S)):
        d2[i]=num
        num=num+1
    #print(d2)
    r=[]
    t1=[]
    for i in r1:
        t1.append(d2.get(i))
    r.append(t1)
    #print(r)
    t2=[]
    for i in r2:
        t2.append(d2.get(i))
    r.append(t2)
    
    return r

def is_sub_tableau_a_column(T,l): # T is rectangular tableau, subtableau 234 in [[1,3,4],[2,5,6]] is considered as in one column, l=[2,3,4]
    r=1
    r1=TableauToMatrix(T) 
    r2=[]
    for i in range(r1.nrows()):
        for j in range(r1.ncols()):
            if IsInList(r1[i,j], l)==1:
                r2.append(i)
    #print(r2)
    r3=removeDuplicates(r2)
    if len(r3)!=len(r2):
        r=0
    return r

def is_sub_tableau_a_column_which_is_standard_tableau(T,l): # T is rectangular tableau, subtableau 234 in [[1,3,4],[2,5,6]] is considered as in one column, l=[2,3,4]
    r=is_sub_tableau_a_column(T,l)
    if r==0:
        return r
    else:
        r1=TableauToMatrix(T) 
        p1=[]

        #print(T)

        for i in range(r1.nrows()):
            for j in range(r1.ncols()):
                if IsInList(r1[i,j], l)==1:
                    p1.append(r1[i,j])
        if sorted(p1)!=p1:
            r=0
    return r


def mutable_cluster_variables_in_foldable_seed_first_column(k,n):
    r=[]
    l=floor((n-k)/2)
    t1=ListAToN_cyclic(l,l+k-2,n)
    t2=[l+k]
    t3=sorted(t1+t2)
    #print(t3)
    r.append(t3)
    if (n-k)%2==0:
        i=2
        p=l
        temp=p+k
        while p>1:
            t1=ListAToN_cyclic(p-1,p+k-3,n)
            t2=[temp]
            t3=sorted(t1+t2)
            #print(i,p,t3)
            r.append(t3)
            i=i+1
            if i>n-k-1:
                break
            t1=ListAToN_cyclic(p-1,p+k-3,n)
            t2=[temp+1]
            t3=sorted(t1+t2)
            temp=temp+1
            #print(i,p,t3)
            r.append(t3)
            p=p-1
            i=i+1
            if i>n-k-1:
                break
        while i<=n-k-1:
            #print(i)
            t1=t3[-1] 
            t2=t3[:-1]
            r.append(t2+[t1+1])
            i=i+1
    else:
        i=2
        p=l
        temp=p+k
        while (1):
            t1=ListAToN_cyclic(p,p+k-2,n)
            t2=[temp+1]
            t3=sorted(t1+t2)
            r.append(t3)
            i=i+1
            if i>n-k-1 or p==1: 
                break
            t1=ListAToN_cyclic(p-1,p+k-3,n)
            t2=[temp+1]
            t3=sorted(t1+t2)
            temp=temp+1
            r.append(t3)
            p=p-1
            i=i+1
            if i>n-k-1 or p==1:
                break
        while i<=n-k-1:
            #print(i)
            t1=t3[-1]
            t2=t3[:-1]
            r.append(t2+[t1+1])
            i=i+1
    return r

def split_list_into_intervals(L): # L is a list of numbers which does not have duplicates
    r=[]
    l=sorted(L)
    i=0
    while i<=len(l)-1:
        t1=[l[i]]
        #print(i)
        j=i
        while j<=len(l)-2:
            if abs(l[j+1]-l[j])==1:
                t1.append(l[j+1])
                j=j+1
            else:
                break
        i=j+1
        #print('i,j',i,j)
        r.append(t1)
    return r

def is_cyclic_interval(L,n): # n=9, 129, 189 are cyclic intervals
    r=0
    l=sorted(L)
    if number_of_gaps_in_list(l)==0:
        r=1
    elif number_of_gaps_in_list(l)>1:
        r=0
    elif number_of_gaps_in_list(l)==1:
        r1=split_list_into_intervals(l)
        if r1[0][0]==1 and r1[1][-1]==n:
            r=1
    return r
        

def split_list_into_intervals_cyclic_intervals(L,n): # L is a list of numbers which does not have duplicates
    r1=split_list_into_intervals(L)
    r=r1
    if len(r1)>1:
        if is_cyclic_interval(r1[0]+r1[-1],n)==1:
            t1=sorted(r1[0]+r1[-1])
            r=[t1]+r1[1:-1]
    return r


def mutable_cluster_variables_in_foldable_seed(k,n):
    r={}
    r1=mutable_cluster_variables_in_foldable_seed_first_column(k,n)
    for i in range(len(r1)):
        p1=repr([i+1,1])
        r[p1]=r1[i]
    
    if (n-k)%2==0:
        for i in [1..n-k-1]:
            for j in [2..k-1]:
                t1=r.get(str([i,j-1]))
                t2=list_to_abcd(t1,n)
                a,b=t2[0]
                c,d=t2[1]
                if (i+j+1)%2==0:
                    g1=sorted(ListAToN_cyclic(a,ModResultOneToN(b-1,n),n))
                    g2=sorted(ListAToN_cyclic(ModResultOneToN(c-1,n),d,n))
                else:
                    g1=sorted(ListAToN_cyclic(ModResultOneToN(a+1,n),b,n))
                    g2=sorted(ListAToN_cyclic(c,ModResultOneToN(d+1,n),n))
                p1=repr([i,j])
                r[p1]=sorted(g1+g2)
    else:
        for i in [1..n-k-1]:
            for j in [2..k-1]:
                t1=r.get(str([i,j-1]))
                t2=list_to_abcd(t1,n)
                a,b=t2[0]
                c,d=t2[1]
                if (i+j+1)%2==0:
                    g1=sorted(ListAToN_cyclic(ModResultOneToN(a+1,n),b,n))
                    g2=sorted(ListAToN_cyclic(c,ModResultOneToN(d+1,n),n))
                else:
                    g1=sorted(ListAToN_cyclic(a,ModResultOneToN(b-1,n),n))
                    g2=sorted(ListAToN_cyclic(ModResultOneToN(c-1,n),d,n))
                p1=repr([i,j])
                r[p1]=sorted(g1+g2)
                
    
    return r

def list_to_abcd(l,n): # l = [a,b] \cup [c,d], return a,b,c,d, l is of the form [2,3,4,8,9] = [2,3,4] \cup [8,9]
    r1=split_list_into_intervals_cyclic_intervals(l,n)
    #print(r1)
    if r1[0][0]==1 and r1[0][-1]==n:
        t1=split_list_into_intervals(r1[0])
        #print(t1)
        a=t1[1][0]
        b=t1[0][-1]
    else:
        a=r1[0][0]
        b=r1[0][-1]
    if r1[1][0]==1 and r1[1][-1]==n:
        t1=split_list_into_intervals(r1[1])
        c=t1[1][0]
        d=t1[0][-1]
    else:
        c=r1[1][0]
        d=r1[1][-1]
    r=[[a,b],[c,d]]
    return r

def cluster_variables_in_a_cluster_Grassmannian(cluster):
    # cluster is of the form [[0, [[1, 2, 4]], 0], [0, [[1, 2, 5]], 1], [0, [[1, 2, 6]], 2], [0, [[1, 3, 4]], 3], [0, [[1, 4, 5]], 4], [0, [[1, 5, 6]], 5], [0, [[2, 3, 4]], 6], [0, [[3, 4, 5]], 7], [0, [[4, 5, 6]], 8], [0, [[1, 2, 3]], 9]]
    r1=TakeIthElements(2,cluster)
    r=[]
    for i in r1:
        if RemoveTrivialTableau3(i,k,n)!=[]:
            r.append(i)
    return r

def evaluation_of_x_coordinates_on_web_matrix_CDFL_on_any_cluster(M,k,n,exchange_matrix,cluster):
    #     k,n=4,8
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
#     M = symbolic_matrix(R,k,n) 
    ma=exchange_matrix
    r1=cluster
    cl_va=cluster_variables_in_a_cluster_Grassmannian(cluster)
    #print(cl_va)
    
    r=[]
    for rr2 in range(len(cl_va)):
        r2=cl_va[rr2]
        for i in range(ma.nrows()):
            if r1[i][1]==r2:
                ii=i
        out=[]
        #print(ma)
        for i in range(ma.nrows()):
            #print(i)
            if ma[ii,i]>0:
                t1=ma[ii,i]*[i]
                out=out+t1
        in_coming=[]
        #print(ma)
        for i in range(ma.nrows()):
            #print(i)
            if ma[ii,i]<0:
                t1=(-ma[ii,i])*[i]
                in_coming=in_coming+t1
        g1=1
        #print(out, in_coming)
        for i in out: 
            g1=g1*PluckerToMinorsPolynomial(M, [[1, r1[i][1]]])
        
        g1=g1.expand()
            
        g2=1
        for j in in_coming:
            g2=g2*PluckerToMinorsPolynomial(M, [[1, r1[j][1]]])
            
        g2=g2.expand()
            
        g1=g2/g1
        #g1=g1/g2
            
        r.append(g1.factor())
    return r

def n_choose_abc(n,a,b,c):
    r=factorial(n)
    r1=factorial(a)
    r2=factorial(b)
    r3=factorial(c)
    r=r/(r1*r2*r3)
    return r

def NumberOfPrimeTableaux2Column(k,n): # 2-column prime tableaux in Gr(k,n), k<=n/2
    r=0
    akn2=1
    for i in [1..k]:
        t1=1
        for j in [1..2]:
            t1=t1*(n-i+j)/(k+2-i-j+1)
        akn2=akn2*t1
    bkn=binomial(n,k)
    r1=0
    for j in [1..k]:
        r1=r1+j*n_choose_abc(n,k-j,2*j,n-k-j)
    bkn=bkn+r1
    r=akn2-bkn
    return r

def does_tableau_has_elements_in_L_not_appear_or_appear_in_the_same_column_half(l,L): # if l=[1,2,7,8], L=[7,8], return 1, if T=[1,2,7,9], L=[7,8], return 0
    r=0
    if SetDifference(L,l)==L or SetDifference(L,l)==[]:
        r=1
    return r

def does_tableau_has_elements_in_L_not_appear_or_appear_in_the_same_column(T,L): # if T=[[1,2,7,8]], L=[7,8], return 1, if T=[[1,2,7,9]], L=[7,8], return 0
    r=1
    for i in T:
        if does_tableau_has_elements_in_L_not_appear_or_appear_in_the_same_column_half(i,L)==0:
            r=0
            break
    return r

def ll_perms_compute_cluster_variables_in_coordinate_ring_of_partial_flag_varieties(lli,typ,rank,max_column,n,repeat,b6,fp1,mat,vertices,L): #  L=[7,8] in the case of F_{2,4,6}, L=[8,9,10] in the case of Gr(5,10) 
    ll=list(map(lambda x: x - 1, lli))
    #print(ll)
    mat1=copy.deepcopy(mat)
    vertices1=copy.deepcopy(vertices)
    
    mutationSequence=[]
    for j1 in [1..repeat]: 
        mutationSequence=mutationSequence+ll
        
    record_sequence=[]
    for j in range(len(mutationSequence)): 
        #print('kk')
        vertices1 = computeEquationsForModulesTableaux_compute_exchange_relations_any_relation(vertices1, mat1, mutationSequence[j],typ,rank)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 
        record_sequence.append(mutationSequence[j]+1) # label of the quiver start from 1
        
        t1=does_tableau_has_elements_in_L_not_appear_or_appear_in_the_same_column(MatrixTakeColumns(vertices1[1]),L)
        if vertices1[1].ncols()>max_column or t1==0:
            vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank) ###why repeat above calculation?
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
            record_sequence.pop()
        else:
            ii0=vertices1[1]
            ii=MatrixTakeColumns(vertices1[1])
            p1=repr(ii)
            p2=b6.get(p1)
            
            #print('p1',p1)
            
            if p2==None: 
                f1=open(fp1, 'a+')
                f1.write(str([ii,record_sequence])+'\n') 
                b6[p1]=[] 
                f1.close()
    return b6

def BraidGroupActionSigmaIOnGVector_CDFL_for_any_sigma(l,g,k,n): # l=[i_1,i_2,...] = sigma_{i_1}... , labels of initial seed are the same as CDFL
    indices = [(i,j) for i in range(k+1) for j in range(n+1)]
    R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
    x = dict(zip(indices, R.gens()))
    M = web_matrix_DFGK_reverse_xij(R,x,k,n)
    #print(evaluation_of_x_coordinates_on_web_matrix_CDFL(M,k,n))
    
    L=ReverseList(l)
    
    t2=copy.deepcopy(M)
    for ii in L:
        t2=BraidGroupActionSigmaIOnMatrixRenomalizeColumns_symbolic(R,ii,k,n,t2)
    #print(latex(t2))
    #print(t2)
    t3=evaluation_of_x_coordinates_on_web_matrix_CDFL(t2,k,n)
    #print(latex(t3))
    #print(latex(evaluation_of_x_coordinates_on_web_matrix_CDFL(M,k,n)))
    #print(t3)
    
    r1=t3
    
#     r1=[]
#     for i in t3:
#         p1=i
#         for j in indices:
#             p1=p1.subs({x[j[0],j[1]]:1/x[j[0],j[1]]})
#         r1.append(p1)
    #print(r1)
    t4=evaluation_tropical_function_rational_list_on_vector_CDFL(R, x, k,n,  r1, g)
    
    r=t4
    return r

def BraidGroupActionSigmaIOnTableau_for_any_sigma(l,T,k,n): # l=[i_1,i_2,...] = sigma_{i_1}..., T[i] is a column, this uses Trop(\sigma_{i_1} ... )
    e1=initial_cluster_variables_CDFL(k,n)
    m=(k-1)*(n-k-1)
    g1=GVectorOfTableau(k,n,T,e1)[0][:m]
    #print(g1)
    r1=BraidGroupActionSigmaIOnGVector_CDFL_for_any_sigma(l,g1,k,n)
    #print(r1)
    r=GVectorToTableau_remove_inverses(r1,e1,k,n)
    
    return r

def list_of_lists_to_monomial_in_Y(R,Y,l):
    r=1
    for i in l:
        r=r*Y[i[0],i[1]]
    return r

# compute truncated q-characters using mutations
def truncated_q_characters_using_mutation_sequence(R,Y,ell,ll,typ,rank): # C_{ell}, ell is integer, ll is the mutation sequence 
#     n1,n2=-300, 300
#     R,Y=define_fraction_field_with_variables_that_have_negative_indices('Y',rank,n1,n2)  
#     typ='A'
#     rank=2
#     ell=5
#     ll=[6,7,8,9,10,0,1,2,3,4,6,7,8,9,0,1,2,3]
#     truncated_q_characters_using_mutation_sequence(R,Y,ell,ll,typ,rank)



    IC=InitialCluster_Yis(typ,rank, ell)
#     ll0=IC[2] # mutable vertices
#     print(ll0)

    print(IC[1])
    
    va1=[]
    for i in IC[1][0]:
        t1=[i[0],list_of_lists_to_monomial_in_Y(R,Y,i[1]),i[2]]
        va1.append(t1)
    variable2_ch = [va1, []]  
    mat1=IC[0]
    vertices1=[IC[1], variable2_ch]
    
    r=[]
    mutationSequence=copy.deepcopy(ll) 
        
    for j in range(len(mutationSequence)): 
        vertices1 = computeEquationsForModulesYis_compute_qcharacter(vertices1, mat1, mutationSequence[j],typ,rank,l,R,Y)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 
        r.append([vertices1[0][1], vertices1[1][1]])
    return r

def RimCoordinate_shift_s(l, s):     
    r=[]
    for i in l:
        t1=i[0]
        t2=i[1]
        r.append([t1, [t2[0], t2[1]+s]])
    return r

def RimCoordinateWithMultiplicityOne_m_rows(l, a, n, m): # l is [[1,2,3,6]] a rank one profile, m is number of rows we take when drawing the module, a is the highest a in the set of [0,a]
    r1=RimCoordinateWithMultiplicityOne(l, a, n)
    r=[]
    #print(r1)
    for i in range(m):
        t1=RimCoordinate_shift_s(r1, -2*i)
        r=r+t1
        #print(t1)
    return r

def direct_sum_of_rank_1_modules(l, level, n, m): # m is number of columns we take when drawing the module, level is a list of numbers, indicating the a in [0,a] of a rank one module P_J
# compute cokernel, from g-vector recover profiles 
# k=4
# a=5
# n=8
# m=3
# t1=direct_sum_of_rank_1_modules([[1,2,3,6],[1,2,4,5],[1,5,6,7],[3,4,5,6],[1,2,7,8]], [a,a,a-2,a-2,a-2,a-2],n,m)
# t2=direct_sum_of_rank_1_modules([[1,2,5,6]], [a-2], n, m)
# t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
# t4=RimCoordinateToRimManyRims(t3, a, n, m)
    r1=[]
    for i in range(len(l)):
        t1=RimCoordinateWithMultiplicityOne_m_rows(l[i], level[i], n, m)
        r1=r1+t1
    r=SortByFirstElementInSecondTerms(combineLikeTerms3(r1))
    return r

def IsTableauSnakeModule(k,l): 
    r0=TableauToSmallGapTableau(k,l)
    r1=CollectMissingElementInSmallGapTableau(r0)
    r=1
    a1=sorted(r1)
    if IsTableauRegular(k,l)==0:
        r=0
    else:
        if a1!=r1 and a1!=ReverseList(r1):
            r=0
    return r

def IsTableauSnakeModule2(k,l):
    r0=TableauToSmallGapTableau(k,l)
    r1=CollectMissingElementInSmallGapTableau(r0)
    r=1
    for i in [0..len(r1)-2]: 
        if IsAAllLessThanB(r1[i], r1[i+1])==0: 
            r=0
            break
    return r

def IsTableauRegular(k,a): 
    a1=TableauToListsIJSorted(k,a)
    p1=removeDuplicates(a1[0])
    p2=removeDuplicates(a1[1])
    r=0
    if len(p1)==len(a1[0]) and len(p2)==len(a1[1]):
        r=1 
    return r

def IsAAllLessThanB(a,b): 
    r=1
    for i in a:
        for j in b:
            if i>=j:
                r=0 
                break 
    return r



def rho_Weyl(n): # [1,1,1,1,1] means omega_1+...
    r=[]
    for i in range(n):
        r.append(1)
    return r

def WeylGroupActionSiActionOnAlphaJ(ii,jj,typ,rank): # si(alphaj)=alphaj-C[j,i]alphai, < alpha_i, alpha_j^{\vee} > = C_{j,i}
    r=[]
    r1=[]
    n=rank
    for i in range(n):
        r.append(0)
        r1.append(0)
    r[jj-1]=1
    C=CartanMatrix([typ,rank])
    #print(C)
    t1=-C[jj-1,ii-1]
    r1[ii-1]=t1
    #print(r,r1)
    r=AddVlueOfListsMany([r,r1])
    return r

def WeylGroupActionSiActionOnCombinationOfAlphaJ(ii,la,typ,rank): # si(alphaj)=alphaj-C[j,i]alphai, < alpha_i, alpha_j^{\vee} > = C_{j,i}
    r=[] 
    n=rank
    for i in range(n):
        r.append(0)
    for j in range(len(la)):
        jj=j+1
        t1=WeylGroupActionSiActionOnAlphaJ(ii,jj,typ,rank) 
        t2=[la[j]*i for i in t1]
        r=AddVlueOfListsMany([r,t2])
    return r

def WeylGroupActionOnCombinationOfAlphaJ(w,la,typ,rank): # si(alphaj)=alphaj-C[j,i]alphai, < alpha_i, alpha_j^{\vee} > = C_{j,i}

    r=copy.deepcopy(la)
    if w==[]:
        r=copy.deepcopy(la)
    else:
        w1=ReverseList(w)
        for i in w1:
            r=WeylGroupActionSiActionOnCombinationOfAlphaJ(i,r, typ, rank)
    return r

def root_to_weight(la, typ,rank): # la = l1 alpha1 + .... = k1 omega1 + ...
    r1=[]
    r=[]
    n=rank
    weights=[]
    for i in range(n):
        r1.append(0)
        r.append(0)
    for j in range(n):
        r2=copy.deepcopy(r1)
        r2[j]=1 # r2=omega_j
        weights.append(r2)
    C=CartanMatrix([typ,rank])
    
    roots=[]
    for i in range(n):   # alpha_i = c1 omega_1 + ...
        t1=copy.deepcopy(r1)
        for j in range(n):
            t2=[C[i,j]*j1 for j1 in weights[j]]
            t1=AddVlueOfListsMany([t1,t2])
        roots.append(t1)
    for i in range(len(la)):
        c1=la[i]
        t1=[c1*j for j in roots[i]]
        r=AddVlueOfListsMany([r, t1])
    return r

def weight_to_root(om, typ,rank): # om = l1 omega_1+... = k1 alpha_1+...
    r1=[]
    r=[]
    n=rank
    roots=[]
    for i in range(n):
        r1.append(0)
        r.append(0)
    for j in range(n):
        r2=copy.deepcopy(r1)
        r2[j]=1 # r2=omega_j
        roots.append(r2)
    C=CartanMatrix([typ,rank])
    C_inv=C^(-1)
    weights=[]
    for i in range(n):   # omega_i = c1 alpha_1 + ...
        t1=copy.deepcopy(r1)
        for j in range(n):
            t2=[C_inv[i,j]*j1 for j1 in roots[j]]
            t1=AddVlueOfListsMany([t1,t2])
        weights.append(t1)
    for i in range(len(om)):
        c1=om[i]
        t1=[c1*j for j in weights[i]]
        r=AddVlueOfListsMany([r, t1])
    return r
    

def WeylGroupActionSiActionOnOmegaJ(ii,jj,typ,rank): # s_i(omega_j)
    n=rank
    C=CartanMatrix([typ,rank]) 
    om0=[]
    for i in range(n):
        om0.append(0)
    om0[jj-1]=1
    om=weight_to_root(om0, typ,rank)
    r1=WeylGroupActionOnCombinationOfAlphaJ([ii],om,typ,rank)
    r=root_to_weight(r1, typ, rank)
    return r

def WeylGroupActionSiAction(ii,la,typ,rank): # si(lambda),  lambda=[lambda1,lambda2,...]=lambda_1 omega_1 + lambda_2 omega_2 + ... 
    r=[]
    n=rank
    for i in range(n):
        r.append(0)
    for i in range(len(la)):
        c1=la[i]
        t1=WeylGroupActionSiActionOnOmegaJ(ii,i+1,typ,rank)
        t2=[c1*j for j in t1]
        r=AddVlueOfListsMany([r,t2])
    return r

def WeylGroupAction(w,la,typ,rank): # w =[1,2,1,3,2] is a product of si, w(la), la=lambda_1 omega_1 + lambda_2 omega_2 + ... 
    r=copy.deepcopy(la)
    if w==[]:
        r=copy.deepcopy(la)
    else:
        w1=ReverseList(w)
        for i in w1:
            r=WeylGroupActionSiAction(i, r, typ, rank)
    return r


def Weyl_character_numerator(R,y,la,typ,rank): # la=lambda_1 omega_1 + lambda_2 omega_2 + ... 
#     R=PolynomialRing(QQ,'y',200)
#     y=R.gens()  # y[i] = e^{omega_i}
    #W = WeylGroup([typ,rank], implementation="permutation")
    
    typ1=typ
    if typ=='B':
        typ1='C'
    elif typ=='C':
        typ1='B'
    
    W = WeylGroup([typ1,rank])
    n=rank
    rho=rho_Weyl(n) # omega_1+...+omega_n
    r=0
    for w1 in W:
        w=w1.reduced_word()
        #print('w',w)
        t0=AddVlueOfListsMany([rho,la])
        t1=WeylGroupAction(w,t0,typ1,rank)
        sgn=(-1)^(len(w))
        r1=1
        for i in range(len(t1)):
            r1=r1*y[i+1]^(t1[i])
        r=r+sgn*r1
        
    return r

def Weyl_character_denominator(R,y,typ,rank): # la=lambda_1 omega_1 + lambda_2 omega_2 + ... 
#     R=PolynomialRing(QQ,'y',200)
#     y=R.gens()  # y[i] = e^{omega_i}
    #W = WeylGroup([typ,rank], implementation="permutation")
    
    typ1=typ
    if typ=='B':
        typ1='C'
    elif typ=='C':
        typ1='B'
    
    W = WeylGroup([typ1,rank])
    n=rank
    rho=rho_Weyl(n) # omega_1+...+omega_n
    r=0
    for w1 in W:
        w=w1.reduced_word()
        #print('w',w)
        t0=rho
        t1=WeylGroupAction(w,t0,typ1,rank)
        sgn=(-1)^(len(w))
        r1=1
        for i in range(len(t1)):
            r1=r1*y[i+1]^(t1[i])
        r=r+sgn*r1
        
    return r

def Weyl_character_formula(R,y,la,typ,rank): # la=lambda_1 omega_1 + lambda_2 omega_2 + ... 
    r1=Weyl_character_numerator(R,y,la,typ,rank)
    r2=Weyl_character_denominator(R,y,typ,rank)
    r=simplify(r1/r2)
        
    return r

def weight_of_a_term_in_character_of_module_of_Lie_algebra(R,y,m1,rank): # m1 = [1, y[1], y[1], y[2]], m1=y1y2y3
    r=[]
    n=rank
    for i in range(n):
        r.append(0)
    #print('m1', m1)
    for i in range(1,len(m1)):
        t1=y.index(m1[i])
        r[t1-1]=r[t1-1]+1
    return r

def highest_weight_term_of_character_of_module_of_Lie_algebra(R,y,p1,typ,rank): # p1=1/y[2] + y[2]/y[1] + y[1]
    r=[]
    r1=change_polynomial_to_list(R,numerator(p1))
    if len(r1)!=0:
        #print(r1[0])
        r=r1[0]
        wr=weight_of_a_term_in_character_of_module_of_Lie_algebra(R,y,r,rank)
        #print(r)
        for i in range(1,len(r1)):
            t1=r1[i]
            wt1=weight_of_a_term_in_character_of_module_of_Lie_algebra(R,y,r1[i],rank)
            t2=compareWeights2(wr,wt1,typ,rank)
            if t2==-1:
                r=t1
                wr=wt1
    r=ProductOfElementsInList(r)
    d1=denominator(p1)
    r=r/d1 

    return r

def monomial_to_weight(R,y,p1,rank): # 2*y[1]*y[2] = [2, 1, 1]
    r1=change_polynomial_to_list(R,numerator(p1))[0]
    r2=change_polynomial_to_list(R,denominator(p1))[0]
    #print(r1,r2)
    c1=r1[0]/r2[0]
    wr1=weight_of_a_term_in_character_of_module_of_Lie_algebra(R,y,r1,rank)
    wr2=weight_of_a_term_in_character_of_module_of_Lie_algebra(R,y,r2,rank)
    r3=[-j for j in wr2]
    r4=AddVlueOfListsMany([wr1,r3])
    r=[c1,r4]
    return r

def decompose_character_as_direct_sum_of_simple_module_of_Lie_algebra(R,y,p1,typ,rank):
    r1=copy.deepcopy(p1)
    wr1=highest_weight_term_of_character_of_module_of_Lie_algebra(R,y,p1,typ,rank)
    r=[]
    while r1!=0:
        #print('r1', r1)
        print(wr1)
        r2=monomial_to_weight(R,y,wr1,rank)
        #print('r2',r2)
        #print(Weyl_character_formula(R,y,r2[1],typ,rank))
        r3=simplify(r2[0]*Weyl_character_formula(R,y,r2[1],typ,rank))
        #print('r3',r3)
        r1=simplify(r1-r3)
        #print('r1',r1)
        wr1=highest_weight_term_of_character_of_module_of_Lie_algebra(R,y,r1,typ,rank)
        r.append(r2)
    return r

def ChTUsingKLPolynomialInSage_in_Grkn(T,k,n):
    g1=ChTUsingKLPolynomialInSage(T,k)
    typ='A'
    rank=k-1
    g2=HighestWeightTableauInPolynomialOfTableaux(g1,typ,rank)
    g2=g2[1]
    r2=TableauDivision(g2,T)

    r0=T
    e1=NonCrossingTupleWithSameContentAsT(r0)
    p1=[]
    for i in [1..len(e1)]:
        M=randomMatrixNonInt(k,n)
        t1=PluckerToMinorsPolynomial2(M, g1)
        t2=PluckerToMinorsPolynomial2(M, [[1,r2]])
        #print(t2)
        t3=t1*t2^(-1)
        #print(g1)
        p1.append([M, t3])
    r=ComputeChTInTermsOfNonCrossingBasisUpToRound100th([ e1, p1 ])
    return r

def find_fixed_points_under_braid_group_action_half(R,x,ii,trop,k,n,l,fp1): # sigma_i, i \in [d-1], SSYT(k,n)
    t1=list(l)
    r=[]
    t2=TransposeOfRectangularTableau(t1)
    t3=BraidGroupActionSigmaIOnTableau_use_tropical_map(R,x,t2,k,n,trop[ii-1])
    if t3==t2:
        f1=open(fp1,'a+')
        f1.write(str(t2)+'\n')
        f1.close() 
        r=[t2]
    return r

def find_fixed_points_under_braid_group_action(R,x,trop,k,n,max_column,fp1): # sigma_i, i \in [d-1], SSYT(k,n)
    r=[]
    d=gcd(k,n)
    for ii in [1..d-1]:
        for jj in [1..max_column]:
            r1=SemistandardTableaux_k_by_m(k,n,jj)
            print(jj,'sigma',ii)
            f1=open(fp1,'a+')
            f1.write(str(jj)+', sigma, '+ str(ii) + '\n')
            f1.close()
            
            N_CPU = multiprocessing.cpu_count()
            p = get_context("fork").Pool(processes=N_CPU) # works on mac
            bb = p.starmap(find_fixed_points_under_braid_group_action_half,[(R,x,ii,trop,k,n,i,fp1) for i in r1])
            
            r2=[]
            for j in bb:
                r2=r2+j
            r.append(r2)
    return r

def SemistandardTableaux_k_by_m(k,n,m): # each tableau is written as rows
    t0=[]
    for i in range(k):
        t0.append(m)
    r=SemistandardTableaux(t0, max_entry=n)
    return r 


def StandardTableaux_k_by_m(k,n,m): # each tableau is written as rows
    t0=[]
    for i in range(k):
        t0.append(m)
    r=StandardTableaux(t0, max_entry=n)
    return r 

def compute_all_subquotients_of_tensor_product_of_modules(l,k,n):
    #  this is to compute all subquotients of L(M1) \otimes L(M2) \otimes \cdots \otimes L(Mn)
    typ='A' # find a more efficient way to compute longest word in S_lambda w S_mu
    rank=k # number of rows of tableaux 
    rank1=k-1
    
    e=[] 
    e1=[]
    e2=[]
    for T1 in l:
        tt3=ChTUsingKLPolynomialInSage(T1,k)
        e.append(tt3)

    f4=MultiplyTwoPolynomialsLists2Many(e) 
    f4=RemoveTrivialTableauxInStandardMonomialPolynomial(f4,k,n)
    f4=combineLikeTerms3(f4) 

    while f4!=[]:
        g1=HighestWeightTableauInPolynomialOfTableaux(f4,typ,rank1)
        print([g1[0], RemoveTrivialTableau(g1[1],k,n)])
    #     t1=[g1[0], TableauToMultiSegments(Y,g1[1]), TableauToModuleHasY(Y,g1[1]), TableauToModuleInMatrixPAdicConventionHasY(Y,TableauToMatrix(g1[1]))]
    #     print(t1) 
        e1.append([g1[0], RemoveTrivialTableau(g1[1],k,n)])
    #     e2.append(sorted(t1[1]))
        T1=g1[1]
        if T1==[]: # trivial module, corresponding to \mathbb{C}
            c2=[[1, []]]
        else:
            c2=ChTUsingKLPolynomialInSage(T1,k)
        c2=Multiplyq2NoChangeValue(c2, g1[0])
        f4=PolynomialE1SubtractE2EiArePolynomialsInStandardMonomialsInTermsOfTableaux(f4,c2,k,n)

    r=e1
    return e1

def ComputeExpansionInIrreducibleModules(L0,L1,k,n): # Gr(k,n) L0 = [[1,[[1,2,3],[2,3,5]]], [-2,[[1,2,6]]], ...] is a polynomial in Pluckers, L1[i][0]/L1[i][1] is ch(T), L1[i][1] is product of frozen, L1[i][0], L1[i[1]] are polynomials in Pluckers 
    g1=L1
    m=len(L1)
    p1=np.zeros([m,m])
    p2=np.zeros([m])
    for j1 in range(m): 
        M=randomMatrixNonInt(k,n)
        p2[j1]=PluckerToMinorsPolynomial2(M, L0)
        for p in [0..m-1]:
            #print(g1[p])
            t1=PluckerToMinorsPolynomial2(M, g1[p][0])
            t2=PluckerToMinorsPolynomial2(M, g1[p][1])
            p1[j1, p]=t1/t2
         
    #t1=np.linalg.solve( p1,p2 )
    #print('t1',t1)
    #t1=scipy.linalg.inv(p1).dot(p2)
    t1=np.linalg.lstsq(p1, p2, rcond=None)[0]
    #t1=np.linalg.lstsq(p1, p2, rcond=None)[0]
    #t1=scipy.linalg.lstsq(p1, p2)[0] # ??? check
    t5=[]
    #t5.append([1, i[0]])
    for q in [0..len(g1)-1]:
        t5.append([round(t1[q],5), g1[q]])
    r=RemoveZeroTerms(t5) 
    
    return r


def Speh_module_multi_segments(a,b):
    r=Speh_module_multi_segments_half(a-1,b)
    return r

def Speh_module_multi_segments_shift(a,b,c):
    r1=Speh_module_multi_segments(a,b)
    r=shift_multi_segments(r1, c)
    return r

def divide_list_into_tuples_of_Kirillov_Reshetikhin(l, k): # l is a list of integers of length mk
    r=list(candidates_KR(Counter(sorted(l)), k))
    r=removeDuplicatesListOfLists(r)
    return r

def candidates_KR(content, k, candidate=()): 
    if len(content) < k:
        if len(content) == 0 and candidate:
            yield candidate
        else:
            return
        
    for I in itertools.combinations(content, k):
        # tuples in candidate are in non-decreaseing order
        if candidate and I < candidate[-1]:
            continue
            
        # new tuple is non-crossing with the other tuples
        if number_of_gaps_in_list(I)<=1:
            yield from candidates_KR(content - Counter(I), k, candidate + (I,))

def Kirillov_Reshetikhin_in_tableau_to_Spehab(T): # return [a,b,c] means sp(a,b)|cdot|^c, speh(a,b) has center at (0,0), T is one column tableau, KR module
    t2=TableauToMultiSegments('Y',T)
    a=0
    b=0
    if t2!=[]:
        a=t2[0][1]-t2[0][0]+1
        b=len(t2)
    sn=flatten(t2)
    if sn!=[]:
        sn1=max(sn)
        sn2=min(sn)
    else:
        sn1=0
        sn2=0
    if [a,b]!=[0,0]: 
        r=[a,b,sn2+(sn1-sn2)/2]
    else:
        r=[0,0,0]
    return r

def linear_combination_of_product_of_Kirillov_Reshetikhin_in_tableau_to_Spehab(l): # return [[1,[a,b,c]], [-2,[a1,b1,c1]], ...], [a,b,c] means sp(a,b)|cdot|^c, speh(a,b) has center at (0,0), T is one column tableau, KR module
    r=[]
    for i in l:
        t2=[]
        for j in i[1]:
            T=[j]
            t1=Kirillov_Reshetikhin_in_tableau_to_Spehab(T)
            t2.append(t1)
        t3=[i[0], t2]
        r.append(t3)
    return r

def is_Kirillov_Reshetikhin_module_or_one_column_trivial_module_as_tableau(T): # T is a one column tableaux
    r=1
    if len(T)>1 or len(T)==0:
        r=0
    else:
        t1=number_of_gaps_in_list(T[0])
        if t1>1:
            r=0
    return r

def Speh_module_as_quantum_affine_module(a,b,shift_segments):
    t1=Speh_module_multi_segments(a,b)
    t2=shift_multi_segments(t1,shift_segments)
    r=multisegment_to_module(t2)
    return r

def Speh_module_as_tableau(a,b,shift_segments,k,n,shift_module):
    t1=Speh_module_as_quantum_affine_module(a,b,shift_segments)
    t2=shift_the_ith_entries(t1,2,shift_module)
    r=ModulesToTableau(t2,k,n)
    return r

def number_of_gaps_in_list(L):
    r=0
    l=sorted(L)
    for i in range(len(l)-1):
        if abs(l[i+1]-l[i])>1:
            r=r+1
    return r

def is_Kirillov_Reshetikhin_module_as_tableau(T):
    r=1
    if len(T)>1 or len(T)==0:
        r=0
    else:
        t1=number_of_gaps_in_list(T[0])
        if t1==0 or t1>1:
            r=0
    return r

def is_Kirillov_Reshetikhin_module_as_tableau_list(l):
    r=1
    for i in l:
        if is_Kirillov_Reshetikhin_module_as_tableau(i)==0:
            r=0
            break
    return r

def ComputeExpansionInPolynomialInPluckers(L0,L1,k,n): # Gr(k,n) L0 = [[1,[[1,2,3],[2,3,5]]], [-2,[[1,2,6]]], ...] is a polynomial in Plucker, L1 = [[[1,2,5],[2,3,6]], ...] is a list of Pluckers 
    g1=L1
    m=len(L1)
    p1=np.zeros([m,m])
    p2=np.zeros([m])
    for j1 in range(m): 
        M=randomMatrix(k,n,3,23)
        p2[j1]=PluckerToMinorsPolynomial2(M, L0)
        for p in [0..m-1]:
            #print(g1[p])
            p1[j1, p]=PluckerToMinors2(M, [1, g1[p]]) 
         
    #t1=np.linalg.solve( p1,p2 )
    #print('t1',t1)
    #t1=scipy.linalg.inv(p1).dot(p2)
    t1=np.linalg.lstsq(p1, p2, rcond=None)[0]
    #t1=np.linalg.lstsq(p1, p2, rcond=None)[0]
    #t1=scipy.linalg.lstsq(p1, p2)[0] # ??? check
    t5=[]
    #t5.append([1, i[0]])
    for q in [0..len(g1)-1]:
        t5.append([round(t1[q],5), g1[q]])
    r=RemoveZeroTerms(t5) 
    
    return r

def is_sign_invariant_multisegments(l): # \sum_i [ai,bi]=\sum_i [-bi,-ai]
    r=0
    r1=[]
    for i in l:
        r1.append([-i[1],-i[0]])
    if SetDifference(l,r1)==[] and SetDifference(r1,l)==[]:
        r=1
    return r

def shift_the_ith_entries(l, i, a):
    r=[]
    for ii in l:
        t1=copy.deepcopy(ii)
        t1[i-1]=ii[i-1]+a
        r.append(t1)
    return r

def are_all_almost_integers(l, ep):
    r=1
    for i in l:
        if abs(i-int(i))>ep:
            r=0
            break
    return r


def Speh_module_multi_segments_half(a,b): # each segment is of length a and there are b segments, centered at (0,0)
    r=[]
    r1=[]
    r2=[]
    if b%2==0:
        k=b/2
        x=(1-a)/2
        y=(1+a)/2
        for i in range(k):
            t1=[x+i, y+i]
            t2=[-y-i,-x-i]
            r1.append(t1)
            r2.append(t2)
        r=r1+r2
    elif b%2==1:
        k=(b-1)/2
        t1=[-a/2, a/2]
        r.append(t1)
        for i in range(k):
            t1=[-a/2+i+1, a/2+i+1]
            t2=[-a/2-i-1, a/2-i-1]
            r1.append(t1)
            r2.append(t2)
        r=r+r1+r2
    r=sorted(r)
    return r

def ListAToN_any_number(a, n): # start from a, n numbers
    r=[]
    for i in range(n):
        r.append(a+i)
    return r

def shift_segment(l, a):
    u,v=l
    r=[u+a,v+a]
    return r

def shift_multi_segments(l,a):
    r=[]
    for i in l:
        r.append(shift_segment(i,a))
    return r

def multisegment_to_module(l): # [a,b] is sent to Y_{b-a+1,a+b-1}
    r=[]
    for i in l:
        a,b=i
        r.append([b-a+1,a+b-1])
    r=sorted(r)
    return r

def BraidGroupActionSigmaIOnTableau_use_tropical_map_for_any_sigma(R,x,T,k,n,l_trop): # T[i] is a column, BraidGroupActionSigmaIOnGVector_CDFL_for_any_sigma, there is another map BraidGroupActionSigmaIOnTableau_for_any_sigma
            # trop.append(prepare_tropical_map_CDFL(ii,k,n))
        #trop.append(prepare_tropical_map_CDFL_inverse(ii,k,n)) # if computing sigma_i^{-1}
#     indices = [(i,j) for i in range(k+1) for j in range(n+1)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
# l_trop=[trop[0], trop[1], ...]
    r=copy.deepcopy(T)
    #print(r)
    for i in ReverseList(l_trop):
        r=BraidGroupActionSigmaIOnTableau_use_tropical_map(R,x,r,k,n,i)
        #print('r',r)
    return r 

def BraidGroupActionSigmaIOnGVector_CDFL_use_tropical_map_for_any_sigma(R,x,g,k,n,l_trop): # sigma_ii on g, labels of initial seed are the same as CDFL, this is Trop(\sigma_{i_1})Trop(\sigma_{i_2})..., there is another map BraidGroupActionSigmaIOnGVector_CDFL_for_any_sigma
    # trop.append(prepare_tropical_map_CDFL(ii,k,n))
    #trop.append(prepare_tropical_map_CDFL_inverse(ii,k,n)) # if computing sigma_i^{-1}
#     indices = [(i,j) for i in range(k+1) for j in range(n+1)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
# l_trop=[trop[0], trop[1], ...]
    r=copy.deepcopy(g)
    for i in ReverseList(l_trop):
        r=BraidGroupActionSigmaIOnGVector_CDFL_use_tropical_map(R,x,r,k,n,i)
    return r 

def ChangePolynomialInPluckerToNoncrossingBasisGrknSimToGrkn(T,l,k,n): # l = \tilde{ch}(T) in C[Gr(k,n,\sim)] to ch(T) in C[Gr(k,n)]
    #l=ChTUsingKLPolynomialInSage(T, k)
    sT=TableauToSmallGapTableau(k, T)
    #print(sT)
    fT=TableauDivision(sT,T)
    
    r=[]
    if l!=[]: 
        e1=NonCrossingTupleWithSameContentAsT_new(T)
        
        p1=[]
        for i in [0..len(e1)]:
            M=randomMatrixNonInt(k,n)
            g1=PluckerToMinorsPolynomial2(M, l)/PluckerToMinorsPolynomial2(M, [[1,fT]])
            p1.append([M, g1])
        
        r=ComputeChTInTermsOfNonCrossingBasis([ e1, p1 ])
        
    return r


def ChangePolynomialInPluckerToTableauBasisGrknSimToGrkn(T,l,k,n): # l = \tilde{ch}(T) in C[Gr(k,n,\sim)] to ch(T) in C[Gr(k,n)]
    #l=ChTUsingKLPolynomialInSage(T, k)
    sT=TableauToSmallGapTableau(k, T)
    fT=TableauDivision(sT,T)
    
    r=[]
    if l!=[]: 
        e1=TableauxWithSameContentAsT_new(T)
        
        p1=[]
        for i in [0..len(e1)]:
            M=randomMatrixNonInt(k,n)
            g1=PluckerToMinorsPolynomial2(M, l)/PluckerToMinorsPolynomial2(M, [[1,fT]])
            p1.append([M, g1])
        
        r=ComputeChTInTermsOfNonCrossingBasis([ e1, p1 ])
        
    return r

def quantum_integer(q,n):
    r=(q^n-q^(-n))/(q-q^(-1))
    return r

def Uqsl2hat_interchange_EF_and_K(q,E,F,K,r3): # KF=q^(-2)FK
    r4=[]
    for i in r3:
        t1=i[1]
        t2=i
        for j in range(len(t1)-1):
            if [t1[j],t1[j+1]]==[F,K]:
                t1[j]=K
                t1[j+1]=F
                #print('t1',t1)
                t2=[i[0]*q^2, t1]
            elif [t1[j],t1[j+1]]==[F,1/K]:
                t1[j]=1/K
                t1[j+1]=F
                t2=[i[0]*q^(-2), t1]
            elif [t1[j],t1[j+1]]==[E,K]:
                t1[j]=K
                t1[j+1]=E
                t2=[i[0]*q^(-2), t1]
            elif [t1[j],t1[j+1]]==[E,1/K]:
                t1[j]=1/K
                t1[j+1]=E
                t2=[i[0]*q^2, t1]
            #print('t2', t2)
        r4.append(t2)
    r=r4
    return r4

def Uqsl2hat_remove_K_KInverse(q,E,F,K,r3): # KF=q^(-2)FK
    r4=[]
    for i in r3:
        t1=i[1]
        t2=i
        j=0 
        while j<len(t1)-1:
            if [t1[j],t1[j+1]]==[1/K,K] or [t1[j],t1[j+1]]==[K,1/K]:
                t1.pop(j)
                t1.pop(j)
                t2=[i[0], t1] 
                j=0
            else:
                j=j+1
        r4.append(t2)
    r=r4
    return r4

def ListAToN_cyclic(a,b,n):
    if a<=b:
        r=ListAToN(a,b)
    elif a>b:
        r=ListAToN(a,n)+ListAToN(1,b)
    return r

def is_integer_approximately(a, l):
    r=0
    if abs(a-int(a))<l:
        r=1
    return r

def is_integer_approximately_list(a, l):
    r=1
    for i in a:
        if is_integer_approximately(i, l)==0:
            r=0
            break
    return r

def ChangePolynomialInPluckerToTableauBasis(l,k,n):
    r=[]
    if l!=[]:
        r0=l[0][1]
        e1=TableauxWithSameContentAsT_new(r0)
        
        p1=[]
        for i in [0..len(e1)]:
            M=randomMatrixNonInt(k,n)
            g1=PluckerToMinorsPolynomial2(M, l)
            p1.append([M, g1])
        
        r=ComputeChTInTermsOfNonCrossingBasis([ e1, p1 ])
        
    return r

def change_polynomial_in_plucker_to_polynomial_in_fundamental_modules(p): # p is a polynomial in plucker
    r=[]
    for i in p:
        t1=ReverseList(SortBySecondTerms(TableauToModule('Y', i[1])))
        r.append([i[0], t1])
    return r

def TakeExponentsOfPolynomialLaurent_with_coeffs_indices(R,p, l): # l = [x11, x12, x21, x22, x31, x32] order of variables
#    k,n=4,8
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    
    p1=TakeExponentsOfPolynomial_with_coeffs_indices(R,p.numerator(),l)
    #print(p1)
    p2=TakeExponentsOfPolynomial_with_coeffs_indices(R,p.denominator(),l)[0]
    p3=[]
    for i in p2[1]:
        t1=[-i[0], i[1], i[2]]
        p3.append(t1)
    p4=[[p2[0], p3]]
    
    r1=MultiplyTwoPolynomialsLists2(p1, p4)
    r=[]
    for i in r1:
        t1=i[0]
        t2=combineLikeTerms3(i[1]) 
        r.append([t1,t2]) 
        
    return r 

def TakeExponentsOfMonomial_with_coeffs_indices_list(m, l): # m = [2, x11, x12]  l = [[x11,[1,1]], [x12,[1,2]], ...] order of variables
#    k,n=4,8
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    
    r2=m
    #print('r2', r2)
    r=[m[0]]
    r1=[]
    for i in l:
        ind=i[1]
        #print('ind', ind)
        t1=0
        for j in r2:
            if i[0]==j:
                t1=t1+1
        #print('t1', t1)
        if t1!=0:
            r1.append([t1]+ind)
        #print('r1',r1)
    r.append(r1)
    return r

def TakeExponentsOfPolynomial_with_coeffs_indices(R,p, l): # l = [x11, x12, x21, x22, x31, x32] order of variables
#    k,n=4,8
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    r=[]
    p1=change_polynomial_to_list(R,p)
    #print('p1', p1)
    for i in p1:
        r.append(TakeExponentsOfMonomial_with_coeffs_indices_list(i,l))
    #print('r',r)
    
    return r

def SortByIthTerms(ii, l): # ii starts from 1, from small to large
    r=[]

    if l!=[]: 
        t1=[]
        for i in l:
            t1.append(i[ii-1])

        t1=sorted(t1)

        r=[] 
        for i in [min(t1)..max(t1)]:
            for j in l:
                if j[ii-1]==i:
                    r.append(j)
    return r

def StarProductOfYipYjsPolynomial2_parallel_without_write_to_file(t, p1, p2, N, typ, rank, v): # p1,p2 are polynomials in terms of lists
   
    N_CPU = multiprocessing.cpu_count()
    p = get_context("fork").Pool(processes=N_CPU) # works on mac
    bb = p.starmap(StarProductOfYipYjsMonomialInTermsOfList2_without_write_to_file,[(t, M1, M2, N, typ, rank, v) for M1 in p1 for M2 in p2])
   
    r=[]
    for i in bb:
        r.append([i[0], sorted(i[1])])
    r=combineLikeTerms3(r)
       
    return r
    
def StarProductOfYipYjsMonomialInTermsOfList2_without_write_to_file(t, M1, M2, N, typ, rank, v):
    r1 = M1
    r2 = M2
    r3 = 1
    c1=r1[1]
    c2=r2[1]
    #print(c1, c2)
   
    for Y1 in c1:
        for Y2 in c2:
            #print('Y1,Y2',Y1,Y2)
            t1 = StarProductOfYipYjsCoeff2(t, Y1, Y2, N, typ, rank, v)
            #print('Y1,Y2',Y1,Y2)
            r3 *= t1
           
    c3=combineLikeTerms3(c1+c2)
    c3=sorted(c3)
    r = [r3*r1[0]*r2[0], c3]
   
   
    return r



def StarProductOfYipYjsMonomialInTermsOfList2(t, M1, M2, N, typ, rank, v,fp):
    r1 = M1
    r2 = M2
    r3 = 1
    c1=r1[1]
    c2=r2[1]
    #print(c1, c2)
   
    for Y1 in c1:
        for Y2 in c2:
            #print('Y1,Y2',Y1,Y2)
            t1 = StarProductOfYipYjsCoeff2(t, Y1, Y2, N, typ, rank, v)
            #print('Y1,Y2',Y1,Y2)
            r3 *= t1
           
    c3=combineLikeTerms3(c1+c2)
    c3=sorted(c3)
    r = [r3*r1[0]*r2[0], c3]
   
    f1=open(fp, 'a+')
    f1.write(str(r)+'\n')
    f1.close()
   
   
    return r


def UsualProductOfYipYjsMonomialInTermsOfList2(t, M1, M2, N, typ, rank, v,fp):
    r1 = M1
    r2 = M2
    r3 = 1
    c1=r1[1]
    c2=r2[1]
           
    c3=combineLikeTerms3(c1+c2)
    c3=sorted(c3)
    r = [r3*r1[0]*r2[0], c3]
   
    f1=open(fp, 'a+')
    f1.write(str(r)+'\n')
    f1.close()
   
   
    return r


def UsualProductOfYipYjsMonomialInTermsOfList2_without_write_to_file(t, M1, M2, N, typ, rank, v,fp):
    r1 = M1
    r2 = M2
    r3 = 1
    c1=r1[1]
    c2=r2[1]
           
    c3=combineLikeTerms3(c1+c2)
    c3=sorted(c3)
    r = [r3*r1[0]*r2[0], c3]
   
    return r



def StarProductOfYipYjsMonomialInTermsOfList2_parallel(t, M1, M2, N, typ, rank, v):
    r1 = M1
    r2 = M2
    r3 = 1
    c1=r1[1]
    c2=r2[1]
    #print(c1, c2)
    c3=combineLikeTerms3(c1+c2)
    c3=sorted(c3)
    
    N_CPU = multiprocessing.cpu_count()
    p = get_context("fork").Pool(processes=N_CPU) # works on mac
    bb = p.starmap(StarProductOfYipYjsCoeff2,[(t, Y1, Y2, N, typ, rank, v) for Y1 in c1 for Y2 in c2])
    
    for i in bb:
        r3=r3*i
        
    r = [r3*r1[0]*r2[0], c3]
    return r

def StarProductOfYipYjsPolynomial2(t, p1, p2, N, typ, rank, v): # p1,p2 are polynomials in terms of lists
    r3 = []
    #print(p1,p2)
    for M1 in p1:
        for M2 in p2:
            t1 = StarProductOfYipYjsMonomialInTermsOfList2_without_write_to_file(t, M1, M2, N, typ, rank, v)
            #print('M1',M1,'M2',M2,'t1',t1)
            r3.append(t1)
    #print(p1,p2)
    r = r3
    return r

def StarProductOfYipYjsPolynomialList2(t, l, N, typ, rank, v):
    if len(l) == 0:
        r = 0
    elif len(l) == 1:
        r = l[0]
    else:
        r = l[0]
        for i in range(1, len(l)):
            r = StarProductOfYipYjsPolynomial2(t, r, l[i], N, typ, rank, v)
    return r

def StarProductOfYipYjsPolynomialList2_parallel(t, l, N, typ, rank, v, fp):
    if len(l) == 0:
        r = 0
    elif len(l) == 1:
        r = l[0]
    else:
        r = l[0]
        for i in range(1, len(l)):
            r = StarProductOfYipYjsPolynomial2_parallel(t, r, l[i], N, typ, rank, v, fp)
    return r

def StarProductOfYipYjsPolynomial2_parallel(t, p1, p2, N, typ, rank, v, fp): # p1,p2 are polynomials in terms of lists
   
    N_CPU = multiprocessing.cpu_count()
    p = get_context("fork").Pool(processes=N_CPU) # works on mac
    bb = p.starmap(StarProductOfYipYjsMonomialInTermsOfList2,[(t, M1, M2, N, typ, rank, v, fp) for M1 in p1 for M2 in p2])
   
    r=[]
    for i in bb:
        r.append([i[0],sorted(i[1])])
    r=combineLikeTerms3(r)
       
    return r

def UsualProductOfYipYjsPolynomial2_parallel(t, p1, p2, N, typ, rank, v, fp): # p1,p2 are polynomials in terms of lists
   
    N_CPU = multiprocessing.cpu_count()
    p = get_context("fork").Pool(processes=N_CPU) # works on mac
    bb = p.starmap(UsualProductOfYipYjsMonomialInTermsOfList2,[(t, M1, M2, N, typ, rank, v, fp) for M1 in p1 for M2 in p2])
   
    r=[]
    for i in bb:
        r.append(i)
       
    return r

def UsualProductOfYipYjsPolynomial2_parallel_without_write_to_file(t, p1, p2, N, typ, rank, v, fp): # p1,p2 are polynomials in terms of lists
   
    N_CPU = multiprocessing.cpu_count()
    p = get_context("fork").Pool(processes=N_CPU) # works on mac
    bb = p.starmap(UsualProductOfYipYjsMonomialInTermsOfList2_without_write_to_file,[(t, M1, M2, N, typ, rank, v, fp) for M1 in p1 for M2 in p2])
   
    r=[]
    for i in bb:
        r.append(i)
       
    return r

def is_dominant_monomial(l): # l=[1, [[-1,2,3], [1,2,1]]]
    r=1
    for i in l[1]:
        if i[0]<0:
            r=0
            break
    return r

def all_dominant_monomials_in_polynomial(l):
    r=[]
    for i in l:
        if is_dominant_monomial(i)==1:
            r.append(i)
    return r

def CoefficientOftildeCij(i, j, n, N, typ, rank): # i,j star from 1
    var('z')
    qm = QuantumCartanMatrix(z, typ, rank)
    #print(qm)
    iqm = qm.inverse()
    t1 = iqm[i-1, j-1]
    t2 = taylor(t1, z, 0, N)
    r = t2.coefficient(z, n)
    return r

def diCartan(i, typ, rank): # i stars from 1
    n = rank
    r = 1
    if typ == 'B' and i < rank:
        r = 2
    elif typ == 'C' and i == rank:
        r = 2
    elif typ == 'F' and i < 3:
        r = 2
    elif typ == 'G' and i == 1:
        r = 3
    return r

def QuantumCartanMatrixIJ(z, i, j, typ, rank): # i,j star from 0
    di = diCartan(i+1, typ, rank)
    cij = CartanMatrixSelfDefined(typ, rank)[i, j]
    if i == j:
        r = z^di + z^(-di)
    elif i != j:
        r = (z^cij - z^(-cij))/(z - 1/z)
    r = r.simplify()
    return r

def QuantumCartanMatrix(z, typ, rank):
    r = Matrix(SR, rank, rank)
    #print(r)
    for i in range(rank):
        for j in range(rank):
            r[i, j] = QuantumCartanMatrixIJ(z, i, j, typ, rank)
    return r

def StarProductOfYipYjsCoeff2(t, Y1, Y2, N, typ, rank, v): # Y1=[-2,1,3]=Y_{1,3}^{-2}
    e1,i,p=Y1
    e2,j,s=Y2
    t1 = e1 * e2
    r1 = t1 * Nipjs2(i, p, j, s, N, typ, rank, v)
    r = t^r1
    return r
    
def Nipjs2(i, p, j, s, N, typ, rank, v): # i,j star from 1
    if v == 'Nakajima':
        r1 = CoefficientOftildeCij(i, j, s - p - 1, N, typ, rank)
        r2 = CoefficientOftildeCij(i, j, p - s - 1, N, typ, rank)
        r = r1 - r2
    else:
        r1 = CoefficientOftildeCij(i, j, s - p - 1, N, typ, rank)
        r2 = CoefficientOftildeCij(i, j, p - s - 1, N, typ, rank)
        r3 = CoefficientOftildeCij(i, j, s - p + 1, N, typ, rank)
        r4 = CoefficientOftildeCij(i, j, p - s + 1, N, typ, rank)
        r = 1/2 * r2 + 1/2 * r3 - 1/2 * r1 - 1/2 * r4
    return r


 
        

def X_coordinates(k,n,mat, a1): # a1=vertices1[0], vertices1 is the cluster obtained from InitialClusterWithoutNumericalMatrices(rank,n) or its mutations, mat is the matrix of the quiver, product of incoming / product of outgoing
    r=[]
    n1=mat.nrows()
    n2=mat.ncols()
    
    for p1 in a1:
        ii=p1[2]
        if RemoveTrivialTableau3(p1[1],k,n)!=[]:
            incoming=[]
            outgoing=[]
            for i in range(n1):
                if mat[i,ii]>0:
                    #print(a1[i][1])
                    incoming.append(a1[i][1])
                elif mat[i,ii]<0:
                    outgoing.append(a1[i][1])
            r.append([incoming,outgoing])
            #print(p1[1], [incoming,outgoing])
    return r

def random_complex_number(a,b):
    t0=random.randint(a,b)
    t1= random.randint(a,b)
    r=complex(t0,t1)
    return r

def AuslanderReitenQuiverUsingMutationsTwoRows(k,n,T,max_column,max_step,repeat,R,x,trop): # only works for infinite types, T is a tableau, T[i] is a column
    r=[]
    r1=u_variable_exchange_relation(k,n,T,max_column,max_step,repeat,R,x,trop)
    current_row=Marsh_Scott_twist_on_tableau_use_tropical_map_one_row(R,x,T,k,n,trop)
    #print(current_row)
    #print('r1',r1)
    r2=r1[2]
    for i in r2:
        t1=are_all_frozen(i,k,n)
        if t1==0:
            nu=i
            break
    if len(nu)==1:
        t1=nu[0]
    elif RemoveTrivialTableau3(nu[0],k,n)==[]:
        t1=nu[1] 
    elif RemoveTrivialTableau3(nu[1],k,n)==[]:
        t1=nu[0] 
    elif len(nu[0])<len(nu[1]):
        t1=nu[1] 
    elif len(nu[0])>len(nu[1]):
        t1=nu[0] 
    next_row=Marsh_Scott_twist_on_tableau_use_tropical_map_one_row(R,x,t1,k,n,trop)
    r=[current_row, next_row]
    
    return r 

def AuslanderReitenQuiverUsingMutationsThreeRows(k,n,T,max_column,max_step,repeat,R,x,trop): # only works for infinite types, T is a tableau, T[i] is a column
    r=[]
    r1=u_variable_exchange_relation(k,n,T,max_column,max_step,repeat,R,x,trop)
    current_row=Marsh_Scott_twist_on_tableau_use_tropical_map_one_row(R,x,T,k,n,trop)
    #print(current_row)
    #print('r1',r1)
    r2=r1[2]
    for i in r2:
        t1=are_all_frozen(i,k,n)
        if t1==0:
            nu=i
            break
    t2=0
    if len(nu)==1:
        t1=nu[0]
    elif RemoveTrivialTableau3(nu[0],k,n)==[]:
        t1=nu[1]
        t2=nu[0]
    elif RemoveTrivialTableau3(nu[1],k,n)==[]:
        t1=nu[0]
        t2=nu[1]
    elif len(nu[0])<len(nu[1]):
        t1=nu[1]
        t2=nu[0]
    elif len(nu[0])>len(nu[1]):
        t1=nu[0]
        t2=nu[1]
    next_row=Marsh_Scott_twist_on_tableau_use_tropical_map_one_row(R,x,t1,k,n,trop)
    if t2==0:
        previous_row=[]
    elif RemoveTrivialTableau3(t2,k,n)==[]:
        previous_row=[t2]
    else:
        previous_row=Marsh_Scott_twist_on_tableau_use_tropical_map_one_row(R,x,t2,k,n,trop)
    r=[previous_row, current_row, next_row]
    
    return r 

def RimCoordinateWithMultiplicityOne(l,m,n): # start from coorindate [0,m]
    r1=RimCoordinate(l,m,n)
    #print('r1',r1)
    r=[]
    for i in r1:
        r.append([1, i])
    return r

def largest_y_coorindate_rim_coorindates(l): # l=[[1, [0,3]], ...]
    if l==[]:
        r=-1000000
    else:
        r=-1000000
        for i in l:
            if i[1][1]>r:
                r=i[1][1]
    return r

def add_all_y_coorindates_by_b(l,b): # l=[[1, [0,3]], ...]
    r=[]
    for i in l:
        r.append([i[0], [i[1][0], i[1][1]+b]])
    return r

def RZagier(n,z): # polylogarithm, Clean Single-Valued Polylogarithms paper
    w=0
    for k in [0..n-1]:
        t1=2^k*bernoulli(k)[k]/factorial(k)*polylog( n-k , z )*log(abs(z))^k
        w=w+t1
    if n%2==0:
        r=CC(w).imag()
    else:
        r=CC(w).real()
    r=N(r)
    return r

def multipy_Laurent_monomial_in_terms_of_list(a,b): # a=[list, list], b=[list, list]
    r=[UnionOfLists(a[0], b[0]), UnionOfLists(a[1], b[1])]
    return r

def power_of_Laurent_monomial_in_terms_of_list(a,n): # a=[list, list] 
    r=[[],[]]
    if n>0:
        r=a
        for i in range(n-1):
            r=[UnionOfLists(r[0], a[0]), UnionOfLists(r[1], a[1])]
    elif n<0:
        r=a
        for i in range(-n-1):
            r=[UnionOfLists(r[0], a[0]), UnionOfLists(r[1], a[1])]
        r=[r[1],r[0]]
    return r
def remove_common_factors_list_of_lists(a): # a=[list, list, ...]
    t1=common_factors_in_list_of_lists(a)
    r=[]
    for i in a:
        t2=SetDifferenceListDifference3(i, t1)
        #print(t2)
        r.append(t2)
    return r

def wedge_product(a): # a[i] is one form f(b) db + g(c) dc + ..., compute a[1] \wedge a[2] \wedge ...
    if len(a)==1:
        return a
    else:
        k=len(a)
        n=len(a[0])
        r=[]
        for i in itertools.combinations(range(n),k):
            t1=0
            for j1 in itertools.permutations(i):
                t2=1
                for j in range(k):
                    t2=t2*a[j][i[j1[j]]]
                t2=(-1)^NumberOfInversions(j1)*t2
                t1=(t1+t2).factor() 
            r.append(t1)
        return r

def PK_factor(l):
    r=[]
    for i in [0..len(l)-3]:
        r.append([l[i], l[i+1]])
    r.append([l[-1], l[0]])
    return r

def PK_factor_minor(M,l): # 1/Delta()/Delta()...
    r=1
    for i in l:
        r=r*1/PluckerToMinorsPolynomial(M, [[1,[i]]]).expand().factor()
    return r

def map_Fkn_of_tableau_scattering_amplitudes(T,k,n): # T[i] are columns, \mathcal{F}^{(k)}_n(y) = \sum_{J \in \binom{\lbrack n\rbrack}{k}^{nf}}\omega_J(y)\mathfrak{h}_J
    indices = [(i,j) for i in range(k) for j in range(n)]
    R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
    x = dict(zip(indices, R.gens()))

    r1=tableau_to_alphaij(T)
    r2=alphaij_to_list_list_scattering_amplitudes(r1,k,n)
    vv=AddVlueOfListsMany(r2)
    r=[]
    for J in plucker_non_frozen(k,n):
        r3=map_omega_J_scattering_amplitude_quantum_affine_result_in_tropical_form(k,n,J)
        r3=combineLikeTerms3(r3)
        #print(r3)
        r4=0
        for i in r3:
            t3=evaluation_tropical_function_on_vector_CDFL_k_minus1_n_minus_k(R,x,k,n,i[1],vv)
            #print(i[1])
            #print(i[0], t3)
            r4=r4+i[0]*t3
            
        if r4!=0:
            r.append([r4, J])
    return r

def evaluation_tropical_function_on_vector_CDFL_k_minus1_n_minus_k(R, x, k,n,  f1, vv): 
    # f1 =  [[1, x_1_1, x_2_3], [1, x_1_1], [1, x_2_3], [1]]) means min( x11+x23, x11, x23, 0 ) )
    # f1 = [[2, x11]] means 2 + x11
    # f1 = [[2, x11], [3]] means min(2+x11, 0) ???
    # list_x = [x11,x12,x13,x21,x22,x23,x31,x32,x33]
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    
    r=0
    value_x={}
    
#    print('vv, len(vv)', vv, len(vv))
    
    for i in range(k-1):
        for j in range(n-k):
            #print(i,j)
            value_x[repr(x[i+1,j+1])]=vv[i*(n-k)+j]
#    print(value_x)
    r1=[]
    for i in f1:
        t1=0
        for j in [1..len(i)-1]:
            t1=t1+value_x[repr(i[j])]
        r1.append(t1)
    #print(r1)
    r=min(r1)
    return r 


def set_UJ_scattering_amplitude_quantum_affine(k,n,J): # the set U(J) in scattering amplitudes paper
    r=[]
    t1=[]
    for i in range(k):
        t1.append([0,1])
    r1=CartesianProductOfListsInAList(t1)
    for i in r1:
        t1=[]
        sn=1
        for j in range(k):
            t1.append(ModResultOneToN((i[j]+J[j]),n))
            if IsInList(ModResultOneToN((J[j]+1),n), J)==1 and i[j]!=0:
                sn=0
                break
        if sn==1:
            t2=removeDuplicates(t1)
            r.append(sorted(t2))
#             if len(t2)==k:
#                 r.append(sorted(t2))
        
    return r


def eta_J_scattering_amplitudes(J,k,n): # eta_J = -1/n \sum_{I \in ([n] choose k)} min( L_1(e_I-e_J), ..., L_n(e_I-e_J) ) s_I
    r=[]
    for I1 in NchooseM(ListAToN(1,n),k):
        ej=e_J_vector(J,n)
        ei=e_J_vector(I1,n)
        #print(ej,ei)
        t1=SubtractVlueOfTwoLists(ei,ej)
        t2=[]
        for j in [1..n]:
            t3=function_Lj_scattering_amplitudes(t1, j, n)
            t2.append(t3)
        t3=min(t2)
        r.append([t3*(-1/n), I1])
    return r

def representative_Marsh_Scott_twist_tableaux(l, k, n): # l is a list of tableaux
    R,x,trop=Marsh_Scott_twist_on_tableau_use_tropical_map_preparation(k,n)
    
    r=[]
    for i in l:
        t1=i
        t2=Marsh_Scott_twist_on_tableau_use_tropical_map_one_row(R,x,t1,k,n,trop)
        if IntersectionOfLists(r, t2)==[]:
            r.append(i)
    return r

def representative_promotion_evacuation_Marsh_Scott_twist_tableaux(l, k, n): # l is a list of tableaux
    R,x,trop=Marsh_Scott_twist_on_tableau_use_tropical_map_preparation(k,n)
    
    r=[]
    for i in l:
        t1=TableauToMatrix(i)
        t2=PromotionOfTableauNTimesInMatrix(n, t1, n)
        t3=MatrixTakeColumnsList(t2)
        
        p1=EvacuationOfTableauNTimesInMatrix(n, t1, n)
        p2=MatrixTakeColumnsList(p1)
        
        g1=Marsh_Scott_twist_on_tableau_use_tropical_map_one_row(R,x,i,k,n,trop)
        
        if IntersectionOfLists(r, t3)==[] and IntersectionOfLists(r, p2)==[] and IntersectionOfLists(r, g1)==[]:
            r.append(i)
    return r

def representative_promotion_evacuation_tableaux(l, n): # l is a list of tableaux
    r=[]
    for i in l:
        t1=TableauToMatrix(i)
        t2=PromotionOfTableauNTimesInMatrix(n, t1, n)
        t3=MatrixTakeColumnsList(t2)
        
        p1=EvacuationOfTableauNTimesInMatrix(n, t1, n)
        p2=MatrixTakeColumnsList(p1)
        
        if IntersectionOfLists(r, t3)==[] and IntersectionOfLists(r, p2)==[]:
            r.append(i)
    return r

def ChTUsingKLPolynomialInSage(T1, k):
    
    fp1="pairsOfWords.txt"
    fp3="HL8results18.txt"
    fp5="tableaux.txt"
    
    t2=TableauToSmallGapTableau(k,T1)
    #print(t2)
    t3=TableauToListsIJ(k,t2)
    #print(t3)
    A=t3[0]
    B=t3[1] 
    kk=len(A)
    
    print(kk)
    
    typ='A'
    rank=k

#     fp2="CProgramCodes.txt"
#     F1 = open(fp2,'w')
#     F1.write("./a.out ")
#     F1.write(str(kk))
#     F1.write(" r4.txt | awk '{print $NF}' > HL8results18.txt")
#     F1.close()

    W = SymmetricGroup(kk)

    w=computePairsOfElementsInSymmetricGroupToBeRunOnCProgramOfGregFirstPart(T1,typ,rank,W)
    
    #print('w', w)

    w0=W.long_element()

    #print(w0.reduced_word()) 
    
#     F1 = open(fp1,'w')
#     F1.write(str(''))
#     F1.close()
    
#     F1 = open(fp5,'w')
#     F1.write(str(''))
#     F1.close()
    
    N_CPU = multiprocessing.cpu_count()
    p = get_context("fork").Pool(processes=N_CPU) # works on mac
    bb = p.starmap(ChTUsingKLPolynomialInSageHalfUseCProgram,[(k,i,w,w0,W,T1) for i in iter(W)])
                
    r1=[]
    for i in bb:
        r1=r1+i
    
    r2=[]
    r3=[]
    r4=[]
    for t1 in r1:
        r2.append(t1[0])
        p1=t1[1][0]
        p2=t1[1][1]
        r3.append([p1,p2])
        r4.append(t1[2])
    
    F1 = open(fp1,'w')
    for i in r3:
        p1=i[0]
        p2=i[1]
        #print(p1,p2)
        for j in p1:
            F1.write(chr(j+48))
        F1.write(str(' '))
        for j in p2:
            F1.write(chr(j+48))
        F1.write(str('\n'))
    F1.close()
    
    cmd = "./a.out "+str(kk)+" pairsOfWords.txt | awk '{print $NF}' > HL8results18.txt"
    os.system(cmd)
    
    with open(fp3, 'r') as fp:
        L = [sage_eval(line) for line in fp.readlines() if line.strip()]
        
    t1=[]
    for i in L:
        if type(i)==Integer:
            t1.append([i])
        else:
            t2=[]
            for j in i:
                t2.append(j)
            t1.append(t2)
    PL=ListToPolynomialInQList(t1,1)
    
    #print('PL',PL)
    #print(len(PL), len(r2))
    r=[]
    for i in range(len(r2)):
        #print(PL[i])
        if PL[i]!=0:
            t1=r2[i]*PL[i]
            #print(PL[i], t1)
            r.append([t1, r4[i]])
            
    r=combineLikeTerms3(r)
    
    return r



def ChTUsingKLPolynomialInSageHalfUseCProgram(k,l,w,w0,W,T1): 
    
    fp1="pairsOfWords.txt" 
    
    r=[] 
    t1=ShiftByA(l.tuple(),0)
    t2=SymmetricGroupActionOnTableauxComputeCh(k,t1,T1)
    #print('t1,t2',[t1,t2])
    if t2!=[]:
        p1=(l*w0).tuple()
        p2=(w*w0).tuple()
        #print(p1,p2)
        p1=ShiftByA(p1,-1)
        p2=ShiftByA(p2,-1)
        

        t4=(-1)^((l*w).length())
        r.append([t4, [p1,p2], t2])
    return r

def ChTUsingKLPolynomialInSage2(T1, k):
        
    fp1="pairsOfWords.txt"
    fp3="HL8results18.txt"
    fp5="tableaux.txt"
    
    t2=TableauToSmallGapTableau(k,T1)
    #print(t2)
    t3=TableauToListsIJ(k,t2)
    #print(t3)
    A=t3[0]
    B=t3[1] 
    kk=len(A)
    
    #print(kk)
    
    typ='A'
    rank=k

#     fp2="CProgramCodes.txt"
#     F1 = open(fp2,'w')
#     F1.write("./a.out ")
#     F1.write(str(kk))
#     F1.write(" r4.txt | awk '{print $NF}' > HL8results18.txt")
#     F1.close()

    W = SymmetricGroup(kk)

    w=computePairsOfElementsInSymmetricGroupToBeRunOnCProgramOfGregFirstPart(T1,typ,rank,W)
    
    #print('w', w)

    w0=W.long_element()

    #print(w0.reduced_word()) 
    
#     F1 = open(fp1,'w')
#     F1.write(str(''))
#     F1.close()
    
#     F1 = open(fp5,'w')
#     F1.write(str(''))
#     F1.close()
    
    N_CPU = multiprocessing.cpu_count()
    p = get_context("fork").Pool(processes=N_CPU) # works on mac
    bb = p.starmap(ChTUsingKLPolynomialInSageHalfUseCProgram,[(k,i,w,w0,W,T1) for i in iter(W)])
                
    r2=[] 
    r4=[]
    
    F1 = open(fp1,'w')
    for i in bb:
        for t1 in i:
            r2.append(t1[0])
            r4.append(t1[2])
            
            p1=t1[1][0]
            p2=t1[1][1]
            
            for j in p1:
                F1.write(chr(j+48))
            F1.write(str(' '))
            for j in p2:
                F1.write(chr(j+48))
            F1.write(str('\n'))
    F1.close()
    
    cmd = "./a.out "+str(kk)+" pairsOfWords.txt | awk '{print $NF}' > HL8results18.txt"
    os.system(cmd)
    
    with open(fp3, 'r') as fp:
        L = [sage_eval(line) for line in fp.readlines() if line.strip()]
        
    r=[]
    for i in range(len(L)):
        ii=L[i]
        if type(ii)==Integer:
            pl=ii
        else:
            t2=[]
            for j in ii:
                t2.append(j)
            pl=ListToPolynomialInQ(t2,1)
        if pl!=0:
            t1=r2[i]*pl
            r.append([t1, r4[i]])
            
    r=combineLikeTerms3(r)
    
    return r


def removeAnElementInList(i, l):
#     r=[]
#     for j in range(len(l)):
#         if (j!=i):
#             r.append(l[j])

    r=copy.deepcopy(l)
    r.pop(i)
    
    return r

def web_matrix_DFGK_reverse_xij(R,x,k, n): # 
#     k,n=4,8
#     indices = [(i,j) for i in range(k+1) for j in range(n+1)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    r1=Matrix(SR, k,n)
    r2=Matrix(SR, k,n-k)

    for i in range(k):
        r1[i,i]=1
    for i in range(k):
        r1[i,k]=1
    
    for i1 in range(k):
        i=i1+1
        for j1 in range(n-k):
            j=j1+1
            t1=0
            for p1 in itertools.combinations_with_replacement(ListAToN(0,j-1), k-i):
                la=ReverseList(sorted(p1))
                #print(la)
                t2=1
                #print(k-i, la)
                for a1 in [1..k-i]:
                    a=a1-1
                    for b in [1..la[a]]:
                        #t2=t2*x[a1,b]
                        t2=t2*1/x[a1,b]
                        #print(t2)
                t1=t1+t2
            t1=(-1)^(k+i)*t1
            r2[i1,j1]=t1
    #print(r2)
    
    for i in [0..k-1]:
        for j in [k..n-1]:
            r1[i,j]=r2[i,j-k]
    r=r1
    return r 


def reflection_action_action_on_g_vector_CDFL_2(g,k,n): #   labels of initial seed are the same as CDFL, has some problem, theta(g(T)) is not equal to g(theta(T))
    indices = [(i,j) for i in range(k+1) for j in range(n+1)]
    R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
    x = dict(zip(indices, R.gens()))
    M = web_matrix_DFGK_reverse_xij(R,x,k,n)
    #print(M)
    #print(evaluation_of_x_coordinates_on_web_matrix_CDFL(M,k,n))
    t2=reflection_action_on_matrix_symbolic_2(R,k,n, M) 
    #print(t2)
    t3=evaluation_of_x_coordinates_on_web_matrix_CDFL(t2,k,n)
    #print(latex(t3))
    
    #r1=t3
    
#     r1=[]
#     for i in t3:
#         p1=i
#         for j in indices:
#             p1=p1.subs({x[j[0],j[1]]:1/x[j[0],j[1]]})
#         r1.append(p1)
    
#     r1=[]     # reflection need to reverse arrows of cluster
#     for i in t3:
#         p1=1/i
#         r1.append(p1)
    
    r1=t3
    
    t4=evaluation_tropical_function_rational_list_on_vector_CDFL(R, x, k,n,  r1, g)
    
    r=t4
    return r

def reflection_action_action_on_tableau_2(T,k,n): # T[i] is a column, has some problem, theta(g(T)) is not equal to g(theta(T))
    e1=initial_cluster_variables_CDFL(k,n)
    m=(k-1)*(n-k-1)
    g1=GVectorOfTableau(k,n,T,e1)[0][:m]
    #print(g1)
    r1=reflection_action_action_on_g_vector_CDFL_2(g1,k,n)
    #print(r1)
    r=GVectorToTableau_remove_inverses(r1,e1,k,n)
    
    return r 


def reflection_action_on_matrix_symbolic_2(R,k,n, M): # (v1,v2,...,vn) mapsto (-1)^( {k choose 2} ) (vn, ..., v1), has some problem, theta(g(T)) is not equal to g(theta(T))
    
    l=MatrixToTableauTakeColumns(M)
    r1=ReverseList(l)
    F=R.fraction_field()
    r2=TableauToMatrix_symbolic(F,r1)
    #print(r2)
    r=r2
    m=binomial(k,2)
    for i in range(k):
        for j in range(n):
            r[i,j]=(-1)^( m )*r2[i,j]
            
    #print(r)
    return r


def CompatibilityDegreePairTableauxUsingChT(k,n,TT1,TT2): # TT1[i] are columns, for Grassmannian case
    typ='A' # find a more efficient way to compute longest word in S_lambda w S_mu
    rank=k # number of rows of tableaux 
    rank1=k-1

    t1=TT1
    t2=TT2
    cc1=TableauToModule('Y',t1)
    cc2=TableauToModule('Y',t2) 
    
    t1=[cc1,cc2]

    d=[]
    for i in t1:
        t2=ModuleToTableau('Y',i,k,n)
        d.append(t2)

    e=[] 
    e1=[]
    e2=[]
    for T1 in d:
        tt3=ChTUsingKLPolynomialInSage(T1,k)
        e.append(tt3)

    f4=MultiplyTwoPolynomialsLists2Many(e) 
    f4=RemoveTrivialTableauxInStandardMonomialPolynomial(f4,k,n)
    f4=combineLikeTerms3(f4)
    
    while f4!=[]:
        g1=HighestWeightTableauInPolynomialOfTableaux(f4,typ,rank1)
    #    print([g1[0], RemoveTrivialTableau(g1[1],k,n)])
    #     t1=[g1[0], TableauToMultiSegments(Y,g1[1]), TableauToModuleHasY(Y,g1[1]), TableauToModuleInMatrixPAdicConventionHasY(Y,TableauToMatrix(g1[1]))]
    #     print(t1) 
        e1.append([g1[0], RemoveTrivialTableau(g1[1],k,n)])
    #     e2.append(sorted(t1[1]))
        T1=g1[1]
        if T1==[]: # trivial module, corresponding to \mathbb{C}
            c2=[[1, []]]
        else:
            c2=ChTUsingKLPolynomialInSage(T1,k)
        c2=Multiplyq2NoChangeValue(c2, g1[0])
        f4=PolynomialE1SubtractE2EiArePolynomialsInStandardMonomialsInTermsOfTableaux(f4,c2,k,n)
    
    #print(e1)
    r=len(e1)-1
    
    return r


def are_hJ_basis_representative_equal_to_another_hJ_basis_representative(v1,v2,k,n): #    v1, v2 in h_J basis equal? v1-v2 is in the linearlity space?
    a1=[]
    for i in v1:
        t1=i[0]
        J=i[1]
        t2=h_J_scattering_amplitudes(J,k,n)
        t2=Multiplyq2NoChangeValue(t2, t1)
        a1=a1+t2
    a1=combineLikeTerms3(a1)
    
    a2=[]
    for i in v2:
        t1=i[0]
        J=i[1]
        t2=h_J_scattering_amplitudes(J,k,n)
        t2=Multiplyq2NoChangeValue(t2, t1)
        a2=a2+t2
    a2=combineLikeTerms3(a2)
        
    r3=linearlity_space_scattering_amplitudes(k,n)
    num=-1
    c1=[]
    va=[]
    a=VariableGenerator('a')
    for i in range(n):
        t1=r3[i]
        num=num+1
        t1=Multiplyq2NoChangeValue(t1, a[num])
        c1=c1+t1
        va.append(a[num])
    c1=combineLikeTerms3(c1+Multiplyq2NoChangeValue(a1,-1)+a2)
    c2=TakeIthElements(1, c1)
    c3=solve(c2, va, solution_dict=True)
    
    if c3!=[]:
        t6=[]
        for q in [0..n-1]:
            t6.append([c3[0][a[q]], r3[q-len(r2)]]) 

        r=t6
    else:
        r=0
    return r


def plucker_all(k,n):
    r=NchooseM(ListAToN(1,n),k)
    return r

def map_Fkn_of_tableau_scattering_amplitudes_use_PJeJ(T,k,n): # T[i] are columns, Fkn(y)=\sum_{J \in n choose k} P_J(y)e^J
    indices = [(i,j) for i in range(k+1) for j in range(n+1)]
    R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
    x = dict(zip(indices, R.gens()))
    wM=web_matrix(R,x,k,n)
    
    r1=tableau_to_alphaij(T)
    r2=alphaij_to_list_list_scattering_amplitudes(r1,k,n)
    vv=AddVlueOfListsMany(r2)
    r=[]
    for J in plucker_all(k,n):
        
        t1=PluckerToMinorsPolynomial(wM, [[1,[J]]])
        t2=change_polynomial_to_list(R,t1)
        #print(t2)
        r3=evaluation_tropical_function_on_vector_CDFL_k_minus1_n_minus_k(R,x,k,n,t2,vv)
        #print('r3',r3)
        if r3!=0:
            r4=[r3, J] # this is P_J(y)e^J
            r.append(r4)
    return r 

def linearlity_space_scattering_amplitudes(k,n):
    r=[]
    r1=plucker_all(k,n)
    for j in [1..n]:
        r2=[]
        for J in r1:
            if IsInList(j, J)==1:
                t1=[1, J]
                r2.append(t1)
        r.append(r2)
    return r

def change_from_eJ_basis_to_hJ_basis(l,k,n): # change from e^J basis to h_J basis, scattering amplitudes
    r3=linearlity_space_scattering_amplitudes(k,n)
    num=-1
    c1=[]
    va=[]
    a=VariableGenerator('a')
    r2=plucker_non_frozen(k,n)
    for J in r2:
        t1=h_J_scattering_amplitudes(J,k,n)
        num=num+1
        #print(num,J)
        t1=Multiplyq2NoChangeValue(t1, a[num])
        c1=c1+t1
        va.append(a[num])
    for i in range(n):
        t1=r3[i]
        num=num+1
        t1=Multiplyq2NoChangeValue(t1, a[num])
        c1=c1+t1
        va.append(a[num])
    c1=combineLikeTerms3(c1+Multiplyq2NoChangeValue(l,-1))
    c2=TakeIthElements(1, c1)
    c3=solve(c2, va, solution_dict=True)
    
    t5=[]
    for q in [0..len(r2)-1]:
        if c3[0][a[q]]!=0:
            t5.append([c3[0][a[q]], r2[q]]) 
    t6=[]
    for q in [len(r2)..len(r2)+n-1]:
        t6.append([c3[0][a[q]], r3[q-len(r2)]]) 
    
    r=[t5, t6]
    return r

def alphaij_to_fundamental_tableau(i,j,k): # [j,j+k]\setminus \{i+j\} mapsto e_{i,j}
    r=ListAToN(j,j+k)
    r=SetDifference(r, [i+j])
    return r

def alphaij_list_to_tableau(l,k,n): # [j,j+k]\setminus \{i+j\} mapsto e_{i,j}
    r1=[]
    for a in l:
        i=a[0]
        j=a[1]
        t1=alphaij_to_fundamental_tableau(i,j,k)
        r1.append(t1)
    r1=RemoveTrivialTableau(r1, k, n)
    r=SortRowsOfTableau(r1)
    
    return r

def alphaij_list_to_noncrossing_tuple(l,k,n): # [j,j+k]\setminus \{i+j\} mapsto e_{i,j}
    r1=alphaij_list_to_tableau(l,k,n)
    r=NonCrossingTupleWithSameContentAndWeightAsT(r1)
    
    return r

def function_Lj_scattering_amplitudes(a, j, n): # L_j(a) = a_{j+1}+2a_{j+2}+...+(n-1)a_{j-1}
    r=0
    for i in [1..n-1]:
        t1=ModResultOneToN(j+i, n)
        t2=i*a[t1-1]
        r=r+t2
    return r

def e_J_vector(J,n): # e_J = \sum_j e_j, e_1 = (1, 0, 0, ...)
    r=[0]*n
    for i in J:
        r[i-1]=1
    return r

def h_J_scattering_amplitudes(J,k,n): # h_J = -1/n \sum_{I \in ([n] choose k)} min( L_1(e_J-e_I), ..., L_n(e_J-e_I) ) e^I
    r=[]
    for I1 in NchooseM(ListAToN(1,n),k):
        ej=e_J_vector(J,n)
        ei=e_J_vector(I1,n)
        #print(ej,ei)
        t1=SubtractVlueOfTwoLists(ej,ei)
        t2=[]
        for j in [1..n]:
            t3=function_Lj_scattering_amplitudes(t1, j, n)
            t2.append(t3)
        t3=min(t2)
        r.append([t3*(-1/n), I1])
    return r
    
    

def map_omega_J_scattering_amplitude_quantum_affine_result_in_PM(k,n,J): # the map omega_J in scattering amplitudes paper, \omega_J = \sum_{M\in \mathcal{U}(J)} (-1)^{(k-\#(M\cap J)+1) }P_J(y), P_J(y) is tropical Plucker function

    UJ=set_UJ_scattering_amplitude_quantum_affine(k,n,J)
    indices = [(i,j) for i in range(k+1) for j in range(n+1)]
    R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
    x = dict(zip(indices, R.gens()))
    wM=web_matrix(R,x,k,n)
    #print(wM)
    #print(UJ)
    r0=[]
    for uj in UJ:
        t1=(-1)^( k - len(IntersectionOfLists(uj, J)) + 1 )
        r0.append([t1, uj])
    r=combineLikeTerms3(r0)
    
    return r

def map_omega_J_scattering_amplitude_quantum_affine_result_in_tropical_form(k,n,J): # the map omega_J in scattering amplitudes paper, \omega_J = \sum_{M\in \mathcal{U}(J)} (-1)^{(k-\#(M\cap J)+1) }P_J(y), P_J(y) is tropical Plucker function
 
    UJ=set_UJ_scattering_amplitude_quantum_affine(k,n,J)
    indices = [(i,j) for i in range(k+1) for j in range(n+1)]
    R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
    x = dict(zip(indices, R.gens()))
    wM=web_matrix(R,x,k,n)
    #print(wM)
    #print(UJ)
    r=[]
    for uj in UJ:
        t1=(-1)^( k - len(IntersectionOfLists(uj, J)) + 1 )
        tt2=t1*PluckerToMinorsPolynomial(wM, [[1,[uj]]])
        t3=change_polynomial_to_list(R,tt2)
        t4=t3[0][0]
        t5=[]
        for i in t3:
            t5.append([1]+removeAnElementInList(0,i))
        t6=[t4, sorted(t5)] # [-1, [[1,x_1_1, x_2_2], [3,x_1_2, x_2_2]]] means -min( x11+x22, x12+x22 ), the coefficient in [3,x_1_2, x_2_2] does not matter
        r.append(t6)
    r=combineLikeTerms3(r)
    return r

def common_factors_in_two_lists(l): # l = [a,b], a=[c,d,..], c=[[1,2,4],[3,5,6]], d=[[1,2,4]] and so on
    
    r1=l[0]
    r2=l[1]
    r=IntersectionOfLists_count_multiplicity(r1,r2)
    
    return r

def common_factors_in_list_of_lists(l): # l = [a,b], a=[c,d,..], c=[[1,2,4],[3,5,6]], d=[[1,2,4]] and so on
    r=[]
    if len(l)==1:
        r=l
    elif len(l)>1:
        r=l[0]
        for i in range(1,len(l)):
            #print(r)
            r=IntersectionOfLists_count_multiplicity(r,l[i])

            if r==[]:
                break
    return r

def plucker_non_frozen(k,n):
    r1=NchooseM(ListAToN(1,n),k)
    r=[]
    for i in r1:
        if RemoveTrivialTableau3([i], k,n)!=[]:
            r.append(i)
    return r


def position_of_element_in_list(a, l): # position of 1 in [1,2,3] is 0
    r=-1
    for i in range(len(l)):
        if l[i]==a:
            r=i
            break
    return r



def map_omega_J_scattering_amplitude_quantum_affine(k,n,J): # the map omega_J in scattering amplitudes paper, \omega_J = \sum_{M\in \mathcal{U}(J)} (-1)^{(k-\#(M\cap J)+1) }P_J(y), P_J(y) is tropical Plucker function
    r=[]
    UJ=set_UJ_scattering_amplitude_quantum_affine(k,n,J)
    indices = [(i,j) for i in range(k+1) for j in range(n+1)]
    R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
    x = dict(zip(indices, R.gens()))
    wM=web_matrix(R,x,k,n)
    #print(wM)
    #print(UJ)
    for uj in UJ:
        t1=(-1)^( k - len(IntersectionOfLists(uj, J)) + 1 )
        t2=t1*PluckerToMinorsPolynomial(wM, [[1,[uj]]])
        r.append(t2)
    return r

def IntersectionOfListsList_count_multiplicity(l):
    r=[]
    if l!=[]:
        r=l[0]
        for i in [1..len(l)-1]:
            r=IntersectionOfLists_count_multiplicity(r, l[i])
    return r
    
def IntersectionOfLists_count_multiplicity(a,b):
    r=[]
    r1=removeDuplicates(a)
    #print('r1',r1)
    for i in r1:
        t1=NumberOfA(a,i)
        t2=NumberOfA(b,i)
        t3=min(t1,t2)
        #print(t3)
        for j in range(t3):
            r.append(i)
    #print('r',r)
    return r

def remove_common_factors_quotient_in_lists(l): # l = a/b, a,b are polynomials represented by lists
    a=l[0]
    b=l[1]
    r1=TakeIthElements(2, a)
    r2=TakeIthElements(2, b)
    p1=IntersectionOfListsList_count_multiplicity(r1)
    p2=IntersectionOfListsList_count_multiplicity(r2)
    p3=IntersectionOfLists_count_multiplicity(p1,p2)
    #print(p1)
    a1=[]
    for i in a:
        t1=i[0]
        t2=i[1]
        t2=SetDifferenceListDifference3(t2, p3)
        a1.append([t1, t2])
    b1=[]
    for i in b:
        t1=i[0]
        t2=i[1]
        t2=SetDifferenceListDifference3(t2, p3)
        b1.append([t1, t2])
        
    r=[a1,b1]
    
    return r
    
    
def remove_common_factors_in_two_lists(l): # l = [a,b], a=[c,d,..], c=[[1,2,4],[3,5,6]], d=[[1,2,4]] and so on
    
    r1=l[0]
    r2=l[1]
    p1=IntersectionOfLists_count_multiplicity(r1,r2)
    #print(p1)
     
    a1=SetDifferenceListDifference3(r1, p1)
    b1=SetDifferenceListDifference3(r2, p1) 
        
    r=[a1,b1]
    
    return r

def union_of_two_pairs_of_lists(a,b): # a=[c,d],b=[e,f],r=[c+e,d+f]
    
    a1=a[0]+b[0]
    b1=a[1]+b[1]
    
    r=[a1,b1]
    
    return r
    
def power_of_pair_of_lists(a, n): # a=[c,d], r=[c^n, d^n]

    if n==0:
        r=[ [], [] ]
    elif n>0:
        r=a
        for i in range(n-1):
            r=union_of_two_pairs_of_lists(r,a)
    return r

def AddTwoRationalFunctionsLists2_simplified(a,b): # a[0]/a[1] + b[0]/b[1], remove common factors in the result
    if a==[] or b==[]:
        return a+b
    else:
        r=AddTwoRationalFunctionsLists2(a,b)
        r=remove_common_factors_quotient_in_lists(r)
    return r

def MultiplyTwoRationalFunctionsLists2_simplified(a,b):
    if a==[] or b==[]:
        r=[]
    else:
        r=MultiplyTwoRationalFunctionsLists2(a,b)
        r=remove_common_factors_quotient_in_lists(r)
    return r

def PolynomialE1SubtractE2EiArePolynomialsInStandardMonomialsInTermsOfTableaux(e1,e2,k,n): # e1-e2
    e3=e1+Multiplyq2NoChangeValue(e2,-1)
    e4=RemoveTrivialTableauxInStandardMonomialPolynomial(e3,k,n)
    r=combineLikeTerms3(e4)
    
    return r

def RemoveTrivialTableauxInStandardMonomial(M,k,n): # M = [[...],[...]], each [...] is a Plucker coordinate
    r=[]
    r1=PluckersWhichAreOne(k,n)
    for i in M:
        if IsInList(i, r1)==0:
            r.append(i)
    r=sorted(r)
    return r

def RemoveTrivialTableauxInStandardMonomialPolynomial(P,k,n): # each term in P is [coeff, M], M = [[...],[...]], each [...] is a Plucker coordinate
    r=[]
    for i in P:
        r.append( [i[0], RemoveTrivialTableauxInStandardMonomial(i[1],k,n)] )
    return r

def MultiplyTwoPolynomialsLists2Many(e): # e is a list of polynomials in terms lists
    if e==[]:
        r=[]
    elif len(e)==1:
        r=e[0]
    elif len(e)>=2:
        r=MultiplyTwoPolynomialsLists2(e[0],e[1])
        for i in [2..len(e)-1]:
            r=MultiplyTwoPolynomialsLists2(r,e[i])
    
    return r

def IsCompatiblePairTableauxUsingChT(k,n,TT1,TT2): # TT1[i] are columns, for Grassmannian case
    typ='A' # find a more efficient way to compute longest word in S_lambda w S_mu
    rank=k # number of rows of tableaux 
    rank1=k-1

    t1=TT1
    t2=TT2
    cc1=TableauToModule('Y',t1)
    cc2=TableauToModule('Y',t2) 
    
    t1=[cc1,cc2]

    d=[]
    for i in t1:
        t2=ModuleToTableau('Y',i,k,n)
        d.append(t2)

    e=[] 
    e1=[]
    e2=[]
    for T1 in d:
        tt3=ChTUsingKLPolynomialInSage(T1,k)
        e.append(tt3)

    f4=MultiplyTwoPolynomialsLists2Many(e) 
    f4=RemoveTrivialTableauxInStandardMonomialPolynomial(f4,k,n)
    f4=combineLikeTerms3(f4)
 
    g1=HighestWeightTableauInPolynomialOfTableaux(f4,typ,rank1)
#    print([g1[0], RemoveTrivialTableau(g1[1],k,n)])
#     t1=[g1[0], TableauToMultiSegments(Y,g1[1]), TableauToModuleHasY(Y,g1[1]), TableauToModuleInMatrixPAdicConventionHasY(Y,TableauToMatrix(g1[1]))]
#     print(t1) 
    e1.append([g1[0], RemoveTrivialTableau(g1[1],k,n)])
#     e2.append(sorted(t1[1]))
    T1=g1[1]
    if T1==[]: # trivial module, corresponding to \mathbb{C}
        c2=[[1, []]]
    else:
        c2=ChTUsingKLPolynomialInSage(T1,k)
    c2=Multiplyq2NoChangeValue(c2, g1[0])
    f4=PolynomialE1SubtractE2EiArePolynomialsInStandardMonomialsInTermsOfTableaux(f4,c2,k,n)
    
    if f4==[]:
        r=1
    elif f4!=[]:
        r=0
        
    return r

def Marsh_Scott_twist_on_tableau_use_tropical_map_preparation(k,n):
    indices = [(i,j) for i in range(k+1) for j in range(n+1)]
    R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
    x = dict(zip(indices, R.gens()))
    trop=prepare_tropical_map_CDFL_Marsh_Scott_twist(k,n) 
    
    return [R,x,trop]

def u_variable_exchange_relation(k,n,T,max_column,max_step,repeat,R,x,trop): # T is a tableau, T[i] is a column
#     T=[[3, 4, 6, 7]]
#     k,n=4,9
#     R,x,trop=Marsh_Scott_twist_on_tableau_use_tropical_map_preparation(k,n)
    
    a=T
    b=Marsh_Scott_twist_on_tableau_use_tropical_map(R,x,T,k,n,trop)
    #print('b',b)
    r=compute_exchange_relation_parallel(k,n,a,b,max_column,max_step,repeat)
    return r

def are_all_frozen(l,k,n): # frozen including 129
    r=1
    for i in l:
        t1=RemoveTrivialTableau3(i,k,n)
        if t1!=[]:
            r=0 # l has a nonfrozen factor i
            break
    return r

def u_variable(k,n,T,max_column,max_step,repeat,R,x,trop): # T is a tableau, T[i] is a column
#     T=[[3, 4, 6, 7]]
#     k,n=4,9
#     R,x,trop=Marsh_Scott_twist_on_tableau_use_tropical_map_preparation(k,n)
    
    r1=u_variable_exchange_relation(k,n,T,max_column,max_step,repeat,R,x,trop)
    de=[r1[0], r1[1]]
    r2=r1[2]
    for i in r2:
        t1=are_all_frozen(i,k,n)
        if t1==0:
            nu=i
            break
    r=[T,nu,de]
    return r

def compute_exchange_relation_parallel(k,n,a,b,max_column,max_step,repeat): # max_step, repeat are some integers large enough to ensure that we get the answer
    typ=1
    rank=k
    sizeColumn=n-k
    ll0=[] 
    for i in range(1,k):
        for j in range(1,n-k):
            ll0.append((i-1)*sizeColumn+j)
    
    exchange_relation=[]
    sn=0
    while (1):  #...have split the ComputeClusterVariablesInGrkn function up into part we wish to parallelise 'll_perms' and the remainder
        #Generate a list of permutations, then run above generation function with them on different cores
        lls = [np.random.permutation(ll0) for iii in range(max_step)]
        
        #print('minn,maxn',minn,maxn)
        
        #F1 = open(fp1,'a+') 
        N_CPU = multiprocessing.cpu_count()
        p = get_context("fork").Pool(processes=N_CPU) # works on mac
        bb = p.starmap(ll_perms_compute_exchange_relations,[(lls[i], typ, rank, max_column, n, repeat,a,b) for i in range(len(lls))])        
        
        for i in bb:
            if i!=[]:
                exchange_relation=i
                sn=1
        if sn==1:
            break
    r=exchange_relation
    return r

def TranslateTableauInMatrixToProfileForCanonialTableau(l):

    n=l.nrows()
    m=l.ncols()

    r=[]
    for j in range(m-1,-1,-1): 
        t1=[]
        for i in [0..n-1]:
            t1.append(l[i,j]) 
        r.append(t1)
    return r

def ComputeCutPoint(l):
    n=l.nrows()
    m=l.ncols()
    r=0
    if m>=2: 
        for i in [0..n-2]:  
            for j in range(m-2, -1, -1):
                if l[i+1,j]<=l[i,j+1]:
                    r=m-j-1
                    break
            if r>0:
                break
    return r
    
def TranslateTableauInMatrixToCanonicalTableau(l): 

    n=l.nrows()
    m=l.ncols()

    if m==1: 
        r=TranslateTableauInMatrixToProfileForCanonialTableau(l)
    else:
        t1=l
        kk=ComputeCutPoint(l)
        for i in [0..n-2]:
            t1=MoveElementsBetweenIAndIPlusOneRows(i, kk,t1)
    r=t1
    return r

def TableauToMatrixRowsToMatrix(l):
    r=0

    if l!=[]:
        n=len(l)
        m=len(l[0])

        r=[]
        r=Matrix(n,m);
        for i in [0..n-1]:  
            for j in [0..m-1]: 
                r[i,j]=l[i][j]
    return r

def MoveElementsBetweenIAndIPlusOneRows(i1,k,l): # cut point is m-k, [1,m-k],[m-k+1,m]
    n=l.nrows()
    m=l.ncols()
    l1=MatrixTakeRows(l)
    if n==1: 
        r=l
    else:
        A=[]
        for j in [m-k..m-1]: 
            A.append(l1[i1][j])
        B=[]
        for j in [0..m-k-1]:
            B.append(l1[i1+1][j])
        A=sorted(A)
        B=sorted(B)
        if len(A)==len(B): 
            C2=A
            D2=B
        elif len(A)<len(B): 
            D2=[]
            for i in [0..len(A)-1]:
                D2.append(B[i]) 
            C2=A
        elif len(A)>len(B):
            C2=[]
            for i in range(len(A)-1, len(A)-len(B)-1, -1):
                C2.append(A[i])
            D2=B 
        C1=SetDifferenceListDifference(l1[i1], C2)
        D1=SetDifferenceListDifference(l1[i1+1],D2)
        r=l1
        r[i1]=sorted(UnionOfLists(C1,D2))
        r[i1+1]=sorted(UnionOfLists(D1,C2))
    r=TableauToMatrixRowsToMatrix(r)

    return r


def TableauToProfileRealRootCaseHalf(k,l): 
    r1=IsCanonicalTableauInTermsOfMatrix(l)
    if r1==1:  
        r=TranslateTableauInMatrixToProfileForCanonialTableau(l)
    else:
        kk=ComputeCutPoint(l)
        #print('kk,l',kk,l)
        t1=TranslateTableauInMatrixToCanonicalTableau(l)
        #print('t1',t1)
        t1=TranslateTableauInMatrixToProfileForCanonialTableau(t1)
        r=CyclicRotationATimes(kk,t1)
    return r

def IntersectionOfListsList(l): 
    r=[]
    if l!=[]:
        r=l[0]
        for i in [0..len(l)-1]:
            r=IntersectionOfLists(r,l[i])
    return r

def TableauToProfileRealRootCase(k,l):
    if DoesTableauCorrespondToRealRoot(k,l)==1: 
        l2=MatrixToTableauTakeColumns(l)
        l1=IntersectionOfListsList(l2)
        r1=[]
        for i in l2:
            r1=append(r1, sort(SetDifferenceListDifference(i, l1))) 
        if r1!=[]:
            k1=len(r1[0])
            r2=TableauToProfileRealRootCaseHalf(k1, TableauToMatrix(r1))
            r=[]
            for i in r2:
                r=append(r, sorted(UnionOfLists(i, l1))) 
        else:
            r=l
    else:
        r=0
    return r

def TableauToProfileRealRootAndNonRigidImaginaryRootCase(k,l): # l is a tableau written in matrix form
    l2=MatrixToTableauTakeColumns(l)
    l1=IntersectionOfListsList(l2)
    r1=[]
    for i in l2:
        r1=append(r1, sorted(SetDifferenceListDifference(i, l1))) 
    if r1!=[]: 
        k1=len(r1[0])
        r2=TableauToProfileRealRootCaseHalf(k1, TableauToMatrix(r1))
        #print('r2',r2)
        r=[]
        for i in r2:
            r=append(r, sorted(UnionOfLists(i, l1)))
    else:
        r=l 
    return r

def TableauToProfileRealRootAndNonRigidImaginaryRootCaseList(k,l): # not completely true for imaginary root case
    r=[]
    for i in l:
        r=append(r, TableauToProfileRealRootAndNonRigidImaginaryRootCase(k,i))
    return r

def CyclicRotationOne(l): # [a,b,c] becomes [b,c,a]
    r1=l
    r=append(removeAnElementInList(0,r1), l[0])
    return r

def CyclicRotationATimes(a,l): # apply the Rotation a times

    r=l
    for i in [1..a]: 
        r=CyclicRotationOne(r)
    return r 

def ProfileToMatrix(l): 
    r=0
    l1=ReverseList(l)
    if l!=[]:
        n=len(l[0])
        m=len(l)
        r=Matrix(n,m)
        for i in [0..n-1]: 
            for j in [0..m-1]: 
                r[i,j]=l1[j][i]
    return r

def ProfileToCanonicalTableau(l):
    r=ProfileToMatrix(l)
    kk=0
    while IsSemiStandardTableau(r)==0: 
        kk=kk+1
        l1=CyclicRotationATimes(kk,l)
        r=ProfileToMatrix(l1) 
    r=[r,kk]
    return r

def IsSemiStandardTableau(a): # a is a tableau written in the matrix form 
    n=a.nrows()
    m=a.ncols()

    r=1
    for i in [0..n-1]: 
        for j in [0..m-2]: 
            if a[i,j]>a[i,j+1]:
                r=0
                break
    for j in [0..m-1]: 
        for i in [0..n-2]:
            if a[i,j]>=a[i+1,j]:
                r=0
                break
    return r

def TranslateCanonicalTableauToNonCanonicalTableau(kk,l):
    n=l.nrows()
    m=l.ncols()

    if m==1:
        r=l
    else:
        t1=l
        if kk>m/2:
            for i in [0..n-2]: 
                t1=MoveElementsBetweenIAndIPlusOneRowsStandardToNonstandard(i, kk,t1) 
                #print('i,kk,t11',i,kk,t1)
        else:
            for i in range(n-2, -1, -1): 
                t1=MoveElementsBetweenIAndIPlusOneRowsStandardToNonstandard(i, kk,t1)
                #print('i,kk,t12',i,kk,t1)
    r=t1
    return r

def MoveElementsBetweenIAndIPlusOneRowsStandardToNonstandard(i1,k,l): # cut point is m-k, [1,m-k],[m-k+1,m]
    n=l.nrows()  # not finished
    m=l.ncols()
    l1=MatrixTakeRows(l)
    if n==1:  
        r=l
        r=TableauToMatrixRowsToMatrix(r)
    else:
        sn=0
        k1=0
        while sn==0: 
            k1=k1+1
            A=[]
            for j in [m-k..m-1]:
                A=append(A, l1[i1][j])
            B=[]
            for j in [0..m-k-1]:
                B=append(B, l1[i1+1][j]) 
            #print('A,B',A,B)
            #print('l1', l1)
            A=sorted(A)
            B=sorted(B)
            #print('A,B',A,B)
            if len(A)==len(B): 
                C2=A
                D2=B
            elif len(A)<len(B): 
                D2=[]
                for i in range(len(B)-k1, len(B)-len(A)-k1, -1):
                    D2=append(D2, B[i]) 
                C2=A
            elif len(A)>len(B):
                C2=[]
                for i in [k1-1..len(B)+k1-2]:
                    C2=append(C2, A[i])
                D2=B
                
            #print('C2,D2',C2,D2)
            C1=SetDifferenceListDifference(l1[i1], C2)
            D1=SetDifferenceListDifference(l1[i1+1], D2)

            r=copy.deepcopy(l1)
            r[i1]=sorted(UnionOfLists(C1,D2))
            r[i1+1]=sorted(UnionOfLists(D1,C2))
            r=TableauToMatrixRowsToMatrix(r)
            #print('r,C1,D2,D1,C2',r,C1,D2,D1,C2)
            if IsSemiStandardTableau(r)==1:
                sn=1
                break
    return r

def AuslanderReitenQuiverOneRowUsingTableaux(k,n,L): # L = [a,b], a is a tableau, a is not a frozen variable, a[i] is a column, b is ch(a) 
    t2=MarshScottTwistForPairTandChT(k,n,L)
    r=[L,t2]
    #print('t2',t2)
    while t2[0]!=L[0]:
        t2=MarshScottTwistForPairTandChT(k,n,t2)
        #print('t2',t2)
        r.append(t2)
    r=ReverseList(r)
    return r

def MarshScottTwistForPairTandChT(k,n,l): #  l = [T, ch(T)], ch(T)=[[1,[[1,2,4],[3,5,6]]], [-1, [[1,2,3],[4,5,6]]]]
    T=l[0]
    ch=l[1]
    r1=MarshScottTwistForPolyInPluckInTermsOfListResultInTableau(k,n,ch) 
    T1=r1[0] # T1 is tau(T) not reduced
    ch1=r1[1] # ch(T1)=ch1
    #print('ch1', ch1)
    
    T3=[]
    T2=T1
    a1=PluckersWhichAreOne2(k,n,n)
    for i in a1:
        if IsTableauAFactorOfB([i], T2)==1:
            t1=TableauDivision(T2,[i])
            #print('t1',t1)
            if IsSemiStandardTableauTIsList(t1)==1:
                T3.append(i)
                T2=t1
        
    #print('T1,T2,T3',T1,T2,T3)
    e1=NonCrossingTupleWithSameContentAsT_new(T2)
    #print('e1',e1)
    p1=[]
    for i in [0..len(e1)-1]:
        M=randomMatrixNonInt(k,n)
        #print('ch1,T3', ch1, T3)
        g1=PluckerToMinorsPolynomial(M, ch1)/PluckerToMinorsPolynomial(M, [[1, T3]])
        p1.append([M, g1])

    ch2=ComputeChTInTermsOfNonCrossingBasis([ e1, p1 ])  # ch2=ch(T2)
    
    r=[T2, ch2]

    return r 

def IsSemiStandardTableauTIsList(T): # a = [[],[]] is a tableau, a[i] are columns 
    a=TableauToMatrix(T)
    n=a.nrows()
    m=a.ncols()
    r=1
    for i in [0..n-1]: 
        for j in [0..m-2]: 
            if a[i,j]>a[i,j+1]:
                r=0
                break

    for j in [0..m-1]: 
        for i in [0..n-2]: 
            if a[i,j]>=a[i+1,j]:
                r=0
                break
    return r

def MarshScottTwistOfPolynomialInPluckerDirectly(l,k,n): # l = [[1,[[],[]]],[-1,[[],[]]]], compute tau(l)
    T=MarshScottTwistForPolyInPluckInTermsOfListReturnTableauWhoseContentSameAsTauT(k,n,l) # T is a tableau whose content is tau(l)
    #print('T',T)
    r5=NonCrossingTupleWithSameContentAsT_new(T)
    n1=len(r5)
    p1=np.zeros([n1,n1])
    p2=np.zeros([n1])
    for j1 in range(n1): 
        M=randomMatrixNonInt(k,n) 
        tM=MarshScottTwist(np.array(M)) 
        p2[j1]=PluckerToMinorsPolynomial(tM, l) 
        for p in [0..n1-1]: 
            p1[j1, p]=PluckerToMinors(M, [1, r5[p]]) 
    t1=np.linalg.solve( p1,p2 )

    t5=[] 
    for q in [0..n1-1]:
        t5.append([int(round(t1[q])), r5[q]])
    r=RemoveZeroTerms(t5) 
    
    return r



def MarshScottTwistForPairTandChTComputeOnlyTauT(k,n,l): #  l = [T, ch(T)], ch(T)=[[1,[[1,2,4],[3,5,6]]], [-1, [[1,2,3],[4,5,6]]]]
    T=l[0]
    ch=l[1]
    r1=MarshScottTwistForPolyInPluckInTermsOfListResultInTableau(k,n,ch) 
    T1=r1[0] # T1 is tau(T) not reduced
    ch1=r1[1] # ch(T1)=ch1
    #print('ch1', ch1)
    
    T3=[]
    T2=T1
    a1=PluckersWhichAreOne2(k,n,n)
    for i in a1:
        if IsTableauAFactorOfB([i], T2)==1:
            t1=TableauDivision(T2,[i])
            if IsSemiStandardTableauTIsList(t1)==1:
                T3.append(i)
                T2=t1
        
    r=T2

    return r 

def MarshScottTwistForPolyInPluckInTermsOfListReturnTableauWhoseContentSameAsTauT(k,n,l): # l=[[1,[[1,2,4],[3,5,6]]], [-1, [[1,2,3],[4,5,6]]]]
    t1=MarshScottTwistForPolyInPluckInTermsOfListResultInList(k,n,[l[0]])
    r=t1[0][1]
    return r

def MarshScottTwistForPolyInPluckInTermsOfListResultInTableau(k,n,l): # l=[[1,[[1,2,4],[3,5,6]]], [-1, [[1,2,3],[4,5,6]]]]
    t1=MarshScottTwistForPolyInPluckInTermsOfListResultInList(k,n,l)
    #print('t1',t1)
    t1=ChangePolynomialInPluckerToNoncrossingBasis(t1,k,n) 
    #print('t1',t1)
    p1=HighestWeightTableauInPolynomialOfTableaux(t1,'A',k-1)
    #print('p1',p1)
    r=[SortRowsOfTableau(p1[1]), t1]
    return r 

def ChangePolynomialInPluckerToNoncrossingBasis(l,k,n):
    r=[]
    if l!=[]:
        r0=l[0][1]
        e1=NonCrossingTupleWithSameContentAsT_new(r0)
        
        p1=[]
        for i in [0..len(e1)]:
            M=randomMatrixNonInt(k,n)
            g1=PluckerToMinorsPolynomial2(M, l)
            p1.append([M, g1])
        
        r=ComputeChTInTermsOfNonCrossingBasis([ e1, p1 ])
        
    return r


def MarshScottTwistForPolyInPluckInTermsOfListResultInList(k,n,l1): # Gr(k,n), ll = [[1, [[],[]]], [-1, [[],[]]]]
    r=[]
    for i in l1:
        t0=i[0]
        t1=i[1]
        r1=[[t0,[]]]
        for j in t1:
            t2=MarshScottTwistOfPlucker(j, k, n)
            r1=MultiplyTwoPolynomialsLists2(r1,t2)
        r=r+r1
            
    return r

def IsRimAAboveB(l1,l2): # l is a list of lists, first rim starts from level a, Gr(k,n), k is the length of l

    l3=sorted(l1)
    l4=sorted(l2)

    r=1
    for i in [0..len(l3)-1]: 
        if l3[i][1]<l4[i][1]:
            r=0
            break
    return r

def SortByFirstElementInFirstTerms(l):
    r=[]
    if l!=[]:
        t1=[]
        for i in l:
            t1=append(t1, i[0][0]) 
        t1=sorted(t1)
        t2=sorted(l)
        r=[]
        for i in range(max(t1), min(t1)-1, -1): 
            for j in t2:
                if j[0][0]==i: 
                    r=append(r, j)
    r=ReverseList(r)
    return r

def TakeSecondTermsInList(L):
    r=[]
    for i in [0..len(L)-1]:
        r.append(L[i][1]) 

    return r
    
def RimCoordinateToRimManyRims(l,a,n,m):
    l1=l
    r=[]
    #print('l1',l1)
    while l1!=[]:
        t2=RimCoordinateToRimOneRim(l1,a,n,m)
        #print('t2',t2)
        if t2==-1:
            #r=[]
            break
        else:
            r.append(t2)
            l1=RimCoordinatesSubtractTopRim(l1,a,n,m)
            #print('r',r)
    return r

# def RimCoordinateToRimManyRims2(l,a,n,m):
#     l1=l
#     r=[]
#     #print('l1',l1)
#     while l1!=[]:
#         t2=RimCoordinateToRimOneRim(l1,a,n,m)
#         #print('t2',t2)
#         if t2==-1:
#             r=[]
#             break
#         else:
#             r.append(t2)
#             l1=RimCoordinatesSubtractTopRim(l1,a,n,m)
#             #print('r',r)
#     return r


def FirstRimOfASetOfCoordinatesWithMultiplicityOne(l1,a,n,m):
    r1=FirstRimOfASetOfCoordinates(l1,a,n,m)
    r=[]
    for i in r1:
        r.append([i[0], 1])
    return r
    
def RimCoordinatesSubtractTopRim(l, a, n, m):
    l1=l
    r1=FirstRimOfASetOfCoordinatesWithMultiplicityOne(l,a,n,m)
    r2=DimensionVectorForLastRim(r1,a,n,m)[0]
    r2=SwichFirstAndSecondElement(r2)
    r=RemoveZeroTerms(combineLikeTerms3(UnionOfLists(l1, Multiplyq2NoChangeValue(r2, -1))))
    return r

def HasNegativeElement(l):
    r=0
    for i in l:
        if i<0:
            r=1
            break 
    return r
    
def RimCoordinateToRimOneRim(l,a,n,m):

    l1=FirstRimOfASetOfCoordinates(l,a,n,m)
    #print('l1',l1)
    p1=TakeSecondTermsInList(l1)
    if HasNegativeElement(p1)==1:
        r=-1
    else:
        r1=SortByFirstElementInFirstTerms(l1)
        r=[]
        for i in [0..n-1]:
            if r1[i][0][1]>r1[i+1][0][1]: 
                r.append(r1[i+1][0][0])
    return r

def AuslanderReitenTranslation(l,a,n,m): # l = [[3, 6, 9], [2, 5, 8], [1, 4, 7]], a,n,m=3,9,2
    t3=TauInverseGLS(l,a,n,m)
    if t3==[]:
        r=[]
    else:
        r=RimCoordinateToRimManyRims(t3,a,n,m)
    return r

def AuslanderReitenQuiverOneRow(L,a,n,m):
    t2=AuslanderReitenTranslation(L,a,n,m)
    if t2==[]:
        r=[]
    else:  
        r=[L,t2]
        while t2!=L and t2!=[]:
            t2=AuslanderReitenTranslation(t2,a,n,m)
            r.append(t2)
    return r

def CoordinateOfProfileWithMultiplicityOne(l,a,n): # l is a list of lists, first rim starts from level a, Gr(k,n), k is the length of l

    r1=CoordinateOfProfile(l,a,n)
    r=[]
    for i in r1: 
        t1=[]
        for j in i:
            t1.append([j, 1])
        
        r.append(t1) 

    return r

def CoordinateOfProfile(l,a,n): # l is a list of lists, first rim starts from level a, Gr(k,n), k is the length of l
    r=[]
    r1=RimCoordinate(l[0],a,n)
    r.append(r1)
    a1=a
    for i in [1..len(l)-1]:
        sn=0
        a2=a1
        while (sn==0):
            t1=RimCoordinate(l[i],a2,n)
            if IsRimAAboveB(r1, t1)==1: 
                r.append(t1)
                r1=t1
                a1=a2
                sn=1
            else:
                a2=a2-1 
    return r

def RimCoordinate(l,a,n): # l is a list, start from level a, Gr(k,n), k is the length of l

    l1=sorted(l)

    r=[[0,a]]
    r1=a
    for i in [1..n]: 
        if IsInList(i,l)==1:
            r1=r1-1
            r.append([i,r1])
        else:
            r1=r1+1
            r.append([i,r1]) 

    return r

def DimensionVectorOfProfileNew(l,a,n,m):
    r1=CoordinateOfProfileWithMultiplicityOne(l,a,n)
    
    #print('r1', r1)
    
    r=DimensionVectorForManyRims(r1,a,n,m)
    #print('r', r)
    return r

def DimensionVectorForManyRims(a1,a,n,m):
    r=[]
    #print('a1',a1)
    for i in a1: 
        #print('i',i)
        t1=DimensionVectorForLastRim(i,a,n,m)[0]
        #print('t1',t1)
        r=r+t1 

    return r

def ElementWithXCoordinateI(i1, l):

    r=[]

    for j in l:
        if j[0][0]==i1: 
            r=j 

    return r

def DimensionVectorForLastRim(a1,a,n,m): # m is the number of rows we take
    r1=a1 
    r=[]
    r3=[]

    for i in [0..n]: 
        p1=[]

        for j in range(ElementWithXCoordinateI(i, r1)[0][1], ElementWithXCoordinateI(i, r1)[0][1]-2*m-2, -2):
            p1=append(p1, [[i, j], ElementWithXCoordinateI(i, r1)[1]]) 
        #print('i,p1',i,p1)
        r=r+p1 

    r=[r,r3]

    return r

def FirstRimOfASetOfCoordinates(l1,a,n,m):
 
    l=SwichFirstAndSecondElement(l1)

    r=[]
    for i in [0..n]: 
        t1=[]

        for j in l:
            if j[0][0]==i:
                t1.append(j) 

        if t1!=[]:
            t2=t1[0]
            for k in t1:
                if k[0][1]>t2[0][1]: 
                    t2=k 
            r.append(t2)

    return r

def FirstRimOfASetOfCoordinatesWithMultiplicityOnePositive(l1,a,n,m):

    r1=FirstRimOfASetOfCoordinates(l1,a,n,m)
    r=[]
    for i in r1:
        if i[1]>0: 
            r.append([i[0], 1])
        else:
            r.append(i) 
    return r

def PeakSetPositiveRemoveOneBoundary(l,a,n):

    r=[]
    r1=[]
    for i in l: 
        if IsPeak(i,l,a,n)==1 and i[1]>0: 
            r.append(i) 
        if IsOnBoundary(i,l,a,n)==1: 
            r1.append(i) 

    r2=IntersectionOfLists(r,r1)

    if len(r2)==2:
        r=SetDifference(r,r2)
        r.append(r2[0]) 
    r=sorted(r)

    return r

def IsPeak(p,l,a,n):

    r=1
    l1=l
    if IsInList(p,l1)==0:
        r=0
    
    r1=BoundaryPoints(l,a,n)
    t1=r1[0]
    t2=r1[1] 

    for i in l:
        if (i[0][0]==p[0][0]+1 and i[0][1]>p[0][1]) or (i[0][0]==p[0][0]-1 and i[0][1]>p[0][1]):
            r=0
        elif (i[0][0]==1 and  p[0][0]==n and i[0][1]>t1[0][1]): 
            r=0
        elif (i[0][0]==n-1 and  p[0][0]==0 and i[0][1]>t2[0][1]):
            r=0
            
    return r

def BoundaryPoints(rim,a,n):
 
    t1=0
    t2=0

    for i in rim: 
        if i[0][0]==0: 
            t1=i
        elif i[0][0]==n: 
            t2=i 
    r=[t1,t2]

    return r

def IsOnBoundary(p,l,a,n):
    r=0
    l1=l
    if IsInList(p,l1)==0: 
        r=0 
    if p[0][0]==n or p[0][0]==0: 
        r=1
    return r

def DimensionVectorForProjectiveCoverOneP(a1,a,n,m):
    r1=RimCoordinateWithMultiplicityForProjectiveCoverOneP(a1,a,n)
    r=DimensionVectorForLastRim(r1,a,n,m)[0]

    return r


def RimCoordinateWithMultiplicityForProjectiveCoverOneP(a1,a,n):
    r=[]
    r.append(a1)
    #print('r',r)
    r1=copy.deepcopy(a1)
    #print('a1[0][0]+a,n',a1[0][0]+a,n)
    if a1[0][0]+a>=n: 
        for i in [a1[0][0]+1..n]:
            r1[0][1]=r1[0][1]-1;
            r.append([[i, r1[0][1]], r1[1]]) 
    elif a1[0][0]+a<n:
        for i in [a1[0][0]+1..a1[0][0]+a]: 
            r1[0][1]=r1[0][1]-1
            r.append([[i,r1[0][1]], r1[1]]) 
            #print('r',r)
        for i in [a1[0][0]+a+1..n]: 
            r1[0][1]=r1[0][1]+1
            r.append([[i,r1[0][1]], r1[1]]) 
            #print('r',r)
    
    r1=copy.deepcopy(a1)
    if a1[0][0]+a<=n: 
        for i in range(a1[0][0]-1, -1, -1): 
            r1[0][1]=r1[0][1]-1
            r.append([[i,r1[0][1]], r1[1]]) 
    elif a1[0][0]+a>n:
        for i in range(a1[0][0]-1, ModResultOneToN(a1[0][0]+a,n)-1, -1):
            r1[0][1]=r1[0][1]-1
            r.append([[i,r1[0][1]], r1[1]]) 
        for i in range(ModResultOneToN(a1[0][0]+a,n)-1, -1, -1):
            r1[0][1]=r1[0][1]+1
            r.append([[i,r1[0][1]], r1[1]]) 
    #print('r',r)
    r=sorted(r)
    return r

def RimCoordinatesSubtractARim(l, L, a, n, m): # l-L
 
    l1=l
    r2=L
    r2=SwichFirstAndSecondElement(r2) 
    r=RemoveZeroTerms(combineLikeTerms3(UnionOfLists(l1, Multiplyq2NoChangeValue(r2, -1))))

    return r

def SortByFirstElementInSecondTerms(l):
    r=[]
    if l!=[]: 
        t1=[]
        for i in l:
            t1.append(i[1][0])
        t1=sorted(t1) 
        t2=sorted(l)
        r=[]
        for i in range(max(t1), min(t1)-1, -1): 
            for j in t2:
                if j[1][0]==i: 
                    r.append(j)
    r=ReverseList(r)

    return r

def SwichFirstAndSecondElement(l): 
    r=[]
    for j in l:
        r.append([j[1],j[0]]) 
    return r

def ProjectiveCover2(l,a,n,m): # l is a list of list, profile

    r=[]
    #print('l,a,n,m',l,a,n,m)
    l1=DimensionVectorOfProfileNew(l,a,n,m)
    #print('l1',l1)
    sn=0

    num=0
    nrow=len(l)

    l1=combineLikeTerms3(SwichFirstAndSecondElement(l1))
 
    p1=TakeFirstElements(l1)
    if HasPositive(p1)==0:
        sn=1 
    while sn==0 and num<=3*nrow+2:
        L=FirstRimOfASetOfCoordinatesWithMultiplicityOnePositive(l1,a,n,m)
        s1=PeakSetPositiveRemoveOneBoundary(L,a,n)
        #print('L',L)
        #print('s1',s1)
        num=num+1
        for i in s1: 
            p1=TakeFirstElements(l1)
            #print('p1',p1)
            if HasPositive(p1)==0: 
                sn=1
            #print('i',i)
            if i[1]>0 and sn==0:
                r.append(i)
                t1=DimensionVectorForProjectiveCoverOneP(i,a,n,m)
                #print('t1',t1)
                l1=RimCoordinatesSubtractARim(l1, t1, a,n,m) # l1-t1
                #print('l1',l1)
        p1=TakeFirstElements(l1)
        if HasPositive(p1)==0: 
            sn=1 

    if num>3*nrow+2:
        r=[] 

    return r

def UnionOfListsManyLists(l):
    r=[]
    for i in l:
        r=r+i
    return r

def TauInverseGLS(l,a,n,m): 

    t3=DimensionVectorOfProfileNew(l,a,n,m) 
    t1=CoordinateOfProfileWithMultiplicityOne(l,a,n)
    t4=ProjectiveCover2(l,a,n,m)
    #print('t4',t4)
    if t4==[]:
        r=[]
    else:
        s1=[]
        for i in t4:
            t1=DimensionVectorForProjectiveCoverOneP(i,a,n,m)
            #print('t1',t1)
            s1.append(SwichFirstAndSecondElement(t1)) 

        r1=UnionOfListsManyLists(s1) 
        r2=SortByFirstElementInSecondTerms(combineLikeTerms3(r1))
        r4=combineLikeTerms3(SwichFirstAndSecondElement(t3))
        r=SortByFirstElementInSecondTerms(RemoveZeroTerms(combineLikeTerms3(UnionOfLists(r2, Multiplyq2NoChangeValue(r4,-1)))))
    return r

def TakeFirstElements(l):

    r=[]
    for i in l:
        r.append(i[0])
     
    return r

def HasPositive(l):

    r=0
    for i in l:
        if i>0:
            r=1
            break
    return r

def ProfileToTableauRealRootCase(k,l1):

    if len(l1)==1: 
        n=len(l1[0])
        r=Matrix(n,1)
        for i in [0..n-1]: 
            r[i,0]=l1[0][i] 
    else:
        if DoesProfileCorrespondToRealRoot(k,l1)==1: 
            r1=IntersectionOfListsList(l1)
            l2=[]
            for i in l1:
                l2.append(SetDifferenceListDifference(i,r1)) 
            k1=len(l1[1])-len(r1)
            r2=ProfileToTableauRealRootCaseHalf(k1,l2)
            r2=MatrixToTableauTakeColumns(r2)
            t2=[]
            for i in r2:
                t2.append(sorted(r1+i)) 
            r=TableauToMatrix(t2)
        else:
            r=0
    return r

def ProfileToListOfTimesOfOccurrenceOfNumbers(a): 
    r=[]
    r1=UnionOfListsManyLists(a)

    for k in [1..max(r1)]:
        t1=0
        for i in r1:
            if i==k:
                t1=t1+1 
        r.append(t1) 
    return r

def DoesProfileCorrespondToRealRoot(k,l): # Gr(k,n)

    r1=ProfileToListOfTimesOfOccurrenceOfNumbers(l)
    r2=QaRoots(k,r1)

    if r2==2:
        r=1
    else:
        r=0
    return r


def IsCanonicalTableauInTermsOfMatrix(l):
    n=l.nrows()
    m=l.ncols()
    r=1
    for i in [1..n-1]: 
        for j in [0..m-2]: 
            if l[i,j]<=l[i-1, j+1]:
                r=0
                break
    return r

def ProfileToTableauRealRootCaseHalf(k,l1):
    m=len(l1)
    #print('l1',l1)
    l=TableauToMatrix(ReverseList(l1))
    
    #print('l',l)
    
    if IsCanonicalTableauInTermsOfMatrix(l)==1 and IsSemiStandardTableau(l)==1:
        r=l
    else:
        r1=ProfileToCanonicalTableau(l1)
        kk=r1[1]
        kk1=m-kk
        #print("m,kk1,r1",m,kk1,r1)
        t1=TranslateCanonicalTableauToNonCanonicalTableau(kk1,r1[0])
        #print(t1)
        r=t1
    return r

def ProfileToTableauRealRootAndNonRigidImaginaryRootCaseList(k,l1):  
    r=[]
    for i in l1:
        t1=ProfileToTableauRealRootAndNonRigidImaginaryRootCase(k,i)
        r.append(t1)
        #print(t1)
    return r

def ProfileToTableauRealRootAndNonRigidImaginaryRootCase(k,l1):  

    if len(l1)==1: 
        n=len(l1[0])
        r=Matrix(n,1)
        for i in [0..n-1]: 
            r[i,0]=l1[0][i] 
    else:
        r1=IntersectionOfListsList(l1)
        l2=[]
        for i in l1:
            g1=sorted(SetDifferenceListDifference(i,r1))
            l2.append(g1)
            #print('i,r1,difference',i,r1,g1)
        k1=len(l1[1])-len(r1)
        r2=ProfileToTableauRealRootCaseHalf(k1,l2)
        r2=MatrixToTableauTakeColumns(r2)
        t2=[]
        for i in r2:
            t2.append(sorted(UnionOfLists(r1,i)))
        r=TableauToMatrix(t2)
    return r


def ARQuiverTableaux(L1,L2, m): # L1, L2 are first two rows, compute new m rows, AuslanderReiten quiver from the first two rows

    r=[L1,L2]
    l1=copy.deepcopy(L1)
    l2=copy.deepcopy(L2)

    for j in [0..m-1]: 
        r1=[]
        for i in [0..len(l2)-2]: 
            t1=UnionOfTwoTableaux(l2[i],l2[i+1])
            #print('t1',t1)
            t2=TableauDivision(t1, l1[i])
            #print('l1[i]',l1[i])
            #print('t2',t2)
            r1=append(r1, t2)
        r1=UnionOfLists([t2], r1)
        #print('r1',r1)
        r=append(r, r1)
        l1=copy.deepcopy(l2)
        l2=copy.deepcopy(r1)
    return r

def ll_perms_compute_exchange_relations(lli,typ,rank,max_column,n,repeat,a,b): 
    b1=[]
    IC=InitialClusterWithoutNumericalMatrices(rank,n)
    mat1=IC[0]
    vertices1=IC[1]
    #print('rr')
    #ll=list(np.array(lli)-1)
    ll=list(map(lambda x: x - 1, lli))
    #print(ll)
    
    mutationSequence=[]
    for j1 in [1..repeat]: ###why do this?, just to repeat the same sequence of mutations, it will give more cluster variables, without this loop is also ok
        mutationSequence=mutationSequence+ll 
        
    for j in range(len(mutationSequence)): 
        #print('kk')
        vertices1 = computeEquationsForModulesTableaux_compute_exchange_relations(vertices1, mat1, mutationSequence[j],typ,rank,a,b)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 

        if vertices1[1].ncols()>max_column:
            vertices1 = computeEquationsForModulesTableaux_compute_exchange_relations(vertices1, mat1, mutationSequence[j],typ,rank,a,b) ###why repeat above calculation?
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
        elif vertices1[2]!=[]:
            #b1.append(vertices1[1]) 
            b1=vertices1[2]
            break
            
#             if MatrixTakeRows(vertices1[1][len(vertices1[1])-1])==[[1,2,2,2],[3,4,4,4],[5,6,6,6],[7,8,8,8],[9,10,10,10]]:
#                 print(vertices1[1][len(vertices1[1])-1])

#     b1=removeDuplicates2(b1)
#     b1=ExtendSetOfTableauxToContainPromotionsInMatrix(b1,n)
#     b1=ExtendSetOfTableauxToContainEvacuationsInMatrix(b1,n)
#     b1=TableauExpansionsInMatrixList(b1, n)
    
     

    return b1

def ARQuiverComputeSecondRowUsingExchangeRelation(l, rank, n, max_column, max_step, repeat): # Gr(rank,n), l is the first row of AR quiver, works for the case that l consists of cluster variables
    r=[]
    typ=1
    for ii in [0..len(l)-2]:
        a=l[ii]
        b=l[ii+1]
        
        k=rank
        sizeColumn=n-k

        ll0=[] 
        for i in range(1,k):
            for j in range(1,n-k):
                ll0.append((i-1)*sizeColumn+j)

        sn=0
        exchange_relation=[]
        while (1):  #...have split the ComputeClusterVariablesInGrkn function up into part we wish to parallelise 'll_perms' and the remainder
            #Generate a list of permutations, then run above generation function with them on different cores
            lls = [np.random.permutation(ll0) for iii in range(max_step)]
            N_CPU = multiprocessing.cpu_count()
            p = get_context("fork").Pool(processes=N_CPU) # works on mac
            bb = p.starmap(ll_perms_compute_exchange_relations,[(lls[i], typ, rank, max_column, n, repeat,a,b) for i in range(len(lls))])        

            for i in bb:
                if i!=[]:
                    exchange_relation=i
                    sn=1
            if sn==1:
                break
        r1=exchange_relation[2]
        #print(r1)
        
        t1=[]
        for i in r1:
            t2=[]
            for j in i:
                if RemoveTrivialTableau2(j,k,n)!=[]:
                    t2=UnionOfTwoTableaux(t2, j)
            t1.append(t2)
        if t1[0]!=[]:
            r.append(t1[0])
        else:
            r.append(t1[1])
    r=[r[-1]]+r
    r=[l,r]
    return r
    

def computeEquationsForModulesTableaux_compute_exchange_relations(variable2, mat, k, typ, rank,a,b): # a*b=c+d, variable2=(variables on quiver, cluster variables obtained so far)
    variable=variable2[0]
    clusterVariables=variable2[1] 
    size=mat.nrows() 
    newVariable=[]
    newVariable2=[]
    old_cluster_variable=copy.deepcopy(variable[k][1])
    c1=[]
    d1=[]
    sn=[]

    for i in range(size):
        if mat[i, k]>0:
            newVariable=UnionOfTwoTableaux( newVariable, PowerOfTableaux(variable[i][1], mat[i,k]) )
            for j in range(mat[i,k]):
                c1.append(variable[i][1])
 
    for i in range(size): 
        if mat[i, k]<0:
            newVariable2= UnionOfTwoTableaux( newVariable2, PowerOfTableaux(variable[i][1], -mat[i,k]) )
            for j in range(-mat[i,k]):
                d1.append(variable[i][1])
 
    variable[k][0]=variable[k][0]+1
    t1=compareWeightsTableaux(newVariable, newVariable2,typ,rank)
 
    if t1==1: 
        variable[k][1]=TableauDivision(newVariable, variable[k][1])
    else:
        variable[k][1]=TableauDivision(newVariable2, variable[k][1])  
        
    clusterVariables=TableauToMatrix(variable[k][1]) 
    
    if sorted([old_cluster_variable, variable[k][1]])==sorted([a,b]):
        sn=[old_cluster_variable, variable[k][1], sorted([sorted(c1), sorted(d1)])]
        #print('sn',sn)
    
    r=[variable, clusterVariables, sn]

    return r

def ARQuiverComputeThirdOrLargerRowUsingExchangeRelation(L, rank, n, max_column, max_step, repeat): # has some problem? Gr(rank,n), L is a part of AR quiver, works for the case that l[-1] consists of cluster variables
    r=[]     
    typ=1
    l0=L[-2]
    l=L[-1]
    for ii in [0..len(l)-2]:
        a=l[ii]
        b=l[ii+1]
        
        k=rank
        sizeColumn=n-k

        ll0=[] 
        for i in range(1,k):
            for j in range(1,n-k):
                ll0.append((i-1)*sizeColumn+j)

        sn=0
        exchange_relation=[]
        while (1):  #...have split the ComputeClusterVariablesInGrkn function up into part we wish to parallelise 'll_perms' and the remainder
            #Generate a list of permutations, then run above generation function with them on different cores
            lls = [np.random.permutation(ll0) for iii in range(max_step)]
            N_CPU = multiprocessing.cpu_count()
            p = get_context("fork").Pool(processes=N_CPU) # works on mac
            bb = p.starmap(ll_perms_compute_exchange_relations,[(lls[i], typ, rank, max_column, n, repeat,a,b) for i in range(len(lls))])        

            for i in bb:
                if i!=[]:
                    exchange_relation=i
                    sn=1
            if sn==1:
                break
        r1=exchange_relation[2]
        #print(r1)
         
        if IsInList(l0[ii], r1[0]):
            t3=TableauDivision(UnionOfTableauxMany(r1[0]),l0[ii])
        elif IsInList(l0[ii], r1[1]):
            t3=TableauDivision(UnionOfTableauxMany(r1[1]),l0[ii])
        r.append(t3) 
        print('t3',TableauToMatrix(t3))
    r=[r[-1]]+r
    r=L+[r]
    return r

def web_matrix_not_normalized(R, x,k, n):
    #indices = [(i,j) for i in range(k) for j in range(n)]
    #R1 = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
    #x = dict(zip(indices, R.gens()))
    r1=Matrix(SR, k,n)
    for i in range(k):
        r1[i,i]=1
    for i in [k..n-1]:
        r1[k-1,i]=x[k,i-k+1]
        #r1[k-1,i]=1
    #print(r1)
    for i in [1..k-1]:
        for j in [1..n-k]:
            t1=0
            for p in itertools.combinations_with_replacement(ListAToN(1,j), k-i): # choose with repetition
                t3=1
                for p1 in [0..len(p)-1]:
                    t3=t3*x[i+p1,p[p1]]
                t3=(-1)^(k+i)*t3
                t1=t1+t3
            t1=t1*x[k,j]
            r1[i-1,k+j-1]=t1
    r=r1
    return r

def split_list_every_part_k(l,k):
    r=[]
    for i in range(ceil(len(l)/k)):
        t1=[]
        for j in range(k):
            if i*k+j<len(l):
                t1.append(l[i*k+j])
            else:
                break
        r.append(t1)
    return r

def prepare_tropical_map_CDFL_including_frozens(ii,k,n): # prepare for tropical map sigma_i
    indices = [(i,j) for i in range(k+1) for j in range(n+1)]
    R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
    R1 = PolynomialRing(QQ, ['y_{}_{}'.format(i,j) for (i,j) in indices])
    x = dict(zip(indices, R.gens()))
    y = dict(zip(indices, R1.gens()))
    #M = web_matrix_DFGK(R,x,k,n)
    M=web_matrix(R,x,k,n)
    #print(M)
    #print(evaluation_of_x_coordinates_on_web_matrix_CDFL_including_frozens(M,k,n))
    t00=evaluation_of_x_coordinates_on_web_matrix_CDFL_including_frozens(M,k,n)
    #print(t00)
    t0=t00[:(k-1)*(n-k-1)]
    t1=t00[-(k-1):]
    #print('t0,t1',t0,t1)
    t0=split_list_every_part_k(t0, k-1)
    t1=split_list_every_part_k(t1, 1)
    #print('t0,t1',t0,t1,len(t0),len(t1))
    ee1=[]
    for i in range(len(t0)):
        ee1=ee1+(t0[i]+t1[i])
        
    #print('ee1', ee1)
    
    Xs = Matrix([[var("x_%s_%s"%(u, v), latex_name="x_{%s,%s}"%(u, v))
              for v in range(1, k+1)]
             for u in range(1, n-k)])
    Ys = Matrix([[var("y_%s_%s"%(u, v), latex_name="y_{%s,%s}"%(u, v))
                  for v in range(1, k+1)]
                 for u in range(1, n-k)])
    #print('Xs, Ys', Xs, Ys)
    va=Xs.list()  # if use mormal order, Xs.list()
    a1=Ys.list()  
     
    #print('va', va)
    
    a3 = list(map(lambda a, b:a-b, ee1, a1))
    
    #print('a3',a3)
    
    Sol = solve(a3, va, solution_dict=True)
    
    #print(Sol)
    for i in Sol:
        t1=list(i.values())
        if IsInList(0, t1)==0:
            a5=t1 
            break 
    
    g1=BraidGroupActionSigmaIOnMatrix_symbolic_not_normalized(R,ii,k,n,M)
    #print(latex(t2))
    g2=evaluation_of_x_coordinates_on_web_matrix_CDFL_including_frozens(g1,k,n)
    #print(t3)
    
    t0=g2[:(k-1)*(n-k-1)]
    t1=g2[-(k-1):]
    t0=split_list_every_part_k(t0, k-1)
    t1=split_list_every_part_k(t1, 1)
    #print('t0,t1',t0,t1)
    ee2=[]
    for i in range(len(t0)):
        ee2=ee2+(t0[i]+t1[i])
    
    r1=[]
    for i in ee2:
        p1=i
        for j in range(1,k):
            for j1 in range(1,n-k+1): 
                p1=p1.subs({x[j,j1]:a5[(j-1)*k+j1-1]})
        p1=(p1.expand()).factor()
        for j in range(1,k):
            for j1 in range(1,n-k+1): 
                p1=p1.subs({y[j,j1]:x[j,j1]})
        r1.append(p1)
    #print(r1) 
    r=prepare_tropical_map_rational_list_CDFL(R, r1)
    return r


def prepare_tropical_map_CDFL_inverse_including_frozens(ii,k,n): # prepare for tropical map sigma_i
    indices = [(i,j) for i in range(k+1) for j in range(n+1)]
    R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
    R1 = PolynomialRing(QQ, ['y_{}_{}'.format(i,j) for (i,j) in indices])
    x = dict(zip(indices, R.gens()))
    y = dict(zip(indices, R1.gens()))
    #M = web_matrix_DFGK(R,x,k,n)
    M=web_matrix(R,x,k,n)
    
    #M=web_matrix_not_normalized(R,x,k,n)
    print(latex(M))
    #print(evaluation_of_x_coordinates_on_web_matrix_CDFL_including_frozens(M,k,n))
    t00=evaluation_of_x_coordinates_on_web_matrix_CDFL_including_frozens(M,k,n)
    print(t00)
    t0=t00[:(k-1)*(n-k-1)]
    t1=t00[-(k-1):]
    #print('t0,t1',t0,t1)
    t0=split_list_every_part_k(t0, k-1)
    t1=split_list_every_part_k(t1, 1)
    #print('t0,t1',t0,t1,len(t0),len(t1))
    ee1=[]
    for i in range(len(t0)):
        ee1=ee1+(t0[i]+t1[i])
        
    #print('ee1', ee1)
    
    Xs = Matrix([[var("x_%s_%s"%(u, v), latex_name="x_{%s,%s}"%(u, v))
              for v in range(1, k+1)]
             for u in range(1, n-k)])
    Ys = Matrix([[var("y_%s_%s"%(u, v), latex_name="y_{%s,%s}"%(u, v))
                  for v in range(1, k+1)]
                 for u in range(1, n-k)])
    #print('Xs, Ys', Xs, Ys)
    va=Xs.list()  # if use mormal order, Xs.list()
    a1=Ys.list()  
     
    #print('va', va)
    
    a3 = list(map(lambda a, b:a-b, ee1, a1))
    
    #print('a3',a3)
    
    Sol = solve(a3, va, solution_dict=True)
    
    #print('Sol', Sol)
    for i in Sol:
        t1=list(i.values())
        if IsInList(0, t1)==0:
            a5=t1 
            break 
    
    g1=BraidGroupActionSigmaI_inverse_OnMatrix_symbolic_not_normalized(R,ii,k,n,M)
    print(latex(g1))
    g2=evaluation_of_x_coordinates_on_web_matrix_CDFL_including_frozens(g1,k,n)
    #print(g2)
    
    t0=g2[:(k-1)*(n-k-1)]
    t1=g2[-(k-1):]
    t0=split_list_every_part_k(t0, k-1)
    t1=split_list_every_part_k(t1, 1)
    #print('t0,t1',t0,t1)
    ee2=[]
    for i in range(len(t0)):
        ee2=ee2+(t0[i]+t1[i])
    
    r1=[]
    for i in ee2:
        p1=i
        for j in range(1,k):
            for j1 in range(1,n-k+1): 
                p1=p1.subs({x[j,j1]:a5[(j-1)*k+j1-1]})
        p1=(p1.expand()).factor()
        for j in range(1,k):
            for j1 in range(1,n-k+1): 
                p1=p1.subs({y[j,j1]:x[j,j1]})
        r1.append(p1)
    print(r1) 
    r=prepare_tropical_map_rational_list_CDFL(R, r1)
    return r

def face_of_tableau_in_polytope(k,n,T,c3): # c3 are list of exponents of the polynomial defining the polytope
    r1=tableau_to_alphaij(T)
    #print(r1)
    r2=alphaij_to_list_list_scattering_amplitudes(r1,k,n)
    r=exponents_that_are_minimized_by_sum_of_functions(r2, c3)
    return r

def is_face_of_tableau_facet_in_polytope(k,n,T,c3): # c3 are list of exponents of the polynomial defining the polytope
    r1=tableau_to_alphaij(T)
    #print(r1)
    r2=alphaij_to_list_list_scattering_amplitudes(r1,k,n)
    r3=exponents_that_are_minimized_by_sum_of_functions(r2, c3)
    r4=np.array(r3)
    r5=np.linalg.matrix_rank(r4) 
    if r5==(k-1)*(n-k-1):
        r=1
    else:
        r=0
    return [r,r5]


def sort_according_to_length(l):
    r=[]
    if l!=[]:
        a1=len(l[0])
        for i in l:
            if len(i)>a1:
                a1=len(i)
        r=[]
        for j in [0..a1]:
            for i in l:
                if len(i)==j:
                    r.append(i)
    return r

def representative_promotion_tableaux(l, n): # l is a list of tableaux
    r=[]
    for i in l:
        t1=TableauToMatrix(i)
        t2=PromotionOfTableauNTimesInMatrix(n, t1, n)
        t3=MatrixTakeColumnsList(t2)
        if IntersectionOfLists(r, t3)==[]:
            r.append(i)
    return r

def representative_evacuation_tableaux(l, n): # l is a list of tableaux
    r=[]
    for i in l:
        t1=TableauToMatrix(i)
        t2=EvacuationOfTableauNTimesInMatrix(n, t1, n)
        t3=MatrixTakeColumnsList(t2)
        if IntersectionOfLists(r, t3)==[]:
            r.append(i)
    return r

def generalized_binomial(n,l):
    r=factorial(n)
    for i in l:
        r=r/factorial(i)
    return r

def matrixMutation_2_cycles(l, k):  # paper cluster algebra with 2-cycles or loops, mutate at k, l=[M,N], M, N are two matrices such that M-N is sign-skew-symmetric
    M=l[0]
    N=l[1]
    a,b=M.nrows(), M.ncols()
    c,d=N.nrows(), N.ncols()
    r1=Matrix(a,b)
    r2=Matrix(c,d)
    
    for i in range(a): 
        for j in range(b): 
            if k==i or k==j:
                r1[i,j]=N[i, j]    
            else: 
                print('r1ij', 1/2*(abs(M[i,k]-N[i,k])*M[k,j]+M[i,k]*abs(M[k,j]-N[k,j])))
                r1[i, j] = M[i, j]+1/2*(abs(M[i,k]-N[i,k])*M[k,j]+M[i,k]*abs(M[k,j]-N[k,j]))
                
    for i in range(c): 
        for j in range(d): 
            if k==i or k==j:
                r2[i,j]=M[i, j]    
            else: 
                print('r2ij', 1/2*(abs(N[i,k]-M[i,k])*N[k,j]+N[i,k]*abs(N[k,j]-M[k,j])))
                r2[i, j] = N[i, j]+1/2*(abs(N[i,k]-M[i,k])*N[k,j]+N[i,k]*abs(N[k,j]-M[k,j]))
     
    r=[r1,r2]
    return r

def computeEquationsForClusterAlgebras_2_cycles_quiver(R, variable2, l, k): # paper cluster algebra with 2-cycles or loops, l=[M,N], M, N are two matrices and N=M^T, variable2=(variables on quiver, cluster variables obtained so far)
    variable=variable2[0]
    clusterVariables=variable2[1] 
    
    M=l[0]
    N=l[1]
    a,b=M.nrows(), M.ncols()
    A=Matrix(a,b)
    
    for i in range(a):
        for j in range(b):
            A[i,j]=max(M[i,j],0)+max(-M[j,i],0)
    newVariable=1
    newVariable2=1
    for i in range(a):
        newVariable=newVariable*variable[i]^(A[i,k])
        newVariable2=newVariable2*variable[i]^(A[k,i])
    
    variable[k]=expand(factor(newVariable+newVariable2)/factor(variable[k]))
    
    clusterVariables=variable[k] 
    
    r=[variable, clusterVariables]

    return r

def split_list_approximately(a, n): # n is the number of parts
    k, m = divmod(len(a), n)
    r=[a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n)]
    return r

def fundamental_tableau_to_alphaij(T): # T=[[1,3,4]] in the paper scattering amplitudes
    k=len(T[0])
    a=T[0][0]
    b=SetDifferenceListDifference(ListAToN(a,a+k),T[0])[0]
    r=[b-a,a]
    return r

def tableau_to_alphaij(T): # T=[[1,3,4], [1,3,5]] in the paper scattering amplitudes
    k=len(T[0])
    r1=TableauToSmallGapTableau(k,T)
    r=[]
    for i in r1:
        t1=fundamental_tableau_to_alphaij([i])
        r.append(t1)
    return r

def alphaij_to_list_scattering_amplitudes(a,k,n):
    i=a[0]
    j=a[1]
    t1=(k-1)*(n-k)
    r=[0]*t1
    t2=(i-1)*(n-k)+(j-1)
    r[t2]=1
    
    return r

def alphaij_to_list_list_scattering_amplitudes(a,k,n):
    r=[]
    for i in a:
        t1=alphaij_to_list_scattering_amplitudes(i,k,n)
        r.append(t1)
    return r

def inner_product_of_vectors(a,b):
    r=0
    for i in range(len(a)):
        r=r+a[i]*b[i]
    return r

def exponents_that_are_minimized(a, l): # a=(1,0,1,0,0) is a vector, l = [l[1], l[2], ...] is a list of exponents, l[i] is a list
    r=[]
    r1=[]
    r2=[]
    for i in l:
        t1=inner_product_of_vectors(a, i)
        r1.append([t1, i])
        r2.append(t1)
    p1=min(r2)
    for i in r1:
        if i[0]==p1:
            r.append(i[1])
    return r

def exponents_that_are_minimized_by_sum_of_functions(a, l): # a=[[1,0,1,0,0], [1,0,0,0,1]] is a sum of functions, l = [l[1], l[2], ...] is a list of exponents, l[i] is a list
    r=[]
    r1=[]
    r2=[]
    for i in l:
        t1=0
        for j in a:
            t2=inner_product_of_vectors(j, i)
            t1=t1+t2
        r1.append([t1, i])
        r2.append(t1)
    p1=min(r2)
    #print(r1)
    #print(r2)
    for i in r1:
        if i[0]==p1:
            r.append(i[1])
    return r

def is_not_weakly_separated_pairwise(l):
    r=1
    r1=NchooseM(l,2) 
    for i in r1: 
        if is_weakly_separated(i[0], i[1])==1:
            r=0
            break
    return r

def is_noncrossing_and_not_weakly_separated(a,b):
    r=1
    if IsNonCrossing(a,b)==0 or IsWeaklySeparated(a,b)==1:
        r=0
    return r

def is_noncrossing_and_not_weakly_separated_pairwise(l):
    r=1
    r1=NchooseM(l,2)
    #print(r1[0])
    for i in r1: 
        if is_noncrossing_and_not_weakly_separated(i[0], i[1])==0:
            r=0
            break
    return r

def BraidGroupActionSigmaIOnPlucker_symbolic_not_normalized(R,ii, p, k,n, M): # p = [1,2,3,8],  
    
    r1=BraidGroupActionSigmaI_inverse_OnMatrix_symbolic_not_normalized(R,ii,k,n, M)

    t1=sub_matrix_general(r1,ListAToN(1,k),p)
    r2=det(t1)
    r3=r2.numerator()
    r4=r2.denominator()
    
    #print('r3,r4',r3,r4)
    
    r5=MinorsToPluckerPolynomial(R,M,r3)
    if r4!=1:
        r6=MinorsToPluckerPolynomial(R,M,r4)
    else:
        r6=[[1,[]]]
    r=[r5,r6]

    return r 


def ContentOfTableauObtainedFromBraidGroupActionPlucker_rational(R, ii, p, k,n, M): # p is a plucker
    #print('M', M)
    r1=BraidGroupActionSigmaI_inverse_OnMatrix_symbolic_not_normalized(R,ii,k,n, M) # r1 is rational
    #print(r1)
    r2=det(sub_matrix_general(r1,ListAToN(1,k),p))
    #print(r2)
    
    g1=r2.numerator()
    g2=r2.denominator()
    
    r=[]
    for a1 in [g1, g2]: 
        t1=str(a1).split('+')[0]
        # print(t1)
        r3=change_polynomial_to_list(R, t1)
        b1=[]
        for j in r3[0]:
            t1=index_of_yij(j) # take indices of yij
            if t1!=[]:
                b1.append(t1[1]+1)
        r.append(b1)
    return r

def BraidGroupActionSigmaI_inverse_OnPlucker_symbolic_not_normalized(R,ii, p, k,n, M): # p = [1,2,3,8],  
    
    r1=BraidGroupActionSigmaIOnMatrix_symbolic_not_normalized(R,ii,k,n, M)
    
    #t1=sub_matrix_general2(R,r1,ListAToN(1,k),p)
    t1=sub_matrix_general(r1,ListAToN(1,k),p)
    r2=det(t1)
    r3=r2.numerator()
    r4=r2.denominator()
    
    #print('r3,r4',r3,r4)
    
    r5=MinorsToPluckerPolynomial(R,M,r3)
    
    if r4!=1:
        r6=MinorsToPluckerPolynomial(R,M,r4)
    else:
        r6=[[1,[]]]
    r=[r5,r6]
    
    #print('r',r)

    return r

def BraidGroupActionSigmaI_inverse_OnMatrix_symbolic_not_normalized(R,i,k,n, M): 

    d=gcd(k,n)
    m=n/d
    l=MatrixToTableauTakeColumns(M)
    #print(l)

    r1=[]
    t1=[]

    for j in [0..n-1]:
        if (j+1) % d !=0: 
            t1.append(l[j])
        else:
            t1.append(l[j])
            r1.append(t1)
            t1=[] 
    #print(r1)
    
    l1=UnionOfLists(l,l)
    
    #print(l1)
    
    r2=[] 
    for j in [0..len(r1)-1]: 
        #print(i,k,m,j,r1[j])
        t1=BraidGroupActionSigmaI_inverse_OnWindow_symbolic_not_normalized(R,i,k, m,j,r1[j], l1)
        r2.append(t1) 
    F=R.fraction_field()
    r=TableauToMatrix_symbolic(F,UnionOfListsMany(r2))

    return r

def BraidGroupActionSigmaIOnWindow_symbolic_not_normalized(R,i1, k, l, l1): # l=[v1,v2,...,vd], r=[v1,v2,...,v_{i-1},v_{i+1},w_1,v_{i+2},...,v_d], l1= l + ... + beginning of original list 
    i=i1-1
    r=l
    r=removeAnElementInList(i,r)
    r.insert(i+1, 0)
    #print(r)
    
    t1=[]
    for j in [i..i+k]:
        if j!=i+1: 
            t1.append(l1[j])
    
    #print(t1)
    #print(TableauToMatrix_symbolic(R,t1))
    m1=det(TableauToMatrix_symbolic(R,t1))
    
    #print(m1)
    
    t1=[]
    for j in [i+1..i+k]:  
        t1.append(l1[j])  
    m2=det(TableauToMatrix_symbolic(R,t1))
    #print(t1)
    #print(m2)
    w1=[]
    for j in [0..k-1]: 
        w1.append(m1*l[i+1][j]/m2-l[i][j]) 
    #print(w1)
    r[i+1]=w1
    #print('r',r)
    
    return r

def BraidGroupActionSigmaIOnMatrix_symbolic_not_normalized(R,i,k,n, M): 

    d=gcd(k,n)
    l=MatrixToTableauTakeColumns(M)
    #print(l)

    r1=[]
    t1=[]

    for j in [0..n-1]:
        if (j+1) % d !=0: 
            t1.append(l[j])
        else:
            t1.append(l[j])
            r1.append(t1)
            t1=[] 
    #print(r1)
    r2=[]
    for j in [0..len(r1)-1]: 
        l1=[]
        for i1 in [j..len(r1)-1]: 
            l1=UnionOfLists(l1, r1[i1]) 
        l1=UnionOfLists(l1, l)
        t1=BraidGroupActionSigmaIOnWindow_symbolic_not_normalized(R,i,k, r1[j], l1)
        r2.append(t1) 
    F=R.fraction_field()
    r=TableauToMatrix_symbolic(F,UnionOfListsMany(r2))

    return r

def BraidGroupActionSigmaI_inverse_OnWindow_symbolic_not_normalized(R,i1, k, m,jj,l, LL): # sigma_i^{-1}, l=[v1,v2,...,vd], r=[v1,v2,...,v_{i-1},v_{i+1},w_1,v_{i+2},...,v_d], l1= l + ... + beginning of original list 
    i=i1-1
    
    d=len(l)
    r=l  
    r=removeAnElementInList(i+1,r)
    r.insert(i, 0)
    #print(r)
    
    L=ReverseList(LL)
    
    #print('L',L)
    
    #print(len(L), (m-jj+1)*d-i+2, (m-jj+1)*d-i+k+3)
    
    t1=[] 
    for j in [(m-jj)*d-i-2..(m-jj)*d-i+k-2]:
        if j!=(m-jj)*d-i-1: 
            #print('L[j]', L[j])
            t1.append(L[j])
    t1=ReverseList(t1)
    
    #print('t1',t1) 
    #print(TableauToMatrix_symbolic(R,t1))
    m1=det(TableauToMatrix_symbolic(R,t1))
    
    #print(len(L), (m-jj+1)*d-i+1, (m-jj+1)*d-i+k)
    
    #print(m1)
    
    t1=[]  
    
    for j in [(m-jj)*d-i-1..(m-jj)*d-i+k-2]:  
        t1.append(L[j])  
        
    t1=ReverseList(t1)
    m2=det(TableauToMatrix_symbolic(R,t1))
    #print(t1)
    #print(m2)
    w1=[]
    for j in [0..k-1]: 
        #w1.append(m1*l[i][j]-m2*l[i+1][j]) 
        w1.append(m1*l[i][j]/m2-l[i+1][j]) 
    #print(w1)
    r[i]=w1   
    #print('r',r)
    
    return r


def AddTwoRationalFunctionsLists2(a,b): # a[0]/a[1] + b[0]/b[1]
    if a==[] or b==[]:
        return a+b
    else:
        d1=MultiplyTwoPolynomialsLists2(a[1],b[1])
        d1=combineLikeTerms3(d1)

        n1=MultiplyTwoPolynomialsLists2(a[0],b[1])
        n2=MultiplyTwoPolynomialsLists2(a[1],b[0])
        n3=n1+n2
        n3=combineLikeTerms3(n3)
        r=[n3, d1] 
    return r

def MultiplyTwoRationalFunctionsLists2(a,b): # a[0]/a[1] * b[0]/b[1]
    if a==[] or b==[]:
        r=[]
    else:
        d1=MultiplyTwoPolynomialsLists2(a[1],b[1])
        n1=MultiplyTwoPolynomialsLists2(a[0],b[0])
        d1=combineLikeTerms3(d1)
        n1=combineLikeTerms3(n1)

        r=[n1, d1] 
    return r


def BraidGroupActionSigmaIOnPluckerSet_symbolic_partial_not_normalized(R,ii, k,n, M, l): # l is a set of pl = [1,2,3,8], return a dictionary d['[1,2,3,8]'] is the sigma_ii of [1,2,3,8]
#     k,n=4,8
#     n1,n2=1,n
#     R=define_ring_with_variables_that_have_negative_indices('x', k,n1-1,n2)[0]
    
    r={}
    for i in l:
        t1=BraidGroupActionSigmaIOnPlucker_symbolic_not_normalized(R,ii, i, k,n, M)
        #print(i,t1)
        r[repr(i)]=t1
    return r

def BraidGroupActionSigmaI_inverse_OnPluckerSet_symbolic_partial_not_normalized(R,ii, k,n, M, l): # l is a set of pl = [1,2,3,8], return a dictionary d['[1,2,3,8]'] is the sigma_ii of [1,2,3,8]
#     k,n=4,8
#     n1,n2=1,n
#     R=define_ring_with_variables_that_have_negative_indices('x', k,n1-1,n2)[0]
    
    r={}
    for i in l:
        t1=BraidGroupActionSigmaI_inverse_OnPlucker_symbolic_not_normalized(R,ii, i, k,n, M)
        #print(i,t1)
        r[repr(i)]=t1
    return r


def BraidGroupActionSigmaIOnPluckerPolynomial_symbolic_NTimes2_not_normalized(R,ii, p, k,n, M,N): # p = [[1,[[1,2,3,8],[2,3,4,8]]], [1,[[1,2,3,8],[2,3,4,8]]]]
    l=[]
    for i in p:
        l=l+i[1]
    
    sigma_i_of_plucker_Set=BraidGroupActionSigmaIOnPluckerSet_symbolic_partial_not_normalized(R,ii, k,n, M,l) 
    #print(sigma_i_of_plucker_Set)
    r=p # N=0
    if N>0:
        for i in range(N):
            r=BraidGroupActionSigmaIOnPluckerPolynomial_symbolic_not_normalized(R,ii, r, k,n, M, sigma_i_of_plucker_Set)
    
    return r

def BraidGroupActionSigmaI_inverse_OnPluckerPolynomial_symbolic_NTimes2_not_normalized(R,ii, p, k,n, M,N): # p = [[1,[[1,2,3,8],[2,3,4,8]]], [1,[[1,2,3,8],[2,3,4,8]]]]
    l=[]
    for i in p:
        l=l+i[1]
    
    sigma_i_of_plucker_Set=BraidGroupActionSigmaI_inverse_OnPluckerSet_symbolic_partial_not_normalized(R,ii, k,n, M,l) 
    #print(sigma_i_of_plucker_Set)
    r=p # N=0
    if N>0:
        for i in range(N):
            r=BraidGroupActionSigmaIOnPluckerPolynomial_symbolic_not_normalized(R,ii, r, k,n, M, sigma_i_of_plucker_Set)
    
    return r

def BraidGroupActionSigmaIOnPluckerPolynomial_symbolic_not_normalized(R,ii, p, k,n, M, sigma_i_of_plucker_Set): # works for both sigma_i and sigma_i^(-1)
    # p = [[1,[[1,2,3,8],[2,3,4,8]]], [1,[[1,2,3,8],[2,3,4,8]]]]
    #k,n=4,8
    #indices = [(i,j) for i in range(k) for j in range(n)]
    #R = PolynomialRing(QQ, ['y_{}_{}'.format(i,j) for (i,j) in indices])
    #y = dict(zip(indices, R.gens()))
    #ii=1
    #M = Matrix(R, k, n, R.gens())
    #sigma_i_of_plucker_Set=BraidGroupActionSigmaIOnPluckerSet_symbolic(R,ii, k,n, M)
    r=[]
    for j in p:
        t1=[[[j[0], []]], [[1, []]]]
        #print('t1',t1)
        for j1 in j[1]:
            t2=sigma_i_of_plucker_Set.get(repr(j1))
            #print(t1)
            #print(t2)
            #t2=polynomial_use_tuple_to_polynomial_use_list(t2)
            #print(t2)
            t1=MultiplyTwoRationalFunctionsLists2(t1,t2)
        r=AddTwoRationalFunctionsLists2(r, t1)
    #r=combineLikeTerms3(r)
    return r

def compute_highest_weight_tableau_not_remove_trivial_factors(a1, k,n,mi,ma): # a1=[[1,[[2,3,4,6],[3,4,5,7]]], ...]
    r=[]
    if a1!=[]:
        if len(a1)==1 and a1[0][1]==[]:
            r=a1
        else:
            r0=a1[0][1]
            e1=TableauxWithSameContentAsT_new(r0)
            p1=[]
            for i in [1..len(e1)]:
                M1=randomMatrix(k,n,mi,ma)
                g1=PluckerToMinorsPolynomial(M1, a1) 
                p1.append([M1, g1])
            e2=ComputeChTInTermsOfNonCrossingBasis([ e1, p1 ])
            g1=HighestWeightTableauInPolynomialOfTableaux(e2, 'A', k-1)
            #print(g1)
            g20=SortRowsOfTableau(g1[1])
            #print(g20)
            g21=RemoveTrivialTableau(g20,k,n)
            g22=TableauDivision(g20, g21)

            g23=RemoveTrivialTableau3(g22,k,n)
            g24=TableauDivision(g22, g23)

            g2=[g1[0], g24+g23+g21]
            r=[g2, e2]
    return r



def reflection_action_on_matrix_symbolic(R,k,n, M): # (v1,v2,...,vn) mapsto (-1)^( {k choose 2} ) (vn, ..., v1), has some problem, theta(g(T)) is not equal to g(theta(T))

    l=MatrixToTableauTakeColumns(M)
    r1=ReverseList(l)
    r2=TableauToMatrix_symbolic(R,r1)
    #print(r2)
    r=r2
    m=binomial(k,2)
    for i in range(k):
        for j in range(n):
            r[i,j]=(-1)^( m )*r2[i,j]
            
    #print(r)
    return r

def prepare_tropical_map_CDFL_reflection_action(k,n): # prepare for tropical map sigma_i, has some problem, theta(g(T)) is not equal to g(theta(T))
    indices = [(i,j) for i in range(k+1) for j in range(n+1)]
    R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
    x = dict(zip(indices, R.gens()))
    M = web_matrix_DFGK_reverse_xij(R,x,k,n)
    #print(evaluation_of_x_coordinates_on_web_matrix_CDFL(M,k,n))
    t2=reflection_action_on_matrix_symbolic(R,k,n,M)
    #print(t2)
    t3=evaluation_of_x_coordinates_on_web_matrix_CDFL(t2,k,n)
    #print(t3)
    r1=t3
    
#     r1=[]
#     for i in t3:
#         p1=i
#         for j in indices:
#             p1=p1.subs({x[j[0],j[1]]:1/x[j[0],j[1]]})
#         r1.append(p1)
    r=prepare_tropical_map_rational_list_CDFL(R, r1)
    return r

def reflection_action_action_on_g_vector_CDFL_use_tropical_map(R,x,g,k,n,trop): # labels of initial seed are the same as CDFL
    # trop = prepare_tropical_map_CDFL_reflection_action(ii,k,n)
#     indices = [(i,j) for i in range(k+1) for j in range(n+1)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    
    r=evaluation_tropical_function_rational_list_on_vector_CDFL_use_tropical_map(R, x, k,n, trop, g)
    
    return r 

def reflection_action_action_on_g_vector_CDFL(g,k,n): #   labels of initial seed are the same as CDFL, has some problem, theta(g(T)) is not equal to g(theta(T))
    indices = [(i,j) for i in range(k+1) for j in range(n+1)]
    R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
    x = dict(zip(indices, R.gens()))
    M = web_matrix_DFGK_reverse_xij(R,x,k,n)
    #print(evaluation_of_x_coordinates_on_web_matrix_CDFL(M,k,n))
    t2=reflection_action_on_matrix_symbolic(R,k,n, M) 
    #print(latex(t2))
    t3=evaluation_of_x_coordinates_on_web_matrix_CDFL(t2,k,n)
    #print(latex(t3))
    
    r1=t3
    
#     r1=[]
#     for i in t3:
#         p1=i
#         for j in indices:
#             p1=p1.subs({x[j[0],j[1]]:1/x[j[0],j[1]]})
#         r1.append(p1)
    t4=evaluation_tropical_function_rational_list_on_vector_CDFL(R, x, k,n,  r1, g)
    
    r=t4
    return r

def reflection_action_action_on_tableau(T,k,n): # T[i] is a column, has some problem, theta(g(T)) is not equal to g(theta(T))
    e1=initial_cluster_variables_CDFL(k,n)
    m=(k-1)*(n-k-1)
    g1=GVectorOfTableau(k,n,T,e1)[0][:m]
    #print(g1)
    r1=reflection_action_action_on_g_vector_CDFL(g1,k,n)
    #print(r1)
    r=GVectorToTableau_remove_inverses(r1,e1,k,n)
    
    return r 

def reflection_action_action_on_tableau_use_tropical_map(R,x,T,k,n,trop): # T[i] is a column, not finished, has some problem, theta(g(T)) is not equal to g(theta(T))
        # trop = prepare_tropical_map_CDFL_reflection_action(k,n) 
#     indices = [(i,j) for i in range(k+1) for j in range(n+1)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    e1=initial_cluster_variables_CDFL(k,n)
    m=(k-1)*(n-k-1)
    g1=GVectorOfTableau(k,n,T,e1)[0][:m]
    #print('g1', g1)
    r1=reflection_action_action_on_g_vector_CDFL_use_tropical_map(R,x,g1,k,n,trop) 
    #print(r1)
    r=GVectorToTableau_remove_inverses(r1,e1,k,n)
    
    return r 

def cyclic_permutation_on_matrix_symbolic(R,k,n, M): # (v1,v2,...,vn) mapsto (v2,v3,...,vn,(-1)^{k-1} v_{1})

    l=MatrixToTableauTakeColumns(M)
    r1=[]
    for i in [1..len(l)-1]:
        r1.append(l[i])
    t1=[]
    for i in l[0]:
        t1.append((-1)^(k-1)*i)
    r1.append(t1) 
    #print(r1)
    r=TableauToMatrix_symbolic(R,r1)

    return r

def cyclic_permutation_on_matrix_symbolic_inverse(R,k,n, M): # (v1,v2,...,vn) mapsto (v2,v3,...,vn,(-1)^{k-1} v_{1})

    l=MatrixToTableauTakeColumns(M)
    
    r1=[]
    
    t1=[]
    for i in l[-1]:
        t1.append((-1)^(k-1)*i)
    r1.append(t1) 
    
    for i in [0..len(l)-2]:
        r1.append(l[i])
    
    #print(r1)
    r=TableauToMatrix_symbolic(R,r1)

    return r

def prepare_tropical_map_CDFL_cyclic_permutation_inverse(k,n): # prepare for tropical version of cyclic inverse shift
    indices = [(i,j) for i in range(k+1) for j in range(n+1)]
    R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
    x = dict(zip(indices, R.gens()))
    M = web_matrix_DFGK_reverse_xij(R,x,k,n)
    #print(evaluation_of_x_coordinates_on_web_matrix_CDFL(M,k,n))
    t2=cyclic_permutation_on_matrix_symbolic(R,k,n,M)
    t3=evaluation_of_x_coordinates_on_web_matrix_CDFL(t2,k,n)
    #print(t3)
    r1=t3
    
#     r1=[]
#     for i in t3:
#         p1=i
#         for j in indices:
#             p1=p1.subs({x[j[0],j[1]]:1/x[j[0],j[1]]})
#         r1.append(p1)
    r=prepare_tropical_map_rational_list_CDFL(R, r1)
    return r

def prepare_tropical_map_CDFL_cyclic_permutation(k,n): #  
    indices = [(i,j) for i in range(k+1) for j in range(n+1)]
    R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
    x = dict(zip(indices, R.gens()))
    M = web_matrix_DFGK_reverse_xij(R,x,k,n)
    #print(evaluation_of_x_coordinates_on_web_matrix_CDFL(M,k,n))
    t2=cyclic_permutation_on_matrix_symbolic_inverse(R,k,n,M)
    t3=evaluation_of_x_coordinates_on_web_matrix_CDFL(t2,k,n)
    #print(t3)
    r1=t3
    
#     r1=[]
#     for i in t3:
#         p1=i
#         for j in indices:
#             p1=p1.subs({x[j[0],j[1]]:1/x[j[0],j[1]]})
#         r1.append(p1)
    r=prepare_tropical_map_rational_list_CDFL(R, r1)
    return r

def cyclic_permutation_action_on_g_vector_CDFL_use_tropical_map(R,x,g,k,n,trop): # labels of initial seed are the same as CDFL
    # trop = prepare_tropical_map_CDFL_cyclic_permutation(ii,k,n)
#     indices = [(i,j) for i in range(k+1) for j in range(n+1)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    
    r=evaluation_tropical_function_rational_list_on_vector_CDFL_use_tropical_map(R, x, k,n, trop, g)
    
    return r 

def cyclic_permutation_action_on_g_vector_CDFL(g,k,n): #   labels of initial seed are the same as CDFL
    indices = [(i,j) for i in range(k+1) for j in range(n+1)]
    R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
    x = dict(zip(indices, R.gens()))
    M = web_matrix_DFGK_reverse_xij(R,x,k,n)
    #print(evaluation_of_x_coordinates_on_web_matrix_CDFL(M,k,n))
    t2=cyclic_permutation_on_matrix_symbolic(R,k,n, M) 
    #print(latex(t2))
    t3=evaluation_of_x_coordinates_on_web_matrix_CDFL(t2,k,n)
    #print(latex(t3))
    
    r1=t3
    
#     r1=[]
#     for i in t3:
#         p1=i
#         for j in indices:
#             p1=p1.subs({x[j[0],j[1]]:1/x[j[0],j[1]]})
#         r1.append(p1)
    t4=evaluation_tropical_function_rational_list_on_vector_CDFL(R, x, k,n,  r1, g)
    
    r=t4
    return r

def cyclic_permutation_action_on_g_vector_CDFL_inverse(g,k,n): #   labels of initial seed are the same as CDFL
    indices = [(i,j) for i in range(k+1) for j in range(n+1)]
    R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
    x = dict(zip(indices, R.gens()))
    M = web_matrix_DFGK_reverse_xij(R,x,k,n)
    #print(evaluation_of_x_coordinates_on_web_matrix_CDFL(M,k,n))
    t2=cyclic_permutation_on_matrix_symbolic_inverse(R,k,n, M) 
    #print(latex(t2))
    t3=evaluation_of_x_coordinates_on_web_matrix_CDFL(t2,k,n)
    #print(latex(t3))
    
    r1=t3
    
#     r1=[]
#     for i in t3:
#         p1=i
#         for j in indices:
#             p1=p1.subs({x[j[0],j[1]]:1/x[j[0],j[1]]})
#         r1.append(p1)
    t4=evaluation_tropical_function_rational_list_on_vector_CDFL(R, x, k,n,  r1, g)
    
    r=t4
    return r

def cyclic_permutation_action_on_tableau_inverse(T,k,n): # T[i] is a column
    e1=initial_cluster_variables_CDFL(k,n)
    m=(k-1)*(n-k-1)
    g1=GVectorOfTableau(k,n,T,e1)[0][:m]
    #print(g1)
    r1=cyclic_permutation_action_on_g_vector_CDFL(g1,k,n)
    #print(r1)
    r=GVectorToTableau_remove_inverses(r1,e1,k,n)
    
    return r 

def cyclic_permutation_action_on_tableau(T,k,n): # T[i] is a column
    e1=initial_cluster_variables_CDFL(k,n)
    m=(k-1)*(n-k-1)
    g1=GVectorOfTableau(k,n,T,e1)[0][:m]
    #print(g1)
    r1=cyclic_permutation_action_on_g_vector_CDFL_inverse(g1,k,n)
    #print(r1)
    r=GVectorToTableau_remove_inverses(r1,e1,k,n)
    
    return r 


def cyclic_permutation_action_on_tableau_use_tropical_map(R,x,T,k,n,trop): # T[i] is a column, not finished
        # trop = prepare_tropical_map_CDFL_cyclic_permutation(k,n)
        # trop = prepare_tropical_map_CDFL_cyclic_permutation_inverse(k,n)
#     indices = [(i,j) for i in range(k+1) for j in range(n+1)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    e1=initial_cluster_variables_CDFL(k,n)
    m=(k-1)*(n-k-1)
    g1=GVectorOfTableau(k,n,T,e1)[0][:m]
    #print('g1', g1)
    r1=cyclic_permutation_action_on_g_vector_CDFL_use_tropical_map(R,x,g1,k,n,trop) 
    #print(r1)
    r=GVectorToTableau_remove_inverses(r1,e1,k,n)
    
    return r 

def ModN(l, n): # ModN(3,3) is 3, Mod(3,3) is 0
    if l%n==0:
        r=n
    else:
        r=l%n
    return r

def CrossProduct_symbolic(l): # l=[p1,p2,...,pk], cross product of p1,p2,...,pk, pi are k+1 \times 1 matrices

    k=len(l)
    n=k+1
    #r=Matrix(n,1)
    r=Matrix(SR, n, 1) 
 
    X=Matrix(SR, k+1, k) 
    
    for i in [0..k]:
        for j in [0..k-1]:
            X[i,j]=l[j][i,0] 

    for i in [1..n]:
        t1=SetDifference(ListAToN(1,n), [i])
        t2=ListAToN(1,k)
        r[i-1,0]=(-1)^(i-1)*Minor(X,t1,t2)
    
    return r

def MarshScottTwistHalf_symbolic(i1,p): # p is a k times n matrix
 
    k,n=p.nrows(), p.ncols()
    #print(k,n)
    P=[]
    for j in range(1,k): 
        t1=Matrix(SR, k+1, 1) 
        t2=ShiftModNKTimes(i1, -1, n, k-j)
        #print('t2', t2)
        for d in range(k):
            t1[d,0]=p[d,t2] 
        #print('t1',t1)
        P.append(t1) 
 
    r=CrossProduct_symbolic(P) 

    return r

def Marsh_Scott_twist_on_matrix_symbolic(p): # p is a k times n matrix
    #print(p) 
    k,n=p.nrows(), p.ncols()
    #print(k,n)
    
    r=Matrix(SR, k, n) 

    for i in range(n): 
        t1=MarshScottTwistHalf_symbolic(i,p)  
        for d in [0..k-1]: 
            r[d,i]=t1[d,0] 
    return r

def prepare_tropical_map_CDFL_Marsh_Scott_twist(k,n): # prepare for tropical map sigma_i
    indices = [(i,j) for i in range(k+1) for j in range(n+1)]
    R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
    x = dict(zip(indices, R.gens()))
    M = web_matrix_DFGK_reverse_xij(R,x,k,n)
    #print(evaluation_of_x_coordinates_on_web_matrix_CDFL(M,k,n))
    t2=Marsh_Scott_twist_on_matrix_symbolic(M) 
    t3=evaluation_of_x_coordinates_on_web_matrix_CDFL(t2,k,n)
    #print(t3)
    r1=t3
    
#     r1=[]
#     for i in t3:
#         p1=i
#         for j in indices:
#             p1=p1.subs({x[j[0],j[1]]:1/x[j[0],j[1]]})
#         r1.append(p1)
    r=prepare_tropical_map_rational_list_CDFL(R, r1)
    return r

def Marsh_Scott_twist_on_g_vector_CDFL_use_tropical_map(R,x,g,k,n,trop): # labels of initial seed are the same as CDFL
    # trop=prepare_tropical_map_CDFL_Marsh_Scott_twist(k,n)
#     indices = [(i,j) for i in range(k+1) for j in range(n+1)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    
    r=evaluation_tropical_function_rational_list_on_vector_CDFL_use_tropical_map(R, x, k,n, trop, g)
    
    return r 

def Marsh_Scott_twist_on_g_vector_CDFL(g,k,n): #   labels of initial seed are the same as CDFL
    indices = [(i,j) for i in range(k+1) for j in range(n+1)]
    R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
    x = dict(zip(indices, R.gens()))
    M = web_matrix_DFGK_reverse_xij(R,x,k,n)
    #print(evaluation_of_x_coordinates_on_web_matrix_CDFL(M,k,n))
    t2=Marsh_Scott_twist_on_matrix_symbolic(M) 
    #print(latex(t2))
    t3=evaluation_of_x_coordinates_on_web_matrix_CDFL(t2,k,n)
    #print(latex(t3))
    
    r1=t3
    
#     r1=[]
#     for i in t3:
#         p1=i
#         for j in indices:
#             p1=p1.subs({x[j[0],j[1]]:1/x[j[0],j[1]]})
#         r1.append(p1)
    t4=evaluation_tropical_function_rational_list_on_vector_CDFL(R, x, k,n,  r1, g)
    
    r=t4
    return r

def Marsh_Scott_twist_on_tableau(T,k,n): # T[i] is a column
    e1=initial_cluster_variables_CDFL(k,n)
    m=(k-1)*(n-k-1)
    g1=GVectorOfTableau(k,n,T,e1)[0][:m]
    #print(g1)
    r1=Marsh_Scott_twist_on_g_vector_CDFL(g1,k,n)
    #print(r1)
    r=GVectorToTableau_remove_inverses(r1,e1,k,n)
    
    return r 

def Marsh_Scott_twist_on_tableau_use_tropical_map(R,x,T,k,n,trop): # T[i] is a column, not finished
       # trop=prepare_tropical_map_CDFL_Marsh_Scott_twist(k,n)
#     indices = [(i,j) for i in range(k+1) for j in range(n+1)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    e1=initial_cluster_variables_CDFL(k,n)
    m=(k-1)*(n-k-1)
    g1=GVectorOfTableau(k,n,T,e1)[0][:m]
    #print('g1', g1)
    r1=Marsh_Scott_twist_on_g_vector_CDFL_use_tropical_map(R,x,g1,k,n,trop) 
    #print(r1)
    r=GVectorToTableau_remove_inverses(r1,e1,k,n)
    
    return r 

def Marsh_Scott_twist_on_tableau_use_tropical_map_one_row(R,x,T,k,n,trop): # T[i] is a column, not finished
       # trop=prepare_tropical_map_CDFL_Marsh_Scott_twist(k,n)
#     indices = [(i,j) for i in range(k+1) for j in range(n+1)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    r=[T]
    while (1):
        r1=Marsh_Scott_twist_on_tableau_use_tropical_map(R,x,r[-1],k,n,trop)
        if r1==r[0]:
            break
        else:
            r.append(r1)
    
    return r 

def Marsh_Scott_twist_on_tableau_one_row(T,k,n): # T[i] is a column
    r=[T]
    while (1):
        r1=Marsh_Scott_twist_on_tableau(r[-1],k,n)
        if r1==r[0]:
            break
        else:
            r.append(r1)
    
    return r 

def GVectorToTableau_fractional(g,e1,k,n): # e1 is initial cluster
    r1=[]
    for i in [0..len(g)-1]:
        if g[i]>0:
            t1=PowerOfTableaux(e1[i],g[i])
            r1=UnionOfTwoTableaux(r1,t1)
    r=[r1]
    r1=[]
    for i in [0..len(g)-1]:
        if g[i]<0:
            t1=PowerOfTableaux(e1[i],-g[i])
            r1=UnionOfTwoTableaux(r1,t1)
            #print(r1)
    #print(r1)
    r.append(r1)
    
    return r

def GVectorToTableau_remove_inverses(g,e1,k,n): # e1 is initial cluster
    r0=GVectorToTableau_fractional(g,e1,k,n)
    #print(r0)
    if r0[1]==[]:
        r=r0[0]
        r=RemoveTrivialTableau3(r,k,n)
    else:
        t1=TableauToModule('Y', r0[0])
        t2=TableauToModule('Y', r0[1])
        r1=[t1,t2]
        #print(t1)
        #print(t2)
        p1=[]
        for i in e1[-(k-1):]:
            t1=TableauToModule('Y', i)
            p1.append([t1[0][0], t1])
        #print(p1)
        r2=[]
        for a1 in p1:
            i=a1[0]
            t1=max_occuring_times_Yis_for_i(i,r1[1])
            if t1>0:
                #print('t1',t1)
                r2=r2+a1[1]*t1
        #print('r2',r2)
        #print('r1[1]',r1[1])
        r2=SetDifferenceListDifference3(r2,r1[1])
        r3=ModulesToTableau(r1[0]+r2,k,n)
        #print('r',r)
        r=RemoveTrivialTableau3(r3,k,n)
        #print('r',r)
    return r


def SetDifferenceListDifference3(A,B): # A-B, can have duplicate elements
    r=[]
    r1=removeDuplicatesListOfLists(A)
    for i in r1:
        t1=NumberOfA(A, i)
        t2=NumberOfA(B, i)
        t1=t1-t2
        #print(t1)
        for j in range(1,t1+1):
            r.append(i)
            
    return r

def max_occuring_times_Yis_for_i(ii,l): # l = [[1, -1], [2, 0], [1, -3]]
    r2=[]
    for j in l:
        if j[0]==ii and IsInList(j[1],r2)==0:
            r2.append(j[1]) 
    r3=[]
    for j in l:
        if j[0]==ii:
            r3.append(j)
    #print(r3)
    #print(r2)
    r1=0
    for j in r2:
        t1=0
        for i in r3:
            if i[1]==j:
                t1=t1+1
        if r1<t1:
            r1=t1
    r=r1
    return r


def BraidGroupActionSigmaIOnTableau(ii,T,k,n): # T[i] is a column
    e1=initial_cluster_variables_CDFL(k,n)
    m=(k-1)*(n-k-1)
    g1=GVectorOfTableau(k,n,T,e1)[0][:m]
    #print(g1)
    r1=BraidGroupActionSigmaIOnGVector_CDFL(ii,g1,k,n)
    #print(r1)
    r=GVectorToTableau_remove_inverses(r1,e1,k,n)
    
    return r 

def BraidGroupActionSigmaIOnTableau_inverse(ii,T,k,n): # T[i] is a column
    e1=initial_cluster_variables_CDFL(k,n)
    m=(k-1)*(n-k-1)
    g1=GVectorOfTableau(k,n,T,e1)[0][:m]
    #print(g1)
    r1=BraidGroupActionSigmaIOnGVector_inverse_CDFL(ii,g1,k,n)
    #print(r1)
    r=GVectorToTableau_remove_inverses(r1,e1,k,n)
    
    return r 


def prepare_tropical_map_rational_CDFL(R, f1):  # f1 is a list of rational functions
    # f1 =  [[1, x_1_1, x_2_3], [1, x_1_1], [1, x_2_3], [1]]) means min( x11+x23, x11, x23, 0 ) )
    # f1 = [[2, x11]] means 2 + x11
    # f1 = [[2, x11], [3]] means min(2+x11, 0) ???
    # t2=BraidGroupActionSigmaIOnMatrixRenomalizeColumns_symbolic(R,ii,k,n,M)
    # f1 = evaluation_of_x_coordinates_on_web_matrix_DFGK(t2,k,n)
    # list_x = [x11,x12,x13,x21,x22,x23,x31,x32,x33], #  
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    
    r2=f1.numerator()
    r3=f1.denominator()
    c1=change_polynomial_to_list(R,r2)
    c2=change_polynomial_to_list(R,r3)
    r=[c1, c2]

    return r

def prepare_tropical_map_rational_list_CDFL(R, f1):  # f1 is a list of rational functions
    # f1 =  [[1, x_1_1, x_2_3], [1, x_1_1], [1, x_2_3], [1]]) means min( x11+x23, x11, x23, 0 ) )
    # f1 = [[2, x11]] means 2 + x11
    # f1 = [[2, x11], [3]] means min(2+x11, 0) ???
    # t2=BraidGroupActionSigmaIOnMatrixRenomalizeColumns_symbolic(R,ii,k,n,M)
    # f1 = evaluation_of_x_coordinates_on_web_matrix_DFGK(t2,k,n)
    # list_x = [x11,x12,x13,x21,x22,x23,x31,x32,x33], #  
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    
    r=[]
    for i in f1:
        t1=prepare_tropical_map_rational_CDFL(R, i)
        r.append(t1)

    return r

def prepare_tropical_map_CDFL(ii,k,n): # prepare for tropical map sigma_i
    indices = [(i,j) for i in range(k+1) for j in range(n+1)]
    R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
    x = dict(zip(indices, R.gens()))
    M = web_matrix_DFGK_reverse_xij(R,x,k,n)
    #print(evaluation_of_x_coordinates_on_web_matrix_CDFL(M,k,n))
    t2=BraidGroupActionSigmaIOnMatrixRenomalizeColumns_symbolic(R,ii,k,n,M)
    #print(latex(t2))
    t3=evaluation_of_x_coordinates_on_web_matrix_CDFL(t2,k,n)
    #print(latex(t3))
    r1=t3
    
#     r1=[]
#     for i in t3:
#         p1=i
#         for j in indices:
#             p1=p1.subs({x[j[0],j[1]]:1/x[j[0],j[1]]})
#         r1.append(p1.factor())
#     print(r1)
#     print(latex(r1))
    r=prepare_tropical_map_rational_list_CDFL(R, r1)
    return r

def prepare_tropical_map_CDFL_inverse(ii,k,n): # prepare for tropical map sigma_i
    indices = [(i,j) for i in range(k+1) for j in range(n+1)]
    R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
    x = dict(zip(indices, R.gens()))
    M = web_matrix_DFGK_reverse_xij(R,x,k,n)
    #print(evaluation_of_x_coordinates_on_web_matrix_CDFL(M,k,n))
    t2=BraidGroupActionSigmaI_inverse_OnMatrixRenomalizeColumns_symbolic(R,ii,k,n,M)
    #print(latex(t2))
    t3=evaluation_of_x_coordinates_on_web_matrix_CDFL(t2,k,n)
    #print(latex(t3))
    r1=t3
    
#     r1=[]
#     for i in t3:
#         p1=i
#         for j in indices:
#             p1=p1.subs({x[j[0],j[1]]:1/x[j[0],j[1]]})
#         r1.append(p1)
#     #print(r1)
    r=prepare_tropical_map_rational_list_CDFL(R, r1)
    return r

def evaluation_tropical_function_rational_on_vector_CDFL_use_tropical_map(R, x, k,n,  f1, vv):  # f1=[c1,c2] means f1=c1/c2, c1, c2 are lists [[1, x_1_1, x_2_3], [1, x_1_1], [1, x_2_3], [1]]
    # f1 =  [[1, x_1_1, x_2_3], [1, x_1_1], [1, x_2_3], [1]] means min( x11+x23, x11, x23, 0 )
    # f1 = [[2, x11]] means 2 + x11
    # f1 = [[2, x11], [3]] means min(2+x11, 0) ???
    # list_x = [x11,x12,x13,x21,x22,x23,x31,x32,x33]
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    
    c1=f1[0]
    c2=f1[1]
    
    g1=evaluation_tropical_function_on_vector_CDFL(R, x, k,n,   c1, vv)
    g2=evaluation_tropical_function_on_vector_CDFL(R, x, k,n,   c2, vv)
    r=g1-g2
    
    return r

def evaluation_tropical_function_rational_list_on_vector_CDFL_use_tropical_map(R, x, k,n,   f1, vv):  # f1 is a list of [c1,c2], c1=[[1, x_1_1, x_2_3], [1, x_1_1], [1, x_2_3], [1]]
    # f1 =  [[1, x_1_1, x_2_3], [1, x_1_1], [1, x_2_3], [1]]) means min( x11+x23, x11, x23, 0 ) )
    # f1 = [[2, x11]] means 2 + x11
    # f1 = [[2, x11], [3]] means min(2+x11, 0) ???
    # t2=BraidGroupActionSigmaIOnMatrixRenomalizeColumns_symbolic(R,ii,k,n,M)
    # f1 = evaluation_of_x_coordinates_on_web_matrix_DFGK(t2,k,n)
    # list_x = [x11,x12,x13,x21,x22,x23,x31,x32,x33], #  
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    
    r=[]
    for i in f1:
        t1=evaluation_tropical_function_rational_on_vector_CDFL_use_tropical_map(R, x, k,n,  i, vv)
        r.append(t1)

    return r

def BraidGroupActionSigmaIOnGVector_CDFL_use_tropical_map(R,x,g,k,n,trop): # sigma_ii on g, labels of initial seed are the same as CDFL
    # trop = prepare_tropical_map_CDFL(ii,k,n)
    #trop = prepare_tropical_map_CDFL_inverse(ii,k,n) # if computing sigma_i^{-1}
#     indices = [(i,j) for i in range(k+1) for j in range(n+1)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    
    r=evaluation_tropical_function_rational_list_on_vector_CDFL_use_tropical_map(R, x, k,n, trop, g)
    
    return r 

def BraidGroupActionSigmaIOnTableau_use_tropical_map(R,x,T,k,n,trop): # T[i] is a column
            # trop = prepare_tropical_map_CDFL(ii,k,n)
        #trop = prepare_tropical_map_CDFL_inverse(ii,k,n) # if computing sigma_i^{-1}
#     indices = [(i,j) for i in range(k+1) for j in range(n+1)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    e1=initial_cluster_variables_CDFL(k,n)
    m=(k-1)*(n-k-1)
    g1=GVectorOfTableau(k,n,T,e1)[0][:m]
    #print(g1)
    r1=BraidGroupActionSigmaIOnGVector_CDFL_use_tropical_map(R,x,g1,k,n,trop) 
    #print(r1)
    r=GVectorToTableau_remove_inverses(r1,e1,k,n)
    
    return r 

#Function for multiprocessing 
def ll_perms_compute_braid_group_action(R,x,T_list,lli,max_column,k,n,repeat, b6,fp1,trop): # trop =[trop1,trop2,...] trop(sigma_1), ...  
    ll0=lli
    r2=copy.deepcopy(T_list)
    #print(ll0)
    #print(r2)
    
    r1=[]
    for i in list(b6.keys()):
        r1.append(sage_eval(i))
    r2=r2+r1
    
    for i in range(repeat):
        r3=[]
        #print(len(r2))
        for j in r2:
            t1=j
            for ii in np.random.permutation(ll0): 
            #for ii in range(len(trop)): 
                #print('ii',ii, np.random.permutation(ll0))
                t1=BraidGroupActionSigmaIOnTableau_use_tropical_map(R,x,t1,k,n,trop[ii])
                if len(t1)<=max_column:
                    #print('t1', t1)
                    r3.append(t1)
                    p1=repr(t1)
                    p2=b6.get(p1)            
                    if p2==None:
                        f1=open(fp1, 'a+')
                        f1.write(str(t1)+'\n')
                        f1.close()
                        b6[p1]=[] 
                        print(len(b6))
                else:
                    break
        r2=r3
        
    return b6

def evaluation_of_x_coordinates_on_web_matrix_CDFL_including_frozens(M,k,n):
#     k,n=4,8
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
#     M = symbolic_matrix(R,k,n) 
    p1=InitialClusterWithoutNumericalMatrices(k,n)
    ini=initial_cluster_variables_CDFL(k,n)
    #print(ini)
    ma=p1[0]
    r1=p1[1][0]
    r=[]
    for rr2 in range(len(ini)):
        r2=ini[rr2]
        for i in range(ma.nrows()):
            if r1[i][1]==r2:
                ii=i
        out=[]      
        #print(ma)
        for i in range(ma.nrows()):
            #print(i)
            if ma[ii,i]>0:
                t1=ma[ii,i]*[i]
                out=out+t1
        in_coming=[]            
        #print(ma)
        for i in range(ma.nrows()):
            #print(i)
            if ma[ii,i]<0:
                t1=(-ma[ii,i])*[i]
                in_coming=in_coming+t1
        g1=1
        #print(out, in_coming)
        for i in out: 
            g1=g1*PluckerToMinorsPolynomial(M, [[1, r1[i][1]]])
            #g1=(g1.expand()).factor()
        g2=1
        for j in in_coming:
            g2=g2*PluckerToMinorsPolynomial(M, [[1, r1[j][1]]])
            #g2=(g2.expand()).factor()
        g1=g2/g1
        #g1=g1/g2
            
        r.append(g1.factor())
        
    return r


def RemoveTrivialTableau2(a1, n,n1): # n is the row of tableau, also remove 189, 129 in Gr39, Gr(n,n1),m1 =m= n1-n+1
 
    r=a1
    m=n1-n+1
    t2=PluckersWhichAreOne2(n,n1,m)

    #print(t2)

    sn=1

    while sn!=0:

        r1=r
        for i in [0..len(t2)-1]:

            #print(t2[i],r)

            if r==[]:
                return r
            elif IsTableauAFactorOfB([t2[i]], r)==1:
                r=TableauDivision(r, [t2[i]])
                #print(r)
                  
        if r1==r or r==[]:
            sn=0
            break
        else:
            sn=1  

    return r

def RemoveTrivialTableau3(a1, n,n1): # n is the row of tableau, also remove 189, 129 in Gr39, Gr(n,n1),m1 =m= n1-n+1
 
    r=a1
    m=n1-n+1
    t2=PluckersWhichAreOne2(n,n1,m)

    #print(t2)

    sn=1

    while sn!=0:

        r1=r
        for i in [0..len(t2)-1]:

            #print(t2[i],r)

            if r==[]:
                return r
            elif IsTableauAFactorOfB([t2[i]], r)==1:
                g1=TableauDivision(r, [t2[i]])
                if g1==[]:
                    r=g1
                    return r
                elif IsSemistandardTableauInMatrix(TableauToMatrix(g1))==1:
                    r=g1
        if r1==r or r==[]:
            sn=0
            break
        else:
            sn=1  

    return r

def initial_cluster_variables_CDFL(k,n): # Grkn
    t1=InitialClusterWithoutNumericalMatrices(k,n)[1][0]
    t2=TakeIthElements(2,t1)
    r1=[]
    for i in t2:
        if RemoveTrivialTableau2(i,k,n)!=[]:
            r1.append(i)
    r2=initial_cluster_variables_DFGK(k,n)
    m=(k-1)*(n-k-1)
    r3=r2[m:]
    r=r1+r3
    
    return r

def BraidGroupActionSigmaIOnPlucker_symbolic(R,ii, p, k,n, M): # p = [1,2,3,8], need to check and correct
    
#     r1=ContentOfTableauObtainedFromBraidGroupActionPlucker(R, ii, p, k,n, M)
#     T=list(split_list(r1, k))
#     #u1=NonCrossingTupleWithSameContentAsT_new(T) 
#     u1=TableauxWithSameContentAsT_new(T)
#     c1=[]
#     for j in range(len(u1)):
#         M1=randomMatrixNonInt(k,n)
#         r1=BraidGroupActionSigmaI_inverse_OnMatrixRenomalizeColumns_symbolic(R,ii,k,n, M1)
#         r2=PluckerToMinorsPolynomial(r1, [[1, [p]]])
#         c1.append([M1, r2])
#     r=ComputeChTInTermsOfNonCrossingBasis([u1, c1])

#     k,n=4,8
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R1 = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     M1=symbolic_matrix(R1,k,n)
#     #print(M1)
#     ii=1
    
    
    r1=BraidGroupActionSigmaI_inverse_OnMatrixRenomalizeColumns_symbolic(R,ii,k,n, M)

    t1=sub_matrix_general(r1,ListAToN(1,k),p)
    r2=det(t1)
    r=MinorsToPluckerPolynomial(R,M,r2)

    return r 

def BraidGroupActionSigmaI_inverse_OnPlucker_symbolic(R,ii, p, k,n, M): # p = [1,2,3,8], need to check and correct
#     #print('M',M)
#     r1=ContentOfTableauObtainedFromBraidGroupActionPlucker(R, ii, p, k,n, M)
#     T=list(split_list(r1, k))
#     #u1=NonCrossingTupleWithSameContentAsT_new(T) 
    
#     #print('r1',r1)
#     #print('T',T)
#     u1=TableauxWithSameContentAsT_new(T)
    
#     #print('u1',u1)
    
#     c1=[]
#     for j in range(len(u1)):
#         M1=randomMatrixNonInt(k,n)
#         r1=BraidGroupActionSigmaIOnMatrixRenomalizeColumns_symbolic(R,ii,k,n, M1)
#         r2=PluckerToMinorsPolynomial(r1, [[1, [p]]])
#         c1.append([M1, r2])
#     r=ComputeChTInTermsOfNonCrossingBasis([u1, c1])
     
#     k,n=4,8
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R1 = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     M1=symbolic_matrix(R1,k,n)
#     #print(M1)
#     ii=1
    
    r1=BraidGroupActionSigmaIOnMatrixRenomalizeColumns_symbolic(R,ii,k,n, M)
    
    #t1=sub_matrix_general2(R,r1,ListAToN(1,k),p)
    t1=sub_matrix_general(r1,ListAToN(1,k),p)
    r2=det(t1)
    r=MinorsToPluckerPolynomial(R,M,r2)
    
    #print('r',r)

    return r



def BraidGroupActionSigmaIOnGVector_inverse_CDFL(ii,g,k,n): # sigma_ii on g, labels of initial seed are the same as CDFL
    indices = [(i,j) for i in range(k+1) for j in range(n+1)]
    R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
    x = dict(zip(indices, R.gens()))
    M = web_matrix_DFGK_reverse_xij(R,x,k,n)
    #print(evaluation_of_x_coordinates_on_web_matrix_CDFL(M,k,n))
    t2=BraidGroupActionSigmaI_inverse_OnMatrixRenomalizeColumns_symbolic(R,ii,k,n,M)
    t3=evaluation_of_x_coordinates_on_web_matrix_CDFL(t2,k,n)
    
    r1=t3
    
#     r1=[]
#     for i in t3:
#         p1=i
#         for j in indices:
#             p1=p1.subs({x[j[0],j[1]]:1/x[j[0],j[1]]})
#         r1.append(p1)
    t4=evaluation_tropical_function_rational_list_on_vector_CDFL(R, x, k,n,  r1, g)
    
    r=t4
    return r 

def BraidGroupActionSigmaIOnGVector_CDFL(ii,g,k,n): # sigma_ii on g, labels of initial seed are the same as CDFL
    indices = [(i,j) for i in range(k+1) for j in range(n+1)]
    R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
    x = dict(zip(indices, R.gens()))
    M = web_matrix_DFGK_reverse_xij(R,x,k,n)
    #print(evaluation_of_x_coordinates_on_web_matrix_CDFL(M,k,n))
    t2=BraidGroupActionSigmaIOnMatrixRenomalizeColumns_symbolic(R,ii,k,n,M)
    #print(latex(t2))
    #print(t2)
    t3=evaluation_of_x_coordinates_on_web_matrix_CDFL(t2,k,n)
    #print(latex(t3))
    #print(latex(evaluation_of_x_coordinates_on_web_matrix_CDFL(M,k,n)))
    #print(t3)
    
    r1=t3
    
#     r1=[]
#     for i in t3:
#         p1=i
#         for j in indices:
#             p1=p1.subs({x[j[0],j[1]]:1/x[j[0],j[1]]})
#         r1.append(p1)
    #print(r1)
    t4=evaluation_tropical_function_rational_list_on_vector_CDFL(R, x, k,n,  r1, g)
    
    r=t4
    return r

def BraidGroupActionSigmaIOnGVector_compatible_with_tableau(ii,g,k,n): #  labels of initial seed the same as DFGK
    indices = [(i,j) for i in range(k+1) for j in range(n+1)]
    R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
    x = dict(zip(indices, R.gens()))
    M = web_matrix_DFGK_reverse_xij(R,x,k,n)
    #print(evaluation_of_x_coordinates_on_web_matrix_compatible_with_tableau(M,k,n))
    t2=BraidGroupActionSigmaIOnMatrixRenomalizeColumns_symbolic(R,ii,k,n,M)
    #print(t2)
    t3=evaluation_of_x_coordinates_on_web_matrix_compatible_with_tableau(t2,k,n)
    #print(t3)
    r1=t3
    
#     r1=[]
#     for i in t3:
#         p1=i
#         for j in indices:
#             p1=p1.subs({x[j[0],j[1]]:1/x[j[0],j[1]]})
#         r1.append(p1)
    t4=evaluation_tropical_function_rational_list_on_vector(R, x, k,n,  r1, g)
    
    r=t4
    return r 

def BraidGroupActionSigmaIOnGVector_inverse_compatible_with_tableau(ii,g,k,n): #  labels of initial seed the same as DFGK
    indices = [(i,j) for i in range(k+1) for j in range(n+1)]
    R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
    x = dict(zip(indices, R.gens()))
    M = web_matrix_DFGK_reverse_xij(R,x,k,n)
    #print(M)
    #print(evaluation_of_x_coordinates_on_web_matrix_compatible_with_tableau(M,k,n))
    t2=BraidGroupActionSigmaI_inverse_OnMatrixRenomalizeColumns_symbolic(R,ii,k,n,M)
    t3=evaluation_of_x_coordinates_on_web_matrix_compatible_with_tableau(t2,k,n)
    r1=t3
    
#     r1=[]
#     for i in t3:
#         p1=i
#         for j in indices:
#             p1=p1.subs({x[j[0],j[1]]:1/x[j[0],j[1]]})
#        r1.append(p1)
    t4=evaluation_tropical_function_rational_list_on_vector(R, x, k,n,  r1, g)
    
    r=t4
    return r 

def evaluation_of_x_coordinates_on_web_matrix_compatible_with_tableau(M,k,n):
#     k,n=4,8
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
#     M = symbolic_matrix(R,k,n) 
    p1=InitialClusterWithoutNumericalMatrices(k,n)
    ini=initial_cluster_variables_DFGK(k,n)
    
    ma=p1[0]
    r1=p1[1][0]
    r=[]
    for rr2 in range(len(ini)-n):
        r2=ini[rr2]
        for i in range(ma.nrows()):
            if r1[i][1]==r2:
                ii=i
        out=[]
        #print(ma)
        for i in range(ma.nrows()):
            #print(i)
            if ma[ii,i]>0:
                t1=ma[ii,i]*[i]
                out=out+t1
        in_coming=[]
        #print(ma)
        for i in range(ma.nrows()):
            #print(i)
            if ma[ii,i]<0:
                t1=(-ma[ii,i])*[i]
                in_coming=in_coming+t1
        g1=1
        #print(out, in_coming)
        for i in out: 
            g1=g1*PluckerToMinorsPolynomial(M, [[1, r1[i][1]]])
        g2=1
        for j in in_coming:
            g2=g2*PluckerToMinorsPolynomial(M, [[1, r1[j][1]]])
        g1=g2/g1
        #g1=g1/g2
            
        r.append(g1.factor())
    return r

def g_vector_in_DFGK_from_mutation_seq_CDFL(typ,k,n,p2):
    # p2 is a sequence from initial quiver
    g1=initial_g_vector_DFGK(k,n)
    m=(k-1)*(n-k-1)
    rank=k-1
    e3=initial_cluster_variables_DFGK(k,n)
    d1=dictionary_CDFL_to_DFGK(k,n)

    IC=InitialClusterWithoutNumericalMatrices(k,n)
    mat1=IC[0] 
    vertices1=IC[1] 

    g1=initial_g_vector_DFGK(k,n)
    matB=initial_exchange_matrix_DFGK(k,n)
    #print(matB)
    matB0=matB
    matC=Matrix(m,m)

    for i in range(m):
        matC[i,i]=1 
    #print(matC)

    r1=g1

    for kk1 in p2:
        kk=d1[kk1] 
        r1=mutation_g_vector_DFGK(kk, r1, matB, matC, matB0) # this is the g vector in DFGK

        matC=matrixMutation_C_matrix(matC, matB, kk)
        matB=matrixMutation(matB,kk)

        vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, kk1,typ,rank)
        mat1 = matrixMutation(mat1, kk1) 

        #print('kk+1', kk+1)
        #print('new cluster variable:',vertices1[1])
        t1=MatrixTakeColumns(vertices1[1])
        t2=GVectorOfTableau(k,n,t1,e3)[0][:m] # this is the g vector in CDFL
        #print('len(t1)', len(t1))
        #print('t1', t1)
        #print('t2.   ', t2)
        #print('r1[kk]', r1[kk])
    #print('t1', t1)
    #print('t2.   ', t2)
    #print('r1[kk]', r1[kk])

    r=r1[kk]
    
    return r



def g_vector_in_DFGK_nonreal_from_mutation_seq_CDFL_origin_quiver(typ,k,n,p2,mutation_number):
    # p2 is a sequence from initial quiver which produce an origin quiver
    # p2=[0, 6, 2, 12, 17, 13, 1, 15, 7, 11, 3, 10, 16, 18, 5, 8, 0, 0, 6, 6, 2, 12, 12, 17, 13, 1, 15, 7, 11, 3, 10, 16, 18, 5, 8, 0, 0, 6, 2, 12, 12, 17, 13, 1]
    g1=initial_g_vector_DFGK(k,n)
    m=(k-1)*(n-k-1)
    rank=k-1
    e3=initial_cluster_variables_DFGK(k,n)
    d1=dictionary_CDFL_to_DFGK(k,n)

    IC=InitialClusterWithoutNumericalMatrices(k,n)
    mat1=IC[0] 
    vertices1=IC[1] 

    g1=initial_g_vector_DFGK(k,n)
    matB=initial_exchange_matrix_DFGK(k,n)
    #print(matB)
    matB0=matB
    matC=Matrix(m,m)

    for i in range(m):
        matC[i,i]=1 
    #print(matC)

    r1=g1

    for kk1 in p2:
        kk=d1[kk1] 
        r1=mutation_g_vector_DFGK(kk, r1, matB, matC, matB0)

        matC=matrixMutation_C_matrix(matC, matB, kk)
        matB=matrixMutation(matB,kk)

        vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, kk1,typ,rank)
        mat1 = matrixMutation(mat1, kk1) 

        #print('kk+1', kk+1)
        #print('new cluster variable:',vertices1[1])
        t1=MatrixTakeColumns(vertices1[1])
        t2=GVectorOfTableau(k,n,t1,e3)[0][:m]
        #print('t2.   ', t2)
        #print('r1[kk]', r1[kk])

    double_arrow=[]
    for i in range(mat1.nrows()):
        for j in range(mat1.ncols()):
            if mat1[i,j]>1 or mat1[i,j]<-1:
                #print(i,j,mat1[i,j])
                double_arrow.append([i,j])

    np2=double_arrow[0]*mutation_number
    for kk1 in np2:
        kk=d1[kk1] 
        r1=mutation_g_vector_DFGK(kk, r1, matB, matC, matB0)

        matC=matrixMutation_C_matrix(matC, matB, kk)
        matB=matrixMutation(matB,kk)

        vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, kk1,typ,rank)
        mat1 = matrixMutation(mat1, kk1) 

        #print('kk+1', kk+1)
        #print('new cluster variable:',vertices1[1])
        t1=MatrixTakeColumns(vertices1[1])
        t2=GVectorOfTableau(k,n,t1,e3)[0][:m]
        #print('t2.   ', t2)
        #print('r1[kk]', r1[kk])
    #print('t2.   ', t2)
    #print('r1[kk]', r1[kk])
    d2=[]
    for i in p2+np2:
        d2.append(d1.get(i)+1)
    #print(ll[3], remove_common_factors_approximately(r1[kk], 12), d2)
    r=remove_common_factors_approximately(r1[kk], mutation_number)
    
    return r

def initial_g_vector_DFGK(k,n): # only mutable part
    m=(k-1)*(n-k-1)
    r=[]
    for i in range(m):
        t1=[]
        for j in range(m):
            if j!=i:
                t1.append(0)
            else:
                t1.append(1)
        r.append(t1)
    return r

def dictionary_DFGK_to_CDFL(k,n): # vertices label in DFGK to CDFL
    IC=InitialClusterWithoutNumericalMatrices(k,n)
    cl=initial_cluster_variables_DFGK(k,n)
    
    #print(cl)
    
    r0=IC[0]
    p1=IC[1][0]
    
    #print(p1)
    
    d1={}
    for i in range(len(cl)):
        for j in p1:
            if cl[i]==j[1]:
                d1[i]=j[2]
    r=d1
    return r

def dictionary_CDFL_to_DFGK(k,n): # vertices label in CDFL to DFGK, CDFL includes frozens
    IC=InitialClusterWithoutNumericalMatrices(k,n)
    cl=initial_cluster_variables_DFGK(k,n)
    
    #print(cl)
    
    r0=IC[0]
    p1=IC[1][0]
    
    #print(p1)
    
    d1={}
    for i in range(len(cl)):
        for j in p1:
            if cl[i]==j[1]:
                d1[j[2]]=i
    r=d1
    return r
    

#Function for multiprocessing 
def ll_perms_compute_nonreal_from_clusters(lli,typ,rank,max_column,n,repeat,b6,b_nonreal,b_nonreal_with_seq,fp1,fp2,fp3,mutation_number): #  
    IC=InitialClusterWithoutNumericalMatrices(rank,n)
    mat1=IC[0]
    vertices1=IC[1]
    e1=initial_cluster_variables_DFGK(rank,n)
    
    #print('e1',e1)
    
    #print('rr')
    #ll=list(np.array(lli)-1)
    ll=list(map(lambda x: x - 1, lli))
    #print(ll)
    
    mutationSequence=[]
    for j1 in [1..repeat]: 
        mutationSequence=mutationSequence+ll 
    mutate_seq=[]
    for j in range(len(mutationSequence)): 
        #print('kk')
        vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 
        mutate_seq.append(mutationSequence[j])
        
        if vertices1[1].ncols()>max_column:
            vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank) ###why repeat above calculation?
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
            mutate_seq.append(mutationSequence[j])
        else:
            ii0=vertices1[1]
            ii=MatrixTakeColumns(vertices1[1])
            p1=repr(ii)
            p2=b6.get(p1)
            
            #print('p1',p1)
            
            if p2==None:
                #print('ii',ii)
                
                c1=ExtendSetOfTableauxToContainPromotionsInMatrix([ii0],n) 
                c1=ExtendSetOfTableauxToContainEvacuationsInMatrix(c1,n) 
                c1=ExtendSetOfTableauxToContainEvacuationsInMatrix(c1,n) 
                c1=ExtendSetOfTableauxToContainPromotionsInMatrix(c1,n) 
                c1=MatrixTakeColumnsList(c1)
                f1=open(fp1, 'a+')
                for ci in c1:
                    g1=repr(ci)
                    g2=b6.get(g1)
                    if g2==None:
                        f1.write(str(ci)+'\n')
                        b6[g1]=[] 
                f1.close()
                
            
            ma0=MatrixTakeRows(mat1)
            if len(SetDifference(flatten(ma0), [1,0,-1]))>=2:
            #if len(SetDifference(flatten(ma0), [1,0,-1]))==2:
                t0=sorted(TakeIthElements(2,vertices1[0]))
                t1=removeAnElementInListList(0,vertices1[0])
                #print('t1',t1)
                t2=[t0, ma0, t1]
                t3=copy.deepcopy(t2)
                g00=FindNonRealModuleUsingQuiverWithMultipleArrows(rank,n,t3,e1,mutation_number)
                #print(g00)
                
                if g00!=[]:
                    g0=g00[0][1]
                    if len(g0)<=max_column: # collect only nonreal modules less or equal to max_column columns
                        g1=repr(g0)
                        g2=b_nonreal.get(g1)
                        
                        z1=repr(t0)
                        z2=b_nonreal_with_seq.get(z1)
                        
                        if z2==None:
                            f1=open(fp3, 'a+')
                            f1.write(str([t0,ma0,t1,g0,mutate_seq])+'\n')
                            b_nonreal_with_seq[z1]=[]
                            f1.close()

                        if g2==None:
                            
                            c1=ExtendSetOfTableauxToContainPromotionsInMatrix([TableauToMatrix(g0)],n) 
                            c1=ExtendSetOfTableauxToContainEvacuationsInMatrix(c1,n) 
                            c1=ExtendSetOfTableauxToContainEvacuationsInMatrix(c1,n) 
                            c1=ExtendSetOfTableauxToContainPromotionsInMatrix(c1,n) 
                            c1=MatrixTakeColumnsList(c1)
                            f1=open(fp2, 'a+')
                            for ci in c1:
                                d1=repr(ci)
                                d2=b_nonreal.get(d1)
                                if d2==None:
                                    f1.write(str(ci)+'\n')
                                    b_nonreal[d1]=[] 
                            f1.close()
    return [b6, b_nonreal, b_nonreal_with_seq]

def initial_exchange_matrix_DFGK(k,n): # only mutable part
    IC=InitialClusterWithoutNumericalMatrices(k,n)
    cl=initial_cluster_variables_DFGK(k,n)
    
    #print(cl)
    
    r0=IC[0]
    p1=IC[1][0]
    
    #print(p1)
    
    d1={}
    for i in range(len(cl)):
        for j in p1:
            if cl[i]==j[1]:
                d1[i]=j[2]
    #print(d1)
    
    n=(k-1)*(n-k-1)
    r1=Matrix(n,n)
    for i in range(n):
        for j in range(n):
            r1[i,j]=r0[d1.get(i), d1.get(j)]
    
    r=Matrix(n,n)
    for i in range(n):
        for j in range(n):
            r[i,j]=-r1[i,j]
    
    return r

def compute_chT_tableaux_basis(T, k,n): # T=[[1,2,3,5],[2,4,6,7]] is a tableau
    d1=T # rows of tableau
    d2=TableauToSmallGapTableau(k,d1)
    d3=TableauDivision(d2,d1)
    #print(d3)
    a1=ChTUsingKLPolynomialInSage(d1,k)
    a2=d3 
    
    #print('a1',a1)
    
    r=[]
    if a1!=[]: 
        e1=TableauxWithSameContentAsT_new(T)
        #print(e1)
        p1=[]
        for i in [1..len(e1)]:
            M1=randomMatrixNonInt(k,n)
            g1=PluckerToMinorsPolynomial(M1, a1)/PluckerToMinorsPolynomial(M1, [[1,a2]])
            p1.append([M1, g1])
        e2=ComputeChTInTermsOfNonCrossingBasis([ e1, p1 ])
        r=e2
    return r

def ChTUsingKLPolynomialInSageHalf(k,l,w,w0,W,T1):
    P.<q> = LaurentPolynomialRing(QQ)
    KL = KazhdanLusztigPolynomial(W,q)
    
    r=[]
    i=l
    #print(i)
    t1=ShiftByA(i.tuple(),0)
    t2=SymmetricGroupActionOnTableauxComputeCh(k,t1,T1)
    if t2!=[]:
        t3=KL.P(i*w0, w*w0)
        t4=(-1)^((i*w).length())
        t5=t4*t3.subs({q:1})
        if t5!=0:
            r.append([t5, t2])
    return r


def ListToPolynomialInQ(l, q):
    r=0

    for i in [0..len(l)-1]:
        r=r+l[i]*q^(i-1)
    return r
    
def ListToPolynomialInQList(l, q):
    r=[]
    for i in l:
        r.append(ListToPolynomialInQ(i, q))
    return r

def SymmetricGroupActionOnTableauxComputeCh(k,w1,T): # Gr(k,n), w is an element in Sm written in one-line notation, nmlen(ii)
    r1=TableauToListsIJSorted(k,T)
    ii=r1[0]
    jj=r1[1]
    w=ShiftByA(w1,-1)
    #print(ii)
    #print(jj)
    
    m=len(ii)
    r=[]
        
    #print(m)   
    #print('w',w)
    for b in range(m):
        #print('b',b)
        if jj[b]>=ii[w[b]] and jj[b]<=ii[w[b]]+k:
            r.append(SetDifference(ListAToN(ii[w[b]], ii[w[b]]+k), [jj[b]]))
        else:
            r=[]
            break;
    r=sorted(r)
    return r

def TableauToListsIJ(k,a): # I is the set of the first element, J is the list of missing elements
    a1=TableauToSmallGapTableau(k,a);
    r1=CollectFirstElementInSmallGapTableau(a1);
    r2=CollectMissingElementInSmallGapTableau(a1)
    r=[r1,r2]
    return r


def TableauToListsIJSortedSLk(Y,a): # I is the set of the first element, J is the list of missing elements
    a1=TableauToFundamentalTableauxSLk(Y,a);
    r1=[]
    for i in a1:
        r1.append(len(i))
    r2=[]
    for i in a1:
        r2.append(max(i)) 
    r=[sorted(r1),sorted(r2)]
    return r


def TableauToListsIJSorted(k,a): # I is the set of the first element, J is the list of missing elements
    a1=TableauToSmallGapTableau(k,a);
    r1=CollectFirstElementInSmallGapTableau(a1);
    r2=CollectMissingElementInSmallGapTableau(a1)
    r=[sorted(r1),sorted(r2)]
    return r

def computePairsOfElementsInSymmetricGroupToBeRunOnCProgramOfGregFirstPart(T1,typ,rank,W): # rank=k, number of rows of tableaux
# T1 is the given tableau  
    k=rank 
    t2=TableauToSmallGapTableau(k,T1)
    #print(t2)
    t3=TableauToListsIJ(k,t2)
    #print(t3)
    A=t3[0]
    B=t3[1]
    
    #kk=len(A)
    
    #Bsorted=sorted(B)
    #r1=LongestPerm(A,sorted(B),B)
    
    #W = WeylGroup(typ+str(kk-1), prefix = 's')
    #W=SymmetricGroup(k)
    
    w=LongestPermWInSBWInverseSA(W,A,B)
    

    return w

def LongestPermWInSBWInverseSA(W,A,B): # longest word in S_j w^{-1} S_i, is it longest word in S_{lambda} w S_{mu}?
    t1=StablizerOfTuple(A)
    t2=StablizerOfTuple(sorted(B))
    s=W.simple_reflections()
    #print(t1,t2)
    t3=[]
    for i in t1:
        t3.append(s[i])
    t4=[]
    for i in t2:
        t4.append(s[i])
    t5=find_permutation2(B,sorted(B))
    w=W.one()
    for i in t5:
        w=w*s[i]
    #print(t4,w,t3)
    r2=LongestPermInDoubleCosetWeylGroup(W,t4,w,t3)
    r=r2
    
    return r

def SymmetricGroupActionOnListSi(i,L): #  w=s_i,  L=[a1,a2,...,an]
    r1=[]
    for j in L:
        r1.append(j)
    
    t1=r1[i-1]
    r1[i-1]=r1[i]
    r1[i]=t1 
     
    r=r1
        
    return r

def StablizerOfTuple(A): # A is a list, weakly increasing
    k = len(A)
    r=[]
    for i in [1..k-1]:
        t1=SymmetricGroupActionOnListSi(i,A)
        if t1==A:
            r.append(i)
    return r

def find_permutation2(L1, L2):
    perm = Word(L1).standard_permutation() / Word(L2).standard_permutation()
    assert [L2[i-1] for i in perm] == L1
    r=perm.reduced_word()
    return r

def LongestPermInDoubleCosetWeylGroup(W,S1,w,S2): # W is a Weyl group, typ='A', rank=4, W=WeylGroup(typ+str(rank), prefix = 's')
    g1=W.subgroup(S1)
    g2=W.subgroup(S2)
    winner = W.one()
    N_CPU = multiprocessing.cpu_count()
    p = get_context("fork").Pool(processes=N_CPU) # works on mac
    winner = max( p.imap_unordered(functools.partial(myprod,g1,w), g2), key=lambda t: t.length() )      
#     for u1 in g1:
#         for u2 in g2:
#             t1=u1*w*u2
#             if t1.length()>winner.length():
#                 winner=t1
    r=winner
    
    return r

def myprod(g1,w,u2):
    return max([t*w*u2 for t in g1], key=lambda t: t.length())


def ShiftByA(l,a):
    r=[]
    for i in l:
        r.append(i+a)
    return r

def ShiftByAList(l,a):
    r=[]
    for i in l:
        r.append(ShiftByA(i,a))
    return r

def initial_exchange_matrix_CDFL_Gr48(): # only mutable part
    r=Matrix([[0,-1,0,-1,1,0,0,0,0],[1,0,-1,0,-1,1,0,0,0],[0,1,0,0,0,-1,0,0,0],[1,0,0,0,-1,0,-1,1,0],[-1,1,0,1,0,-1,0,-1,1],[0,-1,1,0,1,0,0,0,-1],[0,0,0,1,0,0,0,-1,0],[0,0,0,-1,1,0,1,0,-1],[0,0,0,0,-1,1,0,1,0]])
    r1=MatrixTakeRows(r)
    n=len(r1)
    t1=Matrix(n,n)
    for i in range(n):
        t1[i,i]=1
    r2=MatrixTakeRows(t1)
    r=r1+r2
    r=TableauToMatrixTakeRows(r)
    
    return r

def matrixMutation_C_matrix_FZ(mat, mat_B, k):  # mutate at k
    size=mat.nrows()
    r=Matrix(size,size)
    for i in range(size):
        for j in range(size):
            r[i,j]=mat[i,j]
    
    for i in range(size): 
        for j in range(size): 
            
            if j==k:
                r[i,j]=-mat[i, j]    
            else: 
                r[i, j] = mat[i, j] + max( -mat[i,k], 0 )*mat_B[k,j] + mat[i,k]*max(mat_B[k,j], 0)
     
    return r

def mutation_g_vector_FZ2(kk, g, mat_B, mat_B0): # matrix B is 2n by n matrix, g is the list of initial g vectors, mutate at vertex kk (the vertices are labelled by 0, 1, 2, ...), mat_B0 is the initial exchange matrix

    r=copy.deepcopy(g)
    n=len(g)
    t1=[]
    for i in range(n):
        t1.append(0)
    for i in [0..n-1]:
        #print('i,kk',i,kk,max(-mat_B[i,kk], 0), g[i])
        t1=AddVlueOfTwoLists(t1, ScalarMultiplicationForVector(max(mat_B[i,kk], 0), g[i]))
    t2=[]
    for i in range(n):
        t2.append(0)
        
    for j in [0..n-1]:
        bj = []
        for i in range(n):
            bj.append(mat_B0[i,j])
        #print('j,kk',j,kk,max(mat_C[j,kk], 0), bj)
        t2=AddVlueOfTwoLists(t2, ScalarMultiplicationForVector(max(mat_B[n+j,kk], 0), bj))
        
    
    r1=ScalarMultiplicationForVector(-1, g[kk])
    r1=AddVlueOfTwoLists(r1, t1)
    r1=SubtractVlueOfTwoLists(r1, t2)
    
    r[kk]=r1
    
    return r



def matrixMutation_rectangular(mat,  k):  # mutate at k
    a,b=mat.nrows(), mat.ncols()
    r=Matrix(a,b)
    for i in range(a):
        for j in range(b):
            r[i,j]=mat[i,j]
    
    for i in range(a): 
        for j in range(b): 
            
            if k==i or k==j:
                r[i,j]=-mat[i, j]    
            else: 
                r[i, j] = mat[i, j]+1/2*(abs(mat[i,k])*mat[k,j]+mat[i,k]*abs(mat[k,j]))
     
    return r

def evaluation_of_x_coordinates_on_web_matrix_CDFL(M,k,n):
#     k,n=4,8
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
#     M = symbolic_matrix(R,k,n) 
    p1=InitialClusterWithoutNumericalMatrices(k,n)
    ini=initial_cluster_variables_CDFL(k,n)
    
    ma=p1[0]
    r1=p1[1][0]
    r=[]
    for rr2 in range(len(ini)-n):
        r2=ini[rr2]
        for i in range(ma.nrows()):
            if r1[i][1]==r2:
                ii=i
        out=[]
        #print(ma)
        for i in range(ma.nrows()):
            #print(i)
            if ma[ii,i]>0:
                t1=ma[ii,i]*[i]
                out=out+t1
        in_coming=[]
        #print(ma)
        for i in range(ma.nrows()):
            #print(i)
            if ma[ii,i]<0:
                t1=(-ma[ii,i])*[i]
                in_coming=in_coming+t1
        g1=1
        #print(out, in_coming)
        for i in out: 
            g1=g1*PluckerToMinorsPolynomial(M, [[1, r1[i][1]]])
        
        g1=g1.expand()
            
        g2=1
        for j in in_coming:
            g2=g2*PluckerToMinorsPolynomial(M, [[1, r1[j][1]]])
            
        g2=g2.expand()
            
        g1=g2/g1
        #g1=g1/g2
            
        r.append(g1.factor())
    return r


def evaluation_tropical_function_on_vector_CDFL(R, x, k,n,  f1, vv): 
    # f1 =  [[1, x_1_1, x_2_3], [1, x_1_1], [1, x_2_3], [1]]) means min( x11+x23, x11, x23, 0 ) )
    # f1 = [[2, x11]] means 2 + x11
    # f1 = [[2, x11], [3]] means min(2+x11, 0) ???
    # list_x = [x11,x12,x13,x21,x22,x23,x31,x32,x33]
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    
    r=0
    value_x={}
    
    for i in range(k-1):
        for j in range(n-k-1):
            value_x[repr(x[i+1,j+1])]=vv[i*(n-k-1)+j]
    #print(value_x)
    r1=[]
    for i in f1:
        t1=0
        for j in [1..len(i)-1]:
            t1=t1+value_x[repr(i[j])]
        r1.append(t1)
    #print(r1)
    r=min(r1)
    return r

def evaluation_tropical_function_rational_on_vector_CDFL(R, x, k,n,  f1, vv):  # f1 is a rational function
    # f1 =  [[1, x_1_1, x_2_3], [1, x_1_1], [1, x_2_3], [1]]) means min( x11+x23, x11, x23, 0 ) )
    # f1 = [[2, x11]] means 2 + x11
    # f1 = [[2, x11], [3]] means min(2+x11, 0) ???
    # list_x = [x11,x12,x13,x21,x22,x23,x31,x32,x33]
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    
    r2=f1.numerator()
    r3=f1.denominator()
    c1=change_polynomial_to_list(R,r2)
    c2=change_polynomial_to_list(R,r3)
    
    g1=evaluation_tropical_function_on_vector_CDFL(R, x, k,n,   c1, vv)
    g2=evaluation_tropical_function_on_vector_CDFL(R, x, k,n,   c2, vv)
    r=g1-g2
    
    #print(f1)
    #print(c1)
    #print(c2)
    #print(g1,g2,r)

    return r

def evaluation_tropical_function_rational_list_on_vector_CDFL(R, x, k,n,   f1, vv):  # f1 is a list of rational functions
    # f1 =  [[1, x_1_1, x_2_3], [1, x_1_1], [1, x_2_3], [1]]) means min( x11+x23, x11, x23, 0 ) )
    # f1 = [[2, x11]] means 2 + x11
    # f1 = [[2, x11], [3]] means min(2+x11, 0) ???
    # t2=BraidGroupActionSigmaIOnMatrixRenomalizeColumns_symbolic(R,ii,k,n,M)
    # f1 = evaluation_of_x_coordinates_on_web_matrix_DFGK(t2,k,n)
    # list_x = [x11,x12,x13,x21,x22,x23,x31,x32,x33], #  
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    
    r=[]
    for i in f1:
        t1=evaluation_tropical_function_rational_on_vector_CDFL(R, x, k,n,  i, vv)
        r.append(t1)

    return r

# def MinorsToPluckerPolynomial(R,M,l): 
#     #R is a polynomial ring
#     #indices = [(i,j) for i in range(k) for j in range(n)]
#     #R = PolynomialRing(QQ, ['y_{}_{}'.format(i,j) for (i,j) in indices])
#     #y = dict(zip(indices, R.gens()))
#     #for i in range(k):
#     #    for j in range(n):
#     #        var('y'+'_'+str(i)+'_'+str(j))
#     #M=Matrix(R, k, n, R.gens())
#     #l is a polynomial in y_{i,j}
#     # change l = yij ykl + ... to polynomial of Pluckers [[a, [1,2,3],[2,5,8],[4,5,9]], ...]

#     p1=l.expand()
#     #print(p1)
#     #p1=l
#     p2=str(p1).split('+')
#     #print(p2)
#     r=[]
#     j=1
#     while p1!=0 and j<=len(p2):
#         t3=p2[-j]
#         t4=change_polynomial_to_list(R, t3)
#         sn=0
#         for e1 in t4:
#             t2=MinorsToPlucker(R,M,e1)  # get a tableau with coefficient
#             if IsSemistandardTableauInMatrix(TableauToMatrix(t2[1]))==1:
#                 r.append(t2)
#                 #print(t2)
#                 t1=PluckerToMinors(M, t2)
#                 p1=(p1-t1).expand()
#                 p2=str(p1).split('+')
#                 j=1
#                 sn=1
#                 break
#         if sn==0:
#             j=j+1
            
#    return r 

def sub_matrix_general2(R, M, c1, c2):
    m,n=len(c1),len(c2)
    r=Matrix(R, m, n)
    for i in range(m):
        for j in range(n):
            #print(i,j,c1[i],c2[j])
            r[i,j]=M[c1[i]-1, c2[j]-1]
    return r

def ContentOfTableauObtainedFromBraidGroupActionPlucker(R, ii, p, k,n, M): # p is a plucker
    #print('M', M)
    r1=BraidGroupActionSigmaIOnMatrixRenomalizeColumns_symbolic(R,ii,k,n, M)
    #print(r1)
    r2=det(sub_matrix_general(r1,ListAToN(1,k),p))
    #print(r2)
    t1=str(r2).split('+')[0]
    # print(t1)
    r3=change_polynomial_to_list(R, t1)
    r=[]
    for j in r3[0]:
        t1=index_of_yij(j) # take indices of yij
        if t1!=[]:
            r.append(t1[1]+1)
    return r

def BraidGroupActionSigmaI_inverse_OnWindow_symbolic(R,i1, k, m,jj,l, LL): # sigma_i^{-1}, l=[v1,v2,...,vd], r=[v1,v2,...,v_{i-1},v_{i+1},w_1,v_{i+2},...,v_d], l1= l + ... + beginning of original list 
    i=i1-1
    
    d=len(l)
    r=l  
    r=removeAnElementInList(i+1,r)
    r.insert(i, 0)
    #print(r)
    
    L=ReverseList(LL)
    
    #print('L',L)
    
    #print(len(L), (m-jj+1)*d-i+2, (m-jj+1)*d-i+k+3)
    
    t1=[] 
    for j in [(m-jj)*d-i-2..(m-jj)*d-i+k-2]:
        if j!=(m-jj)*d-i-1: 
            #print('L[j]', L[j])
            t1.append(L[j])
    t1=ReverseList(t1)
    
    #print('t1',t1) 
    #print(TableauToMatrix_symbolic(R,t1))
    m1=det(TableauToMatrix_symbolic(R,t1))
    
    #print(len(L), (m-jj+1)*d-i+1, (m-jj+1)*d-i+k)
    
    #print(m1)
    
    t1=[]  
    
    for j in [(m-jj)*d-i-1..(m-jj)*d-i+k-2]:  
        t1.append(L[j])  
        
    t1=ReverseList(t1)
    m2=det(TableauToMatrix_symbolic(R,t1))
    #print(t1)
    #print(m2)
    w1=[]
    for j in [0..k-1]: 
        #w1.append(m1*l[i][j]-m2*l[i+1][j]) 
        w1.append(m1*l[i][j]-m2*l[i+1][j]) 
    #print(w1)
    r[i]=w1   
    
    return r

def BraidGroupActionSigmaI_inverse_OnMatrixRenomalizeColumns_symbolic(R,i,k,n, M): 

    d=gcd(k,n)
    m=n/d
    l=MatrixToTableauTakeColumns(M)
    #print(l)

    r1=[]
    t1=[]

    for j in [0..n-1]:
        if (j+1) % d !=0: 
            t1.append(l[j])
        else:
            t1.append(l[j])
            r1.append(t1)
            t1=[] 
    #print(r1)
    
    l1=UnionOfLists(l,l)
    
    #print(l1)
    
    r2=[] 
    for j in [0..len(r1)-1]: 
        #print(i,k,m,j,r1[j])
        t1=BraidGroupActionSigmaI_inverse_OnWindow_symbolic(R,i,k, m,j,r1[j], l1)
        r2.append(t1) 
    r=TableauToMatrix_symbolic(R,UnionOfListsMany(r2))

    return r

def BraidGroupActionSigmaIOnGVector_inverse2(ii,g,k,n): # sigma_ii on g
    indices = [(i,j) for i in range(k+1) for j in range(n+1)]
    R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
    x = dict(zip(indices, R.gens()))
    M = web_matrix_DFGK(R,x,k,n)
    t2=BraidGroupActionSigmaI_inverse_OnMatrixRenomalizeColumns_symbolic(R,ii,k,n,M)
    t3=evaluation_of_x_coordinates_on_web_matrix_DFGK(t2,k,n)
    t4=evaluation_tropical_function_rational_list_on_vector(R, x, k,n,  t3, g)
    
    r=t4
    return r 

def mutation_g_vector_FZ(kk, g, mat_B): # g is the list of initial g vectors, mutate at vertex kk (the vertices are labelled by 0, 1, 2, ...), mat_B0 is the initial exchange matrix

    r=copy.deepcopy(g)
    r0=copy.deepcopy(r[kk])
    r1=copy.deepcopy(r[kk])
    
    for j in [0..len(r1)-1]:
        if j==kk:
            r1[j]=-r0[j]
        else:
            r1[j]=r0[j]+max(mat_B[j,kk], 0)*r0[kk]-mat_B[j,kk]*min(r0[kk],0)
    
    r[kk]=r1
    
    return r




def BraidGroupActionSigmaI_inverse_OnPluckerSet_symbolic_partial(R,ii, k,n, M, l): # l is a set of pl = [1,2,3,8], return a dictionary d['[1,2,3,8]'] is the sigma_ii of [1,2,3,8]
#     k,n=4,8
#     n1,n2=1,n
#     R=define_ring_with_variables_that_have_negative_indices('x', k,n1-1,n2)[0]
    
    r={}
    #print('l',l)
    for i in l:
        #print('i',i)
        t1=BraidGroupActionSigmaI_inverse_OnPlucker_symbolic(R,ii, i, k,n, M)
        #print(i,t1)
        r[repr(i)]=t1
    return r

def BraidGroupActionSigmaI_inverse_OnPluckerPolynomial_symbolic_NTimes2(R,ii, p, k,n, M,N): # p = [[1,[[1,2,3,8],[2,3,4,8]]], [1,[[1,2,3,8],[2,3,4,8]]]]
    l=[]
    for i in p:
        l=l+i[1]
    
    sigma_i_of_plucker_Set=BraidGroupActionSigmaI_inverse_OnPluckerSet_symbolic_partial(R,ii, k,n, M,l) 
    #print(sigma_i_of_plucker_Set)
    r=p # N=0
    if N>0:
        for i in range(N):
            r=BraidGroupActionSigmaIOnPluckerPolynomial_symbolic(R,ii, r, k,n, M, sigma_i_of_plucker_Set)
    
    return r 

def matrixMutation_C_matrix(mat, mat_B, k):  # mutate at k
    size=mat.nrows()
    r=Matrix(size,size)
    for i in range(size):
        for j in range(size):
            r[i,j]=mat[i,j]
    
    for i in range(size): 
        for j in range(size): 
            
            if j==k:
                r[i,j]=-mat[i, j]    
            else: 
                r[i, j] = mat[i, j] - max( -mat[i,k], 0 )*mat_B[k,j] + mat[i,k]*max(-mat_B[k,j], 0)
     
    return r

def initial_g_vector_DFGK_Gr48(): # only mutable part
    m=9
    r=[]
    for i in range(m):
        t1=[]
        for j in range(m):
            if j!=i:
                t1.append(0)
            else:
                t1.append(1)
        r.append(t1)
    return r

def SubtractVlueOfTwoLists(a,b):
    b1=[-i for i in b]
    r=AddVlueOfTwoLists(a,b1)
    return r

def ScalarMultiplicationForVector(k, a):
    r=[]
    for i in a:
        r.append(k*i)
    return r

def mutation_g_vector_DFGK(kk, g, mat_B, mat_C, mat_B0): # g is the list of initial g vectors, mutate at vertex kk (the vertices are labelled by 0, 1, 2, ...), mat_B0 is the initial exchange matrix

    r=copy.deepcopy(g)
    n=len(g)
    t1=[]
    for i in range(n):
        t1.append(0)
    for i in [0..n-1]:
        #print('i,kk',i,kk,max(-mat_B[i,kk], 0), g[i])
        t1=AddVlueOfTwoLists(t1, ScalarMultiplicationForVector(max(-mat_B[i,kk], 0), g[i]))
    t2=[]
    for i in range(n):
        t2.append(0)
        
    for j in [0..n-1]:
        bj = []
        for i in range(n):
            bj.append(mat_B0[i,j])
        #print('j,kk',j,kk,max(mat_C[j,kk], 0), bj)
        t2=AddVlueOfTwoLists(t2, ScalarMultiplicationForVector(max(mat_C[j,kk], 0), bj))
        
    
    r1=ScalarMultiplicationForVector(-1, g[kk])
    r1=AddVlueOfTwoLists(r1, t1)
    r1=AddVlueOfTwoLists(r1, t2)
    
    r[kk]=r1
    
    return r

def compute_highest_weight_tableau(a1, k,n,mi,ma): # a1=[[1,[[2,3,4,6],[3,4,5,7]]], ...]
    r=[]
    if a1!=[]:
        r0=a1[0][1]
        e1=TableauxWithSameContentAsT_new(r0)
        p1=[]
        for i in [1..len(e1)]:
            M1=randomMatrix(k,n,mi,ma)
            g1=PluckerToMinorsPolynomial(M1, a1) 
            p1.append([M1, g1])
        e2=ComputeChTInTermsOfNonCrossingBasis([ e1, p1 ])
        g1=HighestWeightTableauInPolynomialOfTableaux(e2, 'A', k-1)
        g2=RemoveTrivialTableau(SortRowsOfTableau(g1[1]), k,n)
        r=[g2, e2]
    return r

def TakeExponentsOfMonomial(R,m, l): # l = [x11, x12, x21, x22, x31, x32] order of variables
#    k,n=4,8
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    r1=change_polynomial_to_list(R,m)
    r2=r1[0]
    r=[]
    for i in l:
        t1=0
        for j in r2:
            if i==j:
                t1=t1+1
        r.append(t1)
    return r

def TakeExponentsOfMonomial_list(m, l): # m = [2, x11, x12]  l = [x11, x12, x21, x22, x31, x32] order of variables
#    k,n=4,8
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    
    r2=m
    r=[]
    for i in l:
        t1=0
        for j in r2:
            if i==j:
                t1=t1+1
        r.append(t1)
    return r

def TakeExponentsOfPolynomial(R,p, l): # l = [x11, x12, x21, x22, x31, x32] order of variables
#    k,n=4,8
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    r=[]
    p1=change_polynomial_to_list(R,p)
    for i in p1:
        r.append(TakeExponentsOfMonomial_list(i,l))
    
    return r

def TakeExponentsOfMonomialLaurent(R,m, l): # l = [x11, x12, x21, x22, x31, x32] order of variables
#    k,n=4,8
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    r1=m.numerator()
    r2=m.denominator()
    t1=TakeExponentsOfMonomial(R,r1,l)
    t2=TakeExponentsOfMonomial(R,r2,l)
    r=[]
    for i in range(len(t1)):
        r.append(t1[i]-t2[i])
    
    return r

def TakeExponentsOfPolynomialLaurent(R,p, l): # l = [x11, x12, x21, x22, x31, x32] order of variables
#    k,n=4,8
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    r=[]
    p1=TakeExponentsOfPolynomial(R,p.numerator(),l)
    p2=TakeExponentsOfPolynomial(R,p.denominator(),l)[0]
    for i in p1:
        t1=[]
        for j in range(len(i)):
            t1.append(i[j]-p2[j])
        r.append(t1)
    
    return r

def compute_inverse_map_of_rational_functions(l,k, n): # l is a list of rational function in xij
    Xs = Matrix([[var("x_%s_%s"%(u, v), latex_name="x_{%s,%s}"%(u, v))
              for v in range(1, k)]
             for u in range(1, n-k)])
    Ys = Matrix([[var("y_%s_%s"%(u, v), latex_name="y_{%s,%s}"%(u, v))
                  for v in range(1, k)]
                 for u in range(1, n-k)])
    #print(Xs, Ys)
    va=Xs.transpose().list()  # if use mormal order, Xs.list()
    a1=Ys.transpose().list()  
    
    #print('l', l)
    #print('va', va)
    
    a3 = list(map(lambda a, b:a-b, l, a1))
    
    #print('a3',a3)
    
    Sol = solve(a3, va, solution_dict=True)
    
    #print(Sol)
    for i in Sol:
        t1=list(i.values())
        if IsInList(0, t1)==0:
            a5=t1
            break
    r=a5
    return r

def evaluation_tropical_function_rational_list_on_vector_inverse(R, x, k,n,   f0, vv):  # f1 is a list of rational functions
    # f1 =  [[1, x_1_1, x_2_3], [1, x_1_1], [1, x_2_3], [1]]) means min( x11+x23, x11, x23, 0 ) )
    # f1 = [[2, x11]] means 2 + x11
    # f1 = [[2, x11], [3]] means min(2+x11, 0) ???
    # t2=BraidGroupActionSigmaIOnMatrixRenomalizeColumns_symbolic(R,ii,k,n,M)
    # f0 = evaluation_of_x_coordinates_on_web_matrix_DFGK(t2,k,n)
    # list_x = [x11,x21,x31,x12,x22,x32,x13,x23,x33], # (k-1) times (n-k-1)
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    
    r=[]
    
    f1=compute_inverse_map_of_rational_functions(f0,k, n)           # compute inverse map of f0
    
    indices = [(i,j) for i in range(k) for j in range(n)]
    R1 = PolynomialRing(QQ, ['y_{}_{}'.format(i,j) for (i,j) in indices])
    y = dict(zip(indices, R1.gens()))
    
    for i in f1:
        t1=evaluation_tropical_function_rational_on_vector(R1, y, k,n,  i, vv)
        r.append(t1)

    return r

def BraidGroupActionSigmaIOnGVector(ii,g,k,n): # sigma_ii on g
    indices = [(i,j) for i in range(k+1) for j in range(n+1)]
    R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
    x = dict(zip(indices, R.gens()))
    M = web_matrix_DFGK(R,x,k,n)
    t2=BraidGroupActionSigmaIOnMatrixRenomalizeColumns_symbolic(R,ii,k,n,M)
    t3=evaluation_of_x_coordinates_on_web_matrix_DFGK(t2,k,n)
    t4=evaluation_tropical_function_rational_list_on_vector(R, x, k,n,  t3, g)
    
    r=t4
    return r

def BraidGroupActionSigmaIOnGVector_inverse(ii,g,k,n): # sigma_ii^(-1) on g, maybe this is sigma_ii and the other is sigma_ii^(-1)
    indices = [(i,j) for i in range(k+1) for j in range(n+1)]
    R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
    x = dict(zip(indices, R.gens()))
    M = web_matrix_DFGK(R,x,k,n)
    t2=BraidGroupActionSigmaIOnMatrixRenomalizeColumns_symbolic(R,ii,k,n,M)
    t3=evaluation_of_x_coordinates_on_web_matrix_DFGK(t2,k,n)
    #print('t3',t3)
    t4=evaluation_tropical_function_rational_list_on_vector_inverse(R, x, k,n,   t3, g)
    r=t4
    return r 

def evaluation_tropical_function_on_vector(R, x, k,n,  f1, vv): 
    # f1 =  [[1, x_1_1, x_2_3], [1, x_1_1], [1, x_2_3], [1]]) means min( x11+x23, x11, x23, 0 ) )
    # f1 = [[2, x11]] means 2 + x11
    # f1 = [[2, x11], [3]] means min(2+x11, 0) ???
    # list_x = [x11,x21,x31,x12,x22,x32,x13,x23,x33], # (k-1) times (n-k-1)
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    
    r=0
    value_x={}
    
    for i in range(k-1):
        for j in range(n-k-1):
            value_x[repr(x[j+1,i+1])]=vv[i*(n-k-1)+j]
    #print(value_x)
    r1=[]
    for i in f1:
        t1=0
        for j in [1..len(i)-1]:
            t1=t1+value_x[repr(i[j])]
        r1.append(t1)
    #print(r1)
    r=min(r1)
    return r

def evaluation_tropical_function_rational_on_vector(R, x, k,n,  f1, vv):  # f1 is a rational function
    # f1 =  [[1, x_1_1, x_2_3], [1, x_1_1], [1, x_2_3], [1]]) means min( x11+x23, x11, x23, 0 ) )
    # f1 = [[2, x11]] means 2 + x11
    # f1 = [[2, x11], [3]] means min(2+x11, 0) ???
    # list_x = [x11,x21,x31,x12,x22,x32,x13,x23,x33], # (k-1) times (n-k-1)
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    
    r2=f1.numerator()
    r3=f1.denominator()
    c1=change_polynomial_to_list(R,r2)
    c2=change_polynomial_to_list(R,r3)
    
    g1=evaluation_tropical_function_on_vector(R, x, k,n,   c1, vv)
    g2=evaluation_tropical_function_on_vector(R, x, k,n,   c2, vv)
    r=g1-g2
    
    #print(f1)
    #print(c1)
    #print(c2)
    #print(g1,g2,r)

    return r

def evaluation_tropical_function_rational_list_on_vector(R, x, k,n,   f1, vv):  # f1 is a list of rational functions
    # f1 =  [[1, x_1_1, x_2_3], [1, x_1_1], [1, x_2_3], [1]]) means min( x11+x23, x11, x23, 0 ) )
    # f1 = [[2, x11]] means 2 + x11
    # f1 = [[2, x11], [3]] means min(2+x11, 0) ???
    # t2=BraidGroupActionSigmaIOnMatrixRenomalizeColumns_symbolic(R,ii,k,n,M)
    # f1 = evaluation_of_x_coordinates_on_web_matrix_DFGK(t2,k,n)
    # list_x = [x11,x21,x31,x12,x22,x32,x13,x23,x33], # (k-1) times (n-k-1)
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    
    r=[]
    for i in f1:
        t1=evaluation_tropical_function_rational_on_vector(R, x, k,n,  i, vv)
        r.append(t1)

    return r

def evaluation_of_x_coordinates_on_web_matrix_DFGK(M,k,n):
#     k,n=4,8
#     indices = [(i,j) for i in range(k) for j in range(n)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
#     M = symbolic_matrix(R,k,n) 
    p1=InitialClusterWithoutNumericalMatrices(k,n)
    ini=initial_cluster_variables_DFGK(k,n)
    
    ma=p1[0]
    r1=p1[1][0]
    r=[]
    for rr2 in range(len(ini)-n):
        r2=ini[rr2]
        for i in range(ma.nrows()):
            if r1[i][1]==r2:
                ii=i
        out=[]
        #print(ma)
        for i in range(ma.nrows()):
            #print(i)
            if ma[ii,i]>0:
                t1=ma[ii,i]*[i]
                out=out+t1
        in_coming=[]
        #print(ma)
        for i in range(ma.nrows()):
            #print(i)
            if ma[ii,i]<0:
                t1=(-ma[ii,i])*[i]
                in_coming=in_coming+t1
        g1=1
        #print(out, in_coming)
        for i in out: 
            g1=g1*PluckerToMinorsPolynomial(M, [[1, r1[i][1]]])
        g2=1
        for j in in_coming:
            g2=g2*PluckerToMinorsPolynomial(M, [[1, r1[j][1]]])
        #g1=g2/g1
        g1=g1/g2
            
        r.append(g1.factor())
    return r

def web_matrix_DFGK(R,x, k, n): # need to double check
#     k,n=4,8
#     indices = [(i,j) for i in range(k+1) for j in range(n+1)]
#     R = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
#     x = dict(zip(indices, R.gens()))
    r1=Matrix(SR, k,n)
    r2=Matrix(SR, k,n-k)

    for i in range(k):
        r1[i,i]=1
    for i in range(k):
        r1[i,k]=1
    
    for i1 in range(k):
        i=i1+1
        for j1 in range(n-k):
            j=j1+1
            t1=0
            for p1 in itertools.combinations_with_replacement(ListAToN(0,j-1), k-i):
                la=ReverseList(sorted(p1))
                #print(la)
                t2=1
                #print(k-i, la)
                for a1 in [1..k-i]:
                    a=a1-1
                    for b in [1..la[a]]:
                        t2=t2*x[a1,b]
                        #print(t2)
                t1=t1+t2
            t1=(-1)^(k+i)*t1
            r2[i1,j1]=t1
    #print(r2)
    
    for i in [0..k-1]:
        for j in [k..n-1]:
            r1[i,j]=r2[i,j-k]
    r=r1
    return r 

def ModuleToTableauSLk(Y,l,k): # SLk
    t1=ModuleToMultiSegment(Y,l)
    r=MultisegmentToTableauSLk(k,t1)
    return r

def MultisegmentToTableauSLkOneSegment(k, m): # SLk
 
    a=m[0]
    b=m[1]
    t1=ListAToN(1, -a)
    t1.append(k-b)
    r=[t1]

    return r


def MultisegmentToTableauSLk(k, l): # SLk

    r=[]
    if l!=[]:
        for i in l:
            t1=MultisegmentToTableauSLkOneSegment(k, i)
            #print('t1', t1)
            r=UnionOfTwoTableauxSLk(r, t1)
            
        #print('r', r)
             
        r=RemoveTrivialTableauSLk(r,k) 
        #print(r)
    return r

def PluckersWhichAreOneSLk(k): # SLk 1, 12, 123, ..., are trivial
    r=[]
    for i in [1..k]:
        t1=[]
        for j in [1..i]:
            t1.append(j)
        r.append(t1)
    return r

def IsTableauAFactorOfBSLk(a,b):
    
    r1=a
    r2=b

    r=1;
    for i in range(len(r1)):
        if IsSubsetAsMultiset(r1[i], r2[i])==0:
            r=0
            break
    return r

def RemoveTrivialTableauSLk(a1,k): # SLk
    r=a1
    t2=PluckersWhichAreOneSLk(k);
    sn=1
    while sn!=0:
        r1=r;
        for i in range(len(t2)):
            #print(r, t2[i])
            if r==[]:
                break
            elif IsTableauAFactorOfBSLk([t2[i]], r)==1:
                p1=TableauDivisionSLkResultInRows(r, [t2[i]])
                #print('p1',p1)
                if IsSemistandardTableauInRows(p1)==1:
                    r=TableauDivisionSLk(r, [t2[i]])
                
        if r1==r or r==[]:
            sn=0
            break
        else:
            sn=1
        
    
    return r

def TableauToModuleSLk(Y,k,l): # l is a tableau T, l[i] is column of i of T, SLk
    
    t1=TableauToFundamentalTableauxSLk(Y,l)
    #print(t1)
 
    r=[]
    for i in t1:
        r.append(FundamentalTableauToYiaSLk(i,k)) 

    return r

def FundamentalTableauToYiaSLk(a1,k): # SLk, T_{l,a}, length l, last entry a, T_{l,a}=T^{l+k-a,l} \sim Y_{l+k-a,k-a-l}, weight of T_{l,a} is omiga_(l+k-a)
 
    r=[]
    rank=k-1
    n=rank
    for i in [1..n]:
        r=append(r,0)
        
    a=sorted(a1)
    l=len(a)
    t1=a[len(a)-1]
    r=[l+k-t1,k-t1-l]
    
    return r

def IsPartition1GreaterThanOrEqualPartition2(p1,p2): # p1,p2 are partitions
    l1=p1
    l2=p2

    n=max(len(p1), len(p2))
    for i in [len(p1)..n]:
        l1.append(0) 
    for i in [len(p2)..n]: 
        l2.append(0) 

    r=1 # p1 >= p2

    for i in [1..n]: 
        t1=0
        t2=0
        for j in [0..i-1]:
            t1=t1+l1[j]
            t2=t2+l2[j] 
        if t1<t2: 
            r=0 # p1 is not greater or equal to p2
            break 
    return r

def comparePartitions(p1,p2): # p1,p2 are partitions
    t1=IsPartition1GreaterThanOrEqualPartition2(p1,p2)
    t2=IsPartition1GreaterThanOrEqualPartition2(p2,p1)

    if t1==1: 
        r=1  # p1>=p2
    elif t2==1: 
        r=-1 # p1<=p2
    else:
        r=0 # not comparable 
    return r

def IsTableauxP1GreaterEqualP2SLkU(P1,P2): # P1,P2 are tableaux

    n=-1000
    for i in UnionOfListsMany(P1):
        if n<i:
            n=i
    for i in UnionOfListsMany(P2):
        if n<i:
            n=i 

    r=1 # P1>=P2

    for a in [1..n]: 
        t1=TableauRestrictToElementsLessOrEqualAToPartition(P1,a)
        t2=TableauRestrictToElementsLessOrEqualAToPartition(P2,a)
        r1=comparePartitions(t1,t2)
        if r1==0 or r1==-1: 
            r=0 # P1 is not greater or equal to P2
            break 

    return r

def TableauToMatrixTakeRowsSLkMoreGeneral(a): # send a tableau to a matrix, a[i] are rows
    if a==[]:
        r=[]
    else:
        m=len(a)  # number of rows
        n=len(a[0])
        for i in [1..len(a)-1]:
            if n<len(a[i]):
                n=len(a[i])
        
        r=Matrix(m,n)
        for i in range(m):
            for j in range(n):
                if j<len(a[i]):
                    r[i,j]=a[i][j]
                else:
                    r[i,j]=-10000
    return r

def TableauToMatrixTakeColumnsSLkMoreGeneral(a): # send a tableau to a matrix, a[i] are columns
    
    if a==[]:
        r=[]
    else:
        n=len(a)  # number of columns
        m=len(a[0])
        for i in [1..len(a)-1]:
            if m<len(a[i]):
                m=len(a[i])
        
        r=Matrix(m,n)
        for i in range(m):
            for j in range(n):
                if i<len(a[j]):
                    r[i,j]=a[j][i]
                else:
                    r[i,j]=-10000
    return r 

def TableauToMatrixTakeRowsSLkListMoreGeneral(a): # SLk, a[i] are columns of tableau
    r=[]
    for i in a:
        r.append(TableauToMatrixTakeRowsSLkMoreGeneral(i))
    return r

def TableauToMatrixTakeColumnsSLkListMoreGeneral(a): # SLk, a[i] are columns of tableau
    r=[]
    for i in a:
        r.append(TableauToMatrixTakeColumnsSLkMoreGeneral(i))
    return r

def TableauToPartition(P):

    r=[]
    t1=TableauToMatrixTakeColumnsSLkMoreGeneral(P)
    for i in [0..t1.nrows()-1]: 
        t2=0
        for j in [0..t1.ncols()-1]: 
            if t1[i,j]!=-10000:
                t2=t2+1 
        r.append(t2) 
    return r

def TableauRestrictToElementsLessOrEqualAToPartition(P,a):

    t1=TableauRestrictToElementsLessOrEqualA(P,a)
    r=TableauToPartition(t1)

    return r

def compareWeightsTableauxSLkU(P1,P2): # P1,P2 are tableaux

    t1=IsTableauxP1GreaterEqualP2SLkU(P1,P2)
    t2=IsTableauxP1GreaterEqualP2SLkU(P2,P1)

    if t1==1:  
        r=1 # P1>=P2
    elif t2==1: 
        r=-1 # P1<=P2
    else:
        r=0 # P1, P2 not comparable
     
    return r

def TableauRestrictToElementsLessOrEqualA(P,a):

    r=[]
    for i in P: 
        t1=[]
        for j in i: 
            if j<=a:
                t1.append(j) 
        r.append(t1) 
    return r

# mutation for base affine space, C[SL_k]^U, and C[N]

def InitialCluster_SLk(kk): # initial cluster for SL_kk, only works for kk <= 10

    # quivers are from Keller's mutation app

    if kk==2: 
        mat=Matrix([[0,-1,1],[1,0,-1],[-1,1,0]]) # SL2
    elif kk==3: 
        mat=Matrix([[0,-1,1,0,0,0],[1,0,-1,-1,1,0],[-1,1,0,0,-1,1],[0,1,0,0,-1,0],[0,-1,1,1,0,-1],[0,0,-1,0,1,0]]) # SL3
    elif kk==4:
        mat=Matrix([[0,-1,1,0,0,0,0,0,0,0],[1,0,-1,-1,1,0,0,0,0,0],[-1,1,0,0,-1,1,0,0,0,0],[0,1,0,0,-1,0,-1,1,0,0],[0,-1,1,1,0,-1,0,-1,1,0],[0,0,-1,0,1,0,0,0,-1,1],[0,0,0,1,0,0,0,-1,0,0],[0,0,0,-1,1,0,1,0,-1,0],[0,0,0,0,-1,1,0,1,0,-1],[0,0,0,0,0,-1,0,0,1,0]]) # SL4
    elif kk==5: 
        mat=Matrix([[0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0],[1,0,-1,-1,1,0,0,0,0,0,0,0,0,0,0],[-1,1,0,0,-1,1,0,0,0,0,0,0,0,0,0],[0,1,0,0,-1,0,-1,1,0,0,0,0,0,0,0],[0,-1,1,1,0,-1,0,-1,1,0,0,0,0,0,0],[0,0,-1,0,1,0,0,0,-1,1,0,0,0,0,0],[0,0,0,1,0,0,0,-1,0,0,-1,1,0,0,0],[0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0,0],[0,0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0],[0,0,0,0,0,-1,0,0,1,0,0,0,0,-1,1],[0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0],[0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0],[0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0],[0,0,0,0,0,0,0,0,-1,1,0,0,1,0,-1],[0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0]]) # SL5
    elif kk==6: 
        mat=Matrix([[0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[1,0,-1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[-1,1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,1,0,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,-1,1,1,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,-1,0,1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,1,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0],[0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0],[0,0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0],[0,0,0,0,0,-1,0,0,1,0,0,0,0,-1,1,0,0,0,0,0,0],[0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,-1,1,0,0,0,0],[0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0],[0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0],[0,0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0],[0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,-1,1],[0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0]]) # SL6
    elif kk==7: 
        mat=Matrix([[0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[1,0,-1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[-1,1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,1,0,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,-1,1,1,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,-1,0,1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,1,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,-1,0,0,1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,-1,1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,-1,1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0]]) # SL7
    elif kk==8: 
        mat=Matrix([[0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[1,0,-1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[-1,1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,1,0,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,-1,1,1,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,-1,0,1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,1,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,-1,0,0,1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0]]) # SL8
    elif kk==9: 
        mat=Matrix([[0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[1,0,-1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[-1,1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,1,0,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,-1,1,1,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,-1,0,1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,1,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,-1,0,0,1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0]]) # SL9
    elif kk==10: 
        mat=Matrix([[0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[1,0,-1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[-1,1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,1,0,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,-1,1,1,0,-1,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,-1,0,1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,1,0,0,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,-1,1,0,1,0,-1,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,-1,0,0,1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,1,0,0,0,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,-1,1,0,0,1,0,-1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,1,0,-1,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,1,0,-1,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0,-1,1,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,-1,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,-1,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,-1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,-1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,-1,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,1,0,-1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,1,0]]) # SL10

    vertices0=[]
    n=kk

    for i in [1..n]: 
        for j in [1..i]: 
            t3=ListAToN(j, j+n-i)
            vertices0.append(t3) 
            
    verticesTableaux=[] # Tableaux are represented by matrices
    for i in [0..len(vertices0)-1]:
        verticesTableaux.append([0, [vertices0[i]], i]) # [vertices0[i]] is an one column tableau
        
    mat1 = mat
    clusterVariables=[] 
    vertices1 = [verticesTableaux, clusterVariables] # vertices1[1] store cluster variables, vertices1[0] store variables on quiver
    r=[mat, vertices1]
    
    return r
    
def UnionOfTwoTableauxSLk(a,b): # a[i], b[i] are columns
    if a==[]:
        r=b
    elif b==[]:
        r=a
    else:
        r1=TransposeTableauSLk(a)
        r2=TransposeTableauSLk(b)
        
        m=max(len(r1), len(r2))
        #print('m,len(r1),len(r2)',m,len(r1),len(r2))
        r3=[]
        for i in [0..m-1]:
            #print(i,len(r1),len(r2))
            if i<len(r1) and i<len(r2):
                t2=r1[i]+r2[i]
                #print('t2',t2)
                t2=sorted(t2)
            elif i>=len(r1) and i<len(r2):
                    t2=sorted(r2[i])
            elif i<len(r1) and i>=len(r2):
                    t2=sorted(r1[i])
            r3.append(t2)
        #print('r3',r3)
        r=TransposeTableauSLk(r3)
    return r

def PowerOfTableaux_SLk(T, n): # T=[[1,2,3],[2,5]]
    if n==1:
        r=T
    elif n>1:
        r=T
        for i in range(n-1):
            r=UnionOfTwoTableauxSLk(r, T)
    return r
    
def computeEquationsForModulesTableaux_SLk(variable2, mat, k, kk): # variable2=(variables on quiver, cluster variables obtained so far)
    variable=variable2[0]
    clusterVariables=variable2[1] 
    size=mat.nrows() 
    newVariable=[]
    newVariable2=[]
    newVariableList=[]
    newVariable2List=[]

    for i in range(size):
        if mat[i, k]>0:
            newVariable=UnionOfTwoTableauxSLk(newVariable, PowerOfTableaux_SLk(variable[i][1], mat[i,k]) )
            for j in range(mat[i,k]):
                newVariableList.append(variable[i][1])
 
    for i in range(size): 
        if mat[i, k]<0:
            newVariable2= UnionOfTwoTableauxSLk( newVariable2, PowerOfTableaux_SLk(variable[i][1], -mat[i,k]) )
            for j in range(-mat[i,k]):
                newVariable2List.append(variable[i][1])
 
    variable[k][0]=variable[k][0]+1
    t1=compareWeightsTableauxSLkU(newVariable, newVariable2)
    
    if t1==1: 
        variable[k][1]=TableauDivisionSLk(newVariable, variable[k][1])
    else:
        variable[k][1]=TableauDivisionSLk(newVariable2, variable[k][1])  
    
    #print(variable[k][1])
    clusterVariables=TableauToMatrixTakeColumnsSLkMoreGeneral(variable[k][1])
    #print('new cluster variable', clusterVariables)
    
    r=[variable, clusterVariables]

    return r

def MatrixToTableauTakeColumnsSLk(a): 
 

    n=a.nrows()
    m=a.ncols()

    r=[]
    for i in [0..m-1]:
        t2=[]
        for j in [0..n-1]:
            if a[j,i]>=0:
                t2=append(t2,a[j,i])
        
        r=append(r, t2)
    
    #r=SemistandardTableau(r)
   
    return r

def ll_perms_SLk(lli, kk, max_column, repeat): # SL_kk
    b1=[]
    IC=InitialCluster_SLk(kk)
    mat1=IC[0]
    vertices1=IC[1]

    ll=list(map(lambda x: x - 1, lli))
    
    #print(IC)
    
    mutationSequence=[]
    for j1 in [1..repeat]: ###why do this?, just to repeat the same sequence of mutations, it will give more cluster variables, without this loop is also ok
        mutationSequence=mutationSequence+ll 
        
    for j in range(len(mutationSequence)): 
        vertices1 = computeEquationsForModulesTableaux_SLk(vertices1, mat1, mutationSequence[j], kk)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 

        if vertices1[1].ncols()>max_column:
            vertices1 = computeEquationsForModulesTableaux_SLk(vertices1, mat1, mutationSequence[j], kk)
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
        else:
            #print(vertices1[1])
            b1.append(MatrixToTableauTakeColumnsSLk(vertices1[1]))

    return b1

def TableauDivisionSLk(a,b): # a[i], b[i] are columns
    r1=TableauDivisionSLkResultInRows(a,b)
        
    r=TransposeTableauSLk(r1)
        
    return r

def TableauDivisionSLkResultInRows(a,b): # a[i], b[i] are columns
    r1=TransposeTableauSLk(a)
    r2=TransposeTableauSLk(b)
    
    r3=[]
    m=max(len(r1),len(r2))
    for i in range(m):
        if i<len(r1) and i<len(r2):
            r3.append(sorted(SetDifferenceListDifference(r1[i],r2[i])))
        elif i<len(r1) and i>=len(r2):
            r3.append(r1[i])
        elif i>=len(r1) and i<len(r2):
            r3=[]
            print('tableau not divisible')
            break
        
    r=r3
        
    return r

def sub_matrix_symbolic(R,M,a,b):
    r=Matrix(SR,len(a),len(b))
    for i in range(len(a)):
        for j in range(len(b)):
            r[i,j]=M[a[i]-1, b[j]-1]
    return r


def PluckerToMinors_symbolic(R,A,l):
    r=l[0]
    for i in l[1]:
        r=r*Minor_symbolic(R,A,ListAToN(1, len(i)), i)
        #print(r)
        r=r.expand().factor()
    return r

def PluckerToMinorsPolynomial_symbolic(R,A,l):
    r=0
    for i in l:
        r=r+PluckerToMinors_symbolic(R,A,i)
    r=r.expand().factor()
    return r


def Minor_symbolic(R,M, rows, cols):
    #r=np.linalg.det(subMatrix(M,rows, cols))
    #r=det(subMatrix(M,rows, cols))
    #r=det(sub_matrix_general(M,rows, cols))
    #r=detSelfDefined(subMatrix(M,rows, cols))
    t1=sub_matrix_symbolic(R,M,rows, cols)
    r=det(t1)
    return r

def TableauxWithSameContentAsT_and_smaller_or_equal_weight_new(T, typ,rank): 
    r1=TableauxWithSameContentAsT_new(T)
    r=ChooseTableauxWithSmallOrEqualWeight(T,r1,typ,rank)
    return r

def TableauxWithSameContentAsT_new(T): 
    if T and T[0]:
        r=list(candidates_new(Counter(sorted(flatten(T))), len(T[0])))
        r1=removeDuplicatesListOfLists(r)
        r=[]
        for i in r1:
            t1=SortRowsOfTableau(i)
            r.append(t1)
        r=removeDuplicatesListOfLists(r)
        return r

@functools.lru_cache()  
def candidates_tableau(content, k, candidate=()): 
    if len(content) < k:
        if len(content) == 0 and candidate:
            yield candidate
        else:
            return
        
    for I in itertools.combinations(content, k):
        # tuples in candidate are in non-decreaseing order
        if candidate and I < candidate[-1]:
            continue
            
        # adding I to candidate is semistandard tableau after reordering rows 
        if is_semistandard_after_reordering(candidate, I)==1:
            yield from candidates_tableau(content - Counter(I), k, candidate + (I,))
            
def is_semistandard_after_reordering(l,J): # l is a tuple, l[i] is (2,3,...), J = (1,3,...)
    r1=[]
    for i in l:
        r1.append(list(i))
    r1.append(list(J))
    r1=SortRowsOfTableau(r1)
    r=IsSemistandardTableauInMatrix(TableauToMatrix(r1))
    return r

def TableauxWithSameContentAsT_in_rows(shape,content,frontline=None): # each result is in terms of rows
    if frontline is None:
        frontline = [0] * len(shape)

    if len(content)==0:
        yield [[0]*s for s in shape]
        return

    e = content[0]
    for row,col in enumerate(frontline):
        if col>=shape[row] or (row>0 and frontline[row-1]<=col):
            continue

        frontline[row] += 1

        for t in TableauxWithSameContentAsT_in_rows(shape,content[1:],frontline):
            if row+1==len(shape) or col>=shape[row+1] or e<t[row+1][col]:
                t[row][col] = e
                yield t

        frontline[row] -= 1
    return

def TableauxWithSameContentAsT(T): # each result is in terms of columns, T[i] is column i
    t1=TransposeTableauSLk(T)
    shape=[]
    for i in t1:
        shape.append(len(i))
    #print(shape)
    content=sorted(flatten(T))
    r1=list(TableauxWithSameContentAsT_in_rows(shape,content,frontline=None))
    #print(r1)
    r1=removeDuplicatesListOfLists(r1)
    r=TransposeOfRectangularTableauList(r1)
    return r

def TransposeOfRectangularTableauList(l):
    r=[]
    for i in l:
        t1=TransposeOfRectangularTableau(i)
        r.append(t1)
    return r

def TableauxWithSameContentAsT_and_smaller_or_equal_weight(T,typ,rank): # each result is in terms of columns, T[i] is column i
    r1=TableauxWithSameContentAsT(T)
    r=ChooseTableauxWithSmallOrEqualWeight(T,r1,typ,rank)
    return r

def MinorsToPluckerPolynomial(R,M,l): 
    #R is a polynomial ring
    #indices = [(i,j) for i in range(k) for j in range(n)]
    #R = PolynomialRing(QQ, ['y_{}_{}'.format(i,j) for (i,j) in indices])
    #y = dict(zip(indices, R.gens()))
    #for i in range(k):
    #    for j in range(n):
    #        var('y'+'_'+str(i)+'_'+str(j))
    #M=Matrix(R, k, n, R.gens())
    #l is a polynomial in y_{i,j}
    # change l = yij ykl + ... to polynomial of Pluckers [[a, [1,2,3],[2,5,8],[4,5,9]], ...]

    p1=l.expand()
    #print(p1)
    #p1=l
    p2=str(p1).split('+')
    #print(p2)
    r=[]
    j=1
    while p1!=0 and j<=len(p2):
        t3=p2[-j]
        t4=change_polynomial_to_list(R, t3)
        sn=0
        for e1 in t4:
            t2=MinorsToPlucker(R,M,e1)  # get a tableau with coefficient
            if IsSemistandardTableauInMatrix(TableauToMatrix(t2[1]))==1:
                r.append(t2)
                #print(t2)
                t1=PluckerToMinors(M, t2)
                p1=(p1-t1).expand()
                p2=str(p1).split('+')
                j=1
                sn=1
                break
        if sn==0:
            j=j+1
            
            
    return r

def MinorsToPlucker(R,M,l):
    #R is a polynomial ring
    #indices = [(i,j) for i in range(k) for j in range(n)]
    #R = PolynomialRing(QQ, ['y_{}_{}'.format(i,j) for (i,j) in indices])
    #y = dict(zip(indices, R.gens()))
    #for i in range(k):
    #    for j in range(n):
    #        var('y'+'_'+str(i)+'_'+str(j))
    #M=Matrix(R, k, n, R.gens())
    #l is a monomial in y_{i,j}
    # change l = [1, yij, ykl,...] to Plucker [a, [1,2,3],[2,5,8],[4,5,9]]
    
    r1=l
    r=[r1[0]]
    r2=[]
    for j in r1:
        t1=index_of_yij(j) # take indices of yij
        if t1!=[]:
            r2.append(t1[1]+1)
    #print(r2)
    r3=[]
    num=len(r2)/k
    for i in range(num):
        t1=[]
        for j in range(k):
            t1.append(r2[j*num+i])
        r3.append(t1)
    
    r3=SortRowsOfTableau(r3)
    
    r.append(r3)

    return r

def IsSemistandardTableauInMatrix(a): 
    r1=MatrixTakeRows(a)
    r=IsSemistandardTableauInRows(r1)
        
    return r



def symbolic_matrix(R,k,n):
    #indices = [(i,j) for i in range(k) for j in range(n)]
    #R = PolynomialRing(QQ, ['y_{}_{}'.format(i,j) for (i,j) in indices])
    #y = dict(zip(indices, R.gens()))
    r = Matrix(R, k, n, R.gens())
    return r

def BraidGroupActionSigmaIOnWindow_symbolic(R,i1, k, l, l1): # l=[v1,v2,...,vd], r=[v1,v2,...,v_{i-1},v_{i+1},w_1,v_{i+2},...,v_d], l1= l + ... + beginning of original list 
    i=i1-1
    r=l
    r=removeAnElementInList(i,r)
    r.insert(i+1, 0)
    #print(r)
    
    t1=[]
    for j in [i..i+k]:
        if j!=i+1: 
            t1.append(l1[j])
    
    #print(t1)
    #print(TableauToMatrix_symbolic(R,t1))
    m1=det(TableauToMatrix_symbolic(R,t1))
    
    #print(m1)
    
    t1=[]
    for j in [i+1..i+k]:  
        t1.append(l1[j])  
    m2=det(TableauToMatrix_symbolic(R,t1))
    #print(t1)
    #print(m2)
    w1=[]
    for j in [0..k-1]: 
        w1.append(m1*l[i+1][j]-m2*l[i][j]) 
    #print(w1)
    r[i+1]=w1
    
    return r

def TableauToMatrix_symbolic(R,a): # R = PolynomialRing(QQ, ['y_{}_{}'.format(i,j) for (i,j) in indices])
    m=len(a)
    n=len(a[0])
    F=R.fraction_field()
    r=Matrix(F, n, m)
    #print(r)
    for i in range(n):
        for j in range(m):
            r[i,j]=a[j][i]
    return r
              
def index_of_yij(a): # a = y[i,j]=y_i_j
    r=[int(s) for s in str(a).split('_')[1:]] # take indices of yij
    return r

def BraidGroupActionSigmaIOnMatrixRenomalizeColumns_symbolic(R,i,k,n, M): 

    d=gcd(k,n)
    l=MatrixToTableauTakeColumns(M)
    #print(l)

    r1=[]
    t1=[]

    for j in [0..n-1]:
        if (j+1) % d !=0: 
            t1.append(l[j])
        else:
            t1.append(l[j])
            r1.append(t1)
            t1=[] 
    #print(r1)
    r2=[]
    for j in [0..len(r1)-1]: 
        l1=[]
        for i1 in [j..len(r1)-1]: 
            l1=UnionOfLists(l1, r1[i1]) 
        l1=UnionOfLists(l1, l)
        t1=BraidGroupActionSigmaIOnWindow_symbolic(R,i,k, r1[j], l1)
        r2.append(t1) 
    r=TableauToMatrix_symbolic(R,UnionOfListsMany(r2))

    return r

def change_polynomial_to_list(R, f):
    r=[[c] + sum([[SR(R.gen(i))]*e[i] for i in range(R.ngens()) if e[i] != 0], []) for (e,c) in R(f).dict().items()]
    return r



def ContentOfTableauObtainedFromBraidGroupActionTableau(R, ii, T, k,n, M): # T is a tableau, T[i] is a column
    r=[]
    for pl in T:
        t1=ContentOfTableauObtainedFromBraidGroupActionPlucker(R, ii, pl, k,n, M)
        r=r+t1
    
    return r



def BraidGroupActionSigmaIOnPluckerSet_symbolic(R,ii, k,n, M): # l is a set of pl = [1,2,3,8], return a dictionary d['[1,2,3,8]'] is the sigma_ii of [1,2,3,8]
#     k,n=4,8
#     n1,n2=1,n
#     R=define_ring_with_variables_that_have_negative_indices('x', k,n1-1,n2)[0]
    
    r={}
    l=NchooseM(ListAToN(1,n),k)
    for i in l:
        t1=BraidGroupActionSigmaIOnPlucker_symbolic(R,ii, i, k,n, M)
        #print(i,t1)
        r[repr(i)]=t1
    return r


def MultiplyTwoPolynomialsLists2(a,b):

    r=[]
    for i in a:
        for j in b:
            temp1=i[0]*j[0]
            temp2=sorted(i[1]+j[1])
            r.append([temp1,temp2])
    return r

def BraidGroupActionSigmaIOnPluckerPolynomial_symbolic(R,ii, p, k,n, M, sigma_i_of_plucker_Set): 
    # p = [[1,[[1,2,3,8],[2,3,4,8]]], [1,[[1,2,3,8],[2,3,4,8]]]]
    #k,n=4,8
    #indices = [(i,j) for i in range(k) for j in range(n)]
    #R = PolynomialRing(QQ, ['y_{}_{}'.format(i,j) for (i,j) in indices])
    #y = dict(zip(indices, R.gens()))
    #ii=1
    #M = Matrix(R, k, n, R.gens())
    #sigma_i_of_plucker_Set=BraidGroupActionSigmaIOnPluckerSet_symbolic(R,ii, k,n, M)
    r=[]
    for j in p:
        t1=[[j[0], []]]
        #print('t1',t1)
        for j1 in j[1]:
            #t2=BraidGroupActionSigmaIOnPlucker_symbolic(R,ii, j1, k,n, M)
            t2=sigma_i_of_plucker_Set.get(repr(j1))
            t2=polynomial_use_tuple_to_polynomial_use_list(t2)
            #print(t2)
            t1=MultiplyTwoPolynomialsLists2(t1,t2)
        r=r+t1
    #r=combineLikeTerms3(r)
    return r

def BraidGroupActionSigmaIOnPluckerSet_symbolic_partial(R,ii, k,n, M, l): # l is a set of pl = [1,2,3,8], return a dictionary d['[1,2,3,8]'] is the sigma_ii of [1,2,3,8]
#     k,n=4,8
#     n1,n2=1,n
#     R=define_ring_with_variables_that_have_negative_indices('x', k,n1-1,n2)[0]
    
    r={}
    for i in l:
        t1=BraidGroupActionSigmaIOnPlucker_symbolic(R,ii, i, k,n, M)
        #print(i,t1)
        r[repr(i)]=t1
    return r

def BraidGroupActionSigmaIOnPluckerPolynomial_symbolic_NTimes2(R,ii, p, k,n, M,N): # p = [[1,[[1,2,3,8],[2,3,4,8]]], [1,[[1,2,3,8],[2,3,4,8]]]]
    l=[]
    for i in p:
        l=l+i[1]
    
    sigma_i_of_plucker_Set=BraidGroupActionSigmaIOnPluckerSet_symbolic_partial(R,ii, k,n, M,l) 
    #print(sigma_i_of_plucker_Set)
    r=p # N=0
    if N>0:
        for i in range(N):
            r=BraidGroupActionSigmaIOnPluckerPolynomial_symbolic(R,ii, r, k,n, M, sigma_i_of_plucker_Set)
    
    return r

def define_ring_with_variables_that_have_negative_indices(Y,k,n1,n2): # define ring with variables which have negative indices
    #k,n1,n2=4,-100,2, Y='x'
    indices = [(i,j) for i in range(1,k+1) for j in range(n1, n2)]
    R = PolynomialRing(QQ, [(Y+'_{}_{}').format(i, str(j).replace('-', 'm')) for (i,j) in indices])
    Y = dict(zip(indices, R.gens()))
    r=[R, Y]
    return r

def define_fraction_field_with_variables_that_have_negative_indices(Y,k,n1,n2): # define ring with variables which have negative indices
    #k,n1,n2=4,-100,2, Y='x'
    indices = [(i,j) for i in range(1,k+1) for j in range(n1, n2)]
    R = PolynomialRing(QQ, [(Y+'_{}_{}').format(i, str(j).replace('-', 'm')) for (i,j) in indices])
    Y = dict(zip(indices, R.gens()))
    F=R.fraction_field()
    r=[F, Y]
    return r


def BraidGroupActionSigmaIOnPluckerPolynomial_symbolic_NTimes(R,ii, p, k,n, M,N): # p = [[1,[[1,2,3,8],[2,3,4,8]]], [1,[[1,2,3,8],[2,3,4,8]]]]
    sigma_i_of_plucker_Set=BraidGroupActionSigmaIOnPluckerSet_symbolic(R,ii, k,n, M) 
    #print(sigma_i_of_plucker_Set)
    r=p # N=0
    if N>0:
        for i in range(N):
            r=BraidGroupActionSigmaIOnPluckerPolynomial_symbolic(R,ii, r, k,n, M, sigma_i_of_plucker_Set)
    
    return r

def polynomial_use_tuple_to_polynomial_use_list(l):
    r=[]
    for i in l:
        t1=[]
        for j in i[1]:
            t1.append(list(j))
        r.append([i[0], t1])
    return r

def web_matrix(R, x,k, n):
    #indices = [(i,j) for i in range(k) for j in range(n)]
    #R1 = PolynomialRing(QQ, ['x_{}_{}'.format(i,j) for (i,j) in indices])
    #x = dict(zip(indices, R.gens()))
    r1=Matrix(SR, k,n)
    for i in range(k):
        r1[i,i]=1
    for i in [k..n-1]:
        r1[k-1,i]=1
        #r1[k-1,i]=1
    #print(r1)
    for i in [1..k-1]:
        for j in [1..n-k]:
            t1=0
            for p in itertools.combinations_with_replacement(ListAToN(1,j), k-i): # choose with repetition
                t3=1
                for p1 in [0..len(p)-1]:
                    t3=t3*x[i+p1,p[p1]]
                t3=(-1)^(k+i)*t3
                t1=t1+t3
            r1[i-1,k+j-1]=t1
    r=r1
    return r

def BraidGroupActionSigmaIOnMatrixRenomalizeColumnsNTimes(i,k,n, M, N): 

    r=M
    for j in range(N):
        r=BraidGroupActionSigmaIOnMatrixRenomalizeColumns(i,k,n, r)
    return r
        


def BraidGroupActionSigmaIOnMatrixRenomalizeColumns(i,k,n, M): 

    d=gcd(k,n)
    M1=np.zeros((k,n))
    for i1 in range(k):
        for j in range(n):
            M1[i1,j]=M[i1,j]
    #print(M1)
    l=MatrixToTableauTakeColumnsNPArray(M1)
    #print(l)

    r1=[]
    t1=[]

    for j in [0..n-1]:
        if (j+1) % d !=0: 
            t1.append(l[j])
        else:
            t1.append(l[j])
            r1.append(t1)
            t1=[] 
    #print(r1)
    r2=[]
    for j in [0..len(r1)-1]: 
        l1=[]
        for i1 in [j..len(r1)-1]: 
            l1=UnionOfLists(l1, r1[i1]) 
        l1=UnionOfLists(l1, l)
        t1=BraidGroupActionSigmaIOnWindow(i,k, r1[j], l1)
        r2.append(t1) 
    r=TableauToMatrixFloat(UnionOfListsMany(r2))

    return r

def split_list(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def BraidGroupActionSigmaIOnWindow(i1, k, l, l1): # l=[v1,v2,...,vd], r=[v1,v2,...,v_{i-1},v_{i+1},w_1,v_{i+2},...,v_d], l1= l + ... + beginning of original list 
    i=i1-1
    r=l
    r=removeAnElementInList(i,r)
    r.insert(i+1, 0)
    #print(r)
    
    t1=[]
    for j in [i..i+k]:
        if j!=i+1: 
            t1.append(l1[j])
    
    #print(t1)
            
    m1=np.linalg.det(TableauToMatrixFloat(t1))
    
    #print(m1)
    
    t1=[]
    for j in [i+1..i+k]:  
        t1.append(l1[j])  
    m2=np.linalg.det(TableauToMatrixFloat(t1))
    #print(t1)
    #print(m2)
    w1=[]
    for j in [0..k-1]: 
        w1.append(m1*l[i+1][j]-m2*l[i][j]) 
    #print(w1)
    r[i+1]=w1
    
    return r



def sub_matrix_general(M, c1, c2):
    m,n=len(c1),len(c2)
    r=Matrix(SR, m, n)
    for i in range(m):
        for j in range(n):
            r[i,j]=M[c1[i]-1, c2[j]-1]
    return r

def AddVlueOfListsMany(l):
    r=[]
    if l!=[]:
        n=len(l[0])
        for i in range(n):
            r.append(0)
        for i in l:
            r=AddVlueOfTwoLists(r, i)
    return r

def are_all_non_negative(l):
    r=1
    for i in l:
        if i<0:
            r=0
    return r

def initial_cluster_variables_Yis(typ,rank,l): # mutable variables in the beginning
    r1=InitialCluster_Yis(typ,rank,l)
    r2=r1[2]
    #print(r2, len(r1[1][0]))
    #print(r1[3])
    r3=SetDifference(r1[3],r2)
    #print(r3)
    r=[]
    for i in r2:
        r.append(r1[1][0][i][1])
    for i in r3:
        r.append(r1[1][0][i][1])
    return r

def InitialCluster_Yis(typ, rank, l): # C_l^{Dn}, l \ge 0, at the moment, typ='D'
    if typ=='A':
        n=rank
        p1=n*(l+1)
        
        vertices0=[]

        for i in [1..n]:
            d=distance_of_indices_KR_module_half(i,typ,rank)
            for j in [0..l]:  # j=l are frozens
                t1=KR_module_largest_s(i, i-2, j+1,typ,rank)
                vertices0.append(sorted(t1))
                
        verticesTableaux = [] # 
        for i in range(len(vertices0)):
            verticesTableaux.append([0, vertices0[i], i]) # 

        clusterVariables=[] 
        vertices1 = [verticesTableaux, clusterVariables] # vertices1[1] store cluster variables, vertices1[0] store variables on quiver

        mat=Matrix(p1,p1)
        for i in [0..n-1]:
            for j in [0..l-1]:
                t1=i*(l+1)+j
                mat[t1+1,t1]=1
                mat[t1,t1+1]=-1
        for j in [0..l]:
            for i in [0..n-2]:
                t1=i*(l+1)+j
                t2=(i+1)*(l+1)+j
                mat[t1,t2]=-1
                mat[t2,t1]=1
        for j in [0..l-1]:
            for i in [0..n-2]:
                t1=(i+1)*(l+1)+j+1
                t2=i*(l+1)+j
                mat[t1,t2]=-1
                mat[t2,t1]=1

        mutable_vertices=[]
        for i in [0..n-1]:
            for j in [0..l-1]:
                mutable_vertices.append(i*(l+1)+j)

        frozen_vertices=[]
        for i in [0..n-1]:
            frozen_vertices.append(i*(l+1)+l)

        initial_cluster_indices=mutable_vertices+frozen_vertices

        r=[mat, vertices1, mutable_vertices, initial_cluster_indices]
        
    elif typ=='B':
        n=rank
        p1=2*n*(l+1)
        
        vertices0=[]

        for i in [1..n-1]:
            d=distance_of_indices_KR_module_half(i,typ,rank)
            for j in [0..l]:  # j=l are frozens
                t1=KR_module_largest_s(i, 2*n-2*i, j+1,typ,rank)
                vertices0.append(sorted(t1))
        for i in [n]:
            d=distance_of_indices_KR_module_half(i,typ,rank)
            for j in [0..2*l+1]:  # j=l are frozens
                t1=KR_module_largest_s(i, 1, j+1,typ,rank)
                vertices0.append(sorted(t1))
        for i in range(n-1,0,-1):
            d=distance_of_indices_KR_module_half(i,typ,rank)
            for j in [0..l]:  # j=l are frozens
                t1=KR_module_largest_s(i, 2*i+2-2*n, j+1,typ,rank)
                vertices0.append(sorted(t1))
                
        #print(vertices0)
        
        verticesTableaux = [] # 
        for i in range(len(vertices0)):
            verticesTableaux.append([0, vertices0[i], i]) # 

        clusterVariables=[] 
        vertices1 = [verticesTableaux, clusterVariables] # vertices1[1] store cluster variables, vertices1[0] store variables on quiver
        
        mat=Matrix(p1,p1)
        for i in [0..n-2]:
            for j in [0..l-1]:
                t1=i*(l+1)+j
                mat[t1+1,t1]=1
                mat[t1,t1+1]=-1
        i=n-1
        for j in [0..2*l]:
            t1=i*(l+1)+j
            mat[t1+1,t1]=1
            mat[t1,t1+1]=-1
            
        for i in [n+1..2*n-1]:
            for j in [0..l-1]:
                t1=i*(l+1)+j
                mat[t1+1,t1]=1
                mat[t1,t1+1]=-1
                
        for j in [0..l]:
            for i in [0..n-3]:
                t1=i*(l+1)+j
                t2=(i+1)*(l+1)+j
                mat[t1,t2]=1
                mat[t2,t1]=-1
        for j in [0..l]:
            t1=(n-2)*(l+1)+j
            t2=(n-1)*(l+1)+2*j+1
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..l]:
            t1=(n-1)*(l+1)+2*j
            t2=(n+1)*(l+1)+j
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for i in [n+1..2*n-2]:
            for j in [0..l]:
                t1=i*(l+1)+j
                t2=(i+1)*(l+1)+j
                mat[t1,t2]=1
                mat[t2,t1]=-1
        for j in [0..l-1]:
            for i in [0..n-3]:
                t1=(i+1)*(l+1)+j
                t2=i*(l+1)+j+1
                mat[t1,t2]=1
                mat[t2,t1]=-1
        for j in [0..l-1]:
            t1=(n-1)*(l+1)+2*j+1
            t2=(n-2)*(l+1)+j+1
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..l-1]:
            t1=(n+1)*(l+1)+j
            t2=(n-1)*(l+1)+2*j+2
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..l-1]:
            for i in [n+1..2*n-2]:
                t1=(i+1)*(l+1)+j
                t2=i*(l+1)+j+1
                mat[t1,t2]=1
                mat[t2,t1]=-1
        
        frozen_vertices=[]
        for i in [0..n-2]:
            frozen_vertices.append(i*(l+1)+l)
        i=n-1
        frozen_vertices.append((n+1)*(l+1)-1)
        for i in [n+1..2*n-1]:
            frozen_vertices.append(i*(l+1)+l)
        
        mutable_vertices=SetDifference(ListAToN(0, p1-1), frozen_vertices)

        initial_cluster_indices=mutable_vertices+frozen_vertices

        r=[mat, vertices1, mutable_vertices, initial_cluster_indices]
        
    elif typ=='C':
        n=rank
        p1=(n-1)*(l+1)+ceil(l/2)+ceil((l-1)/2)+2
        
        vertices0=[]

        for i in [1..n-1]:
            d=distance_of_indices_KR_module_half(i,typ,rank)
            for j in [0..l]:  # j=l are frozens
                t1=KR_module_largest_s(i, -i, j+1,typ,rank)
                vertices0.append(sorted(t1))
        i=n
        d=distance_of_indices_KR_module_half(i,typ,rank)
        for j in [0..ceil(l/2)]:  
            t1=KR_module_largest_s(i, -1, j+1,typ,rank)
            vertices0.append(sorted(t1))
        i=n
        d=distance_of_indices_KR_module_half(i,typ,rank)
        for j in [0..ceil((l-1)/2)]:  
            t1=KR_module_largest_s(i, -3, j+1,typ,rank)
            vertices0.append(sorted(t1))
                
        verticesTableaux = [] # 
        for i in range(len(vertices0)):
            verticesTableaux.append([0, vertices0[i], i]) # 

        clusterVariables=[] 
        vertices1 = [verticesTableaux, clusterVariables] # vertices1[1] store cluster variables, vertices1[0] store variables on quiver

        mat=Matrix(p1,p1)
        for i in [0..n-2]:
            for j in [0..l-1]:
                t1=i*(l+1)+j
                mat[t1+1,t1]=1
                mat[t1,t1+1]=-1
        for i in [n-1]:
            for j in [0..ceil(l/2)-1]:
                t1=i*(l+1)+j
                mat[t1+1,t1]=1
                mat[t1,t1+1]=-1
        for i in [n]:
            for j in [0..ceil((l-1)/2)-1]:
                t1=(n-1)*(l+1)+ceil(l/2)+1+j
                mat[t1+1,t1]=1
                mat[t1,t1+1]=-1
        
        for j in [0..l]:
            for i in [0..n-3]:
                t1=i*(l+1)+j
                t2=(i+1)*(l+1)+j
                mat[t1,t2]=1
                mat[t2,t1]=-1
                
        for j in [0..ceil((l-1)/2)]:
            t1=(n-2)*(l+1)+2*j
            t2=(n-1)*(l+1)+ceil(l/2)+1+j
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..ceil(l/2)-1]:
            t1=(n-2)*(l+1)+2*j+1
            t2=(n-1)*(l+1)+j+1
            mat[t1,t2]=1
            mat[t2,t1]=-1
            
        for j in [0..l-1]:
            for i in [0..n-3]:
                t1=(i+1)*(l+1)+j
                t2=i*(l+1)+j+1
                mat[t1,t2]=1
                mat[t2,t1]=-1
                
        for j in [0..ceil(l/2)-1]:
            t1=(n-1)*(l+1)+j
            t2=(n-2)*(l+1)+2*j+1
            mat[t1,t2]=1
            mat[t2,t1]=-1
            
        for j in [0..ceil((l-1)/2)-1]:
            t1=(n-1)*(l+1)+ceil(l/2)+j+1
            t2=(n-2)*(l+1)+2*j+2
            mat[t1,t2]=1
            mat[t2,t1]=-1
  
        frozen_vertices=[]
        for i in [0..n-2]:
            frozen_vertices.append(i*(l+1)+l)
        frozen_vertices=frozen_vertices+[ (n-1)*(l+1)+ceil(l/2), (n-1)*(l+1)+ceil(l/2)+ceil((l-1)/2)+1 ]
        
        mutable_vertices=SetDifference(ListAToN(0, p1-1), frozen_vertices)

        initial_cluster_indices=mutable_vertices+frozen_vertices 
        
        r=[mat, vertices1, mutable_vertices, initial_cluster_indices]
    
    elif typ=='D':
        n=rank
        p1=n*(l+1)

        vertices0=[]
        
        for i in [1..n-1]:
            for j in [0..l]:  # j=l are frozens
                t1=KR_module_largest_s(i, i-2, j+1,typ,rank)
                vertices0.append(sorted(t1))
        i=n
        for j in [0..l]:   # j=l are frozens 
            t1=KR_module_largest_s(i, i-3, j+1,typ,rank)
            vertices0.append(sorted(t1))

        verticesTableaux = [] # 
        for i in range(len(vertices0)):
            verticesTableaux.append([0, vertices0[i], i]) # 

        clusterVariables=[] 
        vertices1 = [verticesTableaux, clusterVariables] # vertices1[1] store cluster variables, vertices1[0] store variables on quiver

        mat=Matrix(p1,p1)
        for i in [0..n-1]:
            for j in [0..l-1]:
                t1=i*(l+1)+j
                mat[t1+1,t1]=1
                mat[t1,t1+1]=-1
        for j in [0..l]:
            for i in [0..n-3]:
                t1=i*(l+1)+j
                t2=(i+1)*(l+1)+j
                mat[t1,t2]=-1
                mat[t2,t1]=1
        for j in [0..l-1]:
            for i in [0..n-3]:
                t1=(i+1)*(l+1)+j+1
                t2=i*(l+1)+j
                mat[t1,t2]=-1
                mat[t2,t1]=1
        i=n-3
        for j in [0..l]:
            t1=i*(l+1)+j
            t2=(i+2)*(l+1)+j
            mat[t1,t2]=-1
            mat[t2,t1]=1

        i=n-3
        for j in [0..l-1]:
            t1=(i+2)*(l+1)+j+1
            t2=i*(l+1)+j
            mat[t1,t2]=-1
            mat[t2,t1]=1

        mutable_vertices=[]
        for i in [0..n-1]:
            for j in [0..l-1]:
                mutable_vertices.append(i*(l+1)+j)

        frozen_vertices=[]
        for i in [0..n-1]:
            frozen_vertices.append(i*(l+1)+l)

        initial_cluster_indices=mutable_vertices+frozen_vertices

        r=[mat, vertices1, mutable_vertices, initial_cluster_indices]
        
    elif typ=='E':  # 
        n=rank
        p1=n*(l+1)
        
        vertices0=[]
        xi=[-1,0,1,2]
        num=-1
        for i in [1,3,4,2]:
            num=num+1
            for j in [0..l]:  # j=l are frozens
                t1=KR_module_largest_s(i, xi[num], j+1,typ,rank)
                vertices0.append(sorted(t1))
        for i in [5..n]:
            for j in [0..l]:  # j=l are frozens
                t1=KR_module_largest_s(i, i-3, j+1,typ,rank)
                vertices0.append(sorted(t1))

        verticesTableaux = [] # 
        for i in range(len(vertices0)):
            verticesTableaux.append([0, vertices0[i], i]) # 

        clusterVariables=[] 
        vertices1 = [verticesTableaux, clusterVariables] # vertices1[1] store cluster variables, vertices1[0] store variables on quiver

        mat=Matrix(p1,p1)
        
        for i in [0..n-1]:
            for j in [0..l-1]:
                t1=i*(l+1)+j
                mat[t1+1,t1]=1
                mat[t1,t1+1]=-1
        for j in [0..l]:
            for i in [0..2]:
                t1=(i+1)*(l+1)+j
                t2=i*(l+1)+j
                mat[t1,t2]=1
                mat[t2,t1]=-1
        i=4
        for j in [0..l]:
            t1=i*(l+1)+j
            t2=(i-2)*(l+1)+j
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..l]:
            for i in [5..n-1]:
                t1=i*(l+1)+j
                t2=(i-1)*(l+1)+j
                mat[t1,t2]=1
                mat[t2,t1]=-1
                
        for j in [0..l-1]:
            for i in [0..2]+[4..n-2]:
                t1=i*(l+1)+j
                t2=(i+1)*(l+1)+j+1
                mat[t1,t2]=1
                mat[t2,t1]=-1
        i=2
        for j in [0..l-1]:
            t1=i*(l+1)+j
            t2=(i+2)*(l+1)+j+1
            mat[t1,t2]=1
            mat[t2,t1]=-1

        frozen_vertices=[]
        for i in [1..n]:
            frozen_vertices.append(i*l+i-1)
        
        mutable_vertices=SetDifference(ListAToN(0, p1-1), frozen_vertices)

        initial_cluster_indices=mutable_vertices+frozen_vertices

        r=[mat, vertices1, mutable_vertices, initial_cluster_indices]
        
    elif typ=='F':
        n=rank
        m1=ceil(l/2)
        m2=floor(l/2)
        p1=2*l+2*m1+2*m2+7
        
        vertices0=[]
        for i in [4,3]:
            for j in [0..l]:  # j=l are frozens
                t1=KR_module_largest_s(i, i-5, j+1,typ,rank)
                vertices0.append(sorted(t1))
        i=2
        for j in [0..m1]:  # j=l are frozens
            t1=KR_module_largest_s(i, -1, j+1,typ,rank)
            vertices0.append(sorted(t1))
        i=1
        for j in [0..m1+1]:  # j=l are frozens
            t1=KR_module_largest_s(i, 1, j+1,typ,rank)
            vertices0.append(sorted(t1))
        i=2
        for j in [0..m2]:  # j=l are frozens
            t1=KR_module_largest_s(i, -3, j+1,typ,rank)
            vertices0.append(sorted(t1))
        i=1
        for j in [0..m2]:  # j=l are frozens
            t1=KR_module_largest_s(i, -5, j+1,typ,rank)
            vertices0.append(sorted(t1))

        verticesTableaux = [] # 
        for i in range(len(vertices0)):
            verticesTableaux.append([0, vertices0[i], i]) # 

        clusterVariables=[] 
        vertices1 = [verticesTableaux, clusterVariables] # vertices1[1] store cluster variables, vertices1[0] store variables on quiver

        mat=Matrix(p1,p1)
        for j in [0..l-1]+[l+1..2*l]+[2*l+2..2*l+m1+1]+[2*l+m1+3..2*l+2*m1+3]+[2*l+2*m1+5..2*l+2*m1+m2+4]+[2*l+2*m1+m2+6..2*l+2*m1+2*m2+5]:
            t1=j
            mat[t1+1,t1]=1
            mat[t1,t1+1]=-1
            
        for j in [0..l]:
            t1=j
            t2=l+1+j
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..m2]:
            t1=l+2*j+1
            t2=2*l+2*m1+5+j
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..m2]:
            t1=2*l+2*m1+5+j
            t2=2*l+2*m1+m2+6+j
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [1..m1]:
            t1=l+2*j
            t2=2*l+2+j
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..m1]:
            t1=2*l+2+j
            t2=2*l+m1+4+j
            mat[t1,t2]=1
            mat[t2,t1]=-1
            
        for j in [0..l-1]:
            t1=l+1+j
            t2=j+1
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..m1-1]:
            t1=2*l+2+j
            t2=l+2*j+2
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..m1]:
            t1=2*l+m1+3+j
            t2=2*l+2+j
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..m2-1]:
            t1=2*l+2*m1+5+j
            t2=l+2*j+3
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..m2-1]:
            t1=2*l+2*m1+m2+6+j
            t2=2*l+2*m1+6+j
            mat[t1,t2]=1
            mat[t2,t1]=-1
        
        
        frozen_vertices=[ l, 2*l+1, 2*l+m1+2, 2*l+2*m1+4, 2*l+2*m1+m2+5, 2*l+2*m1+2*m2+6 ]
        
        mutable_vertices=SetDifference(ListAToN(0, p1-1), frozen_vertices)

        initial_cluster_indices=mutable_vertices+frozen_vertices

        r=[mat, vertices1, mutable_vertices, initial_cluster_indices]
        
    elif typ=='G':
        n=rank
        m1=ceil((l+1)/3)
        m2=ceil(l/3)
        m3=ceil((l-1)/3)
        p1=l+1+m1+m2+m3
        
        vertices0=[]
        i=1
        for j in [0..l]:  # j=l are frozens
            t1=KR_module_largest_s(i, 0, j+1,typ,rank)
            vertices0.append(sorted(t1))
        i=2
        for j in [0..m1-1]:   # 
            t1=KR_module_largest_s(i, -1, j+1,typ,rank)
            vertices0.append(sorted(t1))
        for j in [0..m2-1]:   # 
            t1=KR_module_largest_s(i, -3, j+1,typ,rank)
            vertices0.append(sorted(t1))
        for j in [0..m3-1]:   # 
            t1=KR_module_largest_s(i, -5, j+1,typ,rank)
            vertices0.append(sorted(t1))

        verticesTableaux = [] # 
        for i in range(len(vertices0)):
            verticesTableaux.append([0, vertices0[i], i]) # 

        clusterVariables=[] 
        vertices1 = [verticesTableaux, clusterVariables] # vertices1[1] store cluster variables, vertices1[0] store variables on quiver

        mat=Matrix(p1,p1)
        for j in [0..l-1]:
            t1=j
            mat[t1+1,t1]=1
            mat[t1,t1+1]=-1
        for j in [0..m1-2]:
            t1=l+1+j
            mat[t1+1,t1]=1
            mat[t1,t1+1]=-1
        for j in [0..m2-2]:
            t1=l+1+m1+j
            mat[t1+1,t1]=1
            mat[t1,t1+1]=-1
        for j in [0..m3-2]:
            t1=l+1+m1+m2+j
            mat[t1+1,t1]=1
            mat[t1,t1+1]=-1
        for j in [0..m1-1]:
            t1=j*3
            t2=l+j+1
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..m2-1]:
            t1=j*3+1
            t2=l+m1+j+1
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..m3-1]:
            t1=j*3+2
            t2=l+m1+m2+j+1
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..l]:
            if j % 3 == 0 and j>=3: 
                t1=l+j/3
                t2=j
                mat[t1,t2]=1
                mat[t2,t1]=-1
            elif j%3 == 1 and j>=4:
                t1=l+m1+(j-1)/3
                t2=j
                mat[t1,t2]=1
                mat[t2,t1]=-1
            elif j%3 == 2 and j>=5:
                t1=l+m1+m2+(j-2)/3
                t2=j
                mat[t1,t2]=1
                mat[t2,t1]=-1
    
        frozen_vertices=[ l, l+m1, l+m1+m2, l+m1+m2+m3 ]
        
        mutable_vertices=SetDifference(ListAToN(0, p1-1), frozen_vertices)

        initial_cluster_indices=mutable_vertices+frozen_vertices

        r=[mat, vertices1, mutable_vertices, initial_cluster_indices]
        
    return r

def CartanMatrixSelfDefined(typ, rank):
#     if typ=='E' and rank==6:
#         r=Matrix([[2,0,-1,0,0,0],[0,2,0,-1,0,0],[-1,0,2,-1,0,0],[0,-1,-1,2,-1,0],[0,0,0,-1,2,-1],[0,0,0,0,-1,2]]) # this is the Cartan Matrix in Sage of type E6
#     else:
    
#         r = Matrix(rank, rank)
#         n = rank
#         for i in range(n):
#             if i + 1 <= n-1:
#                 r[i, i + 1] = -1
#             if 0 <= i - 1:
#                 r[i, i - 1] = -1
#             r[i, i] = 2

#         if typ == 'B' or typ == 2:
#             r[n-1, n - 2] = -2
#         elif typ == 'C' or typ == 3:
#             r[n - 2, n-1] = -2
#         elif typ == 'D' or typ == 4:
#             if n == 2:
#                 r[0, 1] = 0
#                 r[1, 0] = 0
#             elif 3 <= n:
#                 r[n - 3, n - 2] = -1
#                 r[n - 3, n-1] = -1
#                 r[n - 2, n - 3] = -1
#                 r[n-1, n - 3] = -1
#                 r[n - 2, n-1] = 0
#                 r[n-1, n - 2] = 0
#         elif typ == 'E' or typ == 5:
#             for k in [[2, 4], [4, 2]]:
#                 r[k[0], k[1]] = -1
#             for k in [[3, 4], [4, 3]]:
#                 r[k[0], k[1]] = 0
#         elif typ == 'F' or typ == 6:
#             r[1, 2] = -2
#         elif typ == 'G' or typ == 7:
#             r[0, 1] = -3
    
    if typ=='A' or typ==1:
        r=CartanMatrix(['A',rank])
    elif typ=='B' or typ==2:
        r=CartanMatrix(['B',rank])
    elif typ=='C' or typ==3:
        r=CartanMatrix(['C',rank])
    elif typ=='D' or typ==4:
        r=CartanMatrix(['D',rank])
    elif typ=='E' or typ==5:
        r=CartanMatrix(['E',rank])
    elif typ=='F' or typ==6:
        r=CartanMatrix(['F',rank])
    elif typ=='G' or typ==7:
        r=CartanMatrix(['G',rank])
     
    return r 

def IsALessOrEqualToBInRightAlignedOrder(a,b): # a, b are two segments
    r=0
    if a[1]<=b[1] and (a[1]<b[1] or a[0]<=b[0]):
        r=1
    
    return r
    
def merge_sortRightAlignOrder(array):
    # If the input array contains fewer than two elements,
    # then return it as the result of the function
    if len(array) < 2:
        return array

    midpoint = len(array) // 2

    # Sort the array by recursively splitting the input
    # into two equal halves, sorting each half and merging them
    # together into the final result
    return mergeRightAlignOrder(
        left=merge_sortRightAlignOrder(array[:midpoint]),
        right=merge_sortRightAlignOrder(array[midpoint:]))
    
def mergeRightAlignOrder(left, right):
    # If the first array is empty, then nothing needs
    # to be merged, and you can return the second array as the result
    if len(left) == 0:
        return right

    # If the second array is empty, then nothing needs
    # to be merged, and you can return the first array as the result
    if len(right) == 0:
        return left

    result = []
    index_left = index_right = 0

    # Now go through both arrays until all the elements
    # make it into the resultant array
    while len(result) < len(left) + len(right):
        # The elements need to be sorted to add them to the
        # resultant array, so you need to decide whether to get
        # the next element from the first or the second array
        if IsALessOrEqualToBInRightAlignedOrder(left[index_left],right[index_right])==1:
            result.append(left[index_left])
            index_left += 1
        else:
            result.append(right[index_right])
            index_right += 1

        # If you reach the end of either array, then you can
        # add the remaining elements from the other array to
        # the result and break the loop
        if index_right == len(right):
            result += left[index_left:]
            break

        if index_left == len(left):
            result += right[index_right:]
            break

    return result

def UnionOfTwoTableauxInMatrix(a,b):
    a1=MatrixToTableauTakeColumns(a)
    b1=MatrixToTableauTakeColumns(b)
    r1=UnionOfTwoTableaux(a1,b1)
    r=TableauToMatrix(r1)
    
    return r

def GVectorToModule_Yis(g,e1): # e1 is initial cluster
    r1=[]
    for i in [0..len(g)-1]:
        if g[i]>0:
            t1=power_of_monomial_Yis(e1[i],g[i])
            r1=r1+t1
    for i in [0..len(g)-1]:
        if g[i]<0:
            t1=power_of_monomial_Yis(e1[i],-g[i])
            r1=monomial_Yis_division(r1, t1)
    #print(r1)
    r=sorted(r1)
    
    return r

def FindNonRealModuleUsingQuiverWithMultipleArrows(k,n,l,e1,mutation_number): # l consists of a cluster, a quiver, and a labeled cluster, e1 is the set of initial cluster variables, e1 can choose a particular order, for example in Gr49, it can be e1=[[[1,2,3,5]],[[1,2,4,5]],[[1,3,4,5]],[[1,2,3,6]],[[1,2,5,6]],[[1,4,5,6]],[[1,2,3,7]],[[1,2,6,7]],[[1,5,6,7]],[[1,2,3,8]],[[1,2,7,8]],[[1,6,7,8]],[[1,2,3,4]],[[2,3,4,5]],[[3,4,5,6]],[[4,5,6,7]],[[5,6,7,8]],[[6,7,8,9]],[[1,2,3,9]],[[1,2,8,9]],[[1,7,8,9]]]
    r1=l
    typ=1
    rank=k 
    r2=r1[1] 
    
    r3=[]
    for i in range(len(r2)):
        for j in range(len(r2[0])):
            if abs(r2[i][j])>=2 and RemoveTrivialTableau2(r1[2][i][0],k,n)!=[] and RemoveTrivialTableau2(r1[2][j][0],k,n)!=[]:
                r3.append(sorted([i,j]))
                #print(i,j,r2[i][j],r1[2][i],r1[2][j])
    r3=removeDuplicatesListOfLists(r3)

    #print('r3',r3) 
    r=[]
    for multi_arrow_v in r3:
        #print('multi_arrow_v', multi_arrow_v)
        mat1=Matrix(r1[1])
        vertices1=[] 
        for i in r1[2]:
            t1=[1]+i
            vertices1.append(t1)
        vertices1=[vertices1, []]
        mutationSequence=multi_arrow_v*mutation_number

        for j in range(len(mutationSequence)): 
            #t1=vertices1[0][mutationSequence[j]][1]
            #print('t1', t1)
            #g2=GVectorOfTableau(k,n,t1,e1)[0]
            #print('g2', g2)
            vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank)
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
            
        #t1=vertices1[0][multi_arrow_v[1]][1]
        t1=vertices1[0][mutationSequence[j]][1]
        
        #print('t1', t1)
        
        g2=GVectorOfTableau(k,n,t1,e1)[0]
        
        p1=remove_common_factors_approximately(g2, mutation_number)
        
        p2=GVectorToTableau(p1,e1,k,n)
        #print('p2',p2)
        #print([multi_arrow_v, p2])
        r.append([multi_arrow_v, p2])
    return r

def FindNonRealModuleUsingQuiverWithMultipleArrows_Yis(typ,rank,L,l,e1,mutation_number): # C_l^{typ}, l \ge 0, L consists of a cluster, a quiver, and a labeled cluster, e1 is the set of initial cluster variables, e1 can choose a particular order, for example in Gr49, it can be e1=[[[1,2,3,5]],[[1,2,4,5]],[[1,3,4,5]],[[1,2,3,6]],[[1,2,5,6]],[[1,4,5,6]],[[1,2,3,7]],[[1,2,6,7]],[[1,5,6,7]],[[1,2,3,8]],[[1,2,7,8]],[[1,6,7,8]],[[1,2,3,4]],[[2,3,4,5]],[[3,4,5,6]],[[4,5,6,7]],[[5,6,7,8]],[[6,7,8,9]],[[1,2,3,9]],[[1,2,8,9]],[[1,7,8,9]]]
    r1=L 
    r2=r1[1] 
    IC=InitialCluster_Yis(typ,rank,l)
    cl=IC[2] # labels of cluster variables
    
    r3=[]
    for i in range(len(r2)):
        for j in range(len(r2[0])):
            if abs(r2[i][j])>=2 and IsInList(i,cl)==1 and IsInList(j,cl)==1:
                r3.append(sorted([i,j]))
                #print(i,j,r2[i][j],r1[2][i],r1[2][j])
    r3=removeDuplicatesListOfLists(r3)

    #print('r3',r3) 
    r=[]
    for multi_arrow_v in r3:
        #print('multi_arrow_v', multi_arrow_v)
        mat1=Matrix(r1[1])
        #print(mat1)
        vertices1=[] 
        #print('r1[2]',r1[2])
        for i in r1[2]:
            t1=[1]+i
            #print('t1',t1)
            vertices1.append(t1)
        vertices1=[vertices1, []]
        #print('vertices1', vertices1)
        mutationSequence=multi_arrow_v*mutation_number
        #print('mutation_sequence, l', mutationSequence, l)
        for j in range(len(mutationSequence)): 
            #print('mutation_sequence[j]', mutationSequence[j])
            #print('vertices1', vertices1)
            #t1=vertices1[0][mutationSequence[j]][1]
            #print('t1', t1)
            #g2=GVectorOfModule_Yis(t1,e1)[0]
            #print('g2', g2) 
            vertices1 = computeEquationsForModulesYis(vertices1, mat1, mutationSequence[j], typ, rank, l)
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
            
        t1=vertices1[0][mutationSequence[j]][1]
        
        #print('t1', t1)
        
        g2=GVectorOfModule_Yis(t1,e1)[0]
        
        p1=remove_common_factors_approximately(g2, mutation_number)
        
        p2=GVectorToModule_Yis(p1,e1)
        #print('p2',p2)
        #print([multi_arrow_v, p2])
        r.append([multi_arrow_v, p2])
    return r

def FindNonRealModuleUsingQuiverWithMultipleArrows_Yis_print_g_vectors(typ,rank,L,l,e1,mutation_number): # C_l^{typ}, l \ge 0, L consists of a cluster, a quiver, and a labeled cluster, e1 is the set of initial cluster variables, e1 can choose a particular order, for example in Gr49, it can be e1=[[[1,2,3,5]],[[1,2,4,5]],[[1,3,4,5]],[[1,2,3,6]],[[1,2,5,6]],[[1,4,5,6]],[[1,2,3,7]],[[1,2,6,7]],[[1,5,6,7]],[[1,2,3,8]],[[1,2,7,8]],[[1,6,7,8]],[[1,2,3,4]],[[2,3,4,5]],[[3,4,5,6]],[[4,5,6,7]],[[5,6,7,8]],[[6,7,8,9]],[[1,2,3,9]],[[1,2,8,9]],[[1,7,8,9]]]
    r1=L 
    r2=r1[1] 
    IC=InitialCluster_Yis(typ,rank,l)
    cl=IC[2] # labels of cluster variables
    
    r3=[]
    for i in range(len(r2)):
        for j in range(len(r2[0])):
            if abs(r2[i][j])>=2 and IsInList(i,cl)==1 and IsInList(j,cl)==1:
                r3.append(sorted([i,j]))
                #print(i,j,r2[i][j],r1[2][i],r1[2][j])
    r3=removeDuplicatesListOfLists(r3)

    #print('r3',r3) 
    r=[]
    for multi_arrow_v in r3:
        #print('multi_arrow_v', multi_arrow_v)
        mat1=Matrix(r1[1])
        #print(mat1)
        vertices1=[] 
        #print('r1[2]',r1[2])
        for i in r1[2]:
            t1=[1]+i
            #print('t1',t1)
            vertices1.append(t1)
        vertices1=[vertices1, []]
        #print('vertices1', vertices1)
        mutationSequence=multi_arrow_v*mutation_number
        #print('mutation_sequence, l', mutationSequence, l)
        for j in range(len(mutationSequence)): 
            print('mutation_sequence[j]', mutationSequence[j])
            #print('vertices1', vertices1)
            t1=vertices1[0][mutationSequence[j]][1]
            #print('t1', t1)
            g2=GVectorOfModule_Yis(t1,e1)[0]
            print('g2', g2) 
            vertices1 = computeEquationsForModulesYis(vertices1, mat1, mutationSequence[j], typ, rank, l)
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
            
        t1=vertices1[0][mutationSequence[j]][1]
        
        #print('t1', t1)
        
        g2=GVectorOfModule_Yis(t1,e1)[0]
        
        p1=remove_common_factors_approximately(g2, mutation_number)
        
        p2=GVectorToModule_Yis(p1,e1)
        #print('p2',p2)
        #print([multi_arrow_v, p2])
        r.append([multi_arrow_v, p2])
    return r
        

def KR_module_largest_s(i,s,k,typ,rank): # Y_{i,s}Y_{i,s-d}...
    d=distance_of_indices_KR_module_half(i,typ,rank)
    r=[]
    for j in [0..k-1]:
        r.append([i,s-2*j*d])
    return r



def UnionOfListsMany(l):
    r=[]
    for i in l:
        r=r+i
    return r

def GVectorOfModule_Yis(p1,r2): # p1 is a module in list form, r2 is initial cluster variables, in Gr48, r2=[[[1, 2, 3, 5]], [[1, 2, 4, 5]], [[1, 3, 4, 5]], [[1, 2, 3, 6]], [[1, 2, 5, 6]], [[1, 4, 5, 6]], [[1, 2, 3, 7]], [[1, 2, 6, 7]], [[1, 5, 6, 7]], [[1, 2, 3, 4]], [[2, 3, 4, 5]], [[3, 4, 5, 6]], [[4, 5, 6, 7]], [[5, 6, 7, 8]], [[1, 6, 7, 8]], [[1, 2, 7, 8]], [[1, 2, 3, 8]]],
    r1=removeDuplicatesListOfLists(UnionOfListsMany(r2)) # r1 are fundamental monomials
    m1=len(r1)
    m2=len(r2)
    gg=np.zeros((m1,m2), dtype=float)
    vv=[]
    #print(gg)
    for i in range(len(r1)):
        #print(r1[i])
        t1=p1.count(r1[i])
        vv.append(t1)
        for j in range(len(r2)):
            #print(r2[j])
            t1=r2[j].count(r1[i])
            gg[i, j]=t1
    vv=np.array(vv)
    #print('vv',vv)
    p6= np.linalg.lstsq(gg, vv, rcond=None)[0]

    #print(p6)
    t3=[]
    t6=[]
    for q in range(len(r2)):
        t2=int(round(p6[q]))
        t6.append([t2, r2[q]])
        t3.append(t2)
   
    r=[t3,t6]
    return r

# mutation of modules of quantum affine algebra of type Dn, C_l^g, l \ge 0, at the moment typ='D'
def computeEquationsForModulesYis(variable2, mat, k, typ, rank, l): # variable2=(variables on quiver, cluster variables obtained so far)
    
    variable=variable2[0]
    clusterVariables=variable2[1] 
    size=mat.nrows() 
    newVariable=[]
    newVariable2=[]
    newVariableList=[]
    newVariable2List=[]

    for i in range(size):
        if mat[i, k]>0:
            #print('i,variable[i]', i,variable[i])
            #print('power_of_monomial_Yis(variable[i][1], mat[i,k])', power_of_monomial_Yis(variable[i][1], mat[i,k]))
            newVariable=newVariable+power_of_monomial_Yis(variable[i][1], mat[i,k])
            #print('newVariable', newVariable)
            for j in range(mat[i,k]):
                newVariableList.append(variable[i][1])
 
    for i in range(size): 
        if mat[i, k]<0:
            newVariable2=newVariable2 + power_of_monomial_Yis(variable[i][1], -mat[i,k])
            for j in range(-mat[i,k]):
                newVariable2List.append(variable[i][1])
 
    variable[k][0]=variable[k][0]+1
    t1=compare_weights_monomial_Yis(newVariable, newVariable2,typ,rank)
    
    #print('t1,newVariable,newVariable2,variable[k][1]',[t1,newVariableList,newVariable2List,variable[k][1]]) 

    if t1==1: 
        variable[k][1]=sorted(monomial_Yis_division(newVariable, variable[k][1]))
    else:
        variable[k][1]=sorted(monomial_Yis_division(newVariable2, variable[k][1])) 
        
    clusterVariables=variable[k][1]
    #print('new cluster variable', clusterVariables)
    
    r=[variable, clusterVariables]

    return r

def remove_common_factors_approximately(l, a): # l[i]/t, t>a
    r1=copy.deepcopy(l)
    r2=list(map(lambda x: abs(x), r1))
    r3=removeDuplicatesListOfLists(sorted(SetDifference(r2, [0])))
    minm=max(r3)
    for i in r3:
        if i<minm and i>a:
            minm=i
        
    #print('minm', minm)
    
    t1=[]
    for i in r1:
        t1.append(round(i/minm, 6))
    r0=t1
    
    multi=1
    while 1:
        r1=[i*multi for i in r0]
        #print(r1)
        r=[]
        for i in r1:
            t1=round(i, 0)
            t2=round(i, 1)
            #print(t1,t2,abs(t1-t2))
            if abs(t1-t2)<0.00001:
                r.append(t1)
            else:
                break
        if len(r)==len(r1):
            break
        else:
            multi=multi+1
    
    return r




#Function for multiprocessing 
def ll_perms_compute_nonreal_from_clusters_Yis(lli,typ,rank,max_degree,repeat,b6,b_nonreal,fp1,fp2,mutation_number,l): #  
    IC=InitialCluster_Yis(typ,rank,l)
    mat1=IC[0]
    vertices1=IC[1]
    e1=initial_cluster_variables_Yis(typ,rank,l)
    
    ll=list(lli)
    #print(ll)
    
    mutationSequence=[]
    for j1 in [1..repeat]: 
        mutationSequence=mutationSequence+ll  
    #print(mutationSequence)
        
    for j in range(len(mutationSequence)): 
        vertices1 = computeEquationsForModulesYis(vertices1, mat1, mutationSequence[j],typ,rank,l)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 
        #print(vertices1[1])
    
        if max_degree_of_Yis_in_monomial(vertices1[1])>max_degree:
            vertices1 = computeEquationsForModulesYis(vertices1, mat1, mutationSequence[j],typ,rank,l)
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
        else:
            ii=vertices1[1]
            p1=repr(ii)
            p2=b6.get(p1)
            
            #print('p1',p1)
            
            if p2==None:
                #print('ii',ii)
                f1=open(fp1, 'a+')
                f1.write(str(ii)+'\n')
                b6[p1]=[] 
                f1.close()
                
            ma0=MatrixTakeRows(mat1)
            if len(SetDifference(flatten(ma0), [1,0,-1]))>=2:
                t0=sorted(TakeIthElements(2,vertices1[0]))
                t1=removeAnElementInListList(0,vertices1[0])
                #print('t1',t1)
                t2=[t0, ma0, t1]
                g00=FindNonRealModuleUsingQuiverWithMultipleArrows_Yis(typ,rank,t2,l,e1,mutation_number)
                #print(g00)
                if g00!=[]:
                    g0=g00[0][1]
                    g1=repr(g0)
                    g2=b_nonreal.get(g1)
                    if g2==None:
                        f1=open(fp2, 'a+')
                        f1.write(str(g0)+'\n')
                        b_nonreal[g1]=[] 
                        f1.close()
                    
#                     if max_degree_of_Yis_in_monomial(g0)<=max_degree: # collect only nonreal modules less or equal to max_column columns
#                         g1=repr(g0)
#                         g2=b_nonreal.get(g1)
#                         if g2==None:
#                             f1=open(fp2, 'a+')
#                             f1.write(str(g0)+'\n')
#                             b_nonreal[g1]=[] 
#                             f1.close()

    return [b6, b_nonreal]

#Function for multiprocessing 
def ll_perms_compute_modules_Yis(lli,typ,rank,max_degree,repeat,b6,fp1,l): #  l \ge 0, at the moment, typ='D'
    
    IC=InitialCluster_Yis(typ,rank, l)
    mat1=IC[0]
    vertices1=IC[1] 
    
    ll=list(lli)
    #print(ll)
    
    mutationSequence=[]
    for j1 in [1..repeat]: 
        mutationSequence=mutationSequence+ll  
    #print(mutationSequence)
        
    for j in range(len(mutationSequence)): 
        vertices1 = computeEquationsForModulesYis(vertices1, mat1, mutationSequence[j],typ,rank,l)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 
        #print(vertices1[1])
        
#         for i1 in range(mat1.nrows()):
#             for j1 in range(mat1.ncols()):
#                 if abs(mat1[i1][j1])>=2:
#                     print(MatrixTakeRows(mat1))
#                     print(str(vertices1)+'\n')
    
        if max_degree_of_Yis_in_monomial(vertices1[1])>max_degree:
            vertices1 = computeEquationsForModulesYis(vertices1, mat1, mutationSequence[j],typ,rank,l)
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
        else:
            ii=vertices1[1]
            p1=repr(ii)
            p2=b6.get(p1)
            
            #print('p1',p1)
            
            if p2==None:
                #print('ii',ii)
                f1=open(fp1, 'a+')
                f1.write(str(ii)+'\n')
                b6[p1]=[] 
                f1.close()
    return b6

def max_degree_of_Yis_in_monomial(l): # l is a list, l = [[2,-2],[2,-2],[1,-1]], max_degree of l is 2.
    r1=removeDuplicatesListOfLists(l)
    r=0
    for i in r1:
        t1=l.count(i)
        if r<t1:
            r=t1
    return r

def product_of_monomial_Yis(a,b):
    r=a+b
    return r
def power_of_monomial_Yis(a,n):
    r=[]
    for i in range(n):
        r=r+a
    return r

def compare_weights_monomial_Yis(P1,P2,typ,rank): # 
    if P1!=[]:
        t1=weight_of_monomial_Yis(P1,rank)
    else:
        t1=[]
        for i in [1..rank]:
            t1.append(0)

    if P2!=[]:
        t2=weight_of_monomial_Yis(P2,rank)
    else:
        t2=[]
        for i in [1..rank]:
            t2.append(0)

    #print(t1,t2)

    r=compareWeights2(t1,t2,typ, rank)

    return r

def weight_of_monomial_Yis(a,rank): # a=[[1,-3],[2,0]]
    n=rank
    
    r=[]
    for i in [0..n-1]:
        r.append(0)
    
    for i in a:
        r[i[0]-1]=r[i[0]-1]+1
        
    return r

def monomial_Yis_division(a,b):
    r=SetDifferenceListDifference2(a,b)
        
    return r

def distance_of_indices_KR_module_half(i,typ,rank): # we use self-defined Cartan matrices, can change to Sage convention later
    r=1
    if typ=='G' and i==2:
        r=3
    elif typ=='B' and i!=rank:
        r=2
    elif typ=='C' and i==rank:
        r=2
    elif typ=='F' and (i==1 or i==2):  # this corresponds to self-defined Cartan matrix of type F4, can change to Sage convention later
        r=2
    return r
            
def KR_module(i,s,k,typ,rank): # Y_{i,s}Y_{i,s+d}...
    d=distance_of_indices_KR_module_half(i,typ,rank)
    r=[]
    for j in [0..k-1]:
        r.append([i,s+2*j*d])
    return r

def ModuleToMultiSegment(Y,l): # l is a list, l[i] is the index of some Y[i,j]
    #print(l)
     
    r=TranslateModuleToMultiSegmentsMonomial(Y,l)

    return r

def TranslateModuleToMultiSegmentsMonomial(Y,l): # l is a list, l[i] is the index of some Y[i,j]
 
    r=[]
    for j in l:
        #print(j)
        r=append(r, TranslateQuantumAffineToLadders(j))
        
    
    #r=SortBySecondTerms(r)
    r=merge_sortRightAlignOrder(r)
    r=list(reversed(r))

    return r


def TranslateQuantumAffineToLadders(l):
    i=l[0]
    s=l[1] 
    r=[(s-i)/2+1/2, (s+i)/2-1/2]

    r=[r[0]+1/2, r[1]+1/2]

    return r


def SortBySecondTerms(l):

    r=[]

    if l!=[]:
        #print(l)

        t1=[]
        for i in l:
            t1=append(t1, i[1])

        t1=sorted(t1)

        #print(t1)

        t2=sorted(l)

        r=[] 
        for i in reversed([min(t1)..max(t1)]):
            for j in t2:
                if j[1]==i:
                    r=append(r, j)
    return r


def ModuleToTableauInMatrix(Y,l,k,n):
    t1=ModuleToMultiSegment(Y,l)
    r=MultisegmentToTableauGrassmannian(k,n,t1)
    return r

def ModuleToTableau(Y,l,k,n):
    t1=ModuleToTableauInMatrix(Y,l,k,n)
    r=MatrixToTableauTakeColumns(t1)
    return r

def MultisegmentToTableauGrassmannianOneSegment(k, m): # Gr(k,n)
 
    a=m[0]
    b=m[1]

    r=Matrix(k,1)
    r1=SetDifference(ListAToN(1-a, 1+k-a), k-b)
    for i in [0..k-1]: 
        r[i,0]=r1[i]

    return r


def MultisegmentToTableauGrassmannian(k,n, l): # Gr(k,n)

    r=0
    if l!=[]:
        r=MultisegmentToTableauGrassmannianOneSegment(k,l[0])
        for i in [1..len(l)-1]:
            t1=MultisegmentToTableauGrassmannianOneSegment(k, l[i])
            r=UnionOfTwoTableauxInMatrix(r, t1)
             
        r=RemoveTrivialTableauInMatrix(r,k,n) 
    return r

def RemoveTrivialTableauInMatrix(a1, n,m): # n is the row of tableau, m can take sufficient large

    t1=MatrixToTableauTakeColumns(a1)
    t2=RemoveTrivialTableau(t1,n,m)
    r=TableauToMatrix(t2)

    return r

def TableauInMatrixToMultiSegments(Y,a): # a is a tableau in matrix   
    k=a.nrows()
    b=MatrixToTableauTakeColumns(a)
    a1=TableauToSmallGapTableau(k,b);
    #print(k,a1)
    r1=CollectFirstElementInSmallGapTableau(a1);
    r2=CollectMissingElementInSmallGapTableau(a1)
    r=[]
    for i in [0..len(r1)-1]:
        t1=[1-r1[i], k-r2[i]]
        r.append(t1)
    return r

def TableauToMultiSegments(Y,l): # l is a tableau T, l[i] is column of i of T, l=[] means trivial tableau, corresponding to trivial module \mathbb{C}
 
    if l==[]:
        r=[]
    else:
        t2=TableauToMatrix(l)
        r=TableauInMatrixToMultiSegments(Y,t2)

    return r

def ProductOfElementsInList(l):
    r=1
    for i in l:
        r=r*i
    return r

def TableauToModulePolynomialOfTableauxWithCoeff(Y,l): #
 
    r=0
    for i in l:
        if i[0]!=0 and i[1]!=[]:
            t1=TableauToModuleHasY(Y,i[1])
            #print(t1)
            r=r+i[0]*ProductOfElementsInList(t1)
       
    return r

def TableauToModulePolynomialOfTableauxWithCoeffHasNoY(Y,l): #
 
    r=[]
    for i in l:
        if i[0]!=0 and i[1]!=[]:
            t1=TableauToModule(Y,i[1])
            #print(t1)
            r.append([i[0], t1])
       
    return r


def TableauToModule(Y,l): # l is a tableau T, l[i] is column of i of T
 
    t1=TableauToMultiSegments(Y,l)
    r=MultiSegmentToModule(t1)

    return r

def TableauToModulePolynomial(Y,l): # l is a polynomial of tableaux, l=[[1,T1],[1,T2],...]
 
    r=[]
    for i in l:
        r.append([i[0], TableauToModule(Y,i[1])]) 

    return r

def TableauToModuleHasY(Y,l): # l is a tableau T, l[i] is column of i of T
 
    t1=TableauToModule(Y,l)
    r=[]
    for i in t1:
        r.append(Y[i[0],i[1]])

    return r

def TableauToModulePolynomialHasY(Y,l): # l is a polynomial of tableaux, l=[[1,T1],[1,T2],...]
 
    r=[]
    for i in l:
        r.append([i[0], TableauToModuleHasY(Y,i[1])]) 

    return r


def TableauInMatrixToModule(Y,l): # l is a tableau T in terms of matrix, l=[] means l is trivial tableau
    
    if l==[]:
        r=[]
    else:

        t1=TableauInMatrixToMultiSegments(Y,l)
        r=MultiSegmentToModule(t1)

    return r

def TableauToModuleInMatrixPolynomial(Y,l): # l is a polynomial of tableaux, l=[[1,T1],[1,T2],...]
 
    r=[]
    for i in l:
        r.append([i[0], TableauToModuleInMatrix(Y,i[1])]) 

    return r

def TableauToModuleInMatrixPolynomialHasY(Y,l): # l is a polynomial of tableaux, l=[[1,T1],[1,T2],...]
 
    r=[]
    for i in l:
        r.append([i[0], TableauToModuleInMatrixHasY(Y,i[1])]) 

    return r


def TableauToModuleInMatrix(Y,l): # l is a tableau T in terms of matrix, l=[] means l is trivial tableau
    if l==[]:
        r=[]
    else:
    
        r=TableauInMatrixToModule(Y,l)

    return r

def TableauToModuleInMatrixHasY(Y,l): # l is a tableau T
 
    r1=TableauToModuleInMatrix(Y,l)
    r=[]
    for i in r1:
        r.append(Y[i[0],i[1]])

    return r


def TableauToModuleInMatrixPAdicConvention(Y,l): # l is a tableau T
 
    r1=TableauToModuleInMatrix(Y,l)
    r=[]
    for i in r1:
        t1=i
        r=append(r, [t1[0], -t1[1]-1]) 

    return r

def TableauToModulePAdicConvention(Y,l): # l is a tableau T
 
    r1=TableauToModule(Y,l)
    r=[]
    for i in r1:
        t1=i
        r=append(r, [t1[0], -t1[1]-1]) 

    return r

def TableauToModulePAdicConventionPolynomial(Y,l): # l is a polynomial of tableaux, l=[[1,T1],[1,T2],...]
 
    r=[]
    for i in l:
        r.append([i[0], TableauToModulePAdicConvention(Y,i[1])]) 

    return r

def TableauToModuleInMatrixPAdicConventionPolynomial(Y,l): # l is a polynomial of tableaux, l=[[1,T1],[1,T2],...]
 
    r=[]
    for i in l:
        r.append([i[0], TableauToModuleInMatrixPAdicConvention(Y,i[1])]) 

    return r


def TableauToModuleInMatrixPAdicConventionHasY(Y,l): # l is a tableau T
    
    if l==[]:
        r=[]
    else:
     
        r1=TableauToModuleInMatrix(Y,l)
        r=[]
        for i in r1:
            t1=i
            r=append(r, Y[t1[0], -t1[1]-1]) 

    return r

def TableauToModulePAdicConventionHasY(Y,l): # l is a tableau T
    
    if l==[]:
        r=[]
    else:
     
        r1=TableauToModule(Y,l)
        r=[]
        for i in r1:
            t1=i
            r=append(r, Y[t1[0], -t1[1]-1]) 

    return r

def TableauToModuleInMatrixPAdicConventionPolynomialHasY(Y,l): # l is a polynomial of tableaux, l=[[1,T1],[1,T2],...]
 
    r=[]
    for i in l:
        r.append([i[0], TableauToModuleInMatrixPAdicConventionHasY(Y,i[1])]) 

    return r

def TableauToModulePAdicConventionPolynomialHasY(Y,l): # l is a polynomial of tableaux, l=[[1,T1],[1,T2],...]
 
    r=[]
    for i in l:
        r.append([i[0], TableauToModulePAdicConventionHasY(Y,i[1])]) 

    return r

def SegmentToModule(l): 
    
    a=l[0]
    b=l[1]
     
    r=[b-a+1,a+b-1]
        
    
    return r

def MultiSegmentToModule(l):  
    
    r=[]
    for i in l:
        r.append(SegmentToModule(i))
        
    
    return r

def ProductOfExponentsInTermsOfListMonomial(a,b): 
    #print('a,b',a,b)
    t0=a[0]*b[0]
    t1=AddVlueOfTwoLists(a[1],b[1]) 
    r=[t0, t1]
    return r

def ProductOfExponentsInTermsOfListPolynomial(a,b): 
    r=[]
    #print('a,b',a,b)
    for i in a:
        for j in b:
            r.append(ProductOfExponentsInTermsOfListMonomial(i,j))
    return r

def combineLikeTerms3(l):

    r=[]
    l2=copy.deepcopy(l)
    
    #print('l',l)
    temp2=[]

    for i in l2:

        if (IsInList(removeAnElementInList(0,i), temp2)==0): 
            t=0
            for j in l2:
                if removeAnElementInList(0,j)==removeAnElementInList(0,i):
                    t=t+j[0]
              
            s=i
            s[0]=t
            r.append(s)
            temp2.append(removeAnElementInList(0,i))
    
    r1=r
    r=[]
    for i in r1:
        if i[0]!=0:
            r.append(i)

    return r

def find_gcd(list):
    x = reduce(gcd, list)
    return x

def IntersectionOfLists(a,b):
    r=[x for x in a if x in b]
    return r

def IntersectionOfLists2(a,b):
 
    a1=ChangeListOfMatricesToSetOfMatrices(a)
    b1=ChangeListOfMatricesToSetOfMatrices(b)
    r=list(a1.intersection(b1))

    return r

def IntersectionOfLists3(a,b):
    
    r=list(set(a) & set(b))

    return r

def FindAllPrimeTableauxParallel(fp,T,k,n): # T[i] is a row
    r=[]
    i=TransposeTableauSLk(T) # i[j] are columns
    #print(i)
    if QaRootsOfTableaux(k, TableauToMatrix(i))<=2:
        if IsTableauPrime(i,n)==1:
            r=[i]
            #print(r)
            F1=open(fp, 'a+')
            F1.write(str(i)+'\n')
            F1.close()
    return r 

def TableauDecompositionIntoPrimes(T, n): # T[i] are columns, need to check more, Gr(k,n)
    r=[T]
    if T!=[]:
        k=len(T[0])
        r1=NonCrossingTupleWithSameContentAndWeightAsT(T)
        #print('r1', r1)
        r1=list_of_tuples_to_list_of_lists(r1)
        #print('r1', r1)
        r2=list(partition(r1))
        for i1 in r2:
            if len(i1)>1:
                i=list_nonsemistandard_tableaux_to_semistandard(i1)
                #print(i1, i, r)
                t1=AreAllTableauxWithQLessOrEqual2(i)
                #print('t1',t1)
                if t1==1:
                    t2=IsCompatiblePairwiseTableaux(i,n)
                    if t2==1:   # compatible means T can be factored as i
                        r=i
                        return r
    return r

def IsTableauPrime(T, n): # T[i] are columns, need to check more
    r=1
    if T!=[]:
        r=1
        k=len(T[0])
        if QaRootsOfTableaux(k,TableauToMatrix(T))>2:
            r=0
            return r
        r1=NonCrossingTupleWithSameContentAndWeightAsT(T)
        #print('r1', r1)
        r1=list_of_tuples_to_list_of_lists(r1)
        #print('r1', r1)
        r2=list(partition(r1))
        for i1 in r2:
            if len(i1)>1:
                i=list_nonsemistandard_tableaux_to_semistandard(i1)
                #print(i1, i, r)
                t1=AreAllTableauxWithQLessOrEqual2(i)
                #print('t1',t1)
                if t1==1:
                    t2=IsCompatiblePairwiseTableaux(i,n)
                    if t2==1:   # compatible means T can be factored as i
                        r=0
                        #print(i1)
                        return r
    return r

def list_of_tuples_to_list_of_lists(l):
    r = [list(elem) for elem in l]
    return r

def list_nonsemistandard_tableaux_to_semistandard(l):
    r=[]
    for i in l:
        t1=SortRowsOfTableau(i)
        r.append(t1)
    return r

def generate_all_semistandard_tableaux(k,n,m): # k rows, m columns, max = n, r[i][j] is a row in tableau r[i]
    t1=[]
    for i in [0..k-1]:
        t1.append(m)
    r=SemistandardTableaux(t1, max_entry=n)
    return r

def IsSemistandardTableauInRows(a): # a[i]'s are rows
    r=1
    
    for i in [0..len(a)-2]:
        if len(a[i])<len(a[i+1]):
            r=0
            return r
    for i in [0..len(a)-1]:
        for j in [0..len(a[i])-2]:
            if a[i][j+1]<a[i][j]:
                r=0
                return r
    r1=TransposeTableauSLk(a)
    for i in [0..len(r1)-1]:
        for j in [0..len(r1[i])-2]:
            if r1[i][j+1]<=r1[i][j]:
                r=0
                return r
        
    return r
    
def TransposeTableauSLk(a): # a[i] are columns
    r=[]
    for j in [0..len(a[0])-1]:
        t1=[]
        for i in [0..len(a)-1]:
            if j<len(a[i]):
                t1.append(a[i][j])
        r.append(t1)

    return r

def AreAllTableauxWithQLessOrEqual2(l): # l[i] is a list, l[i][j] are columns
    r=1
    k=len(l[0][0])
    for i in l:
        i1=TableauToMatrix(i)
        t1=QaRootsOfTableaux(k, i1)
        if t1>2:
            r=0
            return r
    return r

def IsCompatiblePairwiseTableaux(l, n): # l[i] is a tableau, l[i][j] is a column
    r=1
    r1=NchooseM(l,2)
    for r2 in r1:
        t1=TableauToMatrix(r2[0])
        t2=TableauToMatrix(r2[1])
        p1=IsCompatiblePairTableauxInMatrix(t1,t2,n)
        if p1==0:
            r=0
            return r
    return r

def partition(collection):
    if len(collection) == 1:
        yield [ collection ]
        return

    first = collection[0]
    for smaller in partition(collection[1:]):
        # insert `first` in each of the subpartition's subsets
        for n, subset in enumerate(smaller):
            yield smaller[:n] + [[ first ] + subset]  + smaller[n+1:]
        # put `first` in its own subset 
        yield [ [ first ] ] + smaller
        
def FindAllPrimeTableaux(fp,k,n,m): # r[i][j] are columns in r[i]
    r1=generate_all_semistandard_tableaux(k,n,m)
    r=[] 
    for i1 in r1:
        i=TransposeTableauSLk(i1) # i[j] are columns
        #print(i)
        if QaRootsOfTableaux(k, TableauToMatrix(i))<=2:
            if IsTableauPrime(i,n)==1:
                r.append(i)
                print(i)
                F1=open(fp, 'a+')
                F1.write(str(i)+'\n')
                F1.close()
                
    return r 


        

def GVectorToTableau(g,e1,k,n): # e1 is initial cluster
    r1=[]
    for i in [0..len(g)-1]:
        if g[i]>0:
            t1=PowerOfTableaux(e1[i],g[i])
            r1=UnionOfTwoTableaux(r1,t1)
    #print(r1)
    for i in [0..len(g)-1]:
        if g[i]<0:
            t1=PowerOfTableaux(e1[i],-g[i])
            r1=TableauDivision(r1,t1)
            #print(r1)
    #print(r1)
    r=r1
    #r=RemoveTrivialTableau(r,k,n)
    
    return r

class BipartiteGraphMaxMatching:
    def __init__(self,graph): # graph is given by list of lists, it is a bipartite graph, graph[i][j]=1 means xi is connected to yj 
         
        # residual graph
        self.graph = graph
        self.ppl = len(graph)
        self.jobs = len(graph[0])
 
    # A DFS based recursive function
    # that returns true if a matching
    # for vertex u is possible
    def bpm(self, u, matchR, seen):
 
        # Try every job one by one
        for v in range(self.jobs):
 
            # If applicant u is interested
            # in job v and v is not seen
            if self.graph[u][v] and seen[v] == False:
                 
                # Mark v as visited
                seen[v] = True
 
                '''If job 'v' is not assigned to
                   an applicant OR previously assigned
                   applicant for job v (which is matchR[v])
                   has an alternate job available.
                   Since v is marked as visited in the
                   above line, matchR[v]  in the following
                   recursive call will not get job 'v' again'''
                if matchR[v] == -1 or self.bpm(matchR[v], matchR, seen):
                    matchR[v] = u
                    return True
        return False
 
    # Returns maximum number of matching
    def maxBPM(self):
        '''An array to keep track of the
           applicants assigned to jobs.
           The value of matchR[i] is the
           applicant number assigned to job i,
           the value -1 indicates nobody is assigned.'''
        matchR = [-1] * self.jobs
         
        # Count of jobs assigned to applicants
        result = 0
        for i in range(self.ppl):
             
            # Mark all jobs as not seen for next applicant.
            seen = [False] * self.jobs
             
            # Find if the applicant 'u' can get a job
            if self.bpm(i, matchR, seen):
                result += 1
        return result

def matrix_from_edge_set_of_bipartite_graph(l,n,m): # here n=len(l), l = [[2,4,5],[2,3],...] means vertex x1 connect to y2, y4, y5, vertex x2 connect to y2, y3 
    #print('l',l)
    r=[]
    for i in [0..n-1]:
        t1=[]
        for j in [0..m-1]:
            t1.append(0)
        r.append(t1)
    for i in [0..len(l)-1]:
        for j in [0..len(l[i])-1]:
            r[i][l[i][j]]=1
    return r

def IsInList(i,l):
    if (i in l)==True:
        r=1
    else:
        r=0
    return r

def NonCrossingTupleWithSameContentAsT_new(T): 
    if T and T[0]:
        r=list(candidates_new(Counter(sorted(flatten(T))), len(T[0])))
        r=removeDuplicatesListOfLists(r)
        return r

def NonCrossingTupleWithSameContentAsT_new_smaller_or_equal_weight(T, typ,rank): 
    r1=NonCrossingTupleWithSameContentAsT_new(T)
    r=ChooseTableauxWithSmallOrEqualWeight(T,r1,typ,rank)
    return r    

def candidates_new(content, k, candidate=()): 
    if len(content) < k:
        if len(content) == 0 and candidate:
            yield candidate
        else:
            return
        
    for I in itertools.combinations(content, k):
        # tuples in candidate are in non-decreaseing order
        if candidate and I < candidate[-1]:
            continue
            
        # new tuple is non-crossing with the other tuples
        if all(is_noncrossing_new(I, J) for J in candidate):
            yield from candidates_new(content - Counter(I), k, candidate + (I,))
            
@functools.lru_cache()  
def is_noncrossing_new(I,J): # I,J have the same size
    k = len(I)

    for a in range(k - 1):
        for b in range(a + 1, k):
            if (I[a+1:b] == J[a+1:b] and
                    not is_weakly_separated(I[a:b+1], J[a:b+1])):
                return False

    return True
    
def is_weakly_separated(I,J): # I,J have the same size
    r1 = sorted(set(I).difference(J))
    r2 = sorted(set(J).difference(I))

    if not r1 or not r2:
        return True

    if r1[0] >= r2[0]:
        r1, r2 = r2, r1
 
    for i in range(len(r1) - 1):
        if r2[0] < r1[i+1]: 
            return r2[-1] <= r1[i+1]
 
    return r1[-1] <= r2[0]


def InitialClusterWithoutNumericalMatrices(rank,n): # Gr(rank,n)
    sizeColumn=n-rank
    k=sizeColumn    
    k1=rank
    p1=k*rank+1
    mat=Matrix(p1,p1)
    for i in range(p1): 
        i1=i+1
        if i1==1: 
            mat[i,i+k+1]=1
            mat[i, p1-1]=1
            mat[i,i+k]=-1
            mat[i, i+1]=-1
        elif i1>=2 and i1<=k-1: 
            mat[i,i+1]=-1
            mat[i,i+k]=-1
            mat[i,i-1]=1
            mat[i,i+k+1]=1
        elif i1==k: 
            mat[i,i-1]=1
            mat[i,i+k]=-1
        elif i1>k and i1<(rank-1)*k and i1 % k==1: 
            mat[i,i-k]=1
            mat[i,i+k+1]=1
            mat[i,i+1]=-1
            mat[i,i+k]=-1
        elif i1>k and i1<(rank-1)*k and i1 % k >=2 and i1 % k<=k-1:
            mat[i,i-k-1]=-1
            mat[i,i+1]=-1
            mat[i,i+k]=-1
            mat[i,i-k]=1
            mat[i,i-1]=1
            mat[i,i+k+1]=1
        elif i1>=2*k and i1<=(rank-1)*k and i1 % k==0:
            mat[i,i-k-1]=-1
            mat[i,i+k]=-1
            mat[i,i-k]=1
            mat[i,i-1]=1
        elif i1>(rank-1)*k and i1<p1 and i1 % k==1: 
            mat[i,i-k]=1
            mat[i,i+1]=-1
        elif i1>=(rank-1)*k+2 and i1<rank*k:
            mat[i,i-k-1]=-1
            mat[i,i+1]=-1
            mat[i,i-k]=1
            mat[i,i-1]=1
        elif i1==rank*k:
            mat[i,i-k-1]=-1
            mat[i,i-k]=1
            mat[i,i-1]=1
        elif i1==p1:
            mat[i,0]=-1

    vertices0=[]

    for j in range(k1-1,-1,-1):
        for i in range(k1,k1+k):
            t1=list(range(1,j+1))
            t2=list(range(i-k1+j+2,i+2))
            t3=t1+t2
            vertices0.append(t3)

    vertices0.append(list(range(1,k1+1)))  
    verticesTableaux = [] # Tableaux are represented by matrices
    for i in range(len(vertices0)):
        verticesTableaux.append([0, [vertices0[i]], i]) # [vertices0[i]] is an one column tableau
    mat1 = Matrix(p1,p1)
    for i in range(p1):
        for j in range(p1):
            mat1[i,j]=mat[i,j]
    clusterVariables=[] 
    vertices1 = [verticesTableaux, clusterVariables] # vertices1[1] store cluster variables, vertices1[0] store variables on quiver
    r=[mat, vertices1]
    
    return r

def EvacuationOfTableauNTimesTheNthTisListOfLists_List(N,l,n): # l is a list, l[i] is a tableau written as list of lists
    r=[]
    for i in l:
        t1=EvacuationOfTableauNTimesTheNthTisListOfLists(N,i,n)
        r.append(t1)
    return r

def EvacuationOfTableauNTimesTheNthTisListOfLists_ListLabeled(N,l,n): # l is a list, l[i]=[T, i], T is a tableau written as list of lists
    r=[]
    for i in l:
        t1=EvacuationOfTableauNTimesTheNthTisListOfLists(N,i[0],n)
        r.append([t1,i[1]])
    return r

def PromotionOfTableauNTimesTheNthTisListOfLists_List(N,l,n): # l is a list, l[i] is a tableau written as list of lists
    r=[]
    for i in l:
        t1=PromotionOfTableauNTimesTheNthTisListOfLists(N,i,n)
        r.append(t1)
    return r

def PromotionOfTableauNTimesTheNthTisListOfLists_ListLabeled(N,l,n): # l is a list, l[i]=[T, i], T is a tableau written as list of lists
    r=[]
    for i in l:
        t1=PromotionOfTableauNTimesTheNthTisListOfLists(N,i[0],n)
        r.append([t1,i[1]])
    return r


def TakeIthElements(i,l):
    r=[]
    for i1 in l:
        r.append(i1[i-1])
    return r



def removeAnElementInListList(i, l):
 
    r=[]
    for j in l:
        r.append(removeAnElementInList(i, j))
    
    return r

def EvacuationOfPolynomialInPluckerNTimesListForPairList(N,ll,n): 
    r=[]
    for i in ll:
        t1=EvacuationOfPolynomialInPluckerNTimesListForPair(N,i,n)
        r=r+t1
        
    return r

def PromotionOfPolynomialInPluckerNTimesListForPairList(N,ll,n): 
    r=[]
    for i in ll:
        t1=PromotionOfPolynomialInPluckerNTimesListForPair(N,i,n)
        r=r+t1
    
    return r

def PluckerPolynomialTableauExpansionsList(ll,n):
    r=[]
    for i in ll:
        t1=PluckerPolynomialTableauExpansions(i,n)
        r=r+t1
    return r

def ShiftModN(l,a,n): #  l is sent to l+a mod n
    r=Mod(l+a, n)   # range of Mod() is from 0 to n-1
    return r

def ShiftModNKTimes(l,a,n,k):  
    r=l
    r=ShiftModN(r, k*a, n) 
    return r

def CrossProduct(l): # l=[p1,p2,...,pk], cross product of p1,p2,...,pk, pi are k+1 \times 1 matrices

    k=len(l)
    n=k+1
    #r=Matrix(n,1)
    r=np.zeros((n,1), dtype=float)

    #x=Matrix(k+1,k)
    x=np.zeros((k+1,k), dtype=float)
    
    for i in [0..k]:
        for j in [0..k-1]:
            x[i,j]=l[j][i,0]
    #print('x',x)

    for i in [1..n]:
        t1=SetDifference(ListAToN(1,n), [i])
        t2=ListAToN(1,k)
        #print('x,i,t1,t2',x,i,t1,t2,(-1)^(i-1)*Minor(x,t1,t2)) 
        r[i-1,0]=(-1)^(i-1)*Minor(x,t1,t2)
    
    return r

def MarshScottTwistHalf(i1,p): # p is a k times n matrix
 
    k,n=p.shape
    #print(k,n)
    P=[]
    for j in range(1,k): 
        t1=np.zeros((k+1,1), dtype=float)
        #print(t1)
        t2=ShiftModNKTimes(i1, -1, n, k-j)
        #print('t2', t2)
        for d in range(k):
            t1[d,0]=p[d,t2] 
        #print('t1',t1)
        P.append(t1) 
 
    r=CrossProduct(P) 

    return r

def MarshScottTwist(p): # p is a k times n matrix, p is np.array([[],[]])
    #print(p) 
    k,n=p.shape
    #print(k,n)

    r=np.zeros((k,n), dtype=float)
    #r=Matrix(k,n)
    #print(r)

    for i in range(n):
        #print(i, p)
        t1=MarshScottTwistHalf(i,p) 
        #print('t1',t1)
        for d in [0..k-1]: 
            r[d,i]=t1[d,0] 
            
    return r
     
def merge_sort(array):
    # If the input array contains fewer than two elements,
    # then return it as the result of the function
    if len(array) < 2:
        return array

    midpoint = len(array) // 2

    # Sort the array by recursively splitting the input
    # into two equal halves, sorting each half and merging them
    # together into the final result
    return merge(
        left=merge_sort(array[:midpoint]),
        right=merge_sort(array[midpoint:]))
    
def merge(left, right):
    # If the first array is empty, then nothing needs
    # to be merged, and you can return the second array as the result
    if len(left) == 0:
        return right

    # If the second array is empty, then nothing needs
    # to be merged, and you can return the first array as the result
    if len(right) == 0:
        return left

    result = []
    index_left = index_right = 0

    # Now go through both arrays until all the elements
    # make it into the resultant array
    while len(result) < len(left) + len(right):
        # The elements need to be sorted to add them to the
        # resultant array, so you need to decide whether to get
        # the next element from the first or the second array
        if left[index_left] <= right[index_right]:
            result.append(left[index_left])
            index_left += 1
        else:
            result.append(right[index_right])
            index_right += 1

        # If you reach the end of either array, then you can
        # add the remaining elements from the other array to
        # the result and break the loop
        if index_right == len(right):
            result += left[index_left:]
            break

        if index_left == len(left):
            result += right[index_right:]
            break

    return result

def CrossProductResultInMinorIndices(l): # l=[l_1,l_2,l_3,...,l_k], l_i is an integer, denote the l_i-th column of a matrix 
    k=len(l)
    n=k+1
    r=[]
    
    for i in [1..n]:
        t1=SetDifference(ListAToN(1,n), [i])
        r.append([(-1)^(i-1), t1, l])
    
    return r

def MarshScottTwistHalfResultInMinorIndices(i1,k,n):  

    P=[]
    for j in range(1,k):  
        t2=ShiftModNKTimes(i1, -1, n, k-j)+1
        t2=ModResultOneToN(t2,n)
        P.append(t2)
    P1=[]
    for i in [1..n]:
        if IsInList(i, P)==1:
            P1.append(i)
    r=CrossProductResultInMinorIndices(P1) 

    return r

def MarshScottTwistResultInMinorIndices(k,n): 
    r=[]
    for i in [1..k]:
        t1=[]
        for j in [1..n]:
            t1.append(0)
        r.append(t1)      # r[i] are rows of r, r is a k by n matrix written in terms of list of lists
    
    for i in range(n): 
        t1=MarshScottTwistHalfResultInMinorIndices(i,k,n) 
        #print('t1',t1)
        for d in [0..k-1]: 
            r[d][i]=t1[d] 
            
    return r

def TransposeOfRectangularTableau(l): # l=[l_1, l_2, ...], l_i is a list
    r=[]
    if l!=[]:
        n=len(l)
        m=len(l[0])
        for j in [0..m-1]:
            t1=[]
            for i in [0..n-1]:
                t1.append(l[i][j])
            r.append(t1)
    return r

def ContentOfMarshScottTwistOfPlucker(l,k,n): # l=P=[a,b,c,d] are indices of a Plucker coordinate, Grkn, return content of tau(P)
    r1=MarshScottTwistResultInMinorIndices(k,n)
    r=[]
    for i in [0..k-1]: 
        r=r+r1[i][l[i]-1][2]
    r.sort()
    
    return r
 
def MarshScottTwistOfPlucker(l,k,n): # l=P=[a,b,c,d] are indices of a Plucker coordinate, Grkn, return content of tau(P)
 
    r2=ContentOfMarshScottTwistOfPlucker(l,k,n)
    m=len(r2)/k
    r3=[]
    for i in [0..m-1]:
        t1=[]
        for j in [0..k-1]:
            t1.append(r2[i*k+j])
        r3.append(t1)
    r5=NonCrossingTupleWithSameContentAsT_new(r3)
    n1=len(r5)
    p1=np.zeros([n1,n1])
    p2=np.zeros([n1])
    for j1 in range(n1): 
        M=randomMatrixNonInt(k,n) 
        tM=MarshScottTwist(np.array(M)) 
        p2[j1]=PluckerToMinors(tM, [1, [l]]) 
        for p in [0..n1-1]: 
            p1[j1, p]=PluckerToMinors(M, [1, r5[p]]) 
    t1=np.linalg.solve( p1,p2 )

    t5=[] 
    for q in [0..n1-1]:
        t5.append([int(round(t1[q])), r5[q]])
    r=RemoveZeroTerms(t5) 
    
    return r

def MarshScottTwistOfPluckerTopTerm(l,k,n): # l=P=[a,b,c,d] are indices of a Plucker coordinate, Grkn, return content of tau(P)
    r1=MarshScottTwistOfPlucker(l,k,n)
    r=HighestWeightTableauInPolynomialOfTableaux(r1, 'A', k-1)
    r=SortRowsOfTableau(r[1]) 
    return r

def MarshScottTwistOfPluckerTopTermWithoutFrozen(l,k,n): # l=P=[a,b,c,d] are indices of a Plucker coordinate, Grkn, return content of tau(P)
    r1=MarshScottTwistOfPlucker(l,k,n)
    r=HighestWeightTableauInPolynomialOfTableaux(r1, 'A', k-1)
    r=SortRowsOfTableau(r[1])
    r=RemoveTrivialTableau2(r,k,n)
    return r

def IsTotallyPositive(g):
    r=1

    n=g.nrows()
    m=g.ncols()

    for i in [1..min(n,m)]:
        for t1 in NchooseM(ListAToN(1,n), i):
            for t2 in NchooseM(ListAToN(1,m), i):
                t3=Minor(g, sorted(t1), sorted(t2))
                
                if t3<=0:
                    print(t3,sorted(t1),sorted(t2))
                    r=0 
                    break

    return r 

def IsTotallyNonNegative(g):
    r=1

    n=g.nrows()
    m=g.ncols()

    for i in [1..min(n,m)]:
        for t1 in NchooseM(ListAToN(1,n), i):
            for t2 in NchooseM(ListAToN(1,m), i):
                t3=Minor(g, sorted(t1), sorted(t2))
                if t3<0:
                    r=0 
                    break
    return r


def Multiplyq2(l,q):
 
    r=[]
    for i in l:
        temp=i
        temp[0]=simplify(q*temp[0])
        r.append(temp)
 
    return r

def Multiplyq2NoChangeValue(l,q):
 
    r=[]
    for i in l:
        t1=[simplify(q*i[0]), i[1]]
        r.append(t1)
 
    return r

def PluckersWhichAreOne2(n,n1,m): # also remove 189, 129 in Gr39, n is number of rows of tableau, Gr(n,n1)
  
    r=[]
    for i in [1..m]: 
        t1=[]
        for j in [i..n+i-1]:
            t1=append(t1, j)
             
        r=append(r, t1)
         

    for j in [1..n]:
        t1=[]
        for i in [1..j]:
            t1=append(t1, i)
             
        for s in [n1-n+j+1..n1]:
            t1=append(t1, s)
             
        r=append(r, t1)
         
    return r


def PluckersWhichAreOne(n,m):
    r=[]
    for i in range(1,m+1):
        t1=[]
        for j in range (i,n+i):
            t1.append(j)
        r.append(t1)
    return r

def RemoveTrivialTableau(a1, n,m): # n is the row of tableau, m can take sufficient large
    r=a1
    t2=PluckersWhichAreOne(n,m);
    sn=1
    while sn!=0:
        r1=r;
        for i in range(len(t2)):
            #print(r, t2[i])
            if r==[]:
                break
            elif IsTableauAFactorOfB([t2[i]], r)==1:
                
                r=TableauDivision(r, [t2[i]]);
                
        if r1==r or r==[]:
            sn=0
            break
        else:
            sn=1
        
    
    return r

def IsSubsetAsMultiset(a,l):
    r=1
    for i in a:
        t1=NumberOfA(a,i)-NumberOfA(l,i)
        #print(t1)
        if t1>0:
            r=0
            break
    return r

def IsTableauAFactorOfB(a,b):
    t1=TableauToMatrix(a);
    t2=TableauToMatrix(b);
    r1=MatrixTakeRows(t1);
    r2=MatrixTakeRows(t2);

    r=1;
    for i in range(len(r1)):
        if IsSubsetAsMultiset(r1[i], r2[i])==0:
            r=0
            break
    return r

def NumberOfA(l,a):
    r=0;
    for i in l:
        if i==a:
            r=r+1
    return r

def IsCompatiblePairTableauxInMatrix(T1, T2,n):
 

    r=0

    if IsLCTTPrimeNew(T1, T2,n)==1 and IsLCTTPrimeNew(T2, T1,n)==1:
        r=1 

    return r 

def IsLCTTPrimeNew(T1, T2,n):
 
    g1=TableauToSmallGapTableauSeparated(T1)
    g2=TableauToSmallGapTableauSeparated(T2)

    #print(g1,g2)

    r=0

    a=TheSetXTTPrime(T1, T2)
    b=TheSetYTTPrime(T1, T2,n)

    #print(a,b)

    #print(lengthOfList(a),lengthOfList(b))
    if len(a)>len(b):
        r=0
    elif len(a)==0:
        r=1
    else:

        r1=[]
        for pp1 in [0..len(a)-1]:
            p1=a[pp1]
            t1=[]
            for pp2 in [0..len(b)-1]:
                p2=b[pp2]
                i=p1[0]-1
                j=p1[1]-1
                i1=p2[0]-1
                j1=p2[1]-1
                if ( i==i1 and IsWeaklySeparated(g2[j1], g2[j])==0 and g2[j][0,0]<g2[j1][0,0] ) or ( j==j1 and IsWeaklySeparated(g1[i], g1[i1])==0 and g1[i1][0,0]<g1[i][0,0] ):
                    t1.append(pp2)
            
            if t1==[]: # can speed up the computations
                r=0
                return r
                break
            r1.append(t1)

        #print(r1)

        #r3=CartesianProductOfListsInAList(r1)
        #print('len(r3)', len(r3))
        #print(r3)

#         for i in r3:
#             #print(i)
#             t1=removeDuplicatesListOfLists(list(i))
#             #print(i,t1)
#             if len(t1)==len(a):
#                 #print(t1)
#                 r=1
#                 return r
#                 break
        bipartite=matrix_from_edge_set_of_bipartite_graph(r1,len(a),len(b))
        g=BipartiteGraphMaxMatching(bipartite)
        num=g.maxBPM()
        if num==len(a):
            r=1
        else:
            r=0

    return r 
    
def TheSetXTTPrime(T1, T2):

    t1=TableauToSmallGapTableauSeparated(T1)
    t2=TableauToSmallGapTableauSeparated(T2)

    #print(t1,t2)

    r=[]
    for i in [0..lengthOfList(t1)-1]: 
        for j in [0..lengthOfList(t2)-1]: 
            r1=MatrixTakeColumns(t1[i])[0]
            r2=MatrixTakeColumns(t2[j])[0]
            if IsWeaklySeparated(r1,r2)==0 and t2[j][0,0]<t1[i][0,0]:
                r=append(r, [i+1,j+1])

    return r 
    
def TheSetYTTPrime(T1, T2,n):
 
    t1=TableauToSmallGapTableauSeparated(T1)
    t2=TableauToSmallGapTableauSeparated(T2)

    #print(t1,t2)

    r=[]
    for i in [0..lengthOfList(t1)-1]: 
        for j in [0..lengthOfList(t2)-1]:
            r0=t1[i]
            #print('r0',r0)
            g1=SemistandardTableau(MatrixTakeRows(r0))
            #print('g1',g1)
            r1=g1.promotion(n)
            r1=MatrixTakeColumns(Matrix(r1))[0]
            r2=MatrixTakeColumns(t2[j])[0]
            #print('r1,r2',r1,r2)
            if IsWeaklySeparated( r1, r2 )==0 and t2[j][0,0]<=r0[0,0]:
                r=append(r, [i+1,j+1])

    return r 
    
def CartesianProductOfListsInAList(l): # l=[l1,l2,...], l1 is a set written in terms of a list
 
    r=[]
    for i in itertools.product(*l):
        r.append(i)

    return r 

def MatrixToTableauTakeColumnsNPArray(a):  # a is np array
 
    t1=list(a.shape)
    n=t1[0]
    m=t1[1]
    #print(n,m,a)
    r=[]
    for i in [0..m-1]:
        t2=[]
        for j in [0..n-1]:
            t2.append(a[j,i])
        
        r.append(t2)
   

    return r

def MatrixToTableauTakeColumns(a): 
 

    n=a.nrows()
    m=a.ncols()

    r=[]
    for i in [0..m-1]:
        t2=[]
        for j in [0..n-1]:
            t2=append(t2,a[j,i])
        
        r=append(r, t2)
   

    return r

def SortEachRowOfMatrix(l):

    r=[]
    for i in [0..l.nrows()-1]: 
        r1=[]
        for j in [0..l.ncols()-1]:
            r1=append(r1, l[i,j])
             
        r=append(r, sorted(r1))
        

    r=Matrix(r)

    return r 

def SortEachRowOfMatrixList(l):
 
    r=[]
    for i in l:
        r=append(r, SortEachRowOfMatrix(i)) 

    return r 

def CompatibleNTuplesPluckerIncludingFrozenIncludingIEqualJNIsOne(k,n): 
 
 
    t1=NchooseM(ListAToN(1,n),k)
    t2=[]
    for i in t1:
        t2=append(t2, sorted(i))
         

    r=t2

    return r 

def CompatibleNTuplesPluckerIncludingFrozenIncludingIEqualJCasesFaster(N, k,n): # compute n-tuples of plucker which are compatible, the n-tuple can be (I, I, J), (I, I, I).


    if N==1:

        r=CompatibleNTuplesPluckerIncludingFrozenIncludingIEqualJNIsOne(k,n)

    elif N==2:

        t3=CompatibleNTuplesPluckerIncludingFrozenIncludingIEqualJNIsOne(k,n)

        r1=[]
        for i in [0..lengthOfList(t3)-1]:
            for j in [i..lengthOfList(t3)-1]:
                if IsWeaklySeparated(t3[i],t3[j])==1:
                    #print(t3[i],t3[j])
                    r1=append(r1, [t3[i],t3[j]])
                     
        r=r1


    elif N==3:

        t3=CompatibleNTuplesPluckerIncludingFrozenIncludingIEqualJNIsOne(k,n)
        r1=[]

        for i1 in [0..lengthOfList(t3)-1]:
            for i2 in [i1..lengthOfList(t3)-1]:
                for i3 in [i2..lengthOfList(t3)-1]:
                    if IsWeaklySeparated(t3[i1],t3[i2])==1 and IsWeaklySeparated(t3[i2],t3[i3])==1  and IsWeaklySeparated(t3[i1],t3[i3])==1:
                        r1=append(r1, [t3[i1],t3[i2],t3[i3]])
                         

        r=r1

    elif N==4:

        t3=CompatibleNTuplesPluckerIncludingFrozenIncludingIEqualJNIsOne(k,n)
        r1=[]

        for i1 in [0..lengthOfList(t3)-1]:
            for i2 in [i1..lengthOfList(t3)-1]:
                for i3 in [i2..lengthOfList(t3)-1]:
                    for i4 in [i3..lengthOfList(t3)-1]:
                        if IsWeaklySeparated(t3[i1],t3[i2])==1 and IsWeaklySeparated(t3[i2],t3[i3])==1  and IsWeaklySeparated(t3[i3],t3[i4])==1 and IsWeaklySeparated(t3[i1],t3[i3])==1 and IsWeaklySeparated(t3[i1],t3[i4])==1 and IsWeaklySeparated(t3[i2],t3[i4])==1:
                            r1=append(r1, [t3[i1],t3[i2],t3[i3],t3[i4]])
                         

        r=r1

    elif N==5:

        t3=CompatibleNTuplesPluckerIncludingFrozenIncludingIEqualJNIsOne(k,n)
        r1=[]

        for i1 in [0..lengthOfList(t3)-1]:
            for i2 in [i1..lengthOfList(t3)-1]:
                for i3 in [i2..lengthOfList(t3)-1]:
                    for i4 in [i3..lengthOfList(t3)-1]:
                        for i5 in [i4..lengthOfList(t3)-1]:
                            if IsWeaklySeparated(t3[i1],t3[i2])==1 and IsWeaklySeparated(t3[i1],t3[i3])==1 and IsWeaklySeparated(t3[i1],t3[i4])==1 and IsWeaklySeparated(t3[i1],t3[i5])==1 and IsWeaklySeparated(t3[i2],t3[i3])==1 and IsWeaklySeparated(t3[i2],t3[i4])==1         and IsWeaklySeparated(t3[i2],t3[i5])==1 and IsWeaklySeparated(t3[i3],t3[i4])==1 and IsWeaklySeparated(t3[i3],t3[i5])==1 and IsWeaklySeparated(t3[i4],t3[i5])==1:
                                r1=append(r1, [t3[i1],t3[i2],t3[i3],t3[i4],t3[i5]])
        
        r=r1


    elif N==6:

        t3=CompatibleNTuplesPluckerIncludingFrozenIncludingIEqualJNIsOne(k,n)
        r1=[]

        for i1 in [0..lengthOfList(t3)-1]:
            for i2 in [i1..lengthOfList(t3)-1]:
                for i3 in [i2..lengthOfList(t3)-1]:
                    for i4 in [i3..lengthOfList(t3)-1]:
                        for i5 in [i4..lengthOfList(t3)-1]:
                            for i6 in [i5..lengthOfList(t3)-1]:
                                if IsWeaklySeparated(t3[i1],t3[i2])==1 and IsWeaklySeparated(t3[i1],t3[i3])==1 and IsWeaklySeparated(t3[i1],t3[i4])==1 and IsWeaklySeparated(t3[i1],t3[i5])==1        and IsWeaklySeparated(t3[i1],t3[i6])==1 and IsWeaklySeparated(t3[i2],t3[i3])==1         and IsWeaklySeparated(t3[i2],t3[i4])==1 and IsWeaklySeparated(t3[i2],t3[i5])==1         and IsWeaklySeparated(t3[i2],t3[i6])==1 and IsWeaklySeparated(t3[i3],t3[i4])==1 and IsWeaklySeparated(t3[i3],t3[i5])==1 and IsWeaklySeparated(t3[i3],t3[i6])==1         and IsWeaklySeparated(t3[i4],t3[i5])==1 and IsWeaklySeparated(t3[i4],t3[i6])==1 and IsWeaklySeparated(t3[i5],t3[i6])==1:
                                    r1=append(r1, [t3[i1],t3[i2],t3[i3],t3[i4],t3[i5],t3[i6]])
                                
        r=r1 
    return r 

def ExtendSetOfTableauxToContainEvacuationsInMatrix(l,n): # l is a list of tableaux in matrix form
    r=[]
    for i in l:
        t1=EvacuationOfTableauNTimesInMatrix(n,i,n)
        r=r+t1 
    r=removeDuplicatesListOfLists(r)
    
    return r

def ExtendSetOfTableauxToContainEvacuationsInMatrix2(l,n): # l is a list of tableaux in matrix form
    r=[]
    for i in l:
        t1=EvacuationOfTableauNTimesInMatrix(n,i,n)
        r=r+t1 
    r=removeDuplicates2(r)
    
    return r

def EvacuationOfPolynomialInPluckerNTimesListForPair(N,ll,n): # ll=[T,l], T is a list of lists, l = ch(T) = [[1,[[],[]]], [-1,[[],[]]]]  Gr(k,n), collect Evacuations 0 time, 1 time, 2 times, ..., N-1 times
    T=ll[0]
    l=ll[1]
    k=len(T[0])
    
    M=RandomTotallyNonNegativeG(k,n,2,633)
    t1=ll[1]
    g1=PluckerToMinorsPolynomial(M, t1)
    if g1<0:
        #print('t1',t1)
        t1=Multiplyq2(t1,-1)

    r=[[ll[0], t1]]
    
    for i in [1..N-1]:
        #print(T)
        t0=EvacuationOfTableauNTimesTheNthTisListOfLists(i,T,n)
        #print(t0)
        t1=EvacuationOfPolynomialInPluckerNTimes(i,l,n)
        
        
        M=RandomTotallyNonNegativeG(k,n,2,633)
        g1=PluckerToMinorsPolynomial(M, t1)
        if g1<0:
            #print('t1',t1)
            t1=Multiplyq2(t1,-1)
            #print('t1',t1)
        
        r.append([t0,t1])
    #r=removeDuplicatesListOfLists(r) # sometimes has problem when removing duplicates
    return r
    
def EvacuationOfPolynomialInPluckerNTimes(N,l,n): # Gr(k,n), add N to indices and sort, terms in l are noncrossing does not imply that the resulting polynomial has noncrossing terms
    r=[]
    for i in l:
        t1=i[0]
        t2=i[1]
        t3=[]
        for j in t2:
            #print('j',j,TableauToMatrix([j]))
            t4=EvacuationOfTableauNTimesInMatrix(N+1,TableauToMatrix([j]),n)[-1]
            t3.append(MatrixTakeColumns(t4)[0]) 
        r.append([t1, t3]) 
    return r
    
def EvacuationOfTableauNTimesTheNthTisListOfLists(N,T1,n): # T1 is a list of lists
    r2=TableauToMatrix(T1)
    r3=SemistandardTableau(MatrixTakeRows(r2))
    r4=EvacuationOfTableauNTimesTheNth(N,r3,n)
    r5=TableauToMatrixTakeRows(r4)
    r=MatrixTakeColumns(r5)  
    return r


def EvacuationOfTableauNTimesTheNth(N,T1,n): # T1 is a semistandard tableau
    T=T1
    for i in range(N):
        T=T.evacuation(n)
    r=T
    return r
    
def EvacuationOfTableauNTimes(N,T1,n):
    r=[T1]
    T=T1
    for i in range(N-1):
        T=T.evacuation(n)
        #print('T',T)
        r.append(T)
    return r 

def EvacuationOfTableauNTimesInMatrix(N,T1,n):
    t1=MatrixTakeRows(T1)
    t2=SemistandardTableau(t1)
    #print(t2)
    r1=EvacuationOfTableauNTimes(N,t2,n)
    r=[]
    for i in r1:
        r.append(TableauToMatrixTakeRows(i))
        
    return r 

def TableauxToListOfTimesOfOccurrenceOfNumbersNEntries(n,a): # Gr(k,n)
    r=[]
    k=a.nrows() 
    m=a.ncols() 

    r1=[]
    for i in [0..k-1]:  
        for j in [0..m-1]: 
            r1=append(r1, a[i,j]) 

    for j in [1..n]:
        t1=0
        for i in r1:
            if i==j:
                t1=t1+1 
        r=append(r,t1)


    return r
    
def TableauxToListOfTimesOfOccurrenceOfNumbersTableauIsListNEntries(n,a):
    t1=TableauToMatrix(a)
    r=TableauxToListOfTimesOfOccurrenceOfNumbersNEntries(n,t1)
 
    return r

def TakeRowIOfMatrix(i, l):
    r=[]
    for j in [0..l.ncols()-1]:
        r.append(l[i-1,j])
    return r

# def GVectorOfTableau(k,n,p1,r2): # Grkn, p1 is a tableau in list form, p1[i] is column, r2 is initial cluster variables, in Gr48, r2=[[[1, 2, 3, 5]], [[1, 2, 4, 5]], [[1, 3, 4, 5]], [[1, 2, 3, 6]], [[1, 2, 5, 6]], [[1, 4, 5, 6]], [[1, 2, 3, 7]], [[1, 2, 6, 7]], [[1, 5, 6, 7]], [[1, 2, 3, 4]], [[2, 3, 4, 5]], [[3, 4, 5, 6]], [[4, 5, 6, 7]], [[5, 6, 7, 8]], [[1, 6, 7, 8]], [[1, 2, 7, 8]], [[1, 2, 3, 8]]], 
    
#     c=VariableGenerator('c')
#     var=[]
#     for j in [0..len(r2)-1]:
#         var.append(c[j])
#     pp=[]
#     for i in [1..k]:
#         g1=[]
#         for i1 in [0..n-1]:
#             g1.append(0) 
#         p2=TableauxToListOfTimesOfOccurrenceOfNumbersTableauIsListNEntries(n,[TakeRowIOfMatrix(i,TableauToMatrix(p1))])
#         for j in [0..len(r2)-1]:
#             t1=TableauxToListOfTimesOfOccurrenceOfNumbersTableauIsListNEntries(n,[TakeRowIOfMatrix(i,TableauToMatrix(r2[j]))])
#             g1=AddVlueOfTwoLists(g1, [i * c[j] for i in t1]) 
#         p3=AddVlueOfTwoLists(g1, [i*(-1) for i in p2])
#         pp=pp+p3
#     p5=solve(pp, var, solution_dict=True)
#     t3=[] 
#     t5=[]
#     for q in [0..len(r2)-1]:
#         t5.append([p5[0][c[q]], r2[q]])
#         t3.append(p5[0][c[q]])
#     r=[t3,t5]
    
#     return r

def GVectorOfTableau(k,n,p1,r2): # Grkn, p1 is a tableau in list form, p1[i] is column, r2 is initial cluster variables, in Gr48, r2=[[[1, 2, 3, 5]], [[1, 2, 4, 5]], [[1, 3, 4, 5]], [[1, 2, 3, 6]], [[1, 2, 5, 6]], [[1, 4, 5, 6]], [[1, 2, 3, 7]], [[1, 2, 6, 7]], [[1, 5, 6, 7]], [[1, 2, 3, 4]], [[2, 3, 4, 5]], [[3, 4, 5, 6]], [[4, 5, 6, 7]], [[5, 6, 7, 8]], [[1, 6, 7, 8]], [[1, 2, 7, 8]], [[1, 2, 3, 8]]],

    gg=np.zeros((k*n, len(r2)), dtype=float)
    vv=[]
    #print(gg)
    for i in [1..k]:
        p2=TableauxToListOfTimesOfOccurrenceOfNumbersTableauIsListNEntries(n,[TakeRowIOfMatrix(i,TableauToMatrix(p1))])
        vv=vv+p2
        for j in [0..len(r2)-1]:
            t1=TableauxToListOfTimesOfOccurrenceOfNumbersTableauIsListNEntries(n,[TakeRowIOfMatrix(i,TableauToMatrix(r2[j]))])
            #print(t1)
            for j1 in range(len(t1)):
                gg[(i-1)*n+j1, j]=t1[j1]
    vv=np.array(vv)
    p6= np.linalg.lstsq(gg, vv, rcond=None)[0]

    #print(p6)
    t3=[]
    t6=[]
    for q in [0..len(r2)-1]:
        t2=int(round(p6[q]))
        t6.append([t2, r2[q]])
        t3.append(t2)
   
    r=[t3,t6]
    return r


def lengthOfList(l):
    r=len(l)
    return r

def UnionOfLists(a,b):
    r=a+b
    return r
def append(l,i):
    r=l
    r.append(i)
    return r

def TableauToFundamentalTableauxSLkOneColumn(l1): # l1 is a tableau with one column, l1 is in list form
 
    r=[] 
    l=sorted(l1)
    for i in [0..len(l)-1]:
        if l[i]!=i+1 and l[i]<10000:
            t1=ListAToN(1,i)
            t1=append(t1,l[i])
            r=append(r,t1)
      

    return(r) 

def ReverseList(l):
#     r=[]
#     for i in range(len(l)-1,-1,-1):
#         r=r+[l[i]]
    r=l[::-1]
    
    return r


def TableauToFundamentalTableauxSLk(Y,l): # l is a tableau T, l[i] is column of i of T
 

    r=[]

    for i in [0..len(l)-1]: 
        r=UnionOfLists(r, TableauToFundamentalTableauxSLkOneColumn(l[i]))
    r=sorted(r)
    #print(r)
    r=ReverseList(r)

    return(r) 


def TableauToSmallGapTableauSeparated(a): # a is tableau written in terms of a matrix
    a1=MatrixTakeColumns(a)
    #print(a1)
    r1=TableauToSmallGapTableau(6,a1) # TableauToSmallGapTableau(k,a) does not depend on k
    r=[]
    #print(r1)
    for i in r1:
        r.append(TableauToMatrix([i]))
    return r
        
def PABC(a,b,c,n):
    r=[]
    for i in range(n):
        r.append(0)
    r[0]=b
    for i in range(1,a):
        r[i]=r[i-1]+1
    r[a]=r[a-1]+c
    
    for i in range(a+1,n):
        r[i]=r[i-1]+1
    return r

def CollectJumpPointsForPlucker(a1):
    a=sorted(a1)
    n=len(a)
    r=[]
    for i in range(n-1):
        if a[i+1]-a[i]>1:
            r.append(i)
    return r

def PluckerToKR(a1):
    a=sorted(a1)
    n=len(a)
    t2=CollectJumpPointsForPlucker(a);
    #print(t2)
    r=[]
    for i in t2:
        i1=i+1
        t1=PABC(i1, a[i]-i1+1, a[i+1]-a[i],n);
        r.append(t1)
    return r

def FundamentalModulesToPlucker(a1, n): # a1 is the index of Y_{i,s}, n is the number of rows of the tableau
    i=a1[0];
    s=a1[1];

    r=PABC(n-i, (i-s)/2, 2, n);
    return r

def KRToFundamentalModules(a1):
    a=sorted(a1)
    n=len(a)
    t2=CollectJumpPointsForPlucker(a);
    k=t2[0]
    r=[]
    for i in range(a[k]+1, a[k+1]):
        k1=k+1
        i1=i+1
        t1=PABC(k1, a[0]+i1-a[k]-2, 2,n);
        r.append(t1)
    return r




def ModulesToSmallGapTableau(a1, n): # a1 is the list of indices of a module Y_{i1,s1}...Y_{ir,sr} n is the number of rows of tableau 
    r=[]
    for i in a1:
        t1=[FundamentalModulesToPlucker(i, n)]
        print(t1)
        r=UnionOfTwoTableaux(r, t1);
        
    return r

def ModulesToTableau(a1, n,m): # a1 is the list of indices of a module Y_{i1,s1}...Y_{ir,sr} n is the number of rows of tableau 
# m can take sufficient large
    t1=a1
    r=[]
    for i in t1:
        r=UnionOfTwoTableaux(r, [FundamentalModulesToPlucker(i, n)]);
    
    t2=PluckersWhichAreOne(n,m);
    sn=1
    while sn!=0:
        r1=r
        for i in range(len(t2)):
            if IsTableauAFactorOfB([t2[i]], r)==1:
                r=TableauDivision(r, [t2[i]]);
        if r1==r:
            sn=0
            break
        else:
            sn=1
    return r

def PluckerToFundamentalModules(a1):
    t1=PluckerToKR(a1);
    r=[]
    for i in t1:
        t2=KRToFundamentalModules(i);
        r=r+t2
    return r

def TableauToSmallGapTableau(k,a): # a is a list, a[i]'s are columns of a tableau
    t1=[]
    for i in a:
        t2=PluckerToFundamentalModules(i)
        #print(t2)
        t1=t1+t2
    r=sorted(t1)
    return r

def CollectFirstElementInSmallGapTableau(a1):
    r=[]
    for a in a1:
        a2=sorted(a)
        r.append(a2[0])
    return r

def CollectMissingElementInSmallGapTableau(a1):
    r=[]
    for a2 in a1:
        a=sorted(a2)
        n=len(a)
        #print(a)
        #print(a)
        for i in range(n-1):
            if a[i+1]-a[i]>1:
                r.append((a[i]+a[i+1])/2)
                break
    return r 

def HighestWeightTableauInPolynomialOfTableaux(l,typ,rank): # l is a list, l[i]=[coeff, tableau], rank is k-1 in A_{k-1}, Gr(k,n)
    if l==[]:
        r=[]
    else:
        r=l[0]
        for i in [1..len(l)-1]:
            if compareWeightsTableaux2(r[1], l[i][1], typ,rank)==-1:
                
                #print('r[1]',TableauToModuleHasY(Y,r[1]))
                #print('l[i][1]',TableauToModuleHasY(Y,l[i][1]))
                r=l[i]
                
    return r

def compareWeightsTableaux2(P1,P2,typ,rank): # a,b are tableaux, rank is k-1 in A_{k-1}, Gr(k,n)
    if P1!=[]:
        t1=WeightOfTableau2(P1)
    else:
        t1=[]
        for i in [1..rank]:
            t1.append(0)

    if P2!=[]:
        t2=WeightOfTableau2(P2)
    else:
        t2=[]
        for i in [1..rank]:
            t2.append(0)

    #print(t1,t2)

    r=compareWeights2(t1,t2,typ, rank)

    return r

def WeightOfTableau2(a): # a[i] are columns of the tableau a
    #print('a', a)
    m=len(a)
    n=len(a[0])
    
    #print(m,n)
    
    r=[]
    for i in [1..n-1]:
        r.append(0)
    
    #print(r)
    
    for i in [0..m-1]:
        t1=PluckerToMinimalAff2(a[i])
        #print(t1)
        r=AddVlueOfTwoLists(r,t1) 
        
    return r

def PluckerToMinimalAff2(a1):
 
    r=[]
    a=sorted(a1)
    n=len(a)
    for i in range(n-1,0,-1):
        r.append(a[i]-a[i-1]-1)
     
    #print(r)

    return r

def AddVlueOfTwoLists(a,b):
    r=copy.deepcopy(a)
    for i in [0..len(a)-1]:
        r[i]=a[i]+b[i]
    return r

def compareWeights2(a,b,typ,rank): # a,b are lists
    n=len(a)
    t1=Matrix(n,1)
    for i in range(n): 
        t1[i,0]=a[i] 
    t2=Matrix(n,1)
    for i in range(n): 
        t2[i,0]=b[i] 
    r=compareWeights(t1,t2,typ,rank)

    return r


def TupleWhoseUnionIsT(T): # maybe there are other faster way, T is not necessarily semistandard
    """
    Iterator over the non crossing tuples with same content as T.
    """
    if not T or not T[0]:
        return []

    k = len(T[0])
    m = len(T)  # number of columns of T
    content = sorted(flatten(T))

    # Unfortunately, we need to record yielded solutions to avoid duplicates
    #seen = set()
    r=[]
    for c in candidates(Counter(content), k, m):
        #print(c)
        if SortRowsOfTableau(c)==SortRowsOfTableau(T):
            r.append(sorted(c)) 
    r=removeDuplicatesListOfLists(r)
    return r

def NonCrossingTupleWithSameContentAndWeightAsT(T): # maybe there are other faster way, T can be non semistandard
    r1=NonCrossingTupleWithSameContentAsT_new(T)
    for c in r1:
        if SortRowsOfTableau(c)==SortRowsOfTableau(T):
            r=c
            break
    return r

def PromotionOfPolynomialInPluckerNTimesListForPair(N,ll,n): # ll=[T,l], T is a list of lists, l = ch(T) = [[1,[[],[]]], [-1,[[],[]]]]  Gr(k,n), collect promotions 0 time, 1 time, 2 times, ..., N-1 times
    T=ll[0]
    l=ll[1]
    k=len(T[0])
    
    M=RandomTotallyNonNegativeG(k,n,2,633)
    t1=ll[1]
    g1=PluckerToMinorsPolynomial(M, t1)
    if g1<0:
        #print('t1',t1)
        t1=Multiplyq2(t1,-1)

    r=[[ll[0], t1]]
     
    for i in [1..N-1]:
        t0=PromotionOfTableauNTimesTheNthTisListOfLists(i,T,n)
        t1=PromotionOfPolynomialInPluckerNTimes(i,l,n)
        
        t2=t1
        M=RandomTotallyNonNegativeG(k,n,2,633)
        g1=PluckerToMinorsPolynomial(M, t1)
        if g1<0:
            #print('t1',t1)
            t2=Multiplyq2(t1,-1)
            #print('t2',t2)
        
        r.append([t0,t2])
    #r=removeDuplicatesListOfLists(r) # sometimes has problem when removing duplicates
    return r

def PluckerPolynomialTableauExpansionsOneExpansion(l,b,c): # l is [[1, [[1,3,7],[2,5,8]]], [1, [[1,5,7],[2,6,9]]]]
    r=[]
    for i in l:
        t1=i[0]
        t2=i[1]
        t3=TableauExpansionsHalf(t2,b,c)
        r.append([t1, t3])    
    
    return r

# def PluckerPolynomialTableauExpansions(ll,n): # ll = [T, l], l is ch(T) = [[1, [[1,3,7],[2,5,8]]], [1, [[1,5,7],[2,6,9]]]]
#     T=ll[0]
#     l=ll[1]
#     r1=ContentOfTableau(T)
#     m=len(r1)
#     r2=list(itertools.combinations(list(range(1,n+1)), m)) 
#     r=[]
#     for i in r2:
#         t0=TableauExpansionsHalf(T,r1,i)
#         t1=PluckerPolynomialTableauExpansionsOneExpansion(l,r1,i)
#         r.append([t0, t1])
#     return r

def PluckerPolynomialTableauExpansions(ll,n): # ll = [T, l], l is ch(T) = [[1, [[1,3,7],[2,5,8]]], [1, [[1,5,7],[2,6,9]]]]
    
    T=ll[0]
    l=ll[1]
    k=len(T[0])
    
    r1=ContentOfTableau(T)
    m=len(r1)
    r2=list(itertools.combinations(list(range(1,n+1)), m)) 
    r=[]
    for i in r2:
        t0=TableauExpansionsHalf(T,r1,i)
        t1=PluckerPolynomialTableauExpansionsOneExpansion(l,r1,i)
        
        M=RandomTotallyNonNegativeG(k,n,2,633)
        g1=PluckerToMinorsPolynomial(M, t1)
        if g1<0:
            #print('t1',t1)
            t1=Multiplyq2(t1,-1)
            #print('t1',t1)
        
        r.append([t0, t1])
    return r

def TableauExpansionsHalf(l,b,c): # l[i] is ith column of the tableau
    r=[]
    m=len(l[0])
    n=len(l)
    r=[]
    
    #print(b,c)
    
    for i in range(n):
        t1=[]
        for j in range(m):
            t2=b.index(l[i][j])
            t1.append(c[t2])
        r.append(t1)

    return r


def TableauReduceEntriesToOneToN(l):
    b=ContentOfTableau(l)
    c=ListAToN(1,len(b))
    r=TableauExpansionsHalf(l,b,c)
    return r

def TableauExpansions(l,n): # l[i] is a column
    r1=ContentOfTableau(l)
    m=len(r1)
    r2=list(itertools.combinations(list(range(1,n+1)), m)) 
    r=[]
    for i in r2:
        t1=TableauExpansionsHalf(l,r1,i)
        r.append(t1)
    return r

def TableauExpansionsList(l,n): # l is a list of tableaux in list form
    r=[]
    for i in l:
        r=r+TableauExpansions(i,n)
    r=removeDuplicatesListOfLists(r)  
    
    return r

def NumberOfInversions(arr):
    inv_count = 0
    n=len(arr)
    for i in range(n):
        for j in range(i + 1, n):
            if (arr[i] > arr[j]):
                inv_count += 1
  
    return inv_count

def PromotionOfPolynomialInPlucker(l,n): # Gr(k,n), add 1 to indices and sort
    r=[]
    for i in l:
        t1=i[0]
        t2=i[1]
        t3=[]
        for j in t2:
            t4=ShiftEveryIndexByNMondMList(j,1,n)
            t5=sorted(t4)
            t1=t1*(-1)^NumberOfInversions(t4)
            #print(t1)
            t3.append(t5)
        r.append([t1, t3]) 
    return r

def PromotionOfPolynomialInPluckerNTimes(N,l,n): # Gr(k,n), add N to indices and sort, terms in l are noncrossing does not imply that the resulting polynomial has noncrossing terms
    r=[]
    for i in l:
        t1=i[0]
        t2=i[1]
        t3=[]
#         for j in t2:
#             t4=ShiftEveryIndexByNMondMList(j,N,n)
#             t5=sorted(t4)
#             t1=t1*(-1)^NumberOfInversions(t4)
#             #print(t1)
#             t3.append(t5)

        for j in t2:
            t4=PromotionOfTableauNTimesInMatrix(N+1,TableauToMatrix([j]),n)[-1]
            t3.append(MatrixTakeColumns(t4)[0]) 
            
        r.append([t1, t3]) 
    return r
    
def PromotionOfPolynomialInPluckerNTimesList(N,l,n): # Gr(k,n), collect promotions 0 time, 1 time, 2 times, ..., N-1 times
    r=[l]
    for i in [1..N-1]:
        t1=PromotionOfPolynomialInPluckerNTimes(i,l,n)
        r.append(t1)
    
    return r

def ShiftEveryIndexByNMondMList(l,n,m):
    r=[]
    for i in l:
        t1=ModResultOneToN(i+n, m)
        r.append(t1)
    return r

def ShiftEveryIndexByNMondMListOfLists(l,n,m):
    r=[]
    for i in l:
        t1=ShiftEveryIndexByNMondMList(i,n,m)
        r.append(t1)
    return r

def ShiftEveryIndexInPluckerPolynomialByNModM(l, n,m):
    r=[]
    for i in l:
        t1=ShiftEveryIndexByNMondMListOfLists(i[1],n,m)
        r.append([i[0], t1])
    return r

def ModResultOneToN(k,n):
    if k%n!=0:
        r=k%n
    else:
        r=n
    return r

def ChooseTableauxWithSmallOrEqualWeight(T, l,typ,rank):
    r=[]
    for i in l:
        if compareWeightsTableaux(T, i, typ, rank)==1: # weight of T is larger or equal to weight of i
            r.append(i)
    return r

def merge_two_dicts(d1, d2):  
    d={}
    d2_keys_not_in_d1 = d2.keys() - d1.keys()
    d1_keys_not_in_d2 = d1.keys() - d2.keys()
    common_keys = d2.keys() & d1.keys()

    for i in common_keys:
        d[i]= removeDuplicates( d1[i]+d2[i] )
    for i in d1_keys_not_in_d2:
        d[i]=d1[i]
    for i in d2_keys_not_in_d1:
        d[i]=d2[i]
    return d
  
def merge_two_dicts_update(x, y):
    z = x.copy()   # start with keys and values of x
    z.update(y)    # modifies z with keys and values of y
    return z

def detSelfDefined(M):
    M = [row[:] for row in M] # make a copy to keep original M unmodified
    N, sign, prev = len(M), 1, 1
    for i in range(N-1):
        if M[i][i] == 0: # swap with another row having nonzero i's elem
            swapto = next( (j for j in range(i+1,N) if M[j][i] != 0), None )
            if swapto is None:
                return 0 # all M[*][i] are zero => zero determinant
            M[i], M[swapto], sign = M[swapto], M[i], -sign
        for j in range(i+1,N):
            for k in range(i+1,N):
                assert ( M[j][k] * M[i][i] - M[j][i] * M[i][k] ) % prev == 0
                M[j][k] = ( M[j][k] * M[i][i] - M[j][i] * M[i][k] ) // prev
        prev = M[i][i]
    return sign * M[-1][-1]

def UnionOfTableauxMany(a):
 
    r=0

    if a!=[]:
        r=a[0]
        for i in [1..len(a)-1]:
            r=UnionOfTwoTableaux(r, a[i])
         
    return r

# def NonCrossingTupleWithSameContentAsT(T): # T = [[1,2,3,5,8],[2,3,4,8,9]] is a tableau
#     r=[]
#     if T != []:
#         k=len(T[0])
#         m=len(T) # number of columns of T
#         n=0
#         for i in T:
#             if n<max(i):
#                 n=max(i)
    
#     for i in itertools.combinations_with_replacement(itertools.combinations(ListAToN(1,n),k), m):
#         if ContentOfTableauWithMultiplicities(i)==ContentOfTableauWithMultiplicities(T) and IsNonCrossingList(i)==1:
#             r.append(i)
#     #r=removeDuplicatesListOfLists(r) 
    
#     return r

# def NonCrossingTupleWithSameContentAsT(T):
#     """
#     Iterator over the non crossing tuples with same content as T.
#     """
#     if not T or not T[0]:
#         return

#     k = len(T[0])
#     m = len(T)  # number of columns of T
#     content = sorted(flatten(T))

#     # Unfortunately, we need to record yielded solutions to avoid duplicates
#     seen = set()

#     for c in candidates(Counter(content), k, m):
#         if IsNonCrossingList(c):
#             t = tuple(sorted(c))
#             if t not in seen:
#                 yield t
#                 seen.add(t)

def NonCrossingTupleWithSameContentAsT(T):
    """
    Iterator over the non crossing tuples with same content as T.
    """
    if not T or not T[0]:
        return []

    k = len(T[0])
    m = len(T)  # number of columns of T
    content = sorted(flatten(T))

    # Unfortunately, we need to record yielded solutions to avoid duplicates
    #seen = set()
    r=[]
    for c in candidates(Counter(content), k, m):
        if IsNonCrossingList(c):
            r.append(tuple(sorted(c)))
#             if t not in seen:
#                 yield t
#                 seen.add(t)
    r=removeDuplicatesListOfLists(r)
    return r

# def IsNonCrossingList(l): # I1,J1 have the same size
    
#     r=1
#     if len(l)>=2:
#         for i in itertools.combinations(l, 2):
#             if IsNonCrossing(i[0],i[1])==0:
#                 r=0
#                 return r
    
#     return r 

def IsNotWeaklySeparatedForAnyPairInList(L): # any pair is not weakly separated
    r=0
    if len(L)>=2:
        r=1
        for i in itertools.combinations(L, 2):
            if IsWeaklySeparated(i[0],i[1])==1:
                r=0
                return r
    return r

def AtLeastOnePairIsNotWeaklySeparatedInList(L): # at least one pair is not weakly separated
    r=0
    if len(L)>=2:
        r=0
        for i in itertools.combinations(L, 2):
            if IsWeaklySeparated(i[0],i[1])==0:
                r=1
                return r
    return r

def IsNonCrossingList(L):
    return all(IsNonCrossing(I, J) for I, J in itertools.combinations(L, 2))

def candidates(content, k, m):
    """
    Iterator over the m-tuples of k elements from content.

    With this implementation, elements are not repeated in a same tuple.

    INPUT:

    - ``content`` -- a ``Counter`` associated to each possible element it's
      multiplicity

    - ``k`` -- number of elements per tuple

    - ``m`` -- requested number of tuples
    """
    if len(content) < k:
        return
    for I in itertools.combinations(content, k):
        # We ensure that elements inside a k-tuple are sorted
        T = (tuple(sorted(I)), )
        if m == 1:
            yield T
        else:
            for L in candidates(content - Counter(I), k, m - 1):
                yield T + L

def MatrixTakeColumns(a):
    n=a.nrows()
    m=a.ncols()
    r=[]
    for j in range(m):
        t1=a[list(range(n)), [j]]
        t2=[]
        for i in range(n):
            t2.append(t1[i,0])
        r.append(t2)
    return r

# def ComputeChTInTermsOfNonCrossingBasis(l): # noncrossing basis, NonCrossingTupleList is l[0], l[1] is list of pairs (M, value of ch(T) on M)
  
#     c=VariableGenerator('c') 
#     g1=l[0]

#     var=[]
#     for j in [0..len(g1)-1]:
#         var.append(c[j])
    
#     p1=[]
#     for j1 in range(len(l[0])):
#         j=l[1][j1]
#         M=j[0]
#         r3=j[1]
#         r4=0 
#         for p in [0..len(g1)-1]:
#             #print(g1[p])
#             r4=r4+PluckerToMinorsPolynomial(M, [[c[p], g1[p]]]) 
#             #print(r4)
#         #print('r3',r3)
#         #print('r4',r4)
#         p1.append(r3-r4)
#     #print(p1)
#     t1=solve(p1, var, solution_dict=True)
#     #print('t1',t1)
#     t5=[]
#     #t5.append([1, i[0]])
#     for q in [0..len(g1)-1]:
#         t5.append([round(t1[0][c[q]]), g1[q]])
#     t5=RemoveZeroTerms(t5)
#     r=t5
    
#     return r

def ComputeChTInTermsOfNonCrossingBasis(l): # noncrossing basis, NonCrossingTupleList is l[0], l[1] is list of pairs (M, value of ch(T) on M)

    g1=l[0]
    n=len(l[0])
    p1=np.zeros([n,n])
    p2=np.zeros([n])
    for j1 in range(n):
        j=l[1][j1]
        M=j[0] 
        #print('j',j)
        p2[j1]=j[1] 
        for p in [0..n-1]:
            #print(g1[p])
            p1[j1, p]=PluckerToMinors2(M, [1, g1[p]]) 
         
    #t1=np.linalg.solve( p1,p2 )
    #t1=scipy.linalg.inv(p1).dot(p2)
    t1=np.linalg.lstsq(p1, p2, rcond=None)[0]
    #print('t1',t1)
    t5=[]
    #t5.append([1, i[0]])
    for q in [0..len(g1)-1]:
        t5.append([int(round(t1[q])), g1[q]])
    r=RemoveZeroTerms(t5) 
    
    return r
def ComputeChTInTermsOfNonCrossingBasisUpToRound100th(l): # noncrossing basis, NonCrossingTupleList is l[0], l[1] is list of pairs (M, value of ch(T) on M)

    g1=l[0]
    n=len(l[0])
    p1=np.zeros([n,n])
    p2=np.zeros([n])
    for j1 in range(n):
        j=l[1][j1]
        M=j[0] 
        #print('j',j)
        p2[j1]=j[1] 
        for p in [0..n-1]:
            #print(g1[p])
            p1[j1, p]=PluckerToMinors2(M, [1, g1[p]]) 
         
    #t1=np.linalg.solve( p1,p2 )
    #print('t1',t1)
    #t1=scipy.linalg.inv(p1).dot(p2)
    t1=np.linalg.lstsq(p1, p2, rcond=None)[0]
    #t1=np.linalg.lstsq(p1, p2, rcond=None)[0]
    #t1=scipy.linalg.lstsq(p1, p2)[0] # ??? check
    t5=[]
    #t5.append([1, i[0]])
    for q in [0..len(g1)-1]:
        t5.append([round(t1[q],3), g1[q]])
    r=RemoveZeroTerms(t5) 
    
    return r


class VariableGenerator(object): 
     def __init__(self, prefix): 
         self.__prefix = prefix 
     @cached_method 
     def __getitem__(self, key): 
         return SR.var("%s%s"%(self.__prefix,key)) 

def NchooseM(s, m):
    r=list(itertools.combinations(s, m))
    return r

def ContentOfTableauWithMultiplicities(l): # l is tableau
    r=[]
    for i in l:
        for j in i:
            r.append(j)
    r=sorted(r)
    return r

# def IsNonCrossing(I1,J1): # I1,J1 have the same size
#     k=len(I1)
#     I2=sorted(I1)
#     J2=sorted(J1)
#     r=1
#     for a in [0..k-1]:
#         for b in [a+1..k-1]:
#             t1=[]
#             for i in [a..b]:
#                 t1.append(I2[i])
#             t2=[]
#             for j in [a..b]:
#                 t2.append(J2[j])
#             t3=[]
#             for i in [a+1..b-1]:
#                 t3.append(I2[i])
#             t4=[]
#             for j in [a+1..b-1]:
#                 t4.append(J2[j])
             
            
#             if IsWeaklySeparated(t1,t2)==0 and set(t3)==set(t4):
#                 return(0)
#     return r 

def IsNonCrossing(I,J): # I,J have the same size
    k = len(I)

    for a in range(k - 1):
        for b in range(a + 1, k):
            if (I[a+1:b] == J[a+1:b] and
                    not IsWeaklySeparated(I[a:b+1], J[a:b+1])):
                return False

    return True

def IsIncreasingOrdered(l): # is there j such that l_j<...<l_n<l_1<...<l_{j-1}
 

    r=1
    n=len(l)
    for j in [0..n-2]:
        if l[j]>l[j+1]:
            r=0
            break
         
    return r 


# def IsWeaklySeparated(I1,J1): # I1,J1 have the same size
 

#     r1=sorted(SetDifference(I1,J1))
#     r2=sorted(SetDifference(J1,I1))

#     if r1==[] and r2==[]:
#         r=1
#     else:
#         #print(r1,r2)
#         if min(r1)<min(r2):
#             c1=r1
#             c2=r2
#         else: 
#             c1=r2
#             c2=r1 
            
#         r=0

#         for i in [0..len(c1)-1]:
#             t1=[]
#             for j in [0..i]:
#                 t1.append(c1[j])
                
#             t1=t1+c2
#             for j in [i+1..len(c1)-1]:
#                 t1.append(c1[j])
                 
#             if IsIncreasingOrdered(t1)==1:
#                 r=1
#                 break
             
#     return r 

def IsWeaklySeparated(I,J): # I,J have the same size
#     r1 = sorted(set(I).difference(J))
#     r2 = sorted(set(J).difference(I))
    r1=sorted(SetDifference(I,J))
    r2=sorted(SetDifference(J,I))

    if not r1 or not r2:
        return True

    if min(r1) >= min(r2):
        r1, r2 = r2, r1

    # Search for an index i such that list r1[:i+1] + r2 + r1[i+1:] is sorted
    for i in range(len(r1) - 1):
        if r2[0] < r1[i+1]:
            # We check if list r1[:i+1] + r2 + r1[i+1:] is ordered
            return r2[-1] <= r1[i+1]

    # It remains to check if list r1 + r2 is ordered
    return r1[-1] <= r2[0]
 
def QaRoots(k,a):
    r1=0
    for i in a:
        r1=r1+i^2
    r2=0
    for i in a:
        r2=r2+i
    r=r1+(2-k)/(k^2)*r2^2

    return r


def RemoveZeroTerms(l):
 
    r=[]
    for i in l:
        if (i[0]!=0 and i[1]!=0):
            r.append(i)
        
    return r

def QaRootsOfTableaux(k,l): # l is a tableaux written as a matrix
    r1=TableauxToListOfTimesOfOccurrenceOfNumbers(l)
    r=QaRoots(k,r1)

    return r

def TableauToMatrixTakeRows(a): # a[i] are rows
    m=len(a)
    n=len(a[0])
    r=Matrix(m,n)
    for i in range(m):
        for j in range(n):
            r[i,j]=a[i][j]
    return r

def PromotionOfTableauNTimesTheNth(N,T1,n): # T1 is a semistandard tableau
    T=T1
    for i in range(N):
        T=T.promotion(n-1)
    r=T
    return r

def PromotionOfTableauNTimesTheNthTisListOfLists(N,T1,n): # T1 is a list of lists
    r2=TableauToMatrix(T1)
    r3=SemistandardTableau(MatrixTakeRows(r2))
    r4=PromotionOfTableauNTimesTheNth(N,r3,n)
    r5=TableauToMatrixTakeRows(r4)
    r=MatrixTakeColumns(r5)  
    return r

def PromotionOfTableauNTimes(N,T1,n):
    r=[T1]
    T=T1
    for i in range(N-1):
        T=T.promotion(n-1)
        r.append(T)
    return r 

def PromotionOfTableauNTimesInMatrix(N,T1,n):
    t1=MatrixTakeRows(T1)
    t2=SemistandardTableau(t1)
    r1=PromotionOfTableauNTimes(N,t2,n)
    r=[]
    for i in r1:
        r.append(TableauToMatrixTakeRows(i))
        
    return r 

def PluckerToMinimalAff(a1):
    r=[]
    a=sorted(a1)
    n=len(a)
    for i in range(n-1,0,-1):
        r.append(a[i]-a[i-1]-1)
    r.append(a[0]-1)

    return r

def PluckerToMinors(A,l):
    r=l[0]
    #print('l[1]',l[1])
    for i in l[1]:
        #print(i,r)
        r=r*Minor(A, ListAToN(1, len(i)), i) 
        #r=r.expand().factor()
        r=r.factor()
    return r

def PluckerToMinorsPolynomial(A,l):
    r=0
    for i in l:
        r=r+PluckerToMinors(A,i) 
        #print(r)
    #r=r.expand().factor()
    r=r.factor()
    return r


def PluckerToMinors2(A,l):
    r=l[0]
    #print('l[1]',l[1])
    for i in l[1]:
        #print(i,r)
        r=r*Minor2(A, ListAToN(1, len(i)), i) 
        #r=r.expand().factor()
    return r

def PluckerToMinorsPolynomial2(A,l):
    r=0
    for i in l:
        r=r+PluckerToMinors2(A,i) 
        #print(r)
    #r=r.expand().factor()
    return r




def subMatrix(m, rows, cols):
    result = []
    for row in rows:
        newRow = []
        for col in cols:
            newRow.append(m[row-1][col-1])
        result.append(newRow)
    #print(result)
    r=Matrix(np.random.rand( len(rows), len(cols) ))
    for i in range(len(rows)):
        for j in range(len(cols)):
            r[i,j]=result[i][j]
    
    return r


def Minor2(M, rows, cols):
    #r=np.linalg.det(subMatrix(M,rows, cols))
    r=det(subMatrix(M,rows, cols))
    #r=detSelfDefined(subMatrix(M,rows, cols))
    return r 

def Minor(M, rows, cols):
    #r=np.linalg.det(subMatrix(M,rows, cols))
    #r=det(subMatrix(M,rows, cols))
    r=det(sub_matrix_general(M,rows, cols))
    #r=detSelfDefined(subMatrix(M,rows, cols))
    return r

def InitialCluster(rank,n,MatrixGList): # Gr(rank,n)
    
    sizeColumn=n-rank
    k=sizeColumn    
    k1=rank
    p1=k*rank+1
    mat=Matrix(p1,p1)
    for i in range(p1): 
        i1=i+1
        if i1==1: 
            mat[i,i+k+1]=1
            mat[i, p1-1]=1
            mat[i,i+k]=-1
            mat[i, i+1]=-1
        elif i1>=2 and i1<=k-1: 
            mat[i,i+1]=-1
            mat[i,i+k]=-1
            mat[i,i-1]=1
            mat[i,i+k+1]=1
        elif i1==k: 
            mat[i,i-1]=1
            mat[i,i+k]=-1
        elif i1>k and i1<(rank-1)*k and i1 % k==1: 
            mat[i,i-k]=1
            mat[i,i+k+1]=1
            mat[i,i+1]=-1
            mat[i,i+k]=-1
        elif i1>k and i1<(rank-1)*k and i1 % k >=2 and i1 % k<=k-1:
            mat[i,i-k-1]=-1
            mat[i,i+1]=-1
            mat[i,i+k]=-1
            mat[i,i-k]=1
            mat[i,i-1]=1
            mat[i,i+k+1]=1
        elif i1>=2*k and i1<=(rank-1)*k and i1 % k==0:
            mat[i,i-k-1]=-1
            mat[i,i+k]=-1
            mat[i,i-k]=1
            mat[i,i-1]=1
        elif i1>(rank-1)*k and i1<p1 and i1 % k==1: 
            mat[i,i-k]=1
            mat[i,i+1]=-1
        elif i1>=(rank-1)*k+2 and i1<rank*k:
            mat[i,i-k-1]=-1
            mat[i,i+1]=-1
            mat[i,i-k]=1
            mat[i,i-1]=1
        elif i1==rank*k:
            mat[i,i-k-1]=-1
            mat[i,i-k]=1
            mat[i,i-1]=1
        elif i1==p1:
            mat[i,0]=-1

    vertices0=[]

    for j in range(k1-1,-1,-1):
        for i in range(k1,k1+k):
            t1=list(range(1,j+1))
            t2=list(range(i-k1+j+2,i+2))
            t3=t1+t2
            vertices0.append(t3)

    vertices0.append(list(range(1,k1+1)))  
    verticesTableaux = [] # Tableaux are represented by matrices
    for i in range(len(vertices0)):
        #print('PluckerToMinors(MatrixG, [vertices0[i]])',PluckerToMinors(MatrixG, [vertices0[i]]))
        t1=[]
        for MatrixG in MatrixGList:
            t1.append( PluckerToMinorsPolynomial(MatrixG, [[1,[vertices0[i]]]]) )
        verticesTableaux.append([0, [vertices0[i]], t1, i]) # [vertices0[i]] is an one column tableau
    mat1 = Matrix(p1,p1)
    for i in range(p1):
        for j in range(p1):
            mat1[i,j]=mat[i,j]
    clusterVariables=[] 
    vertices1 = [verticesTableaux, clusterVariables] # vertices1[1] store cluster variables, vertices1[0] store variables on quiver
    r=[mat, vertices1]
    
    return r


def TableauExpansionsInMatrixHalf(l,b,c): # l is tableau in matrix form, b is the content of l, c is a list of numbers
    r=[]
    m=l.nrows()
    n=l.ncols()
    r=Matrix(m,n)
    for i in range(m):
        for j in range(n):
            t1=b.index(l[i,j])+1
            r[i,j]=c[t1-1]
    return r

def TableauExpansionsInMatrix(l,n): # l is tableau in matrix form
    r1=ContentOfTableau(l)
    m=len(r1)
    r2=list(itertools.combinations(list(range(1,n+1)), m)) 
    r=[]
    for i in r2:
        t1=TableauExpansionsInMatrixHalf(l,r1,i)
        r.append(t1)
    return r

def TableauExpansionsInMatrixList(l,n): # l is a list of tableaux in matrix form
    r=[]
    for i in l:
        r=r+TableauExpansionsInMatrix(i,n)
    r=removeDuplicatesListOfLists(r)  
    
    return r

def TableauExpansionsInMatrixList2(l,n): # l is a list of tableaux in matrix form
    r=[]
    for i in l:
        r=r+TableauExpansionsInMatrix(i,n)
    r=removeDuplicates2(r)  
    
    return r


def ContentOfTableau(l): # l is tableau
    r=[]
    for i in l:
        for j in i:
            r.append(j)
    #r=np.unique(r,axis=0)
    r=removeDuplicatesListOfLists(r)
    r=sorted(r)
    return r

def immutabilize(m):
    M = copy.deepcopy(m)
    M.set_immutable()
    return M

def ChangeListOfMatricesToSetOfMatrices(S):
    r={immutabilize(i) for i in S}
    return r

def removeDuplicates(l):
    r=[]
    for i in l:
        if (i in r)==False:
            r.append(i)
    return r

def removeDuplicates2(l): # vary fast, for matrices
    t1=ChangeListOfMatricesToSetOfMatrices(l)
    r=list(dict.fromkeys(t1))
    return r

def removeDuplicatesListOfLists(l): # very fast
    l.sort()
    r=list(l for l,_ in itertools.groupby(l))

    return r


def SetDifference(A,B): # A-B
    r=[d for d in A if d not in B]
    return r 

def SetDifferenceListOfLists(A,B): # A-B, there are other faster ways
    
    r=[d for d in A if d not in B]
    
    return r

def SetDifference2(a,b):
    t1=ChangeListOfMatricesToSetOfMatrices(a)
    t2=ChangeListOfMatricesToSetOfMatrices(b)
    r=t1.difference(t2)
    return r

def SetDifferenceListDifference2(A,B): # A-B, can have duplicate elements
    r=[]
    r1=removeDuplicatesListOfLists(A)
    for i in r1:
        t1=A.count(i)-B.count(i)
        #print(t1)
        for j in range(1,t1+1):
            r.append(i)
            
    return r


def SetDifferenceListDifference(a, b):
    '''
    Return a copy of a with the elements of b removed.

    >>> subtract_multiset([], [0, 1])
    []
    >>> subtract_multiset(["x", "x", "y"], [])
    ['x', 'x', 'y']
    >>> subtract_multiset(["ham", "spam", "eggs", "spam", "spam"], ["spam", "spam"])
    ['ham', 'spam', 'eggs']
    >>> subtract_multiset([0, 0, 1, 1], [0, 1, 1, 1])
    [0]
    >>> subtract_multiset([1, 2, 2, 3], [1, 2, 3, 4])
    [2]
    '''
    diff = Counter(a) - Counter(b)
    return list(diff.elements())


def TableauToMatrix(a):
    r=0
    if a!=[]:
        m=len(a)
        n=len(a[0])
        r=Matrix(n,m)
        for i in range(n):
            for j in range(m):
                r[i,j]=a[j][i]
    return r

def TableauToMatrixFloat(a):
    m=len(a)
    n=len(a[0])
    r=np.zeros((n,m))
    #print(r)
    for i in range(n):
        for j in range(m):
            r[i,j]=a[j][i]
    return r

def TableauToMatrixList(a):
    r=[]
    for i in a:
        r.append(TableauToMatrix(i))
    return r


def MatrixTakeRows(a):
    n=a.nrows()
    m=a.ncols()
    r=[]
    for i in range(n):
        t1=a[[i],list(range(m))]
        t2=[]
        for j in range(m):
            t2.append(t1[0,j])
        r.append(t2)
    return r

def MatrixTakeRowsList(a):
    r=[]
    for i in a:
        r.append(MatrixTakeRows(i))
    return r

def MatrixTakeColumnsList(a):
    r=[]
    for i in a:
        r.append(MatrixTakeColumns(i))
    return r

def TableauDivision(a,b):
#     t1=TableauToMatrix(a)
#     t2=TableauToMatrix(b)  
#     r1=MatrixTakeRows(t1)
#     r2=MatrixTakeRows(t2)
    if b==[]:
        r=a
    else:
        r1=TransposeTableauSLk(a)
        r2=TransposeTableauSLk(b)

    #    print(r1,r2)

        r3=[]
        for i in range(len(r1)):
            t1=sorted(SetDifferenceListDifference(r1[i],r2[i]))
            if len(t1)!=len(r1[i])-len(r2[i]):
                print(a,b)
                print('not divisible')
                break
            else:
                r3.append(t1)
            #print(r1[i],r2[i])
        r=[]
        for i in range(len(r3[0])):
            t1=[]
            for j in range(len(r3)):
                t1.append(r3[j][i])
            r.append(t1)

    return r


def SortRowsOfTableau(l): # l[i] are columns of tableau
    r=[]
    if l!=[]:
        for i in range(len(l[0])):
            r1=[]
            for j in range(len(l)):
                r1.append(l[j][i])
            r.append(sorted(r1))
        r2=r
        r=[]
        for i in range(len(r2[0])):
            r1=[]
            for j in range(len(r2)):
                r1.append(r2[j][i])
            r.append(r1)
    
    return r

def UnionOfTwoTableaux(a,b):
    t1=a+b
    r=SortRowsOfTableau(t1)
    return r


# def UnionOfTwoTableaux(a,b):
#     t1=a+b
#     t2=TableauToMatrix(t1)
#     r=[]
#     for i in range(t2.nrows()):
#         r1=[]
#         for j in range(t2.ncols()):
#             r1.append(t2[i,j])
#         r.append(sorted(r1))
        
#     r2=TableauToMatrix(r);
#     r=[]
#     for i in range(r2.nrows()):
#         r1=[]
#         for j in range(r2.ncols()):
#             r1.append(r2[i,j])
#         r.append(sorted(r1))
#     return r


        
# def PowerOfTableaux(a,n):
#     r=[]
#     if a!=[] and a!=[[]]:
#         for i in range(1,n+1):
#             r=UnionOfTwoTableaux(r,a) 
#     else:
#         r=a 
#     return r

        
def PowerOfTableaux(a,n):
    r=[]
    if a!=[] and a!=[[]]:
        for i in range(n):
            r=r+a
        r=SortRowsOfTableau(r) 
    else:
        r=a 
    return r



def compareWeightsTableaux(P1,P2,typ,rank): # a,b are tableaux
    t1=WeightOfTableau(P1)
    t2=WeightOfTableau(P2)
    r=compareWeights2(t1,t2,typ, rank)

    return r

def WeightOfTableau(a): # a[i] are columns of the tableau a
    m=len(a)
    n=len(a[0])
    r=[]
    for i in range(1,n+1):
        r.append(0)
    for i in range(m):
        t1=PluckerToMinimalAff(a[i])
        #r=list(np.array(r)+np.array(t1))
        r=list(map(add, r, t1))
        
    return r

def compareWeights(a, b, typ, rank): # 
    r=1                             # r=1 means a>=b          
    l=a-b
    c=CartanMatrixSelfDefined(typ, rank)
    for i in range(rank): 
        p=0
        for j in range(rank):
            t1=((transpose(c))^(-1))[j,i]
            p=p+l[j,0]*t1
        if p<0: 
            r=-1              # r=-1 means a is not >= b, it is possible that a<b or a,b are not comparable
            break 
            
    if r==-1:
        for i in range(rank):
            p=0
            for j in range(rank):
                t1=((transpose(c))^(-1))[j,i]
                p=p+l[j,0]*t1
            if p>0: 
                r=0
                break
    return r


def matrixMutation(mat,  k):  # mutate at k
    size=mat.nrows()
    r=Matrix(size,size)
    for i in range(size):
        for j in range(size):
            r[i,j]=mat[i,j]
    
    for i in range(size): 
        for j in range(size): 
            
            if k==i or k==j:
                r[i,j]=-mat[i, j]    
            else: 
                r[i, j] = mat[i, j]+1/2*(abs(mat[i,k])*mat[k,j]+mat[i,k]*abs(mat[k,j]))
     
    return r

def ExtendSetOfTableauxToContainPromotions(l,n): # l is a list of tableaux 
    r=[]
    for i in l:
        t1=PromotionOfTableauNTimes(n,i,n)
        r=r+t1 
    #r=np.unique(r,axis=0)
    r=removeDuplicatesListOfLists(r)

    return r

def ExtendSetOfTableauxToContainPromotionsInMatrix(l,n): # l is a list of tableaux in matrix form
    r=[]
    for i in l:
        t1=PromotionOfTableauNTimesInMatrix(n,i,n)
        r=r+t1 
    r=removeDuplicatesListOfLists(r)
    
    return r


def ExtendSetOfTableauxToContainPromotionsInMatrix2(l,n): # l is a list of tableaux in matrix form
    r=[]
    for i in l:
        t1=PromotionOfTableauNTimesInMatrix(n,i,n)
        r=r+t1 
    r=removeDuplicates2(r)
    
    return r


def TableauxToListOfTimesOfOccurrenceOfNumbers(a):
        r=[]
        n=a.nrows() 
        m=a.ncols() 
        r1=[]
        for i in range(n):  
            for j in range(m): 
                r1.append(a[i,j]) 
        for k in range(1,max(r1)+1):
            t1=0
            for i in r1:
                if i==k:
                    t1=t1+1 
            r.append(t1)  
        return r

def TableauxToListOfTimesOfOccurrenceOfNumbersLengthN(a,N):
    r=[]
    n=a.nrows() 
    m=a.ncols() 
    r1=[]
    for i in range(n):  
        for j in range(m): 
            r1.append(a[i,j]) 
    for k in range(1,N):
        t1=0
        for i in r1:
            if i==k:
                t1=t1+1 
        r.append(t1)
    return r
    
def TableauxToListOfTimesOfOccurrenceOfNumbersLengthNWithContentLessOrEquN(a,N): # compute the occurrences of numbers in i for those i in a such that the numbers in i is less or equal to N
    r=[]
    n=a.nrows() 
    m=a.ncols() 
    r1=[]
    for i in range(n):  
        for j in range(m): 
            r1.append(a[i,j])       
    if max(r1)<=N:
        for k in range(1,N):
            t1=0
            for i in r1:
                if i==k:
                    t1=t1+1 
            r.append(t1)
    return r
    
def TableauxToListOfTimesOfOccurrenceOfNumbersTableauIsList(a):
    t1=TableauToMatrix(a)
    r=TableauxToListOfTimesOfOccurrenceOfNumbers(t1)
 
    return r

def TableauxToListOfTimesOfOccurrenceOfNumbersLengthNTableauIsList(a,N):
    t1=TableauToMatrix(a)
    r=TableauxToListOfTimesOfOccurrenceOfNumbersLengthN(t1,N)
 
    return r

# mutation of Grassmannian cluster variables
def computeEquationsForModulesTableaux(variable2, mat, k, typ, rank): # variable2=(variables on quiver, cluster variables obtained so far)
    variable=variable2[0]
    clusterVariables=variable2[1] 
    size=mat.nrows() 
    newVariable=[]
    newVariable2=[]
    newVariableList=[]
    newVariable2List=[]

    for i in range(size):
        if mat[i, k]>0:
            newVariable=UnionOfTwoTableaux( newVariable, PowerOfTableaux(variable[i][1], mat[i,k]) )
            for j in range(mat[i,k]):
                newVariableList.append(variable[i][1])
 
    for i in range(size): 
        if mat[i, k]<0:
            newVariable2= UnionOfTwoTableaux( newVariable2, PowerOfTableaux(variable[i][1], -mat[i,k]) )
            for j in range(-mat[i,k]):
                newVariable2List.append(variable[i][1])
 
    variable[k][0]=variable[k][0]+1
    t1=compareWeightsTableaux(newVariable, newVariable2,typ,rank)
    
    #print('t1,newVariable,newVariable2,variable[k][1]',[t1,newVariableList,newVariable2List,variable[k][1]]) 

    if t1==1: 
        variable[k][1]=TableauDivision(newVariable, variable[k][1])
    else:
        variable[k][1]=TableauDivision(newVariable2, variable[k][1])  
        
    clusterVariables=TableauToMatrix(variable[k][1])
    #print('new cluster variable', clusterVariables)
    
    r=[variable, clusterVariables]

    return r

def computeEquationsForModulesTableauxWithNumericalMutation(variable2, mat, k, typ, rank): # variable2=(variables on quiver, cluster variables obtained so far)
    variable=variable2[0]
    clusterVariables=variable2[1] 
    size=mat.nrows() 
    newVariable=[]
    newVariable2=[] 

    for i in range(size):
        if mat[i, k]>0:
            newVariable=UnionOfTwoTableaux( newVariable, PowerOfTableaux(variable[i][1], mat[i,k]) ) 
 
    for i in range(size): 
        if mat[i, k]<0:
            newVariable2= UnionOfTwoTableaux( newVariable2, PowerOfTableaux(variable[i][1], -mat[i,k]) ) 
            
    t1=compareWeightsTableaux(newVariable, newVariable2,typ,rank)
 
    if t1==1: 
        variable[k][1]=TableauDivision(newVariable, variable[k][1]) 
    else:
        variable[k][1]=TableauDivision(newVariable2, variable[k][1])   
        
    clusterVariables=TableauToMatrix(variable[k][1])
    
    #print('len(variable[0][2])',len(variable[0][2]))
    for j in range(len(variable[0][2])): # the jth matrix MatrixG
        #print('j',j)
        newVariableN=1
        newVariableN2=1
        for i in range(size):
            if mat[i, k]>0:
                newVariableN=newVariableN*variable[i][2][j]^mat[i,k]
                #print('newVariableN',newVariableN)
        for i in range(size): 
            if mat[i, k]<0:
                newVariableN2=newVariableN2*variable[i][2][j]^(-mat[i,k])
                #print('newVariableN2',newVariableN2)
        variable[k][2][j]=(newVariableN+newVariableN2)/variable[k][2][j]
        #print('j,variable[k][2][j]',j,variable[k][2][j])
    
    r=[variable, clusterVariables, variable[k][2]]

    return r


#Function for multiprocessing
def ll_perms(lli,typ,rank,max_column,n,repeat): 
    b1=[]
    IC=InitialCluster(rank,n)
    mat1=IC[0]
    vertices1=IC[1]
    #print('rr')
    #ll=list(np.array(lli)-1)
    ll=list(map(lambda x: x - 1, lli))
    #print(ll)
    
    mutationSequence=[]
    for j1 in [1..repeat]: ###why do this?, just to repeat the same sequence of mutations, it will give more cluster variables, without this loop is also ok
        mutationSequence=mutationSequence+ll 
        
    for j in range(len(mutationSequence)): 
        #print('kk')
        vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 

        if vertices1[1].ncols()>max_column:
            vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank) ###why repeat above calculation?
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
        else:
            b1.append(vertices1[1]) 
            
#             if MatrixTakeRows(vertices1[1][len(vertices1[1])-1])==[[1,2,2,2],[3,4,4,4],[5,6,6,6],[7,8,8,8],[9,10,10,10]]:
#                 print(vertices1[1][len(vertices1[1])-1])

    b1=removeDuplicatesListOfLists(b1)
    b1=ExtendSetOfTableauxToContainPromotionsInMatrix(b1,n)
    b1=ExtendSetOfTableauxToContainEvacuationsInMatrix(b1,n)
    b1=TableauExpansionsInMatrixList(b1, n)
    
     

    return b1


# def ListAToN(a,n):
#     r=[]
#     for i in range(a,n+1):
#         r.append(i)
#     return r

def ListAToN(a,n):
    r=list(range(a,n+1))
    return r


def randomMatrix(rank,n,minn,maxn): # 

    r=np.random.randint(minn,maxn,(rank, n))
    r=Matrix(r)
    
    return r

def randomMatrixNonInt(rank,n): # rank times n matrix 

    r=np.random.rand(rank,n) 
    r=Matrix(r)

    return r

def RandomTotallyNonNegativeG(k,n, minn, maxn): # k<n

    L=sorted(random.sample(range(minn, maxn), max([k,n])))
    r=Matrix([[xi**i for i in range(len(L))] for xi in L]) 

    return r


def ll_permsWithNumericalMutation(lli,typ,rank,max_column,n,repeat,minn,maxn,NRandomM,CL1,CL2,fp1,fp2,b6,b_noncrossing): 


#def ll_permsWithNumericalMutation(lli,typ,rank,max_column,n,repeat,minn,maxn,NRandomM,CL1,CL2,fp1,b6,b_noncrossing): 
#    b6=b61.copy()
#    b6=b61
#    b_noncrossing=b_noncrossing1.copy()

    #MatrixG=np.random.randint(minn,maxn,(rank, n))
    MatrixGList=[]
    for i in range(NRandomM):
        #t1=RandomTotallyNonNegativeG(k,n,minn,maxn)
        #t1=randomMatrix(k,n,minn,maxn)
        t1=randomMatrixNonInt(k,n)
        MatrixGList.append(t1)
    
    #print('MatrixGList',MatrixGList)
    
    IC=InitialCluster(rank,n,MatrixGList)
    
    #print(IC[1])
    
    mat1=IC[0]
    vertices1=IC[1]
    
    #print('vertices1', vertices1)
    
    
    #print('rr')
    #ll=list(np.array(lli)-1)
    ll=list(map(lambda x: x - 1, lli))
    #print(ll)
    
    mutationSequence=[]
    for j1 in [1..repeat]: 
        mutationSequence=mutationSequence+ll 
    
    F1 = open(fp1,'a+')
    F2 = open(fp2,'a+')
    for j in range(len(mutationSequence)): 
        vertices1 = computeEquationsForModulesTableauxWithNumericalMutation(vertices1, mat1, mutationSequence[j],typ,rank)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 
        
        ii=MatrixTakeColumns(vertices1[1])
        p1=repr(ii)
        p2=b6.get(p1)

#         if vertices1[1].ncols()>max_column or p2==[]:
#             if p2==[]:
#                 print('ii is finished computing', ii)
#             vertices1 = computeEquationsForModulesTableauxWithNumericalMutation(vertices1, mat1, mutationSequence[j],typ,rank) 
#             mat1 = matrixMutation(mat1, mutationSequence[j])
            
        if vertices1[1].ncols()>max_column:
            vertices1 = computeEquationsForModulesTableauxWithNumericalMutation(vertices1, mat1, mutationSequence[j],typ,rank) 
            mat1 = matrixMutation(mat1, mutationSequence[j])
        
        elif vertices1[1].ncols()>=CL1 and vertices1[1].ncols()<=CL2:
            cc=[]
            for i in range(len(MatrixGList)):
                cc.append([ MatrixGList[i], vertices1[2][i] ])
            
            ii=MatrixTakeColumns(vertices1[1])
            p1=repr(ii)
            p2=b6.get(p1)
            #print('p1,p2',p1,p2)
            
            if p2==None:
                ii1=ContentOfTableauWithMultiplicities(ii)
                p3=repr(ii1)
                p4=b_noncrossing.get(p3)
                if p4==None:
                    u0=NonCrossingTupleWithSameContentAsT_new(ii)
                    print('length of noncrossing tuples', len(u0))
                    b_noncrossing[p3]=u0
                    #print(ii, u0)
                    F2.write(str([ii1, u0])+'\n')
                    
                    
                else:
                    print('b_noncrossing already occur')
                    u0=b_noncrossing[p3]
                
                print('b_noncrossing', len(b_noncrossing))
                
                u1=ChooseTableauxWithSmallOrEqualWeight(ii,u0,typ,rank)
                print('length of noncrossing tuples for ii', len(u1))
                
                
                print('cc', len(cc))
                if len(cc)<=len(u1)+3:
                    t2=[u1, cc] 
                    b6[p1]=t2
                elif len(cc)>len(u1)+3:
                    t3=ComputeChTInTermsOfNonCrossingBasis([u1,cc])
                    print('length of variables and length of solutions', len(u1), len(t3))
                    #print(ii,t3)
                    #ee1=EvacuationOfPolynomialInPluckerNTimesListForPairList(n,[[ii,t3]],n)
                    #ee1=PluckerPolynomialTableauExpansionsList(ee1,n)
                    #ee1=EvacuationOfPolynomialInPluckerNTimesListForPairList(n,ee1,n)
                    #ee1=PromotionOfPolynomialInPluckerNTimesListForPairList(n,ee1,n)
                  
                    ee1=PromotionOfPolynomialInPluckerNTimesListForPairList(n,[[ii,t3]],n)
                
                    print('ee1', len(ee1))
                    for i in ee1:
                        q1=repr(i[0])
                        q2=b6.get(q1)
                        if q2!=[]:
                            F1.write(str(i)+'\n')
                            b6[q1]=[]  
                            print('number of cluster variables obtained', len(b6))
                    
            elif len(p2)==2 and len(p2[1])<=len(p2[0])+3:
                t1=removeDuplicatesListOfLists(p2[1]+cc)
                print('len(t1)',len(t1),ii)
                if len(t1) != len(p2[1]) + len(cc):  # avoid singular matrix when solving the linear equation to get coefficients
                    b6[p1]=[p2[0], cc]
                else:
                    if len(t1)<=len(p2[0])+3:
                        b6[p1]=[p2[0], t1]
                    else:
                        t2=ComputeChTInTermsOfNonCrossingBasis([p2[0],t1])
                        #ee1=EvacuationOfPolynomialInPluckerNTimesListForPairList(n,[[ii,t2]],n)                        
                        #ee1=PluckerPolynomialTableauExpansionsList(ee1,n)
                        #ee1=EvacuationOfPolynomialInPluckerNTimesListForPairList(n,ee1,n)
                        #ee1=PromotionOfPolynomialInPluckerNTimesListForPairList(n,ee1,n)
                        
                        ee1=PromotionOfPolynomialInPluckerNTimesListForPairList(n,[[ii,t2]],n)
                        
                        print('ee1', len(ee1))
                        for i in ee1:
                            q1=repr(i[0])
                            q2=b6.get(q1)
                            if q2!=[]:
                                F1.write(str(i)+'\n')
                                b6[q1]=[]  
                                print('number of cluster variables obtained', len(b6))
                                
                            
                            
                        
    F1.close()
    F2.close()
                
 
            

#     b1=removeDuplicates2(b1)
#     b1=ExtendSetOfTableauxToContainPromotionsInMatrix(b1,n)
#     b1=TableauExpansionsInMatrixList(b1, n)
    
     

    return [b6,b_noncrossing]

def initial_cluster_variables_DFGK(k,n): # Grkn
    t1=InitialClusterWithoutNumericalMatrices(k,n)[1][0]
    t2=TakeIthElements(2,t1)
    m=n-k
    r1=[]
    for i in [0..m-2]:
        r2=[]
        for j in [0..k-2]:
            r2.append(t2[j*m+i])
            #print(t2[j*m+i])
        r1=r1+r2
    for i in [m]+[0..m-1]:
            r2=[]
            for j in [k-1]:
                r2.append(t2[j*m+i])
                #print(t2[j*m+i])
            r1=r1+r2
    for i in [m-1]:
            r2=[]
            for j in [0..k-2]:
                r2.append(t2[j*m+i])
                #print(t2[j*m+i])
            r1=r1+r2
    r=r1
    return r



#Function for multiprocessing 
def ll_perms_compute_nonreal_from_clusters_nonreal_only(lli,typ,rank,max_column,n,repeat,b_nonreal,fp2,mutation_number): #  
    IC=InitialClusterWithoutNumericalMatrices(rank,n)
    mat1=IC[0]
    vertices1=IC[1]
    e1=initial_cluster_variables_DFGK(rank,n)
    
    #print('e1',e1)
    
    #print('rr')
    #ll=list(np.array(lli)-1)
    ll=list(map(lambda x: x - 1, lli))
    #print(ll)
    
    mutationSequence=[]
    for j1 in [1..repeat]: 
        mutationSequence=mutationSequence+ll 
    
    for j in range(len(mutationSequence)): 
        #print('kk')
        vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 
        
        if vertices1[1].ncols()>max_column:
            vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank) ###why repeat above calculation?
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
        else:
            ma0=MatrixTakeRows(mat1)
            if len(SetDifference(flatten(ma0), [1,0,-1]))>=2:
                t0=sorted(TakeIthElements(2,vertices1[0]))
                t1=removeAnElementInListList(0,vertices1[0])
                #print('t1',t1)
                t2=[t0, ma0, t1]
                g00=FindNonRealModuleUsingQuiverWithMultipleArrows(rank,n,t2,e1,mutation_number)
                #print(g00)
                if g00!=[]:
                    g0=g00[0][1]
                    if len(g0)<=max_column: # collect only nonreal modules less or equal to max_column columns
                        g1=repr(g0)
                        g2=b_nonreal.get(g1)

                        if g2==None:
                            
                            c1=ExtendSetOfTableauxToContainPromotionsInMatrix([TableauToMatrix(g0)],n) 
                            c1=ExtendSetOfTableauxToContainEvacuationsInMatrix(c1,n) 
                            c1=ExtendSetOfTableauxToContainEvacuationsInMatrix(c1,n) 
                            c1=ExtendSetOfTableauxToContainPromotionsInMatrix(c1,n) 
                            c1=MatrixTakeColumnsList(c1)
                            f1=open(fp2, 'a+')
                            for ci in c1:
                                d1=repr(ci)
                                d2=b_nonreal.get(d1)
                                if d2==None:
                                    f1.write(str(ci)+'\n')
                                    b_nonreal[d1]=[] 
                            f1.close()
    return b_nonreal


def has_elements_same_next_to_each_other(l):
    r=0
    for i in range(len(l)-1):
        if l[i]==l[i+1]:
            r=1
            return r
    return r



def ll_perms_compute_clusters_record_mutation_sequence(lli,typ,rank,max_column,n,repeat,b6,bcluster,T1): #  
    IC=InitialClusterWithoutNumericalMatrices(rank,n)
    mat1=IC[0]
    vertices1=IC[1]
    #print('rr')
    #ll=list(np.array(lli)-1)
    ll=list(map(lambda x: x - 1, lli))
    #print(ll)
    
    ll=list(np.random.permutation(ll))
    mutationSequence=[]
    for j1 in [1..repeat]: 
        mutationSequence=mutationSequence+ll 
    
    mutate_seq=[]
    for j in range(len(mutationSequence)): 
        #print('kk')
        vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 
        mutate_seq.append(mutationSequence[j])
        
        if vertices1[1].ncols()>max_column:
            vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank) ###why repeat above calculation?
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
            del mutate_seq[-1]
        else:
            ii=MatrixTakeColumns(vertices1[1])
            p1=repr(ii)
            p2=b6.get(p1)

            if p2==None:
                #print('ii',ii)
                b6[p1]=[ii,mutate_seq] 

            t0=sorted(TakeIthElements(2,vertices1[0]))
            g1=repr(t0)
            if SetDifference(T1,t0)==[]:
                g2=bcluster.get(g1)

                if g2==None or len(mutate_seq)<len(g2[1]):
                    #print(mutate_seq)
                    t1=removeAnElementInListList(0,vertices1[0])
                    t2=[t0, MatrixTakeRows(mat1), t1]
                    bcluster[g1]=[t2,mutate_seq]


    return [b6, bcluster]

#Function for multiprocessing 
def ll_perms_compute_clusters(lli,typ,rank,max_column,n,repeat,b6,bcluster): #  
    IC=InitialClusterWithoutNumericalMatrices(rank,n)
    mat1=IC[0]
    vertices1=IC[1]
    #print('rr')
    #ll=list(np.array(lli)-1)
    ll=list(map(lambda x: x - 1, lli))
    #print(ll)
    
    mutationSequence=[]
    for j1 in [1..repeat]: 
        mutationSequence=mutationSequence+ll 
    
    for j in range(len(mutationSequence)): 
        #print('kk')
        vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 
        
        if vertices1[1].ncols()>max_column:
            vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank) ###why repeat above calculation?
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
        else:
            ii=MatrixTakeColumns(vertices1[1])
            p1=repr(ii)
            p2=b6.get(p1)

            if p2==None:
                #print('ii',ii)
                b6[p1]=ii 

            t0=sorted(TakeIthElements(2,vertices1[0]))
            g1=repr(t0)
            g2=bcluster.get(g1)

            if g2==None:
                t1=removeAnElementInListList(0,vertices1[0])
                t2=[t0, MatrixTakeRows(mat1), t1]
                bcluster[g1]=t2
 
     
    return [b6, bcluster]


#Function for multiprocessing 
def ll_perms_compute_clusters_fix_steps(lli,typ,rank,n,b6,bcluster,num_steps,fp1,fp2): #  
    IC=InitialClusterWithoutNumericalMatrices(rank,n)
    mat1=IC[0]
    vertices1=IC[1]
    #print('rr')
    #ll=list(np.array(lli)-1)
    ll=list(map(lambda x: x - 1, lli))
    #print(ll)
    
    if has_elements_same_next_to_each_other(ll)==0:
        mutationSequence=ll

        for j in range(len(mutationSequence)): 
            #print('kk')
            vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank)
            mat1 = matrixMutation(mat1, mutationSequence[j]) 

            ii=MatrixTakeColumns(vertices1[1])
            p1=repr(ii)
            p2=b6.get(p1)

            if p2==None:
                #print('ii',ii)
                f1=open(fp1, 'a+')
                f1.write(str(ii)+'\n')
                f1.close()
                b6[p1]=[]

            t0=sorted(TakeIthElements(2,vertices1[0]))
            g1=repr(t0)
            g2=bcluster.get(g1)

            if g2==None:
                f1=open(fp2, 'a+')
                f1.write(str(t0)+'\n')
                f1.close()
                bcluster[g1]=[]


    return [b6, bcluster]


#Function for multiprocessing 
def ll_perms_compute_cluster_variables_only(lli,typ,rank,max_column,n,repeat,b6,fp1): #  
    IC=InitialClusterWithoutNumericalMatrices(rank,n)
    mat1=IC[0]
    vertices1=IC[1]
    #e1=initial_cluster_variables_DFGK(rank,n)
    
    #print('e1',e1)
    
    #print('rr')
    #ll=list(np.array(lli)-1)
    ll=list(map(lambda x: x - 1, lli))
    #print(ll)
    
    mutationSequence=[]
    for j1 in [1..repeat]: 
        mutationSequence=mutationSequence+ll 
    
    for j in range(len(mutationSequence)): 
        #print('kk')
        vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 
        
        if vertices1[1].ncols()>max_column:
            vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank) ###why repeat above calculation?
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
        else:
            ii0=vertices1[1]
            ii=MatrixTakeColumns(vertices1[1])
            p1=repr(ii)
            p2=b6.get(p1)
            
            #print('p1',p1)
            
            if p2==None:
                #print('ii',ii)
                
                c1=ExtendSetOfTableauxToContainPromotionsInMatrix([ii0],n) 
                c1=ExtendSetOfTableauxToContainEvacuationsInMatrix(c1,n) 
                c1=ExtendSetOfTableauxToContainEvacuationsInMatrix(c1,n) 
                c1=ExtendSetOfTableauxToContainPromotionsInMatrix(c1,n) 
                c1=MatrixTakeColumnsList(c1)
                f1=open(fp1, 'a+')
                for ci in c1:
                    g1=repr(ci)
                    g2=b6.get(g1)
                    if g2==None:
                        f1.write(str(ci)+'\n')
                        b6[g1]=[] 
                f1.close()
    return b6

def ll_permsWithNumericalMutation_without_expansion(lli,typ,rank,max_column,n,repeat,minn,maxn,NRandomM,CL1,CL2,fp1,fp2,b6,b_noncrossing): 


#def ll_permsWithNumericalMutation(lli,typ,rank,max_column,n,repeat,minn,maxn,NRandomM,CL1,CL2,fp1,b6,b_noncrossing): 
#    b6=b61.copy()
#    b6=b61
#    b_noncrossing=b_noncrossing1.copy()

    #MatrixG=np.random.randint(minn,maxn,(rank, n))
    MatrixGList=[]
    for i in range(NRandomM):
        #t1=RandomTotallyNonNegativeG(k,n,minn,maxn)
        #t1=randomMatrix(k,n,minn,maxn)
        t1=randomMatrixNonInt(k,n)
        MatrixGList.append(t1)
    
    #print('MatrixGList',MatrixGList)
    
    IC=InitialCluster(rank,n,MatrixGList)
    
    #print(IC[1])
    
    mat1=IC[0]
    vertices1=IC[1]
    
    #print('vertices1', vertices1)
    
    
    #print('rr')
    #ll=list(np.array(lli)-1)
    ll=list(map(lambda x: x - 1, lli))
    #print(ll)
    
    mutationSequence=[]
    for j1 in [1..repeat]: 
        mutationSequence=mutationSequence+ll 
    
    F1 = open(fp1,'a+')
    F2 = open(fp2,'a+')
    for j in range(len(mutationSequence)): 
        vertices1 = computeEquationsForModulesTableauxWithNumericalMutation(vertices1, mat1, mutationSequence[j],typ,rank)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 
        
        ii=MatrixTakeColumns(vertices1[1])
        p1=repr(ii)
        p2=b6.get(p1)

#         if vertices1[1].ncols()>max_column or p2==[]:
#             if p2==[]:
#                 print('ii is finished computing', ii)
#             vertices1 = computeEquationsForModulesTableauxWithNumericalMutation(vertices1, mat1, mutationSequence[j],typ,rank) 
#             mat1 = matrixMutation(mat1, mutationSequence[j])
            
        if vertices1[1].ncols()>max_column:
            vertices1 = computeEquationsForModulesTableauxWithNumericalMutation(vertices1, mat1, mutationSequence[j],typ,rank) 
            mat1 = matrixMutation(mat1, mutationSequence[j])
        
        elif vertices1[1].ncols()>=CL1 and vertices1[1].ncols()<=CL2:
            cc=[]
            for i in range(len(MatrixGList)):
                cc.append([ MatrixGList[i], vertices1[2][i] ])
            
            ii=MatrixTakeColumns(vertices1[1])
            p1=repr(ii)
            p2=b6.get(p1)
            #print('p1,p2',p1,p2)
            
            if p2==None:
                ii1=ContentOfTableauWithMultiplicities(ii)
                p3=repr(ii1)
                p4=b_noncrossing.get(p3)
                if p4==None:
                    u0=NonCrossingTupleWithSameContentAsT_new(ii)
                    print('length of noncrossing tuples', len(u0))
                    b_noncrossing[p3]=u0
                    #print(ii, u0)
                    F2.write(str([ii1, u0])+'\n')
                    
                    
                else:
                    print('b_noncrossing already occur')
                    u0=b_noncrossing[p3]
                
                print('b_noncrossing', len(b_noncrossing))
                
                u1=ChooseTableauxWithSmallOrEqualWeight(ii,u0,typ,rank)
                print('length of noncrossing tuples for ii', len(u1))
                
                
                print('cc', len(cc))
                if len(cc)<=len(u1)+3:
                    t2=[u1, cc] 
                    b6[p1]=t2
                elif len(cc)>len(u1)+3:
                    t3=ComputeChTInTermsOfNonCrossingBasis([u1,cc])
                    print('length of variables and length of solutions', len(u1), len(t3))
                    #print(ii,t3)
                    ee1=[[ii,t3]]
                    #print('ee1', len(ee1))
                    for i in ee1:
                        q1=repr(i[0])
                        q2=b6.get(q1)
                        if q2!=[]:
                            F1.write(str(i)+'\n')
                            b6[q1]=[]  
                            print('number of cluster variables obtained', len(b6))
                    
            elif len(p2)==2 and len(p2[1])<=len(p2[0])+3:
                t1=removeDuplicatesListOfLists(p2[1]+cc)
                print('len(t1)',len(t1),ii)
                if len(t1) != len(p2[1]) + len(cc):  # avoid singular matrix when solving the linear equation to get coefficients
                    b6[p1]=[p2[0], cc]
                else:
                    if len(t1)<=len(p2[0])+3:
                        b6[p1]=[p2[0], t1]
                    else:
                        t2=ComputeChTInTermsOfNonCrossingBasis([p2[0],t1])
                        ee1=[[ii,t2]]
                        
                        #print('ee1', len(ee1))
                        for i in ee1:
                            q1=repr(i[0])
                            q2=b6.get(q1)
                            if q2!=[]:
                                F1.write(str(i)+'\n')
                                b6[q1]=[]  
                                print('number of cluster variables obtained', len(b6))
                                
                            
                            
                        
    F1.close()
    F2.close()
                
 
            

#     b1=removeDuplicates2(b1)
#     b1=ExtendSetOfTableauxToContainPromotionsInMatrix(b1,n)
#     b1=TableauExpansionsInMatrixList(b1, n)
    
     

    return [b6,b_noncrossing]


#Function for multiprocessing 
def ll_perms_compute_nonreal_from_clusters_Yis_record_mutation_sequence(lli,typ,rank,max_degree,repeat,b6,b_nonreal,b_nonreal_seq,fp1,fp2,f3,mutation_number,l): # record mutation sequences of origin quiver for the non-real modules 
    IC=InitialCluster_Yis(typ,rank,l)
    mat1=IC[0]
    vertices1=IC[1]
    e1=initial_cluster_variables_Yis(typ,rank,l)
    
    ll=list(lli)
    #print(ll)
    
    mutationSequence=[]
    for j1 in [1..repeat]: 
        mutationSequence=mutationSequence+ll  
    #print(mutationSequence)
    mutate_seq=[]
    for j in range(len(mutationSequence)): 
        vertices1 = computeEquationsForModulesYis(vertices1, mat1, mutationSequence[j],typ,rank,l)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 
        #print(vertices1[1])
        mutate_seq.append(mutationSequence[j])
    
        if max_degree_of_Yis_in_monomial(vertices1[1])>max_degree:
            vertices1 = computeEquationsForModulesYis(vertices1, mat1, mutationSequence[j],typ,rank,l)
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
            mutate_seq.append(mutationSequence[j])
        else:
            ii=vertices1[1]
            p1=repr(ii)
            p2=b6.get(p1)
            
            #print('p1',p1)
            
            if p2==None:
                #print('ii',ii)
                f1=open(fp1, 'a+')
                f1.write(str(ii)+'\n')
                b6[p1]=[] 
                f1.close()
            
            ma0=MatrixTakeRows(mat1)
            #if len(SetDifference(flatten(ma0), [1,0,-1]))>=2:
            if len(SetDifference(flatten(ma0), [1,0,-1]))==2:
                t0=sorted(TakeIthElements(2,vertices1[0]))
                t1=removeAnElementInListList(0,vertices1[0])
                #print('t1',t1)
                t2=[t0, ma0, t1]
                t3=copy.deepcopy(t2)
                g00=FindNonRealModuleUsingQuiverWithMultipleArrows_Yis(typ,rank,t2,l,e1,mutation_number)
                #print(g00)
                                
                if g00!=[]:
                    
#                     g0=g00[0][1]
#                     g1=repr(g0)
#                     g2=b_nonreal.get(g1)
                    
#                     g3=b_nonreal.get(repr(t0))
#                     if g3==None:
#                         f1=open(fp3, 'a+')
#                         f1.write(str([t0,ma0,t1,g0,mutate_seq])+'\n')
#                         b_nonreal_seq[repr(t0)]=[]
#                         f1.close()
                    
#                     if g2==None:
#                         f1=open(fp2, 'a+')
#                         f1.write(str(g0)+'\n')
#                         b_nonreal[g1]=[] 
#                         f1.close()
                    
                    g0=g00[0][1]
                    if max_degree_of_Yis_in_monomial(g0)<=max_degree: # collect only nonreal modules less or equal to max_column columns
                        g1=repr(g0)
                        g2=b_nonreal.get(g1)

                        g3=b_nonreal.get(repr(t0))
                        if g3==None:
                            f1=open(fp3, 'a+')
                            f1.write(str([t0,ma0,t1,g0,mutate_seq])+'\n')
                            b_nonreal_seq[repr(t0)]=[]
                            f1.close()

                        if g2==None:
                            f1=open(fp2, 'a+')
                            f1.write(str(g0)+'\n')
                            b_nonreal[g1]=[] 
                            f1.close()

    return [b6, b_nonreal, b_nonreal_seq]



#Function for multiprocessing 
def ll_perms_compute_cluster_variables_only_with_mutation_seq(lli,typ,rank,max_column,n,repeat,b6,fp1): #  
    IC=InitialClusterWithoutNumericalMatrices(rank,n)
    mat1=IC[0]
    vertices1=IC[1]
    #e1=initial_cluster_variables_DFGK(rank,n)
    
    #print('e1',e1)
    
    #print('rr')
    #ll=list(np.array(lli)-1)
    ll=list(map(lambda x: x - 1, lli))
    #print(ll)
    
    mutationSequence=[]
    for j1 in [1..repeat]: 
        mutationSequence=mutationSequence+ll 
        
    mutation_seq=[]
    
    for j in range(len(mutationSequence)): 
        #print('kk')
        vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 
        mutation_seq.append(mutationSequence[j])
        
        if vertices1[1].ncols()>max_column:
            vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank) ###why repeat above calculation?
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
            mutation_seq.append(mutationSequence[j])
            
        else:
            ii0=vertices1[1]
            ii=MatrixTakeColumns(vertices1[1])
            p1=repr(ii)
            p2=b6.get(p1)
            
            #print('p1',p1)
            
            if p2==None:
                #print('ii',ii)
                
                ci=[ii, mutation_seq]
                f1=open(fp1, 'a+')
                f1.write(str(ci)+'\n')
                b6[p1]=[] 
                f1.close()
    return b6



def ll_perms_compute_fixed_points_of_cluster_variables(R,x,iii,trop,lli,typ,rank,max_column,n,repeat,b6,fp1): # not finished 
    IC=InitialClusterWithoutNumericalMatrices(rank,n)
    mat1=IC[0]
    vertices1=IC[1]
    #e1=initial_cluster_variables_DFGK(rank,n)
    
    #print('e1',e1)
    
    #print('rr')
    #ll=list(np.array(lli)-1)
    ll=list(map(lambda x: x - 1, lli))
    #print(ll)
    
    mutationSequence=[]
    for j1 in [1..repeat]: 
        mutationSequence=mutationSequence+ll 
    
    for j in range(len(mutationSequence)): 
        #print('kk')
        vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 
        
        if vertices1[1].ncols()>max_column:
            vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank) ###why repeat above calculation?
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
        else:
            ii0=vertices1[1]
            ii=MatrixTakeColumns(vertices1[1])
            p1=repr(ii)
            p2=b6.get(p1)
            
            #print('p1',p1)
            
            if p2==None:
                #print('ii',ii)
                c1=ii0
                c1=MatrixTakeColumns(c1)
                k=rank
                c2=BraidGroupActionSigmaIOnTableau_use_tropical_map(R,x,c1,k,n,trop[iii-1])
                if c2==c1:
                    f1=open(fp1, 'a+')
                    g1=repr(c1)
                    g2=b6.get(g1)
                    if g2==None:
                        f1.write(str(c1)+'\n')
                        b6[g1]=[] 
                    f1.close()
    return b6


#Function for multiprocessing 
def ll_perms_compute_braid_group_action_with_Marsh_Scott_twist(R,x,T_list,lli,max_column,k,n,repeat, b6,fp1,trop, tropMS): # trop =[trop1,trop2,...] trop(sigma_1), ...  
    ll0=lli
    r2=copy.deepcopy(T_list)
    #print(ll0)
    #print(r2)
    
    r1=[]
    for i in list(b6.keys()):
        r1.append(sage_eval(i))
    r2=r2+r1
    
    for i in range(repeat):
        r3=[]
        #print(len(r2))
        for j in r2:
            t1=j
            for ii in np.random.permutation(ll0): 
            #for ii in range(len(trop)): 
                #print('ii',ii, np.random.permutation(ll0))
                t1=BraidGroupActionSigmaIOnTableau_use_tropical_map(R,x,t1,k,n,trop[ii])
                if len(t1)<=max_column:
                    #print('t1', t1)
                    r3.append(t1)
                    p1=repr(t1)
                    p2=b6.get(p1)            
                    if p2==None:
                        f1=open(fp1, 'a+')
                        f1.write(str(t1)+'\n')
                        f1.close()
                        b6[p1]=[] 
                        print(len(b6))
                t1=Marsh_Scott_twist_on_tableau_use_tropical_map(R,x,t1,k,n,tropMS)
                if len(t1)<=max_column:
                    #print('t1', t1)
                    r3.append(t1)
                    p1=repr(t1)
                    p2=b6.get(p1)            
                    if p2==None:
                        f1=open(fp1, 'a+')
                        f1.write(str(t1)+'\n')
                        f1.close()
                        b6[p1]=[] 
                        print(len(b6))
                
        r2=r3
        
    return b6



#Function for multiprocessing 
def ll_perms_compute_cluster_variables_regular_modules(lli,typ,rank,max_column,n,repeat,b6,fp1): #  
    IC=InitialClusterWithoutNumericalMatrices(rank,n)
    mat1=IC[0]
    vertices1=IC[1]
    #e1=initial_cluster_variables_DFGK(rank,n)
    
    #print('e1',e1)
    
    #print('rr')
    #ll=list(np.array(lli)-1)
    ll=list(map(lambda x: x - 1, lli))
    #print(ll)
    
    mutationSequence=[]
    for j1 in [1..repeat]: 
        mutationSequence=mutationSequence+ll 
    
    for j in range(len(mutationSequence)): 
        #print('kk')
        vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 
        
        if vertices1[1].ncols()>max_column:
            vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank) ###why repeat above calculation?
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
        else:
            ii0=vertices1[1]
            ii=MatrixTakeColumns(vertices1[1])
            if IsTableauRegular(rank,ii)==1:
                p1=repr(ii)
                p2=b6.get(p1)

                #print('p1',p1)

                if p2==None:
                    #print('ii',ii)

                    c1=[ii]
                    f1=open(fp1, 'a+')
                    for ci in c1:
                        g1=repr(ci)
                        g2=b6.get(g1)
                        if g2==None:
                            f1.write(str(ci)+'\n')
                            b6[g1]=[] 
                    f1.close()
    return b6


def TableauToModuleList(Y, l): # Y is 'Y'
    r=[]
    for i in l:
        r.append(TableauToModule(Y, i))
    return r

def ll_perms_compute_cluster_variables_regular_modules_exchange_relation_also_regular(lli,typ,rank,max_column,n,repeat,b6,fp1): #  
    IC=InitialClusterWithoutNumericalMatrices(rank,n)
    mat1=IC[0]
    vertices1=IC[1]
    #e1=initial_cluster_variables_DFGK(rank,n)
    
    #print('e1',e1)
    
    #print('rr')
    #ll=list(np.array(lli)-1)
    ll=list(map(lambda x: x - 1, lli))
    #print(ll)
    
    mutationSequence=[]
    for j1 in [1..repeat]: 
        mutationSequence=mutationSequence+ll 
    
    for j in range(len(mutationSequence)): 
        #print('kk')
        vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 
        
        ii0=vertices1[1]
        ii=MatrixTakeColumns(vertices1[1])
        
        if vertices1[1].ncols()>max_column or IsTableauRegular(rank,ii)==0:
            vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank) ###why repeat above calculation?
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
        else:
            p1=repr(ii)
            p2=b6.get(p1)

            if p2==None:
                f1=open(fp1, 'a+')
                f1.write(str(ii)+'\n')
                b6[p1]=[] 
                f1.close()
    return b6

def computeEquationsForModulesTableaux_compute_exchange_relations_any_relation(variable2, mat, k, typ, rank): # variable2=(variables on quiver, cluster variables obtained so far)
    variable=variable2[0]
    clusterVariables=variable2[1] 
    size=mat.nrows() 
    newVariable=[]
    newVariable2=[]
    old_cluster_variable=copy.deepcopy(variable[k][1])
    c1=[]
    d1=[]
    sn=[]

    for i in range(size):
        if mat[i, k]>0:
            newVariable=UnionOfTwoTableaux( newVariable, PowerOfTableaux(variable[i][1], mat[i,k]) )
            for j in range(mat[i,k]):
                c1.append(variable[i][1])
 
    for i in range(size): 
        if mat[i, k]<0:
            newVariable2= UnionOfTwoTableaux( newVariable2, PowerOfTableaux(variable[i][1], -mat[i,k]) )
            for j in range(-mat[i,k]):
                d1.append(variable[i][1])
 
    variable[k][0]=variable[k][0]+1
    t1=compareWeightsTableaux(newVariable, newVariable2,typ,rank)
 
    if t1==1: 
        variable[k][1]=TableauDivision(newVariable, variable[k][1])
    else:
        variable[k][1]=TableauDivision(newVariable2, variable[k][1])  
        
    clusterVariables=TableauToMatrix(variable[k][1]) 
    
    sn=[old_cluster_variable, variable[k][1], sorted([sorted(c1), sorted(d1)])]
    
    r=[variable, clusterVariables, sn]

    return r

def ll_perms_compute_cluster_variables_regular_modules_exchange_relation_also_regular_with_exchange_relations(lli,typ,rank,max_column,n,repeat,b6,fp1): #  
    IC=InitialClusterWithoutNumericalMatrices(rank,n)
    mat1=IC[0]
    vertices1=IC[1]
    #e1=initial_cluster_variables_DFGK(rank,n)
    
    #print('e1',e1)
    
    #print('rr')
    #ll=list(np.array(lli)-1)
    ll=list(map(lambda x: x - 1, lli))
    #print(ll)
    
    mutationSequence=[]
    for j1 in [1..repeat]: 
        mutationSequence=mutationSequence+ll 
    
    record_sequence=[]
    for j in range(len(mutationSequence)): 
        #print('kk')
        vertices1 = computeEquationsForModulesTableaux_compute_exchange_relations_any_relation(vertices1, mat1, mutationSequence[j],typ,rank)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 
        record_sequence.append(mutationSequence[j]+1) # label of the quiver start from 1
        
        ii0=vertices1[1]
        ii=MatrixTakeColumns(vertices1[1])
        
        if vertices1[1].ncols()>max_column or IsTableauRegular(rank,ii)==0:
            vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank) ###why repeat above calculation?
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
            record_sequence.pop()
        else:
            p1=repr(ii)
            p2=b6.get(p1)

            if p2==None:
                p3=vertices1[2]
                p4=TableauToModule('Y',ii)
                p5=TableauToModule('Y',p3[0])
                p6=TableauToModule('Y',p3[1])
                p7=TableauToModuleList('Y',p3[2][0])
                p8=TableauToModuleList('Y',p3[2][1])
                f1=open(fp1, 'a+')
                f1.write(str([ii,p3,record_sequence,p4,[p5,p6,[p7,p8]]])+'\n') 
                b6[p1]=[] 
                f1.close()
    return b6




# mutation of modules of quantum affine algebra of type Dn, C_l^g, l \ge 0, at the moment typ='D'
def computeEquationsForModulesYis_compute_qcharacter(variable0, mat, k, typ, rank, l, R, Y): # variable2=(variables on quiver, cluster variables obtained so far)
    
    #print(variable0)
    variable2=variable0[0]
    variable2_ch=variable0[1]
    variable=variable2[0]
    clusterVariables=variable2[1] 
    size=mat.nrows() 
    newVariable=[]
    newVariable2=[] 
    
    variable_ch=variable2_ch[0]
    clusterVariables_ch=variable2_ch[1] 
    newVariable_ch=1
    newVariable2_ch=1
    
    #print(variable2, variable2_ch)

    for i in range(size):
        if mat[i, k]>0:
            #print('i,variable[i]', i,variable[i])
            #print('power_of_monomial_Yis(variable[i][1], mat[i,k])', power_of_monomial_Yis(variable[i][1], mat[i,k]))
            newVariable=newVariable+power_of_monomial_Yis(variable[i][1], mat[i,k])
            #print('newVariable', newVariable)
            #print(variable_ch[i])
            for j in range(mat[i,k]):
                newVariable_ch=newVariable_ch*variable_ch[i][1]
 
    for i in range(size): 
        if mat[i, k]<0:
            newVariable2=newVariable2 + power_of_monomial_Yis(variable[i][1], -mat[i,k])
            for j in range(-mat[i,k]):
                newVariable2_ch=newVariable2_ch*variable_ch[i][1]
 
    variable[k][0]=variable[k][0]+1
    variable_ch[k][0]=variable_ch[k][0]+1
    variable_ch[k][1]=simplify((newVariable_ch+newVariable2_ch)/variable_ch[k][1])
    
    t1=compare_weights_monomial_Yis(newVariable, newVariable2,typ,rank) 

    if t1==1: 
        variable[k][1]=sorted(monomial_Yis_division(newVariable, variable[k][1]))
    else:
        variable[k][1]=sorted(monomial_Yis_division(newVariable2, variable[k][1])) 
        
    clusterVariables=variable[k][1]
    clusterVariables_ch=variable_ch[k][1]
    #print('new cluster variable', clusterVariables)
    
    r=[[variable, clusterVariables], [variable_ch, clusterVariables_ch]]

    return r

#Function for multiprocessing 
def ll_perms_compute_modules_Yis_compute_qcharacters(lli,typ,rank,max_degree,repeat,b6,fp1,l,variable2_ch,R,Y): #  l \ge 0, at the moment, typ='D'
# variable2_ch = [[[0, ch[[1, -1]], 0],
#   [0, ch[[1, -3], [1, -1]], 1],
#   [0, ch[[2, 0]], 2],
#   [0, ch[[2, -2], [2, 0]], 3],
#   [0, ch[[3, 1]], 4],
#   [0, ch[[3, -1], [3, 1]], 5],
#   [0, ch[[4, 1]], 6],
#   [0, ch[[4, -1], [4, 1]], 7]],
#  []]    

    #print(variable2_ch)
    
    IC=InitialCluster_Yis(typ,rank, l)
    mat1=IC[0]
    variable_ch=copy.deepcopy(variable2_ch)
    vertices1=[IC[1], variable_ch] 
    
    #print(vertices1)
    
    ll=list(lli)
    #print(ll)
    
    mutationSequence=[]
    for j1 in [1..repeat]: 
        mutationSequence=mutationSequence+ll  
    #print(mutationSequence)
        
    for j in range(len(mutationSequence)): 
        vertices1 = computeEquationsForModulesYis_compute_qcharacter(vertices1, mat1, mutationSequence[j],typ,rank,l,R,Y)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 
        #print(vertices1[1])
        
#         for i1 in range(mat1.nrows()):
#             for j1 in range(mat1.ncols()):
#                 if abs(mat1[i1][j1])>=2:
#                     print(MatrixTakeRows(mat1))
#                     print(str(vertices1)+'\n')
    
        if max_degree_of_Yis_in_monomial(vertices1[0][1])>max_degree:
            vertices1 = computeEquationsForModulesYis_compute_qcharacter(vertices1, mat1, mutationSequence[j],typ,rank,l,R,Y)
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
        else:
            ii=vertices1[0][1]  # highest weight
            ii2=vertices1[1][1] # qcharacter
            p1=repr(ii)
            p2=b6.get(p1)
        
            #print('p1',p1)
            
            if p2==None:
                #print('ii',ii)
                f1=open(fp1, 'a+')
                f1.write(str([ii, ii2])+'\n')
                #b6[p1]=[] 
                b6[p1]=[[ii,ii2]] 
    return b6
               
    
    
#Function for multiprocessing, partial flag varieties 
def ll_perms_compute_cluster_variables_only_given_initial_seed(lli,typ,rank,max_column,n,repeat,b6,fp1,mat,vertices): #  
    ll=list(map(lambda x: x - 1, lli))
    #print(ll)
    mat1=copy.deepcopy(mat)
    vertices1=copy.deepcopy(vertices)
    
    mutationSequence=[]
    for j1 in [1..repeat]: 
        mutationSequence=mutationSequence+ll 
    
    for j in range(len(mutationSequence)): 
        #print('kk')
        vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 
        
        if vertices1[1].ncols()>max_column:
            vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank) ###why repeat above calculation?
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
        else:
            ii0=vertices1[1]
            ii=MatrixTakeColumns(vertices1[1])
            p1=repr(ii)
            p2=b6.get(p1)
            
            #print('p1',p1)
            
            if p2==None:
                f1=open(fp1, 'a+')
                f1.write(str(ii)+'\n')
                b6[p2]=[]
                f1.close()
    return b6



def ll_perms_compute_clusters_given_initial_seed(lli,typ,rank,max_column,n,repeat,b6,bcluster,mat1,vertices1): #  need to check more examples 
    ll=list(map(lambda x: x - 1, lli))
    #print(ll)
    
    mutationSequence=[]
    for j1 in [1..repeat]: 
        mutationSequence=mutationSequence+ll 
    
    for j in range(len(mutationSequence)): 
        #print('kk')
        vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 
        
        if vertices1[1].ncols()>max_column:
            vertices1 = computeEquationsForModulesTableaux(vertices1, mat1, mutationSequence[j],typ,rank) ###why repeat above calculation?
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
        else:
            ii=MatrixTakeColumns(vertices1[1])
            p1=repr(ii)
            p2=b6.get(p1)

            if p2==None:
                #print('ii',ii)
                b6[p1]=ii 

            t0=sorted(TakeIthElements(2,vertices1[0]))
            g1=repr(t0)
            g2=bcluster.get(g1)

            if g2==None:
                t1=removeAnElementInListList(0,vertices1[0])
                t2=[t0, MatrixTakeRows(mat1), t1]
                bcluster[g1]=t2
 
     
    return [b6, bcluster]


def ChTUsingKLPolynomialInSage_in_Grkn_tableaux_basis(T,k,n):
    g1=ChTUsingKLPolynomialInSage(T,k)
    typ='A'
    rank=k-1
    g2=HighestWeightTableauInPolynomialOfTableaux(g1,typ,rank)
    g2=g2[1]
    r2=TableauDivision(g2,T)

    r0=T
    e1=TableauxWithSameContentAsT_new(r0)
    p1=[]
    for i in [1..len(e1)]:
        M=randomMatrixNonInt(k,n)
        t1=PluckerToMinorsPolynomial2(M, g1)
        t2=PluckerToMinorsPolynomial2(M, [[1,r2]])
        #print(t2)
        t3=t1*t2^(-1)
        #print(g1)
        p1.append([M, t3])
    r=ComputeChTInTermsOfNonCrossingBasisUpToRound100th([ e1, p1 ])
    return r

In [4]:
# # Gr39 
 
# fp1='/Users/jianrongli/Dropbox/Omer/NumericalSmallRankModulesGr39R60ClusterVariables.txt'
# with open(fp1, 'r') as fp:
#     L = [sage_eval(line) for line in fp.readlines() if line.strip()]

In [8]:
# L0=removeDuplicatesListOfLists(L)
# len(L0)

# L1=[]
# for i in L0:
#     if len(i)==4:  
#         L1.append(i)
# len(L1)

In [9]:
# the following are cluster variables for Gr(3,9) corresponding to tableaux with 4 columns
L1=[[[1, 2, 4], [1, 2, 5], [3, 6, 8], [3, 7, 9]],
 [[1, 2, 4], [1, 2, 5], [3, 6, 8], [4, 7, 9]],
 [[1, 2, 4], [1, 4, 7], [3, 5, 8], [3, 6, 9]],
 [[1, 2, 4], [1, 5, 7], [2, 5, 8], [3, 6, 9]],
 [[1, 2, 4], [1, 5, 7], [2, 6, 8], [3, 6, 9]],
 [[1, 2, 4], [1, 5, 7], [2, 6, 8], [3, 7, 9]],
 [[1, 2, 4], [1, 5, 7], [3, 5, 8], [3, 6, 9]],
 [[1, 2, 4], [1, 5, 7], [3, 5, 8], [4, 6, 9]],
 [[1, 2, 4], [1, 5, 7], [3, 6, 8], [3, 6, 9]],
 [[1, 2, 4], [1, 5, 7], [3, 6, 8], [3, 7, 9]],
 [[1, 2, 4], [1, 5, 7], [3, 6, 8], [4, 6, 9]],
 [[1, 2, 4], [1, 5, 7], [3, 6, 8], [4, 7, 9]],
 [[1, 2, 4], [1, 5, 8], [2, 6, 8], [3, 7, 9]],
 [[1, 2, 4], [1, 5, 8], [2, 6, 9], [3, 7, 9]],
 [[1, 2, 4], [1, 5, 8], [3, 6, 8], [3, 7, 9]],
 [[1, 2, 4], [1, 5, 8], [3, 6, 8], [4, 7, 9]],
 [[1, 2, 4], [1, 5, 8], [3, 6, 9], [3, 7, 9]],
 [[1, 2, 4], [1, 5, 8], [3, 6, 9], [4, 7, 9]],
 [[1, 2, 4], [2, 4, 7], [3, 5, 8], [3, 6, 9]],
 [[1, 2, 4], [2, 5, 7], [3, 5, 8], [3, 6, 9]],
 [[1, 2, 4], [2, 5, 7], [3, 5, 8], [4, 6, 9]],
 [[1, 2, 4], [2, 5, 7], [3, 6, 8], [3, 6, 9]],
 [[1, 2, 4], [2, 5, 7], [3, 6, 8], [3, 7, 9]],
 [[1, 2, 4], [2, 5, 7], [3, 6, 8], [4, 6, 9]],
 [[1, 2, 4], [2, 5, 7], [3, 6, 8], [4, 7, 9]],
 [[1, 2, 4], [2, 5, 8], [3, 6, 8], [3, 7, 9]],
 [[1, 2, 4], [2, 5, 8], [3, 6, 8], [4, 7, 9]],
 [[1, 2, 4], [2, 5, 8], [3, 6, 9], [3, 7, 9]],
 [[1, 2, 4], [2, 5, 8], [3, 6, 9], [4, 7, 9]],
 [[1, 2, 4], [3, 5, 7], [3, 5, 8], [4, 6, 9]],
 [[1, 2, 4], [3, 5, 7], [3, 6, 8], [4, 6, 9]],
 [[1, 2, 4], [3, 5, 7], [3, 6, 8], [4, 7, 9]],
 [[1, 2, 4], [3, 5, 8], [3, 6, 8], [4, 7, 9]],
 [[1, 2, 4], [3, 5, 8], [3, 6, 9], [4, 7, 9]],
 [[1, 2, 5], [1, 2, 5], [3, 6, 8], [4, 7, 9]],
 [[1, 2, 5], [1, 2, 6], [3, 7, 8], [4, 7, 9]],
 [[1, 2, 5], [1, 2, 6], [3, 7, 8], [4, 8, 9]],
 [[1, 2, 5], [1, 2, 6], [3, 7, 9], [4, 8, 9]],
 [[1, 2, 5], [1, 3, 5], [4, 6, 8], [4, 7, 9]],
 [[1, 2, 5], [1, 3, 6], [2, 3, 7], [4, 8, 9]],
 [[1, 2, 5], [1, 3, 6], [2, 4, 7], [4, 8, 9]],
 [[1, 2, 5], [1, 3, 6], [2, 4, 7], [5, 8, 9]],
 [[1, 2, 5], [1, 3, 6], [3, 6, 8], [4, 7, 9]],
 [[1, 2, 5], [1, 3, 6], [3, 7, 8], [4, 7, 9]],
 [[1, 2, 5], [1, 3, 6], [3, 7, 8], [4, 8, 9]],
 [[1, 2, 5], [1, 3, 6], [3, 7, 9], [4, 8, 9]],
 [[1, 2, 5], [1, 3, 6], [4, 6, 8], [4, 7, 9]],
 [[1, 2, 5], [1, 3, 6], [4, 6, 8], [5, 7, 9]],
 [[1, 2, 5], [1, 3, 6], [4, 7, 8], [4, 7, 9]],
 [[1, 2, 5], [1, 3, 6], [4, 7, 8], [4, 8, 9]],
 [[1, 2, 5], [1, 3, 6], [4, 7, 8], [5, 7, 9]],
 [[1, 2, 5], [1, 3, 6], [4, 7, 8], [5, 8, 9]],
 [[1, 2, 5], [1, 3, 6], [4, 7, 9], [4, 8, 9]],
 [[1, 2, 5], [1, 3, 6], [4, 7, 9], [5, 8, 9]],
 [[1, 2, 5], [1, 5, 7], [3, 6, 8], [4, 6, 9]],
 [[1, 2, 5], [1, 5, 7], [3, 6, 8], [4, 7, 9]],
 [[1, 2, 5], [1, 5, 8], [3, 6, 8], [4, 7, 9]],
 [[1, 2, 5], [1, 5, 8], [3, 6, 9], [4, 7, 9]],
 [[1, 2, 5], [1, 6, 7], [3, 6, 8], [4, 7, 9]],
 [[1, 2, 5], [1, 6, 8], [3, 6, 8], [4, 7, 9]],
 [[1, 2, 5], [1, 6, 8], [3, 6, 9], [4, 7, 9]],
 [[1, 2, 5], [1, 6, 8], [3, 7, 8], [4, 7, 9]],
 [[1, 2, 5], [2, 3, 5], [4, 6, 8], [4, 7, 9]],
 [[1, 2, 5], [2, 3, 6], [3, 6, 8], [4, 7, 9]],
 [[1, 2, 5], [2, 3, 6], [3, 7, 8], [4, 7, 9]],
 [[1, 2, 5], [2, 3, 6], [3, 7, 8], [4, 8, 9]],
 [[1, 2, 5], [2, 3, 6], [3, 7, 9], [4, 8, 9]],
 [[1, 2, 5], [2, 3, 6], [4, 6, 8], [4, 7, 9]],
 [[1, 2, 5], [2, 3, 6], [4, 6, 8], [5, 7, 9]],
 [[1, 2, 5], [2, 3, 6], [4, 7, 8], [4, 7, 9]],
 [[1, 2, 5], [2, 3, 6], [4, 7, 8], [4, 8, 9]],
 [[1, 2, 5], [2, 3, 6], [4, 7, 8], [5, 7, 9]],
 [[1, 2, 5], [2, 3, 6], [4, 7, 8], [5, 8, 9]],
 [[1, 2, 5], [2, 3, 6], [4, 7, 9], [4, 8, 9]],
 [[1, 2, 5], [2, 3, 6], [4, 7, 9], [5, 8, 9]],
 [[1, 2, 5], [2, 5, 7], [3, 6, 8], [4, 6, 9]],
 [[1, 2, 5], [2, 5, 7], [3, 6, 8], [4, 7, 9]],
 [[1, 2, 5], [2, 5, 8], [3, 6, 8], [4, 7, 9]],
 [[1, 2, 5], [2, 5, 8], [3, 6, 9], [4, 7, 9]],
 [[1, 2, 5], [2, 6, 7], [3, 6, 8], [4, 7, 9]],
 [[1, 2, 5], [2, 6, 8], [3, 6, 8], [4, 7, 9]],
 [[1, 2, 5], [2, 6, 8], [3, 6, 9], [4, 7, 9]],
 [[1, 2, 5], [2, 6, 8], [3, 7, 8], [4, 7, 9]],
 [[1, 2, 5], [3, 5, 7], [3, 6, 8], [4, 6, 9]],
 [[1, 2, 5], [3, 5, 7], [3, 6, 8], [4, 7, 9]],
 [[1, 2, 5], [3, 5, 7], [4, 6, 8], [4, 6, 9]],
 [[1, 2, 5], [3, 5, 7], [4, 6, 8], [4, 7, 9]],
 [[1, 2, 5], [3, 5, 8], [3, 6, 8], [4, 7, 9]],
 [[1, 2, 5], [3, 5, 8], [3, 6, 9], [4, 7, 9]],
 [[1, 2, 5], [3, 5, 8], [4, 6, 8], [4, 7, 9]],
 [[1, 2, 5], [3, 6, 7], [3, 6, 8], [4, 7, 9]],
 [[1, 2, 5], [3, 6, 8], [3, 6, 8], [4, 7, 9]],
 [[1, 2, 5], [3, 6, 8], [3, 7, 9], [4, 7, 9]],
 [[1, 2, 5], [3, 6, 8], [3, 7, 9], [4, 8, 9]],
 [[1, 2, 5], [3, 6, 8], [4, 6, 9], [4, 7, 9]],
 [[1, 2, 5], [3, 6, 8], [4, 6, 9], [5, 7, 9]],
 [[1, 2, 5], [3, 6, 8], [4, 7, 8], [4, 7, 9]],
 [[1, 2, 5], [3, 6, 8], [4, 7, 8], [5, 7, 9]],
 [[1, 2, 5], [3, 6, 8], [4, 7, 9], [4, 7, 9]],
 [[1, 2, 5], [3, 6, 8], [4, 7, 9], [4, 8, 9]],
 [[1, 2, 5], [3, 6, 8], [4, 7, 9], [5, 7, 9]],
 [[1, 2, 5], [3, 6, 8], [4, 7, 9], [5, 8, 9]],
 [[1, 2, 6], [1, 3, 6], [2, 4, 7], [5, 8, 9]],
 [[1, 2, 6], [1, 3, 6], [4, 7, 8], [5, 7, 9]],
 [[1, 2, 6], [1, 3, 6], [4, 7, 8], [5, 8, 9]],
 [[1, 2, 6], [1, 3, 6], [4, 7, 9], [5, 8, 9]],
 [[1, 2, 6], [1, 3, 7], [2, 4, 7], [5, 8, 9]],
 [[1, 2, 6], [1, 3, 7], [4, 7, 8], [5, 8, 9]],
 [[1, 2, 6], [1, 3, 7], [4, 7, 9], [5, 8, 9]],
 [[1, 2, 6], [1, 3, 7], [4, 8, 9], [5, 8, 9]],
 [[1, 2, 6], [2, 3, 6], [4, 7, 8], [5, 7, 9]],
 [[1, 2, 6], [2, 3, 6], [4, 7, 8], [5, 8, 9]],
 [[1, 2, 6], [2, 3, 6], [4, 7, 9], [5, 8, 9]],
 [[1, 2, 6], [2, 3, 7], [4, 7, 8], [5, 8, 9]],
 [[1, 2, 6], [2, 3, 7], [4, 7, 9], [5, 8, 9]],
 [[1, 2, 6], [2, 3, 7], [4, 8, 9], [5, 8, 9]],
 [[1, 2, 6], [3, 6, 8], [4, 7, 8], [5, 7, 9]],
 [[1, 2, 6], [3, 6, 8], [4, 7, 9], [5, 7, 9]],
 [[1, 2, 6], [3, 6, 8], [4, 7, 9], [5, 8, 9]],
 [[1, 2, 6], [3, 7, 8], [4, 7, 9], [5, 8, 9]],
 [[1, 3, 5], [1, 3, 6], [2, 4, 7], [4, 8, 9]],
 [[1, 3, 5], [1, 3, 6], [2, 4, 7], [5, 8, 9]],
 [[1, 3, 5], [1, 4, 6], [2, 4, 7], [5, 8, 9]],
 [[1, 3, 5], [2, 3, 5], [4, 6, 8], [4, 7, 9]],
 [[1, 3, 5], [2, 3, 6], [2, 4, 7], [4, 8, 9]],
 [[1, 3, 5], [2, 3, 6], [2, 4, 7], [5, 8, 9]],
 [[1, 3, 5], [2, 3, 6], [4, 6, 8], [4, 7, 9]],
 [[1, 3, 5], [2, 3, 6], [4, 6, 8], [5, 7, 9]],
 [[1, 3, 5], [2, 3, 6], [4, 7, 8], [4, 7, 9]],
 [[1, 3, 5], [2, 3, 6], [4, 7, 8], [4, 8, 9]],
 [[1, 3, 5], [2, 3, 6], [4, 7, 8], [5, 7, 9]],
 [[1, 3, 5], [2, 3, 6], [4, 7, 8], [5, 8, 9]],
 [[1, 3, 5], [2, 3, 6], [4, 7, 9], [4, 8, 9]],
 [[1, 3, 5], [2, 3, 6], [4, 7, 9], [5, 8, 9]],
 [[1, 3, 5], [2, 4, 6], [4, 6, 8], [5, 7, 9]],
 [[1, 3, 5], [2, 4, 6], [4, 7, 8], [5, 7, 9]],
 [[1, 3, 5], [2, 4, 6], [4, 7, 8], [5, 8, 9]],
 [[1, 3, 5], [2, 4, 6], [4, 7, 9], [5, 8, 9]],
 [[1, 3, 6], [1, 3, 6], [2, 4, 7], [5, 8, 9]],
 [[1, 3, 6], [1, 3, 7], [2, 4, 7], [5, 8, 9]],
 [[1, 3, 6], [1, 3, 7], [2, 4, 8], [2, 5, 9]],
 [[1, 3, 6], [1, 4, 6], [2, 4, 7], [5, 8, 9]],
 [[1, 3, 6], [1, 4, 7], [2, 4, 8], [2, 5, 9]],
 [[1, 3, 6], [1, 4, 7], [2, 4, 8], [3, 5, 9]],
 [[1, 3, 6], [1, 4, 7], [2, 4, 8], [5, 8, 9]],
 [[1, 3, 6], [1, 4, 7], [2, 4, 9], [5, 8, 9]],
 [[1, 3, 6], [1, 4, 7], [2, 5, 7], [5, 8, 9]],
 [[1, 3, 6], [1, 4, 7], [2, 5, 7], [6, 8, 9]],
 [[1, 3, 6], [1, 4, 7], [2, 5, 8], [2, 5, 9]],
 [[1, 3, 6], [1, 4, 7], [2, 5, 8], [2, 6, 9]],
 [[1, 3, 6], [1, 4, 7], [2, 5, 8], [3, 5, 9]],
 [[1, 3, 6], [1, 4, 7], [2, 5, 8], [3, 6, 9]],
 [[1, 3, 6], [1, 4, 7], [2, 5, 8], [5, 8, 9]],
 [[1, 3, 6], [1, 4, 7], [2, 5, 8], [6, 8, 9]],
 [[1, 3, 6], [1, 4, 7], [2, 5, 9], [5, 8, 9]],
 [[1, 3, 6], [1, 4, 7], [2, 5, 9], [6, 8, 9]],
 [[1, 3, 6], [2, 3, 6], [2, 4, 7], [5, 8, 9]],
 [[1, 3, 6], [2, 3, 6], [4, 7, 8], [5, 7, 9]],
 [[1, 3, 6], [2, 3, 6], [4, 7, 8], [5, 8, 9]],
 [[1, 3, 6], [2, 3, 6], [4, 7, 9], [5, 8, 9]],
 [[1, 3, 6], [2, 3, 7], [2, 4, 8], [5, 8, 9]],
 [[1, 3, 6], [2, 3, 7], [2, 4, 9], [5, 8, 9]],
 [[1, 3, 6], [2, 3, 7], [4, 7, 8], [5, 8, 9]],
 [[1, 3, 6], [2, 3, 7], [4, 7, 9], [5, 8, 9]],
 [[1, 3, 6], [2, 3, 7], [4, 8, 9], [5, 8, 9]],
 [[1, 3, 6], [2, 4, 6], [2, 5, 7], [5, 8, 9]],
 [[1, 3, 6], [2, 4, 6], [3, 5, 7], [5, 8, 9]],
 [[1, 3, 6], [2, 4, 6], [4, 7, 8], [5, 7, 9]],
 [[1, 3, 6], [2, 4, 6], [4, 7, 8], [5, 8, 9]],
 [[1, 3, 6], [2, 4, 6], [4, 7, 9], [5, 8, 9]],
 [[1, 3, 6], [2, 4, 6], [5, 7, 8], [5, 7, 9]],
 [[1, 3, 6], [2, 4, 6], [5, 7, 8], [5, 8, 9]],
 [[1, 3, 6], [2, 4, 6], [5, 7, 9], [5, 8, 9]],
 [[1, 3, 6], [2, 4, 7], [2, 4, 7], [5, 8, 9]],
 [[1, 3, 6], [2, 4, 7], [2, 4, 8], [3, 5, 9]],
 [[1, 3, 6], [2, 4, 7], [2, 4, 8], [5, 8, 9]],
 [[1, 3, 6], [2, 4, 7], [2, 4, 9], [5, 8, 9]],
 [[1, 3, 6], [2, 4, 7], [2, 5, 7], [5, 8, 9]],
 [[1, 3, 6], [2, 4, 7], [2, 5, 7], [6, 8, 9]],
 [[1, 3, 6], [2, 4, 7], [2, 5, 8], [3, 5, 9]],
 [[1, 3, 6], [2, 4, 7], [2, 5, 8], [3, 6, 9]],
 [[1, 3, 6], [2, 4, 7], [2, 5, 8], [5, 8, 9]],
 [[1, 3, 6], [2, 4, 7], [2, 5, 8], [6, 8, 9]],
 [[1, 3, 6], [2, 4, 7], [2, 5, 9], [5, 8, 9]],
 [[1, 3, 6], [2, 4, 7], [2, 5, 9], [6, 8, 9]],
 [[1, 3, 6], [2, 4, 7], [3, 4, 7], [5, 8, 9]],
 [[1, 3, 6], [2, 4, 7], [3, 4, 8], [5, 8, 9]],
 [[1, 3, 6], [2, 4, 7], [3, 4, 9], [5, 8, 9]],
 [[1, 3, 6], [2, 4, 7], [3, 5, 7], [5, 8, 9]],
 [[1, 3, 6], [2, 4, 7], [3, 5, 7], [6, 8, 9]],
 [[1, 3, 6], [2, 4, 7], [3, 5, 8], [5, 8, 9]],
 [[1, 3, 6], [2, 4, 7], [3, 5, 8], [6, 8, 9]],
 [[1, 3, 6], [2, 4, 7], [3, 5, 9], [5, 8, 9]],
 [[1, 3, 6], [2, 4, 7], [3, 5, 9], [6, 8, 9]],
 [[1, 3, 6], [2, 4, 7], [4, 7, 8], [5, 8, 9]],
 [[1, 3, 6], [2, 4, 7], [4, 7, 9], [5, 8, 9]],
 [[1, 3, 6], [2, 4, 7], [5, 8, 9], [5, 8, 9]],
 [[1, 3, 6], [2, 4, 7], [5, 8, 9], [6, 8, 9]],
 [[1, 3, 7], [1, 4, 7], [2, 5, 8], [2, 6, 9]],
 [[1, 3, 7], [1, 4, 7], [2, 5, 8], [3, 6, 9]],
 [[1, 3, 7], [1, 4, 7], [2, 5, 8], [6, 8, 9]],
 [[1, 3, 7], [1, 4, 7], [2, 5, 9], [6, 8, 9]],
 [[1, 3, 7], [1, 4, 8], [2, 5, 8], [2, 6, 9]],
 [[1, 3, 7], [1, 4, 8], [2, 5, 8], [3, 6, 9]],
 [[1, 3, 7], [1, 4, 8], [2, 5, 9], [2, 6, 9]],
 [[1, 3, 7], [1, 4, 8], [2, 5, 9], [3, 6, 9]],
 [[1, 3, 7], [1, 4, 8], [2, 5, 9], [6, 8, 9]],
 [[1, 3, 7], [2, 4, 7], [2, 5, 8], [3, 6, 9]],
 [[1, 3, 7], [2, 4, 7], [2, 5, 8], [6, 8, 9]],
 [[1, 3, 7], [2, 4, 7], [2, 5, 9], [6, 8, 9]],
 [[1, 3, 7], [2, 4, 7], [3, 5, 8], [6, 8, 9]],
 [[1, 3, 7], [2, 4, 7], [3, 5, 9], [6, 8, 9]],
 [[1, 3, 7], [2, 4, 7], [5, 8, 9], [6, 8, 9]],
 [[1, 3, 7], [2, 4, 8], [2, 5, 8], [3, 6, 9]],
 [[1, 3, 7], [2, 4, 8], [2, 5, 9], [6, 8, 9]],
 [[1, 3, 7], [2, 4, 8], [3, 5, 9], [6, 8, 9]],
 [[1, 4, 6], [1, 4, 7], [2, 5, 8], [3, 5, 9]],
 [[1, 4, 6], [1, 4, 7], [2, 5, 8], [3, 6, 9]],
 [[1, 4, 6], [1, 5, 7], [2, 5, 8], [3, 6, 9]],
 [[1, 4, 6], [2, 4, 6], [3, 5, 7], [5, 8, 9]],
 [[1, 4, 6], [2, 4, 7], [2, 5, 8], [3, 5, 9]],
 [[1, 4, 6], [2, 4, 7], [2, 5, 8], [3, 6, 9]],
 [[1, 4, 6], [2, 4, 7], [3, 5, 7], [5, 8, 9]],
 [[1, 4, 6], [2, 4, 7], [3, 5, 7], [6, 8, 9]],
 [[1, 4, 6], [2, 4, 7], [3, 5, 8], [3, 5, 9]],
 [[1, 4, 6], [2, 4, 7], [3, 5, 8], [3, 6, 9]],
 [[1, 4, 6], [2, 4, 7], [3, 5, 8], [5, 8, 9]],
 [[1, 4, 6], [2, 4, 7], [3, 5, 8], [6, 8, 9]],
 [[1, 4, 6], [2, 4, 7], [3, 5, 9], [5, 8, 9]],
 [[1, 4, 6], [2, 4, 7], [3, 5, 9], [6, 8, 9]],
 [[1, 4, 6], [2, 5, 7], [2, 5, 8], [3, 6, 9]],
 [[1, 4, 6], [2, 5, 7], [3, 5, 7], [6, 8, 9]],
 [[1, 4, 6], [2, 5, 7], [3, 5, 8], [6, 8, 9]],
 [[1, 4, 6], [2, 5, 7], [3, 5, 9], [6, 8, 9]],
 [[1, 4, 7], [1, 4, 7], [2, 5, 8], [3, 6, 9]],
 [[1, 4, 7], [1, 4, 8], [2, 5, 8], [3, 6, 9]],
 [[1, 4, 7], [1, 4, 8], [2, 5, 9], [3, 6, 9]],
 [[1, 4, 7], [1, 5, 7], [2, 5, 8], [3, 6, 9]],
 [[1, 4, 7], [1, 5, 7], [2, 6, 8], [3, 6, 9]],
 [[1, 4, 7], [1, 5, 8], [2, 5, 8], [3, 6, 9]],
 [[1, 4, 7], [1, 5, 8], [2, 6, 9], [3, 6, 9]],
 [[1, 4, 7], [1, 5, 8], [2, 6, 9], [3, 7, 9]],
 [[1, 4, 7], [2, 4, 7], [2, 5, 8], [3, 6, 9]],
 [[1, 4, 7], [2, 4, 7], [3, 5, 8], [3, 6, 9]],
 [[1, 4, 7], [2, 4, 7], [3, 5, 8], [6, 8, 9]],
 [[1, 4, 7], [2, 4, 7], [3, 5, 9], [6, 8, 9]],
 [[1, 4, 7], [2, 4, 8], [2, 5, 8], [3, 6, 9]],
 [[1, 4, 7], [2, 4, 8], [3, 5, 9], [3, 6, 9]],
 [[1, 4, 7], [2, 4, 8], [3, 5, 9], [6, 8, 9]],
 [[1, 4, 7], [2, 5, 7], [2, 5, 8], [3, 6, 9]],
 [[1, 4, 7], [2, 5, 7], [3, 5, 8], [6, 8, 9]],
 [[1, 4, 7], [2, 5, 7], [3, 5, 9], [6, 8, 9]],
 [[1, 4, 7], [2, 5, 7], [3, 6, 8], [3, 6, 9]],
 [[1, 4, 7], [2, 5, 7], [3, 6, 8], [4, 6, 9]],
 [[1, 4, 7], [2, 5, 7], [3, 6, 8], [6, 8, 9]],
 [[1, 4, 7], [2, 5, 7], [3, 6, 9], [6, 8, 9]],
 [[1, 4, 7], [2, 5, 8], [2, 5, 9], [3, 6, 9]],
 [[1, 4, 7], [2, 5, 8], [2, 6, 8], [3, 6, 9]],
 [[1, 4, 7], [2, 5, 8], [2, 6, 8], [3, 7, 9]],
 [[1, 4, 7], [2, 5, 8], [2, 6, 9], [3, 6, 9]],
 [[1, 4, 7], [2, 5, 8], [2, 6, 9], [3, 7, 9]],
 [[1, 4, 7], [2, 5, 8], [3, 5, 8], [3, 6, 9]],
 [[1, 4, 7], [2, 5, 8], [3, 5, 8], [4, 6, 9]],
 [[1, 4, 7], [2, 5, 8], [3, 5, 9], [3, 6, 9]],
 [[1, 4, 7], [2, 5, 8], [3, 5, 9], [4, 6, 9]],
 [[1, 4, 7], [2, 5, 8], [3, 5, 9], [6, 8, 9]],
 [[1, 4, 7], [2, 5, 8], [3, 6, 8], [3, 6, 9]],
 [[1, 4, 7], [2, 5, 8], [3, 6, 8], [3, 7, 9]],
 [[1, 4, 7], [2, 5, 8], [3, 6, 8], [4, 6, 9]],
 [[1, 4, 7], [2, 5, 8], [3, 6, 8], [4, 7, 9]],
 [[1, 4, 7], [2, 5, 8], [3, 6, 9], [3, 6, 9]],
 [[1, 4, 7], [2, 5, 8], [3, 6, 9], [3, 7, 9]],
 [[1, 4, 7], [2, 5, 8], [3, 6, 9], [4, 6, 9]],
 [[1, 4, 7], [2, 5, 8], [3, 6, 9], [4, 7, 9]],
 [[1, 4, 8], [1, 5, 8], [2, 6, 9], [3, 7, 9]],
 [[1, 4, 8], [2, 5, 8], [2, 6, 9], [3, 7, 9]],
 [[1, 4, 8], [2, 5, 8], [3, 6, 9], [3, 7, 9]],
 [[1, 4, 8], [2, 5, 8], [3, 6, 9], [4, 7, 9]],
 [[1, 5, 7], [2, 5, 7], [3, 6, 8], [4, 6, 9]],
 [[1, 5, 7], [2, 5, 8], [3, 6, 8], [4, 6, 9]],
 [[1, 5, 7], [2, 5, 8], [3, 6, 8], [4, 7, 9]],
 [[1, 5, 7], [2, 5, 8], [3, 6, 9], [4, 6, 9]],
 [[1, 5, 7], [2, 5, 8], [3, 6, 9], [4, 7, 9]],
 [[1, 5, 7], [2, 6, 8], [3, 6, 8], [4, 7, 9]],
 [[1, 5, 7], [2, 6, 8], [3, 6, 9], [4, 7, 9]],
 [[1, 5, 8], [2, 5, 8], [3, 6, 9], [4, 7, 9]],
 [[1, 5, 8], [2, 6, 8], [3, 6, 9], [4, 7, 9]],
 [[1, 5, 8], [2, 6, 8], [3, 7, 9], [4, 7, 9]]]

In [10]:
n=9
p1=representative_promotion_tableaux(L1, n)
len(p1)

34

In [11]:
k,n=3,9
ii=0
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3


([[1, 2, 4], [1, 2, 5], [3, 6, 8], [3, 7, 9]],
 [[1, 2, 4], [1, 2, 5], [1, 6, 7], [1, 7, 8], [3, 4, 5], [3, 4, 5], [1, 2, 9]],
 [[1, 2, 7], [1, 4, 5], [1, 4, 5]])

In [12]:
# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a,a-2,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a-2,a-2,a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)
t4, ContentOfTableauWithMultiplicities(t4)==ContentOfTableauWithMultiplicities(p1[ii])

([[1, 3, 7], [1, 3, 6], [2, 5, 9], [2, 4, 8]], True)

In [13]:
k,n=3,9
ii=1
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3

([[1, 2, 4], [1, 2, 5], [3, 6, 8], [4, 7, 9]],
 [[1, 2, 4], [1, 2, 5], [1, 6, 7], [1, 7, 8], [3, 4, 5], [4, 5, 6], [1, 2, 9]],
 [[1, 2, 7], [1, 4, 5], [1, 5, 6]])

In [14]:
# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a,a-2,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a-2,a-2,a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)
t4, ContentOfTableauWithMultiplicities(t4)==ContentOfTableauWithMultiplicities(p1[ii])


([[1, 4, 7], [1, 3, 6], [2, 5, 9], [2, 4, 8]], True)

In [15]:
k,n=3,9
ii=2
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3

([[1, 2, 4], [1, 4, 7], [3, 5, 8], [3, 6, 9]],
 [[1, 2, 4], [1, 2, 8], [1, 5, 6], [1, 6, 7], [3, 4, 5], [3, 4, 5], [1, 2, 9]],
 [[1, 2, 5], [1, 2, 6], [1, 4, 5]])

In [16]:
# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a-2,a-2,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a-2,a-2,a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)
t4, ContentOfTableauWithMultiplicities(t4)==ContentOfTableauWithMultiplicities(p1[ii])


([[1, 3, 6], [3, 5, 9], [2, 4, 8], [1, 4, 7]], True)

In [17]:
k,n=3,9
ii=3
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3

([[1, 2, 4], [1, 5, 7], [2, 5, 8], [3, 6, 9]],
 [[1, 2, 4],
  [1, 2, 8],
  [1, 5, 6],
  [1, 5, 6],
  [1, 6, 7],
  [2, 3, 4],
  [3, 4, 5],
  [1, 2, 9]],
 [[1, 2, 6], [1, 2, 6], [1, 3, 4], [1, 4, 5]])

In [18]:
# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a-2,a-2,a-2,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a-2,a-2,a-2,a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)
t4, ContentOfTableauWithMultiplicities(t4)==ContentOfTableauWithMultiplicities(p1[ii])

([[1, 3, 6], [2, 5, 9], [2, 5, 8], [1, 4, 7]], True)

In [19]:
k,n=3,9
ii=4
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3


([[1, 2, 4], [1, 5, 7], [2, 6, 8], [3, 6, 9]],
 [[1, 2, 4],
  [1, 2, 8],
  [1, 5, 6],
  [1, 6, 7],
  [1, 6, 7],
  [2, 3, 4],
  [3, 4, 5],
  [1, 2, 9]],
 [[1, 2, 6], [1, 2, 7], [1, 3, 4], [1, 4, 5]])

In [20]:
# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a-2,a-2,a-2,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a-2,a-2,a-2,a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)
t4, ContentOfTableauWithMultiplicities(t4)==ContentOfTableauWithMultiplicities(p1[ii])


([[1, 3, 6], [2, 6, 9], [2, 5, 8], [1, 4, 7]], True)

In [ ]:
k,n=3,9
ii=5
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3

In [ ]:
# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a-2,a-2,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a-2,a-2,a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)
t4, ContentOfTableauWithMultiplicities(t4)==ContentOfTableauWithMultiplicities(p1[ii])


In [ ]:
k,n=3,9
ii=6
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3


In [ ]:
# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a-2,a-2,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a-2,a-2,a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)
t4, ContentOfTableauWithMultiplicities(t4)==ContentOfTableauWithMultiplicities(p1[ii])


In [ ]:
k,n=3,9
ii=7
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3

In [ ]:
# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a-2,a-2,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a-2,a-2,a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)


In [ ]:
k,n=3,9
ii=8
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3

In [ ]:
# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a-2,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a-2,a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)
t4, ContentOfTableauWithMultiplicities(t4)==ContentOfTableauWithMultiplicities(p1[ii])


In [ ]:
k,n=3,9
ii=9
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3

In [ ]:
# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)
t4, ContentOfTableauWithMultiplicities(t4)==ContentOfTableauWithMultiplicities(p1[ii])


In [ ]:
k,n=3,9
ii=10
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3

In [ ]:
# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a-2,a-2,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a-2,a-2,a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)
t4, ContentOfTableauWithMultiplicities(t4)==ContentOfTableauWithMultiplicities(p1[ii])


In [ ]:
k,n=3,9
ii=11
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3

In [ ]:
# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a-2,a-2,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a-2,a-2,a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)
t4, ContentOfTableauWithMultiplicities(t4)==ContentOfTableauWithMultiplicities(p1[ii])


In [ ]:
k,n=3,9
ii=12
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3

In [ ]:
# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a-2,a-2,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a-2,a-2,a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)
t4, ContentOfTableauWithMultiplicities(t4)==ContentOfTableauWithMultiplicities(p1[ii])


In [ ]:
k,n=3,9
ii=13
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3


In [ ]:
# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a-2,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a-2,a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)
t4, ContentOfTableauWithMultiplicities(t4)==ContentOfTableauWithMultiplicities(p1[ii])


In [ ]:
k,n=3,9
ii=14
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3


In [ ]:
# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a-2,a-2,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a-2,a-2,a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)
t4, ContentOfTableauWithMultiplicities(t4)==ContentOfTableauWithMultiplicities(p1[ii])


In [ ]:
k,n=3,9
ii=15
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3

In [ ]:
# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a-2,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a-2,a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)
t4, ContentOfTableauWithMultiplicities(t4)==ContentOfTableauWithMultiplicities(p1[ii])


In [ ]:
k,n=3,9
ii=16
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3

In [ ]:
# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a-2,a-2,a-2,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a-2,a-2,a-2,a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)
t4, ContentOfTableauWithMultiplicities(t4)==ContentOfTableauWithMultiplicities(p1[ii])



In [ ]:
k,n=3,9
ii=17
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3

In [ ]:
# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a-2,a-2,a-2,a-2,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a-2,a-2,a-2,a-2,a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)
t4, ContentOfTableauWithMultiplicities(t4)==ContentOfTableauWithMultiplicities(p1[ii])


In [ ]:
k,n=3,9
ii=18
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3

In [ ]:
# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a-2,a-2,a-2,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a-2,a-2,a-2,a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)
t4, ContentOfTableauWithMultiplicities(t4)==ContentOfTableauWithMultiplicities(p1[ii])



In [ ]:
k,n=3,9
ii=19
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3


In [ ]:
# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a-2,a-2,a-2,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a-2,a-2,a-2,a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)
t4, ContentOfTableauWithMultiplicities(t4)==ContentOfTableauWithMultiplicities(p1[ii])

In [ ]:
k,n=3,9
ii=20
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3

In [ ]:
# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a-2,a-2,a-2,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a-2,a-2,a-2,a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)
t4, ContentOfTableauWithMultiplicities(t4)==ContentOfTableauWithMultiplicities(p1[ii])

In [ ]:
k,n=3,9
ii=21
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3

In [ ]:
# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a-2,a-2,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a-2,a-2,a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)
t4, ContentOfTableauWithMultiplicities(t4)==ContentOfTableauWithMultiplicities(p1[ii])


In [ ]:
k,n=3,9
ii=22
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3

In [ ]:
# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a-2,a-2,a-2,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a-2,a-2,a-2,a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)
t4, ContentOfTableauWithMultiplicities(t4)==ContentOfTableauWithMultiplicities(p1[ii])


In [ ]:
k,n=3,9
ii=23
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3

In [ ]:
# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a-2,a-2,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a-2,a-2,a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)
t4, ContentOfTableauWithMultiplicities(t4)==ContentOfTableauWithMultiplicities(p1[ii])


In [ ]:
k,n=3,9
ii=24
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3


In [ ]:
# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a-2,a-2,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a-2,a-2,a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)
t4, ContentOfTableauWithMultiplicities(t4)==ContentOfTableauWithMultiplicities(p1[ii])

In [ ]:
k,n=3,9
ii=25
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3


In [ ]:
# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a-2,a-2,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a-2,a-2,a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)
t4, ContentOfTableauWithMultiplicities(t4)==ContentOfTableauWithMultiplicities(p1[ii])


In [ ]:
k,n=3,9
ii=26
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3

In [ ]:
# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a-2,a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)
t4, ContentOfTableauWithMultiplicities(t4)==ContentOfTableauWithMultiplicities(p1[ii])


In [ ]:
k,n=3,9
ii=27
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3

In [ ]:
# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a-2,a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)
t4, ContentOfTableauWithMultiplicities(t4)==ContentOfTableauWithMultiplicities(p1[ii])


In [ ]:
k,n=3,9
ii=28
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3

In [ ]:
# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a,a,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a,a-2,a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)
t4, ContentOfTableauWithMultiplicities(t4)==ContentOfTableauWithMultiplicities(p1[ii])

In [ ]:
k,n=3,9
ii=29
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3

In [ ]:
# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a,a,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a,a-2,a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)
t4, ContentOfTableauWithMultiplicities(t4)==ContentOfTableauWithMultiplicities(p1[ii])

In [ ]:
k,n=3,9
ii=30
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3

In [ ]:
# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a,a-2,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a-2,a-2,a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)
t4, ContentOfTableauWithMultiplicities(t4)==ContentOfTableauWithMultiplicities(p1[ii])


In [ ]:
k,n=3,9
ii=31
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3


In [ ]:
# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a-2,a-2,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a-2,a-2,a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)
t4, ContentOfTableauWithMultiplicities(t4)==ContentOfTableauWithMultiplicities(p1[ii])


In [ ]:
k,n=3,9
ii=32
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3


In [ ]:
# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a-2,a-2,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a-2,a-2,a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)
t4, ContentOfTableauWithMultiplicities(t4)==ContentOfTableauWithMultiplicities(p1[ii])


In [ ]:
k,n=3,9
ii=33
e1=initial_cluster_variables_CDFL(k,n)
r1=GVectorOfTableau(k,n,p1[ii],e1)[1]
r2=[]
r3=[]
for i in r1:
    if i[0]>0:
        for j in range(i[0]):
            r2.append(i[1][0])
    else:
        for j in range(-i[0]):
            r3.append(i[1][0])
p1[ii], r2, r3


In [ ]:

# compute cokernel, from g-vector recover profiles 
k=3
a=5
n=9
m=3
t1=direct_sum_of_rank_1_modules(r2, [a,a-2,a-2,a-2,a-2,a-2,a-2,a-2,a-2],n,m)
t2=direct_sum_of_rank_1_modules(r3, [a-2,a-2,a-2,a-2,a-2], n, m)
t3=combineLikeTerms3(t1+Multiplyq2NoChangeValue(t2,-1))
t4=RimCoordinateToRimManyRims(t3, a, n, m)
t4, ContentOfTableauWithMultiplicities(t4)==ContentOfTableauWithMultiplicities(p1[ii])